In [5]:
import os
import sys
import tempfile
import gzip
import math
from functools import partial
from textwrap import wrap
from subprocess import Popen, PIPE
from itertools import groupby, islice
from multiprocessing import Pool

from Bio.SearchIO.HmmerIO import Hmmer3TextParser as HMMERParser


In [6]:
all_germlines = {'J': {'H': {'human': {'IGHJ2*01': '------------------------------------------------------------------------------------------------------------------FDLWGRGTLVTVSS', 'IGHJ1*01': '------------------------------------------------------------------------------------------------------------------FQHWGQGTLVTVSS', 'IGHJ5*01': '------------------------------------------------------------------------------------------------------------------FDSWGQGTLVTVSS', 'IGHJ5*02': '------------------------------------------------------------------------------------------------------------------FDPWGQGTLVTVSS', 'IGHJ4*01': '------------------------------------------------------------------------------------------------------------------FDYWGQGTLVTVSS', 'IGHJ4*02': '------------------------------------------------------------------------------------------------------------------FDYWGQGTLVTVSS', 'IGHJ4*03': '------------------------------------------------------------------------------------------------------------------FDYWGQGTLVTVSS', 'IGHJ6*01': '------------------------------------------------------------------------------------------------------------------MDVWGQGTTVTVSS', 'IGHJ6*04': '------------------------------------------------------------------------------------------------------------------MDVWGKGTTVTVSS', 'IGHJ3*01': '------------------------------------------------------------------------------------------------------------------FDVWGQGTMVTVSS', 'IGHJ3*02': '------------------------------------------------------------------------------------------------------------------FDIWGQGTMVTVSS'}, 'mouse': {'IGHJ3*01': '------------------------------------------------------------------------------------------------------------------FAYWGQGTLVTVSA', 'IGHJ1*01': '------------------------------------------------------------------------------------------------------------------FDVWGAGTTVTVSS', 'IGHJ1*02': '------------------------------------------------------------------------------------------------------------------FDVWGAGTTVTVSS', 'IGHJ1*03': '------------------------------------------------------------------------------------------------------------------FDVWGTGTTVTVSS', 'IGHJ2*02': '------------------------------------------------------------------------------------------------------------------FDYWGQGTSLTVSS', 'IGHJ2*03': '------------------------------------------------------------------------------------------------------------------FDYWGQGTSLTVSS', 'IGHJ2*01': '------------------------------------------------------------------------------------------------------------------FDYWGQGTTLTVSS', 'IGHJ4*01': '------------------------------------------------------------------------------------------------------------------MDYWGQGTSVTVSS'}, 'rabbit': {'IGHJ1*01': '------------------------------------------------------------------------------------------------------------------LDPWGTGTLVTISS', 'IGHJ4*01': '------------------------------------------------------------------------------------------------------------------FNLWGPGTLVTVSS', 'IGHJ4*02': '------------------------------------------------------------------------------------------------------------------FNIWGPGTLVTVSS', 'IGHJ6*01': '------------------------------------------------------------------------------------------------------------------MDLWGPGTLVTVSS', 'IGHJ6*02': '------------------------------------------------------------------------------------------------------------------MDPWGPGTLVTVSS', 'IGHJ2*01': '------------------------------------------------------------------------------------------------------------------FDPWGPGTLVTVSS', 'IGHJ2*02': '------------------------------------------------------------------------------------------------------------------FDPWGPGTLVTVSS', 'IGHJ3*02': '------------------------------------------------------------------------------------------------------------------LDPWGQGTLVTVSS', 'IGHJ3*01': '------------------------------------------------------------------------------------------------------------------LDLWGQGTLVTVSS', 'IGHJ5*01': '------------------------------------------------------------------------------------------------------------------LDLWGQGTLVTVSS', 'IGHJ5*02': '------------------------------------------------------------------------------------------------------------------LDLWGQGTLVTVSS'}, 'rhesus': {'IGHJ5-1*01': '------------------------------------------------------------------------------------------------------------------FDVWGPGVLVTVSS', 'IGHJ5-2*01': '------------------------------------------------------------------------------------------------------------------LDVWGQGVLVTVSS'}, 'pig': {'IGHJ4*01': '------------------------------------------------------------------------------------------------------------------LESWGQGTLVYDAS', 'IGHJ3*01': '------------------------------------------------------------------------------------------------------------------LHSWGRGVEVTVSS', 'IGHJ2*01': '------------------------------------------------------------------------------------------------------------------LDHWGRGVLVTVSS', 'IGHJ5*01': '------------------------------------------------------------------------------------------------------------------MDLWGPGVEVVVSS', 'IGHJ1*01': '------------------------------------------------------------------------------------------------------------------LDSWGQGILVTVSS'}, 'alpaca': {'IGHJ6*01': '------------------------------------------------------------------------------------------------------------------FGSWGQGTQVTVSS', 'IGHJ2*01': '------------------------------------------------------------------------------------------------------------------LEVWGQGTLVTVSS', 'IGHJ5*01': '------------------------------------------------------------------------------------------------------------------FEYWGQGTLVTVS-', 'IGHJ4*01': '------------------------------------------------------------------------------------------------------------------YDYWGQGTQVTVSS', 'IGHJ7*01': '------------------------------------------------------------------------------------------------------------------MDYWGKGTLVTVSS', 'IGHJ3*01': '------------------------------------------------------------------------------------------------------------------LDAWGQGTLVTVSS'}}, 'K': {'human': {'IGKJ3*01': '-------------------------------------------------------------------------------------------------------------------FTFGPGTKVDIK-', 'IGKJ4*01': '-------------------------------------------------------------------------------------------------------------------LTFGGGTKVEIK-', 'IGKJ1*01': '-------------------------------------------------------------------------------------------------------------------WTFGQGTKVEIK-', 'IGKJ5*01': '-------------------------------------------------------------------------------------------------------------------ITFGQGTRLEIK-', 'IGKJ2*01': '-------------------------------------------------------------------------------------------------------------------YTFGQGTKLEIK-'}, 'mouse': {'IGKJ5*01': '-------------------------------------------------------------------------------------------------------------------LTFGAGTKLELK-', 'IGKJ1*02': '-------------------------------------------------------------------------------------------------------------------PTFGGGTKLEIN-', 'IGKJ4*02': '-------------------------------------------------------------------------------------------------------------------FTFGTGTKLEIK-', 'IGKJ4*01': '-------------------------------------------------------------------------------------------------------------------FTFGSGTKLEIK-', 'IGKJ2*02': '-------------------------------------------------------------------------------------------------------------------YTFGSGTKLEMK-', 'IGKJ2*03': '-------------------------------------------------------------------------------------------------------------------YTFGSGTKLEIK-', 'IGKJ1*01': '-------------------------------------------------------------------------------------------------------------------WTFGGGTKLEIK-', 'IGKJ2*01': '-------------------------------------------------------------------------------------------------------------------YTFGGGTKLEIK-'}, 'rat': {'IGKJ2-2*01': '-------------------------------------------------------------------------------------------------------------------DTFGAGTKLELK-', 'IGKJ2-1*01': '-------------------------------------------------------------------------------------------------------------------NTFGAGTKLELK-', 'IGKJ2-3*01': '-------------------------------------------------------------------------------------------------------------------YTFGAGTKLELK-', 'IGKJ5*01': '-------------------------------------------------------------------------------------------------------------------LTFGSGTKLEIK-', 'IGKJ4*01': '-------------------------------------------------------------------------------------------------------------------FTFGSGTKLEIK-', 'IGKJ1*01': '-------------------------------------------------------------------------------------------------------------------WTFGGGTKLELK-'}, 'rabbit': {'IGKJ1-2*01': '------------------------------------------------------------------------------------------------------------------YNAFGGGTEVVVK-', 'IGKJ1-2*02': '------------------------------------------------------------------------------------------------------------------YNAFGGGTEVVVK-', 'IGKJ1-2*03': '------------------------------------------------------------------------------------------------------------------YNTFGGGTKVVVE-', 'IGKJ1-1*03': '-------------------------------------------------------------------------------------------------------------------WAFGAGTNVEIK-', 'IGKJ2-2*01': '------------------------------------------------------------------------------------------------------------------SNTFGAGTKVEIK-', 'IGKJ2-1*01': '-------------------------------------------------------------------------------------------------------------------LTFGAGTKVEIK-', 'IGKJ2-3*01': '-------------------------------------------------------------------------------------------------------------------ITFGKGTKLEIK-'}, 'rhesus': {'IGKJ1*01': '-------------------------------------------------------------------------------------------------------------------WTFGQGTKVEIK-'}, 'pig': {'IGKJ5*01': '-------------------------------------------------------------------------------------------------------------------ITFGEGTSVEIE-', 'IGKJ5*02': '-------------------------------------------------------------------------------------------------------------------ITFGEGTSVEIE-', 'IGKJ2*02': '-------------------------------------------------------------------------------------------------------------------NGFGAGTKLELK-', 'IGKJ2*01': '-------------------------------------------------------------------------------------------------------------------YGFGAGTKLELK-', 'IGKJ3*01': '-------------------------------------------------------------------------------------------------------------------FTFGSGTKVEPK-', 'IGKJ4*01': '-------------------------------------------------------------------------------------------------------------------VVFGSGTKLEIK-', 'IGKJ4*02': '-------------------------------------------------------------------------------------------------------------------VVFGSGTKLEIK-', 'IGKJ1*01': '-------------------------------------------------------------------------------------------------------------------WTFGQGTKLELK-'}}, 'L': {'human': {'IGLJ1*01': '-------------------------------------------------------------------------------------------------------------------YVFGTGTKVTVL-', 'IGLJ6*01': '-------------------------------------------------------------------------------------------------------------------NVFGSGTKVTVL-', 'IGLJ7*02': '-------------------------------------------------------------------------------------------------------------------AVFGGGTQLTAL-', 'IGLJ7*01': '-------------------------------------------------------------------------------------------------------------------AVFGGGTQLTVL-', 'IGLJ3*02': '-------------------------------------------------------------------------------------------------------------------WVFGGGTKLTVL-', 'IGLJ2*01': '-------------------------------------------------------------------------------------------------------------------VVFGGGTKLTVL-', 'IGLJ3*01': '-------------------------------------------------------------------------------------------------------------------VVFGGGTKLTVL-'}, 'mouse': {'IGLJ3*01': '-------------------------------------------------------------------------------------------------------------------FIFGSGTKVTVL-', 'IGLJ2*01': '-------------------------------------------------------------------------------------------------------------------YVFGGGTKVTVL-', 'IGLJ1*01': '-------------------------------------------------------------------------------------------------------------------WVFGGGTKLTVL-'}, 'rat': {'IGLJ1*01': '-------------------------------------------------------------------------------------------------------------------PVFGGGTKLTVL-', 'IGLJ3*01': '-------------------------------------------------------------------------------------------------------------------PVFGGGTKLTVL-'}, 'rabbit': {'IGLJ5*01': '-------------------------------------------------------------------------------------------------------------------YVFGGGTQLTVT-', 'IGLJ6*01': '-------------------------------------------------------------------------------------------------------------------VVFGGGTQLTVT-'}, 'rhesus': {'IGLJ3*01': '-------------------------------------------------------------------------------------------------------------------VLFGGGTRLTVL-', 'IGLJ2*01': '-------------------------------------------------------------------------------------------------------------------GLFGGGTRLTVL-', 'IGLJ5*01': '-------------------------------------------------------------------------------------------------------------------WVFGEGTKLTIL-', 'IGLJ6*01': '-------------------------------------------------------------------------------------------------------------------DVFGSGTKLTVL-', 'IGLJ1*01': '-------------------------------------------------------------------------------------------------------------------YIFGAGTRLTVL-'}, 'pig': {'IGLJ2*01': '-------------------------------------------------------------------------------------------------------------------NIFGGGTHLTVL-', 'IGLJ2*02': '-------------------------------------------------------------------------------------------------------------------NIFGGGTHLTVL-', 'IGLJ3*01': '-------------------------------------------------------------------------------------------------------------------VPFGGGTHLTVL-', 'IGLJ4*01': '-------------------------------------------------------------------------------------------------------------------DRFGRGTRLSVL-'}}, 'A': {'human': {'TRAJ7*01': '------------------------------------------------------------------------------------------------------------------RLAFGKGNQVVVIP', 'TRAJ20*01': '------------------------------------------------------------------------------------------------------------------KLSFGAGTTVTVRA', 'TRAJ16*01': '------------------------------------------------------------------------------------------------------------------KLLFARGTMLKVDL', 'TRAJ16*02': '------------------------------------------------------------------------------------------------------------------KLLFARGTMLKVDL', 'TRAJ41*01': '------------------------------------------------------------------------------------------------------------------ALNFGKGTSLLVTP', 'TRAJ18*01': '------------------------------------------------------------------------------------------------------------------RLYFGRGTQLTVWP', 'TRAJ46*01': '------------------------------------------------------------------------------------------------------------------KLTFGTGTRLAVRP', 'TRAJ13*01': '------------------------------------------------------------------------------------------------------------------KVTFGIGTKLQVIP', 'TRAJ13*02': '------------------------------------------------------------------------------------------------------------------KVTFGTGTKLQVIP', 'TRAJ17*01': '------------------------------------------------------------------------------------------------------------------KLTFGGGTRVLVKP', 'TRAJ39*01': '------------------------------------------------------------------------------------------------------------------MLTFGGGTRLMVKP', 'TRAJ22*01': '------------------------------------------------------------------------------------------------------------------QLTFGSGTQLTVLP', 'TRAJ57*01': '------------------------------------------------------------------------------------------------------------------KLVFGKGTKLTVNP', 'TRAJ50*01': '------------------------------------------------------------------------------------------------------------------KVIFGPGTSLSVIP', 'TRAJ15*01': '------------------------------------------------------------------------------------------------------------------ALIFGKGTTLSVSS', 'TRAJ15*02': '------------------------------------------------------------------------------------------------------------------ALIFGKGTHLSVSS', 'TRAJ56*01': '------------------------------------------------------------------------------------------------------------------KLTFGKGITLSVRP', 'TRAJ11*01': '------------------------------------------------------------------------------------------------------------------TLTFGKGTMLLVSP', 'TRAJ10*01': '------------------------------------------------------------------------------------------------------------------KLTFGTGTQLKVEL', 'TRAJ44*01': '------------------------------------------------------------------------------------------------------------------KLTFGTGTRLQVTL', 'TRAJ28*01': '------------------------------------------------------------------------------------------------------------------QLTFGKGTKLSVIP', 'TRAJ24*02': '------------------------------------------------------------------------------------------------------------------KLQFGAGTQVVVTP', 'TRAJ24*01': '------------------------------------------------------------------------------------------------------------------KFEFGAGTQVVVTP', 'TRAJ24*03': '------------------------------------------------------------------------------------------------------------------KFQFGAGTQVVVTP', 'TRAJ31*01': '------------------------------------------------------------------------------------------------------------------RLMFGDGTQLVVKP', 'TRAJ45*01': '------------------------------------------------------------------------------------------------------------------GLTFGKGTHLIIQP', 'TRAJ38*01': '------------------------------------------------------------------------------------------------------------------KLIWGLGTSLAVNP', 'TRAJ8*01': '------------------------------------------------------------------------------------------------------------------KLVFGTGTRLLVSP', 'TRAJ49*01': '------------------------------------------------------------------------------------------------------------------QFYFGTGTSLTVIP', 'TRAJ35*01': '------------------------------------------------------------------------------------------------------------------VLHCGSGTQVIVLP', 'TRAJ6*01': '------------------------------------------------------------------------------------------------------------------IPTFGRGTSLIVHP', 'TRAJ14*01': '------------------------------------------------------------------------------------------------------------------TFIFGSGTRLSVKP', 'TRAJ36*01': '------------------------------------------------------------------------------------------------------------------NLFFGTGTRLTVIP', 'TRAJ12*01': '------------------------------------------------------------------------------------------------------------------KLIFGSGTRLLVRP', 'TRAJ4*01': '------------------------------------------------------------------------------------------------------------------KLIFGAGTRLAVHP', 'TRAJ29*01': '------------------------------------------------------------------------------------------------------------------PLVFGKGTRLSVIA', 'TRAJ43*01': '------------------------------------------------------------------------------------------------------------------DMRFGAGTRLTVKP', 'TRAJ26*01': '------------------------------------------------------------------------------------------------------------------NFVFGPGTRLSVLP', 'TRAJ9*01': '------------------------------------------------------------------------------------------------------------------KTIFGAGTRLFVKA', 'TRAJ40*01': '------------------------------------------------------------------------------------------------------------------KYIFGTGTRLKVLA', 'TRAJ52*01': '------------------------------------------------------------------------------------------------------------------KLTFGQGTILTVHP', 'TRAJ27*01': '------------------------------------------------------------------------------------------------------------------KSTFGDGTTLTVKP', 'TRAJ37*01': '------------------------------------------------------------------------------------------------------------------KLIFGQGTTLQVKP', 'TRAJ37*02': '------------------------------------------------------------------------------------------------------------------KLIFGQGTTLQVKP', 'TRAJ34*01': '------------------------------------------------------------------------------------------------------------------KLIFGTGTRLQVFP', 'TRAJ32*01': '------------------------------------------------------------------------------------------------------------------KLIFGTGTLLAVQP', 'TRAJ32*02': '------------------------------------------------------------------------------------------------------------------KLIFGTGTLLAVQP', 'TRAJ42*01': '------------------------------------------------------------------------------------------------------------------NLIFGKGTKLSVKP', 'TRAJ53*01': '------------------------------------------------------------------------------------------------------------------KLTFGKGTLLTVNP', 'TRAJ47*01': '------------------------------------------------------------------------------------------------------------------KLVFGAGTILRVKS', 'TRAJ54*01': '------------------------------------------------------------------------------------------------------------------KLVFGQGTRLTINP', 'TRAJ3*01': '------------------------------------------------------------------------------------------------------------------KIIFGSGTRLSIRP', 'TRAJ5*01': '------------------------------------------------------------------------------------------------------------------ALTFGSGTRLQVQP', 'TRAJ30*01': '------------------------------------------------------------------------------------------------------------------KIIFGKGTRLHILP', 'TRAJ23*01': '------------------------------------------------------------------------------------------------------------------KLIFGQGTELSVKP', 'TRAJ23*02': '------------------------------------------------------------------------------------------------------------------KLIFGQGTELSVKP', 'TRAJ48*01': '------------------------------------------------------------------------------------------------------------------KLTFGTGTRLTIIP', 'TRAJ33*01': '------------------------------------------------------------------------------------------------------------------QLIWGAGTKLIIKP', 'TRAJ21*01': '------------------------------------------------------------------------------------------------------------------KFYFGSGTKLNVKP'}, 'mouse': {'TRAJ13*01': '------------------------------------------------------------------------------------------------------------------YQRFGTGTKLQVVP', 'TRAJ26*01': '------------------------------------------------------------------------------------------------------------------GLTFGLGTRVSVFP', 'TRAJ50*01': '------------------------------------------------------------------------------------------------------------------KLVFGQGTSLSVVP', 'TRAJ16*01': '------------------------------------------------------------------------------------------------------------------KLVFGQGTILKVYL', 'TRAJ9*01': '------------------------------------------------------------------------------------------------------------------KLTFGTGTSLLVDP', 'TRAJ2*01': '------------------------------------------------------------------------------------------------------------------KLTFGEGTQVTVIS', 'TRAJ2*02': '------------------------------------------------------------------------------------------------------------------KLTFGEGTQVTVIS', 'TRAJ18*01': '------------------------------------------------------------------------------------------------------------------RLHFGAGTQLIVIP', 'TRAJ58*01': '------------------------------------------------------------------------------------------------------------------KLSFGKGAKLTVSP', 'TRAJ22*01': '------------------------------------------------------------------------------------------------------------------QLIFGSGTQLTVMP', 'TRAJ17*01': '------------------------------------------------------------------------------------------------------------------KLTFGIGTRVLVRP', 'TRAJ57*01': '------------------------------------------------------------------------------------------------------------------KLIFGEGTKLTVSS', 'TRAJ39*01': '------------------------------------------------------------------------------------------------------------------KLTFGGGTRLTVRP', 'TRAJ21*01': '------------------------------------------------------------------------------------------------------------------VLYFGSGTKLTVEP', 'TRAJ30*01': '------------------------------------------------------------------------------------------------------------------KVIFGKGTHLHVLP', 'TRAJ15*01': '------------------------------------------------------------------------------------------------------------------ALIFGTGTTVSVSP', 'TRAJ56*01': '------------------------------------------------------------------------------------------------------------------KLTFGQGTVLSVIP', 'TRAJ11*01': '------------------------------------------------------------------------------------------------------------------KLTFGKGTVLLVSP', 'TRAJ28*01': '------------------------------------------------------------------------------------------------------------------RLTFGKGTKFSLIP', 'TRAJ24*01': '------------------------------------------------------------------------------------------------------------------KLQFGTGTQVVVTP', 'TRAJ31*01': '------------------------------------------------------------------------------------------------------------------RIFFGDGTQLVVKP', 'TRAJ45*01': '------------------------------------------------------------------------------------------------------------------RLTFGKGTQLIIQP', 'TRAJ38*01': '------------------------------------------------------------------------------------------------------------------KLIWGLGTSLVVNP', 'TRAJ12*01': '------------------------------------------------------------------------------------------------------------------KVVFGSGTRLLVSP', 'TRAJ49*01': '------------------------------------------------------------------------------------------------------------------NFYFGKGTSLTVIP', 'TRAJ6*01': '------------------------------------------------------------------------------------------------------------------KPTFGKGTSLVVHP', 'TRAJ5*01': '------------------------------------------------------------------------------------------------------------------QLTFGRGTRLQVYA', 'TRAJ4*02': '------------------------------------------------------------------------------------------------------------------KLTFGAGTRLAVCP', 'TRAJ43*01': '------------------------------------------------------------------------------------------------------------------APRFGAGTKLSVKP', 'TRAJ40*01': '------------------------------------------------------------------------------------------------------------------KYVFGAGTRLKVIA', 'TRAJ52*01': '------------------------------------------------------------------------------------------------------------------KLTFGHGTILRVHP', 'TRAJ27*01': '------------------------------------------------------------------------------------------------------------------KLTFGDGTVLTVKP', 'TRAJ37*01': '------------------------------------------------------------------------------------------------------------------KLIFGLGTTLQVQP', 'TRAJ34*01': '------------------------------------------------------------------------------------------------------------------KVVFGTGTRLQVSP', 'TRAJ32*01': '------------------------------------------------------------------------------------------------------------------KLIFGIGTLLSVKP', 'TRAJ53*01': '------------------------------------------------------------------------------------------------------------------KLTFGKGTLLTVTP', 'TRAJ42*01': '------------------------------------------------------------------------------------------------------------------KLTFGKGTKLSVKS', 'TRAJ33*01': '------------------------------------------------------------------------------------------------------------------QLIWGSGTKLIIKP', 'TRAJ23*01': '------------------------------------------------------------------------------------------------------------------KLIFGQGTKLSIKP', 'TRAJ35*02': '------------------------------------------------------------------------------------------------------------------ALTFGSGTKVIVLP', 'TRAJ35*01': '------------------------------------------------------------------------------------------------------------------ALTFGSGTKVIPCL', 'TRAJ48*01': '------------------------------------------------------------------------------------------------------------------KITFGAGTKLTIKP'}}, 'B': {'human': {'TRBJ1-6*01': '------------------------------------------------------------------------------------------------------------------PLHFGNGTRLTVT-', 'TRBJ1-6*02': '------------------------------------------------------------------------------------------------------------------PLHFGNGTRLTVT-', 'TRBJ1-1*01': '------------------------------------------------------------------------------------------------------------------EAFFGQGTRLTVV-', 'TRBJ1-3*01': '------------------------------------------------------------------------------------------------------------------TIYFGEGSWLTVV-', 'TRBJ2-2*01': '------------------------------------------------------------------------------------------------------------------ELFFGEGSRLTVL-', 'TRBJ1-5*01': '------------------------------------------------------------------------------------------------------------------PQHFGDGTRLSIL-', 'TRBJ2-4*01': '------------------------------------------------------------------------------------------------------------------IQYFGAGTRLSVL-', 'TRBJ2-6*01': '------------------------------------------------------------------------------------------------------------------VLTFGAGSRLTVL-', 'TRBJ1-2*01': '------------------------------------------------------------------------------------------------------------------GYTFGSGTRLTVV-', 'TRBJ2-5*01': '------------------------------------------------------------------------------------------------------------------TQYFGPGTRLLVL-', 'TRBJ2-3*01': '------------------------------------------------------------------------------------------------------------------TQYFGPGTRLTVL-', 'TRBJ2-1*01': '------------------------------------------------------------------------------------------------------------------EQFFGPGTRLTVL-', 'TRBJ2-7*01': '------------------------------------------------------------------------------------------------------------------EQYFGPGTRLTVT-', 'TRBJ1-4*01': '------------------------------------------------------------------------------------------------------------------KLFFGSGTQLSVL-'}, 'mouse': {'TRBJ1-6*01': '------------------------------------------------------------------------------------------------------------------PLYFGMGTRLTVT-', 'TRBJ1-1*01': '------------------------------------------------------------------------------------------------------------------EVFFGKGTRLTVV-', 'TRBJ1-3*01': '------------------------------------------------------------------------------------------------------------------TLYFGEGSRLIVV-', 'TRBJ2-2*01': '------------------------------------------------------------------------------------------------------------------QLYFGEGSKLTVL-', 'TRBJ1-2*01': '------------------------------------------------------------------------------------------------------------------DYTFGSGTRLLVI-', 'TRBJ1-5*01': '------------------------------------------------------------------------------------------------------------------APLFGEGTRLSVL-', 'TRBJ1-5*03': '------------------------------------------------------------------------------------------------------------------AQHFGEGTRLSVL-', 'TRBJ2-4*01': '------------------------------------------------------------------------------------------------------------------TQYFGAGTRLTVL-', 'TRBJ2-5*01': '------------------------------------------------------------------------------------------------------------------TQYFGPGTRLLVL-', 'TRBJ2-3*01': '------------------------------------------------------------------------------------------------------------------TLYFGSGTRLTVL-', 'TRBJ2-1*01': '------------------------------------------------------------------------------------------------------------------EQFFGPGTRLTVL-', 'TRBJ2-7*01': '------------------------------------------------------------------------------------------------------------------EQYFGPGTRLTVL-', 'TRBJ2-6*01': '------------------------------------------------------------------------------------------------------------------EQYFGSGTRLTVI-', 'TRBJ1-4*01': '------------------------------------------------------------------------------------------------------------------RLFFGHGTKLSVL-', 'TRBJ1-4*02': '------------------------------------------------------------------------------------------------------------------RLFFGHGTKLSVL-'}}, 'G': {'human': {'TRGJP2*01': '------------------------------------------------------------------------------------------------------------------IKTFAKGTRLIVTS', 'TRGJP1*01': '------------------------------------------------------------------------------------------------------------------FKIFAEGTKLIVTS', 'TRGJ1*01': '------------------------------------------------------------------------------------------------------------------KKLFGSGTTLVVT-', 'TRGJ1*02': '------------------------------------------------------------------------------------------------------------------KKLFGSGTTLVVT-', 'TRGJ2*01': '------------------------------------------------------------------------------------------------------------------KKLFGSGTTLVVT-', 'TRGJP*01': '------------------------------------------------------------------------------------------------------------------IKVFGPGTKLIIT-'}, 'mouse': {'TRGJ4*01': '------------------------------------------------------------------------------------------------------------------VKIFAKGTKLVVIP', 'TRGJ1*01': '------------------------------------------------------------------------------------------------------------------HKVFAEGTKLIVIP', 'TRGJ2*01': '------------------------------------------------------------------------------------------------------------------HKVFAEGTKLIVIP', 'TRGJ3*01': '------------------------------------------------------------------------------------------------------------------HKVFAEGTKLIVIP'}}, 'D': {'human': {'TRDJ3*01': '------------------------------------------------------------------------------------------------------------------QMFFGTGIKLFVEP', 'TRDJ4*01': '------------------------------------------------------------------------------------------------------------------PLIFGKGTYLEVQQ', 'TRDJ2*01': '------------------------------------------------------------------------------------------------------------------QLFFGKGTQLIVEP', 'TRDJ1*01': '------------------------------------------------------------------------------------------------------------------KLIFGKGTRVTVEP'}, 'mouse': {'TRDJ2*02': '------------------------------------------------------------------------------------------------------------------TDVFGTGIELFVEP', 'TRDJ2*01': '------------------------------------------------------------------------------------------------------------------QMFFGTGIELFVEP', 'TRDJ1*01': '------------------------------------------------------------------------------------------------------------------KLVFGQGTQVTVEP'}}}, 'V': {'H': {'human': {'IGHV1-18*01': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYGISWVRQAPGQGLEWMGWISAY--NGNTNYAQKLQ-GRVTMTTDTSTSTAYMELRSLRSDDTAVYYCAR----------------------', 'IGHV1-18*03': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYGISWVRQAPGQGLEWMGWISAY--NGNTNYAQKLQ-GRVTMTTDTSTSTAYMELRSLRSDDMAVYYCAR----------------------', 'IGHV1-18*04': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYGISWVRQAPGQGLEWMGWISAY--NGNTNYAQKLQ-GRVTMTTDTSTSTAYMELRSLRSDDTAVYYCAR----------------------', 'IGHV1-2*01': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TGYYMHWVRQAPGQGLEWMGRINPN--SGGTNYAQKFQ-GRVTSTRDTSISTAYMELSRLRSDDTVVYYCAR----------------------', 'IGHV1-2*02': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TGYYMHWVRQAPGQGLEWMGWINPN--SGGTNYAQKFQ-GRVTMTRDTSISTAYMELSRLRSDDTAVYYCAR----------------------', 'IGHV1-2*04': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TGYYMHWVRQAPGQGLEWMGWINPN--SGGTNYAQKFQ-GWVTMTRDTSISTAYMELSRLRSDDTAVYYCAR----------------------', 'IGHV1-2*06': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TGYYMHWVRQAPGQGLEWMGRINPN--SGGTNYAQKFQ-GRVTMTRDTSISTAYMELSRLRSDDTAVYYCAR----------------------', 'IGHV1-24*01': 'QVQLVQSGA-EVKKPGASVKVSCKVSGYTL----TELSMHWVRQAPGKGLEWMGGFDPE--DGETIYAQKFQ-GRVTMTEDTSTDTAYMELSSLRSEDTAVYYCAT----------------------', 'IGHV1-3*01': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYAMHWVRQAPGQRLEWMGWINAG--NGNTKYSQKFQ-GRVTITRDTSASTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-3*02': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYAMHWVRQAPGQRLEWMGWSNAG--NGNTKYSQEFQ-GRVTITRDTSASTAYMELSSLRSEDMAVYYCAR----------------------', 'IGHV1-3*03': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYAMHWVRQAPGQRLEWMGWINAG--NGNTKYSQEFQ-GRVTITRDTSASTAYMELSSLRSEDMAVYYCAR----------------------', 'IGHV1-45*01': 'QMQLVQSGA-EVKKTGSSVKVSCKASGYTF----TYRYLHWVRQAPGQALEWMGWITPF--NGNTNYAQKFQ-DRVTITRDRSMSTAYMELSSLRSEDTAMYYCAR----------------------', 'IGHV1-45*02': 'QMQLVQSGA-EVKKTGSSVKVSCKASGYTF----TYRYLHWVRQAPGQALEWMGWITPF--NGNTNYAQKFQ-DRVTITRDRSMSTAYMELSSLRSEDTAMYYCAR----------------------', 'IGHV1-45*03': 'QMQLVQSGA-EVKKTGSSVKVSCKASGYTF----TYRYLHWVRQAPRQALEWMGWITPF--NGNTNYAQKFQ-DRVTITRDRSMSTAYMELSSLRSEDTAMYYCAR----------------------', 'IGHV1-46*01': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYYMHWVRQAPGQGLEWMGIINPS--GGSTSYAQKFQ-GRVTMTRDTSTSTVYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-46*02': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----NSYYMHWVRQAPGQGLEWMGIINPS--GGSTSYAQKFQ-GRVTMTRDTSTSTVYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-46*03': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYYMHWVRQAPGQGLEWMGIINPS--GGSTSYAQKFQ-GRVTMTRDTSTSTVYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-46*04': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYYMHWVRQAPGQGLEWMGIINPS--GGSTSYAQKLQ-GRVTMTRDTSTSTVYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-58*01': 'QMQLVQSGP-EVKKPGTSVKVSCKASGFTF----TSSAVQWVRQARGQRLEWIGWIVVG--SGNTNYAQKFQ-ERVTITRDMSTSTAYMELSSLRSEDTAVYYCAA----------------------', 'IGHV1-58*02': 'QMQLVQSGP-EVKKPGTSVKVSCKASGFTF----TSSAMQWVRQARGQRLEWIGWIVVG--SGNTNYAQKFQ-ERVTITRDMSTSTAYMELSSLRSEDTAVYYCAA----------------------', 'IGHV1-69*01': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGTANYAQKFQ-GRVTITADESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*02': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYTISWVRQAPGQGLEWMGRIIPI--LGIANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*04': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGRIIPI--LGIANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*05': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGTANYAQKFQ-GRVTITTDESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*06': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGTANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*08': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYTISWVRQAPGQGLEWMGRIIPI--LGTANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*09': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGRIIPI--LGIANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*10': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--LGIANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*11': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGRIIPI--LGTANYAQKFQ-GRVTITADESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*12': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGTANYAQKFQ-GRVTITADESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*13': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGTANYAQKFQ-GRVTITADESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*14': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGTANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*15': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGRIIPI--FGTANYAQKFQ-GRVTITADESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*16': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYTISWVRQAPGQGLEWMGGIIPI--LGTANYAQKFQ-GRVTITTDESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69*17': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGIANYAQKFQ-GRVTITADKSTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-69-2*01': 'EVQLVQSGA-EVKKPGATVKISCKVSGYTF----TDYYMHWVQQAPGKGLEWMGLVDPE--DGETIYAEKFQ-GRVTITADTSTDTAYMELSSLRSEDTAVYYCAT----------------------', 'IGHV1-69D*01': 'QVQLVQSGA-EVKKPGSSVKVSCKASGGTF----SSYAISWVRQAPGQGLEWMGGIIPI--FGTANYAQKFQ-GRVTITADESTSTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-8*01': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYDINWVRQATGQGLEWMGWMNPN--SGNTGYAQKFQ-GRVTMTRNTSISTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV1-8*03': 'QVQLVQSGA-EVKKPGASVKVSCKASGYTF----TSYDINWVRQATGQGLEWMGWMNPN--SGNTGYAQKFQ-GRVTITRNTSISTAYMELSSLRSEDTAVYYCAR----------------------', 'IGHV2-26*01': 'QVTLKESGP-VLVKPTETLTLTCTVSGFSLS--NARMGVSWIRQPPGKALEWLAHIFSN---DEKSYSTSLK-SRLTISKDTSKSQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-26*02': 'QVTLKESGP-VLVKPTETLTLTCTVSGFSLS--NARMGVSWIRQPPGKALEWLAHIFSN---DEKSYSTSLK-SRLTISKDTSKSQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-26*03': 'QVTLKESGP-VLVKPTETLTLTCTISGFSLS--NARMGVSWIRQPPGKALEWLAHIFSN---DEKSYSTSLK-SRLTISKDTSKSQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-5*01': 'QITLKESGP-TLVKPTQTLTLTCTFSGFSLS--TSGVGVGWIRQPPGKALEWLALIYWN---DDKRYSPSLK-SRLTITKDTSKNQVVLTMTNMDPVDTATYYCAHR---------------------', 'IGHV2-5*02': 'QITLKESGP-TLVKPTQTLTLTCTFSGFSLS--TSGVGVGWIRQPPGKALEWLALIYWD---DDKRYSPSLK-SRLTITKDTSKNQVVLTMTNMDPVDTATYYCAHR---------------------', 'IGHV2-5*05': 'QITLKESGP-TLVKPTQTLTLTCTFSGFSLS--TSGVGVGWIRQPPGKALEWLALIYWD---DDKRYGPSLK-SRLTITKDTSKNQVVLTMTNMDPVDTATYYCAHR---------------------', 'IGHV2-5*06': 'QITLKESGP-TLVKPTQTLTLTCTFSGFSLS--TSGVGVGWIRQPPGKALEWLALIYWD---DDKRYGPSLK-SRLTITKDTSKNQVVLTMTNMDPVDTATYYCAHR---------------------', 'IGHV2-5*08': 'QVTLKESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMRVSWIRQPPGKALEWLALIYWD---DDKRYSPSLK-SRLTITKDTSKNQVVLTMTNMDPVDTATYYCAHR---------------------', 'IGHV2-5*09': 'QVTLKESGP-TLVKPTQTLTLTCTFSGFSLS--TSGVGVGWIRQPPGKALEWLALIYWD---DDKRYGPSLK-SRLTITKDTSKNQVVLTMTNMDPVDTATYYCAHR---------------------', 'IGHV2-70*01': 'QVTLRESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMCVSWIRQPPGKALEWLALIDWD---DDKYYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*04': 'QVTLKESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMRVSWIRQPPGKALEWLARIDWD---DDKFYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*10': 'QVTLKESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMRVSWIRQPPGKALEWIARIDWD---DDKYYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*11': 'RVTLRESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMCVSWIRQPPGKALEWLARIDWD---DDKYYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*12': 'QITLKESGP-TLVKPTQTLTLTCTFSGFSLS--TSGMCVSWIRQPPGKALEWLALIDWD---DDKYYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCAHR---------------------', 'IGHV2-70*13': 'QVTLRESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMCVSWIRQPPGKALEWLALIDWD---DDKYYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*15': 'QVTLRESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMCVSWIRQPPGKALEWLARIDWD---DDKYYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*16': 'QVTLKESGP-VLVKPTQTLTLTCTFSGFSLS--TSGMCVSWIRQPPGKALEWLARIDWD---DDKFYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*17': 'QVTLRESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMCVSWIRQPPGKALEWLARIDWD---DDKFYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*18': 'QVTLRESGP-ALVKPTQTLTLTCTFSGFSLS--TSEMCVSWVRQPPGKALEWLALIDWD---DDKYYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70*19': 'QVTLRESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMCVSWVRQPPGKALEWLALIDWD---DDKHYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70D*04': 'QVTLKESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMRVSWIRQPPGKALEWLARIDWD---DDKFYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV2-70D*14': 'QVTLKESGP-ALVKPTQTLTLTCTFSGFSLS--TSGMRVSWIRQPPGKALEWLARIDWD---DDKFYSTSLK-TRLTISKDTSKNQVVLTMTNMDPVDTATYYCARI---------------------', 'IGHV3-11*01': 'QVQLVESGG-GLVKPGGSLRLSCAASGFTF----SDYYMSWIRQAPGKGLEWVSYISSS--GSTIYYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-11*03': 'QVQLLESGG-GLVKPGGSLRLSCAASGFTF----SDYYMSWIRQAPGKGLEWVSYISSS--SSYTNYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-11*05': 'QVQLVESGG-GLVKPGGSLRLSCAASGFTF----SDYYMSWIRQAPGKGLEWVSYISSS--SSYTNYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-11*06': 'QVQLVESGG-GLVKPGGSLRLSCAASGFTF----SDYYMSWIRQAPGKGLEWVSYISSS--SSYTNYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-13*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYDMHWVRQATGKGLEWVSAIGTA---GDTYYPGSVK-GRFTISRENAKNSLYLQMNSLRAGDTAVYYCAR----------------------', 'IGHV3-13*02': 'EVHLVESGG-GLVQPGGALRLSCAASGFTF----SNYDMHWVRQATGKGLEWVSANGTA---GDTYYPGSVK-GRFTISRENAKNSLYLQMNSLRAGDTAVYYCAR----------------------', 'IGHV3-13*03': 'EVQLVESGG-GLVQPGGSLRLSCAACGFTF----SSYDMHWVRQATGKGLEWVSAIGTA---GDTYYPGSVK-GQFTISRENAKNSLYLQMNSLRAGDTAVYYCAR----------------------', 'IGHV3-13*04': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYDMHWVRQATGKGLEWVSAIGTA---GDTYYPGSVK-GRFTISRENAKNSLYLQMNSLRAGDTAVYYCAR----------------------', 'IGHV3-13*05': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYDMHWVRQATGKGLEWVSAIGTA---GDPYYPGSVK-GRFTISRENAKNSLYLQMNSLRAGDTAVYYCAR----------------------', 'IGHV3-15*01': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SNAWMSWVRQAPGKGLEWVGRIKSKTDGGTTDYAAPVK-GRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTT----------------------', 'IGHV3-15*02': 'EVQLVESGG-ALVKPGGSLRLSCAASGFTF----SNAWMSWVRQAPGKGLEWVGRIKSKTDGGTTDYAAPVK-GRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTT----------------------', 'IGHV3-15*03': 'EVQLVESAG-ALVQPGGSLRLSCAASGFTC----SNAWMSWVRQAPGKGLEWVGRIKSKANGGTTDYAAPVK-GRFTISRVDSKNTLYLQMNSLKTEDTAVYYCTT----------------------', 'IGHV3-15*04': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SNAWMSWVRQAPGKGLEWVGRIESKTDGGTTDYAAPVK-GRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTT----------------------', 'IGHV3-15*05': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SNAWMSWVRQAPGKGLEWVGRIKSKTDGGTTDYAAPVK-GRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTT----------------------', 'IGHV3-15*06': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SNAWMSWVRQAPGKGLEWVGRIKSKTDGGTTNYAAPVK-GRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTT----------------------', 'IGHV3-15*07': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SNAWMNWVRQAPGKGLEWVGRIKSKTDGGTTDYAAPVK-GRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTT----------------------', 'IGHV3-15*08': 'EVQLVESAG-GLVQPGGSLRLSCAASGFTC----SNAWMSWVRQAPGKGLEWVGCIKSKANGGTTDYAAPVK-GRFTISRDDSKNTLYLQMISLKTEDTAVYYCTT----------------------', 'IGHV3-20*01': 'EVQLVESGG-GVVRPGGSLRLSCAASGFTF----DDYGMSWVRQAPGKGLEWVSGINWN--GGSTGYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTALYHCAR----------------------', 'IGHV3-20*04': 'EVQLVESGG-GVVRPGGSLRLSCAASGFTF----DDYGMSWVRQAPGKGLEWVSGINWN--GGSTGYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTALYYCAR----------------------', 'IGHV3-21*01': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SSYSMNWVRQAPGKGLEWVSSISSS--SSYIYYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-21*02': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SSYSMNWVRQAPGKGLEWVSSISSS--SSYIYYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-21*03': 'EVQLVESGG-GLVKPGGSLRLSCAASGFTF----SSYSMNWVRQAPGKGLEWVSSISSS--SSYIYYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-23*01': 'EVQLLESGG-GLVQPGGSLRLSCAASGFTF----SSYAMSWVRQAPGKGLEWVSAISGS--GGSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-23*02': 'EVQLLESGG-GLVQPGGSLRLSCAASGFTF----SSYAMSWVRQAPGKGLEWVSAISGS--GGSTYYGDSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-23*03': 'EVQLLESGG-GLVQPGGSLRLSCAASGFTF----SSYAMSWVRQAPGKGLEWVSVIYSG--GSSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-23*04': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYAMSWVRQAPGKGLEWVSAISGS--GGSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-23D*01': 'EVQLLESGG-GLVQPGGSLRLSCAASGFTF----SSYAMSWVRQAPGKGLEWVSAISGS--GGSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-30*01': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*02': 'QVQLVESGG-GVVQPGGSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAFIRYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-30*03': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*04': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*05': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEGTAVYYCAR----------------------', 'IGHV3-30*06': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*07': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*08': 'QVQLVDSGG-GVVQPGRSLRLSCAASAFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*09': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFAISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*10': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYTDSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*11': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*12': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*13': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNRLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*14': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*15': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMSSLRAEDTAVYYCAR----------------------', 'IGHV3-30*16': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*17': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30*18': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-30*19': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-30-3*02': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-30-3*03': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-33*01': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVIWYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-33*02': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVIWYD--GSNKYYADSAK-GRFTISRDNSTNTLFLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-33*03': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVIWYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-33*04': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVIWYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-33*05': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVISYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-33*06': 'QVQLVESGG-GVVQPGRSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVAVIWYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV3-33*07': 'QVQLVESGG-RVVQPGRSLRLSCAASGFTF----SRYGMYWVRQAPGKGLEWVAVIWYD--GSNKYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-43*01': 'EVQLVESGG-VVVQPGGSLRLSCAASGFTF----DDYTMHWVRQAPGKGLEWVSLISWD--GGSTYYADSVK-GRFTISRDNSKNSLYLQMNSLRTEDTALYYCAKD---------------------', 'IGHV3-43*02': 'EVQLVESGG-GVVQPGGSLRLSCAASGFTF----DDYAMHWVRQAPGKGLEWVSLISGD--GGSTYYADSVK-GRFTISRDNSKNSLYLQMNSLRTEDTALYYCAKD---------------------', 'IGHV3-43D*03': 'EVQLVESGG-VVVQPGGSLRLSCAASGFTF----DDYAMHWVRQAPGKGLEWVSLISWD--GGSTYYADSVK-GRFTISRDNSKNSLYLQMNSLRAEDTALYYCAKD---------------------', 'IGHV3-43D*04': 'EVQLVESGG-VVVQPGGSLRLSCAASGFTF----DDYAMHWVRQAPGKGLEWVSLISWD--GGSTYYADSVK-GRFTISRDNSKNSLYLQMNSLRAEDTALYYCAKD---------------------', 'IGHV3-48*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYSMNWVRQAPGKGLEWVSYISSS--SSTIYYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-48*02': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYSMNWVRQAPGKGLEWVSYISSS--SSTIYYADSVK-GRFTISRDNAKNSLYLQMNSLRDEDTAVYYCAR----------------------', 'IGHV3-48*03': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYEMNWVRQAPGKGLEWVSYISSS--GSTIYYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-49*01': 'EVQLVESGG-GLVQPGRSLRLSCTASGFTF----GDYAMSWFRQAPGKGLEWVGFIRSKAYGGTTEYTASVK-GRFTISRDGSKSIAYLQMNSLKTEDTAVYYCTR----------------------', 'IGHV3-49*02': 'EVQLVESGG-GLVQPGPSLRLSCTASGFTF----GYYPMSWVRQAPGKGLEWVGFIRSKAYGGTTEYAASVK-GRFTISRDDSKSIAYLQMNSLKTEDTAVYYCTR----------------------', 'IGHV3-49*03': 'EVQLVESGG-GLVQPGRSLRLSCTASGFTF----GDYAMSWFRQAPGKGLEWVGFIRSKAYGGTTEYAASVK-GRFTISRDDSKSIAYLQMNSLKTEDTAVYYCTR----------------------', 'IGHV3-49*04': 'EVQLVESGG-GLVQPGRSLRLSCTASGFTF----GDYAMSWVRQAPGKGLEWVGFIRSKAYGGTTEYAASVK-GRFTISRDDSKSIAYLQMNSLKTEDTAVYYCTR----------------------', 'IGHV3-49*05': 'EVQLVESGG-GLVKPGRSLRLSCTASGFTF----GDYAMSWFRQAPGKGLEWVGFIRSKAYGGTTEYAASVK-GRFTISRDDSKSIAYLQMNSLKTEDTAVYYCTR----------------------', 'IGHV3-53*01': 'EVQLVESGG-GLIQPGGSLRLSCAASGFTV----SSNYMSWVRQAPGKGLEWVSVIYSG---GSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-53*02': 'EVQLVETGG-GLIQPGGSLRLSCAASGFTV----SSNYMSWVRQAPGKGLEWVSVIYSG---GSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-53*03': 'EVQLVESGG-GLIQPGGSLRLSCAASGFTV----SSNYMSWVRQPPGKGLEWVSVIYSG---GSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-62*04': 'EVQLVKSGG-GLVQPGGSLRLSCAASGFTF----SSSAMHWVRQAPRKGLEWVSVISTS--GDTVLYTDSVK-GRFTISRDNAQNSLSLQMNSLRAEDMAVYYCVK----------------------', 'IGHV3-64*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEYVSAISSN--GGSTYYANSVK-GRFTISRDNSKNTLYLQMGSLRAEDMAVYYCAR----------------------', 'IGHV3-64*02': 'EVQLVESGE-GLVQPGGSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEYVSAISSN--GGSTYYADSVK-GRFTISRDNSKNTLYLQMGSLRAEDMAVYYCAR----------------------', 'IGHV3-64*03': 'EVQLVESGG-GLVQPGGSLRLSCSASGFTF----SSYAMHWVRQAPGKGLEYVSAISSN--GGSTYYADSVK-GRFTISRDNSKNTLYVQMSSLRAEDTAVYYCVK----------------------', 'IGHV3-64*04': 'QVQLVESGG-GLVQPGGSLRLSCSASGFTF----SSYAMHWVRQAPGKGLEYVSAISSN--GGSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-64*05': 'EVQLVESGG-GLVQPGGSLRLSCSASGFTF----SSYAMHWVRQAPGKGLEYVSAISSN--GGSTYYADSVK-GRFTISRDNSKNTLYVQMSSLRAEDTAVYYCVK----------------------', 'IGHV3-64*07': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYAMHWVRQAPGKGLEYVSAISSN--GGSTYYADSVK-GRFTISRDNSKNTLYLQMGSLRAEDMAVYYCAR----------------------', 'IGHV3-64D*06': 'EVQLVESGG-GLVQPGGSLRLSCSASGFTF----SSYAMHWVRQAPGKGLEYVSAISSN--GGSTYYADSVK-GRFTISRDNSKNTLYLQMSSLRAEDTAVYYCVK----------------------', 'IGHV3-66*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTV----SSNYMSWVRQAPGKGLEWVSVIYSG---GSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-66*02': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTV----SSNYMSWVRQAPGKGLEWVSVIYSG---GSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-66*03': 'EVQLVESGG-GLIQPGGSLRLSCAASGFTV----SSNYMSWVRQAPGKGLEWVSVIYSC---GSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-66*04': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTV----SSNYMSWVRQAPGKGLEWVSVIYSG---GSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-7*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYWMSWVRQAPGKGLEWVANIKQD--GSEKYYVDSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-7*02': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYWMSWVRQAPGKGLEWVANIKQD--GSEKYYVDSVK-GRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-72*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SDHYMDWVRQAPGKGLEWVGRTRNKANSYTTEYAASVK-GRFTISRDDSKNSLYLQMNSLKTEDTAVYYCAR----------------------', 'IGHV3-73*01': 'EVQLVESGG-GLVQPGGSLKLSCAASGFTF----SGSAMHWVRQASGKGLEWVGRIRSKANSYATAYAASVK-GRFTISRDDSKNTAYLQMNSLKTEDTAVYYCTR----------------------', 'IGHV3-73*02': 'EVQLVESGG-GLVQPGGSLKLSCAASGFTF----SGSAMHWVRQASGKGLEWVGRIRSKANSYATAYAASVK-GRFTISRDDSKNTAYLQMNSLKTEDTAVYYCTR----------------------', 'IGHV3-74*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYWMHWVRQAPGKGLVWVSRINSD--GSSTSYADSVK-GRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-74*02': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYWMHWVRQAPGKGLVWVSRINSD--GSSTSYADSVK-GRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-74*03': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYWMHWVRQAPGKGLVWVSRINSD--GSSTTYADSVK-GRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAR----------------------', 'IGHV3-9*01': 'EVQLVESGG-GLVQPGRSLRLSCAASGFTF----DDYAMHWVRQAPGKGLEWVSGISWN--SGSIGYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTALYYCAKD---------------------', 'IGHV3-9*02': 'EVQLVESGG-GLVQPGRSLRLSCAASGFTS----DDYAMHWVRQAPGKGLEWVSGISWN--SGSIGYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDTALYYCAKD---------------------', 'IGHV3-9*03': 'EVQLVESGG-GLVQPGRSLRLSCAASGFTF----DDYAMHWVRQAPGKGLEWVSGISWN--SGSIGYADSVK-GRFTISRDNAKNSLYLQMNSLRAEDMALYYCAKD---------------------', 'IGHV3-NL1*01': 'QVQLVESGG-GVVQPGGSLRLSCAASGFTF----SSYGMHWVRQAPGKGLEWVSVIYSG--GSSTYYADSVK-GRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAK----------------------', 'IGHV4-28*01': 'QVQLQESGP-GLVKPSDTLSLTCAVSGYSIS---SSNWWGWIRQPPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTMSVDTSKNQFSLKLSSVTAVDTAVYYCAR----------------------', 'IGHV4-28*02': 'QVQLQESGP-GLVKPSQTLSLTCAVSGYSIS---SSNWWGWIRQPPGKGLEWIGYIYYS---GSIYYNPSLK-SRVTMSVDTSKNQFSLKLSSVTAVDTAVYYCAR----------------------', 'IGHV4-28*03': 'QVQLQESGP-GLVKPSDTLSLTCAVSGYSIS---SSNWWGWIRQPPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTMSVDTSKNQFSLKLSSVTAVDTAVYYCAR----------------------', 'IGHV4-28*04': 'QVQLQESGP-GLVKPSDTLSLTCAVSGYSIS---SSNWWGWIRQPPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTMSVDTSKNQFSLKLSSVTAVDTGVYYCAR----------------------', 'IGHV4-28*07': 'QVQLQESGP-GLVKPSDTLSLTCAVSGYSIS---SSNWWGWIRQPPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTMSVDTSKNQFSLKLSSVTAVDTAVYYCAR----------------------', 'IGHV4-30-2*01': 'QLQLQESGS-GLVKPSQTLSLTCAVSGGSIS--SGGYSWSWIRQPPGKGLEWIGYIYHS---GSTYYNPSLK-SRVTISVDRSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-30-2*03': 'QLQLQESGS-GLVKPSQTLSLTCAVSGGSIS--SGGYSWSWIRQPPGKGLEWIGSIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-30-2*05': 'QLQLQESGS-GLVKPSQTLSLTCAVSGGSIS--SGGYSWSWIRQPPGKGLEWIGYIYHS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-30-2*06': 'QLQLQESGS-GLVKPSQTLSLTCAVSGGSIS--SGGYSWSWIRQSPGKGLEWIGYIYHS---GSTYYNPSLK-SRVTISVDRSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-30-4*01': 'QVQLQESGP-GLVKPSQTLSLTCTVSGGSIS--SGDYYWSWIRQPPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-30-4*02': 'QVQLQESGP-GLVKPSDTLSLTCTVSGGSIS--SGDYYWSWIRQPPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-30-4*07': 'QVQLQESGP-GLVKPSQTLSLTCAVSGGSIS--SGGYSWSWIRQPPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-31*01': 'QVQLQESGP-GLVKPSQTLSLTCTVSGGSIS--SGGYYWSWIRQHPGKGLEWIGYIYYS---GSTYYNPSLK-SLVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-31*02': 'QVQLQESGP-GLVKPSQTLSLTCTVSGGSIS--SGGYYWSWIRQHPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-31*03': 'QVQLQESGP-GLVKPSQTLSLTCTVSGGSIS--SGGYYWSWIRQHPGKGLEWIGYIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-31*10': 'QVQLQESGP-GLLKPSQTLSLTCTVSGGSIS--SGGYYWSWIRQHPGKGLEWIGCIYYS---GSTYYNPSLK-SRVTISVDPSKNQFSLKPSSVTAADTAVDYCAR----------------------', 'IGHV4-34*01': 'QVQLQQWGA-GLLKPSETLSLTCAVYGGSF----SGYYWSWIRQPPGKGLEWIGEINHS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-34*02': 'QVQLQQWGA-GLLKPSETLSLTCAVYGGSF----SGYYWSWIRQPPGKGLEWIGEINHS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-34*04': 'QVQLQQWGA-GLLKPSETLSLTCAVYGGSF----SGYYWSWIRQPPGKGLEWIGEINHS---GSTNNNPSLK-SRATISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-34*05': 'QVQLQQWGA-GLLKPSETLSLTCAVYGGSF----SGYYWCWIRQPLGKGLEWIGEINHS---GSTNNNPSLK-SRATISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-34*09': 'QVQLQESGP-GLVKPSQTLSLTCAVYGGSF----SGYYWSWIRQPPGKGLEWIGEINHS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-34*10': 'QVQLQESGP-GLVKPSETLSLTCAVYGGSF----SGYYWSWIRQPPGKGLEWIGEINHS---GSTNYNPSLK-SRITMSVDTSKNQFYLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-34*11': 'QVQLQQWGA-GLLKPSETLSLTCAVYGGSV----SGYYWSWIRQPPGKGLEWIGYIYYS---GSTNNNPSLK-SRATISVDTSKNQFSLNLSSVTAADTAVYCCAR----------------------', 'IGHV4-34*12': 'QVQLQQWGA-GLLKPSETLSLTCAVYGGSF----SGYYWSWIRQPPGKGLEWIGEIIHS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-38-2*01': 'QVQLQESGP-GLVKPSETLSLTCAVSGYSIS---SGYYWGWIRQPPGKGLEWIGSIYHS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-38-2*02': 'QVQLQESGP-GLVKPSETLSLTCTVSGYSIS---SGYYWGWIRQPPGKGLEWIGSIYHS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-39*01': 'QLQLQESGP-GLVKPSETLSLTCTVSGGSIS--SSSYYWGWIRQPPGKGLEWIGSIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-39*02': 'QLQLQESGP-GLVKPSETLSLTCTVSGGSIS--SSSYYWGWIRQPPGKGLEWIGSIYYS---GSTYYNPSLK-SRVTISVDTSKNHFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-39*06': 'RLQLQESGP-GLVKPSETLSLTCTVSGGSIS--SSSYYWGWIRQPPGKGLEWIGSIYYS---GSTYYNPSLK-SRVTISVDTSKNQFPLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-39*07': 'QLQLQESGP-GLVKPSETLSLTCTVSGGSIS--SSSYYWGWIRQPPGKGLEWIGSIYYS---GSTYYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-4*01': 'QVQLQESGP-GLVKPPGTLSLTCAVSGGSIS---SSNWWSWVRQPPGKGLEWIGEIYHS---GSTNYNPSLK-SRVTISVDKSKNQFSLKLSSVTAADTAVYCCAR----------------------', 'IGHV4-4*02': 'QVQLQESGP-GLVKPSGTLSLTCAVSGGSIS---SSNWWSWVRQPPGKGLEWIGEIYHS---GSTNYNPSLK-SRVTISVDKSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-4*07': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSI----SSYYWSWIRQPAGKGLEWIGRIYTS---GSTNYNPSLK-SRVTMSVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-4*08': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSI----SSYYWSWIRQPPGKGLEWIGYIYTS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-59*01': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSI----SSYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-59*02': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSV----SSYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-59*07': 'QVQLQESGP-GLVKPSDTLSLTCTVSGGSI----SSYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-59*08': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSI----SSYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-59*10': 'QVQLQQWGA-GLLKPSETLSLTCAVYGGSI----SSYYWSWIRQPAGKGLEWIGRIYTS---GSTNYNPSLK-SRVTMSVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-59*11': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSI----SSHYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-59*13': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSI----SSYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-61*01': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSVS--SGSYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-61*02': 'QVQLQESGP-GLVKPSQTLSLTCTVSGGSIS--SGSYYWSWIRQPAGKGLEWIGRIYTS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-61*03': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSVS--SGSYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNHFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-61*05': 'QLQLQESGP-GLVKPSETLSLTCTVSGGSIS--SSSYYWGWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDKSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-61*08': 'QVQLQESGP-GLVKPSETLSLTCTVSGGSVS--SGGYYWSWIRQPPGKGLEWIGYIYYS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV4-61*09': 'QVQLQESGP-GLVKPSQTLSLTCTVSGGSIS--SGSYYWSWIRQPAGKGLEWIGHIYTS---GSTNYNPSLK-SRVTISVDTSKNQFSLKLSSVTAADTAVYYCAR----------------------', 'IGHV5-10-1*01': 'EVQLVQSGA-EVKKPGESLRISCKGSGYSF----TSYWISWVRQMPGKGLEWMGRIDPS--DSYTNYSPSFQ-GHVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-10-1*02': 'EVQLVQSGA-EVKKPGESLRISCKGSGYSF----TSYWISWVRQMPGKGLEWMGRIDPS--DSYTNYSPSFQ-GHVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-10-1*03': 'EVQLVQSGA-EVKKPGESLRISCKGSGYSF----TSYWISWVRQMPGKGLEWMGRIDPS--DSYTNYSPSFQ-GHVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-10-1*04': 'EVQLVQSGA-EVKKPGESLRISCKGSGYSF----TSYWISWVRQMPGKGLEWMGRIDPS--DSYTNYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-51*01': 'EVQLVQSGA-EVKKPGESLKISCKGSGYSF----TSYWIGWVRQMPGKGLEWMGIIYPG--DSDTRYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-51*02': 'EVQLVQSGA-EVKKPGESLKISCKGSGYSF----TSYWTGWVRQMPGKGLEWMGIIYPG--DSDTRYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-51*03': 'EVQLVQSGA-EVKKPGESLKISCKGSGYSF----TSYWIGWVRQMPGKGLEWMGIIYPG--DSDTRYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-51*04': 'EVQLVQSGA-EVKKPGESLKISCKGSGYSF----TSYWIGWVRQMPGKGLEWMGIIYPG--DSDTRYSPSFQ-GQVTISADKPISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV5-51*07': 'EVQLVQSGA-EVKKPGESLKISCKGSGYSF----TSYWIGWVHQMPGKGLEWMGIIYPG--DSDTRYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYCAR----------------------', 'IGHV6-1*01': 'QVQLQQSGP-GLVKPSQTLSLTCAISGDSVS--SNSAAWNWIRQSPSRGLEWLGRTYYRS-KWYNDYAVSVK-SRITINPDTSKNQFSLQLNSVTPEDTAVYYCAR----------------------', 'IGHV6-1*02': 'QVQLQQSGP-GLVKPSQTLSLTCAISGDSVS--SNSAAWNWIRQSPSRGLEWLGRTYYRS-KWYNDYAVSVK-SRITINPDTSKNQFSLQLNSVTPEDTAVYYCAR----------------------', 'IGHV7-4-1*01': 'QVQLVQSGS-ELKKPGASVKVSCKASGYTF----TSYAMNWVRQAPGQGLEWMGWINTN--TGNPTYAQGFT-GRFVFSLDTSVSTAYLQICSLKAEDTAVYYCAR----------------------', 'IGHV7-4-1*02': 'QVQLVQSGS-ELKKPGASVKVSCKASGYTF----TSYAMNWVRQAPGQGLEWMGWINTN--TGNPTYAQGFT-GRFVFSLDTSVSTAYLQISSLKAEDTAVYYCAR----------------------'}, 'mouse': {'IGHV1-11*01': 'QIQLQQSGA-ELASPGASVTLSCKASGYTF----TDHIMNWVKKRPGQGLEWIGRIYPV--SGETNYNQKFM-GKATFSVDRSSSTVYMVLNSLTSEDPAVYYCGR----------------------', 'IGHV1-12*01': 'QAYLQQSGA-ELVRPGASVKMSCKASGYTF----TSYNMHWVKQTPRQGLEWIGAIYPG--NGDTSYNQKFK-GKATLTVDKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-14*01': 'EFQLQQSGP-ELVKPGASVKMSCKASGYTF----TSYVMHWVKQKPGQGLEWIGYIYPY--NDGTKYNEKFK-GKATLTSDKSSSTAYMELSSLTSEDSAVYYCAR----------------------', 'IGHV1-15*01': 'QVQLQQSGA-ELVRPGASVTLSCKASGYTF----TDYEMHWVKQTPVHGLEWIGAIDPE--TGGTAYNQKFK-GKAILTADKSSSTAYMELRSLTSEDSAVYYCTR----------------------', 'IGHV1-18*01': 'EVQLQQSGP-ELVKPGASVKIPCKASGYTF----TDYNMDWVKQSHGKSLEWIGDINPN--NGGTIYNQKFK-GKATLTVDKSSSTAYMELRSLTSEDTAVYYCAR----------------------', 'IGHV1-19*01': 'EVQLQQSGP-VLVKPGASVKMSCKASGYTF----TDYYMNWVKQSHGKSLEWIGVINPY--NGGTSYNQKFK-GKATLTVDKSSSTAYMELNSLTSEDSAVYYCAR----------------------', 'IGHV1-20*01': 'EVQLQQSGP-ELVKPGDSVKISCKASGYSF----TGYFMNWVMQSHGKSLEWIGRINPY--NGDTFYNQKFK-GKATLTVDKSSSTAHMELRSLTSEDSAVYYCAR----------------------', 'IGHV1-20*02': 'EVQLQQSGP-ELVKPGASVKISCKASGYSF----TGYFMNWVMQSHGKSLEWIGRINPY--NGDTFYNQKFK-GKATLTVDKSSSTAHMELRSLASEDSAVYYCAR----------------------', 'IGHV1-22*01': 'EVQLQQSGP-ELVKPGASVKMSCKASGYTF----TDYNMHWVKQSHGKSLEWIGYINPN--NGGTSYNQKFK-GKATLTVNKSSSTAYMELRSLTSEDSAVYYCAR----------------------', 'IGHV1-26*01': 'EVQLQQSGP-ELVKPGASVKISCKASGYTF----TDYYMNWVKQSHGKSLEWIGDINPN--NGGTSYNQKFK-GKATLTVDKSSSTAYMELRSLTSEDSAVYYCAR----------------------', 'IGHV1-31*01': 'EVQLQQSGP-ELVKPGASVKISCKASGYSF----TGYYMHWVKQSHGNILDWIGYIYPY--NGVSSYNQKFK-GKATLTVDKSSSTAYMELRSLTSEDSAVYYCAR----------------------', 'IGHV1-34*01': 'EVQLQQSGP-ELVKPGASVKMSCKASGYTF----TDYYMHWVKQSHGKSLEWIGYIYPN--NGGNGYNQKFK-GKATLTVDKSSSTAYMELRSLTSEDSAVYYCAR----------------------', 'IGHV1-34*02': 'EVQLQQSGP-ELVKPGDSVKMSCKASGYTF----TDYYMDWVKQSHGKSLEWIGYIYPN--NGGTSYNQKFK-GKATLTVDKSSSTAYMELHSLTSEDSAVYYCAR----------------------', 'IGHV1-36*01': 'EVQLQQSGP-VLVKPGPSVKISCKASGFTF----TDYYMHWVKQSHGKSLEWIGLVYPY--NGGTSYNQKFK-GKATLTVDTSSSTAYMELNSLTSEDSAVYYCAR----------------------', 'IGHV1-37*01': 'EVQLQQSGP-ELVKPGASVKISCKASGYSF----TGYFMNWVKQSHGKSLEWIGRINPY--NGDTFYNQKFK-GKATLTVDKSSSTAHMELLSLTSEDFAVYYCAR----------------------', 'IGHV1-39*01': 'EFQLQQSGP-ELVKPGASVKISCKASGYSF----TDYNMNWVKQSNGKSLEWIGVINPN--YGTTSYNQKFK-GKATLTVDQSSSTAYMQLNSLTSEDSAVYYCAR----------------------', 'IGHV1-4*01': 'QVQLQQSGA-ELARPGASVKMSCKASGYTF----TSYTMHWVKQRPGQGLEWIGYINPS--SGYTKYNQKFK-DKATLTADKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-4*02': 'QVQLQQSAA-ELARPGASVKMSCKASGYTF----TSYTMHWVKQRPGQGLEWIGYINPS--SGYTEYNQKFK-DKTTLTADKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-42*01': 'EVQLQQSGP-ELVKPGASVKISCKASGYSF----TGYYMNWVKQSPEKSLEWIGEINPS--TGGTTYNQKFK-AKATLTVDKSSSTAYMQLKSLTSEDSAVYYCAR----------------------', 'IGHV1-43*01': 'EVKLQQSGP-ELVKPGASVKISCKASGYSF----TGYYMHWVKQSSEKSLEWIGEINPS--TGGTSYNQKFK-GKATLTVDKSSSTAYMQLKSLTSEDSAVYYCAR----------------------', 'IGHV1-47*01': 'QVQLQQSGA-ELVKPGASVKMSCKASGYTF----TTYPIEWMKQNHGKSLEWIGNFHPY--NDDTKYNEKFK-GKATLTVEKSSSTVYLELSRLTSDDSAVYYCAR----------------------', 'IGHV1-49*01': 'QRELQQSGA-ELVRPGSSVKLSCKDSYFAF----MASAMHWVKQRPGHGLEWIGSFTMY--SDATEYSENFK-GKATLTANTSSSTAYMELSSLTSEDSAVYYCAR----------------------', 'IGHV1-5*01': 'EVQLQQSGT-VLARPGASVKMSCKTSGYTF----TSYWMHWVKQRPGQGLEWIGAIYPG--NSDTSYNQKFK-GKAKLTAVTSASTAYMELSSLTNEDSAVYYCTR----------------------', 'IGHV1-50*01': 'QVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWMQWVKQRPGQGLEWIGEIDPS--DSYTNYNQKFK-GKATLTVDTSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-52*01': 'QVQLQQPGA-ELVRPGSSVKLSCKASGYTF----TSYWMHWVKQRPIQGLEWIGNIDPS--DSETHYNQKFK-DKATLTVDKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-53*01': 'QVQLQQPGT-ELVKPGASVKLSCKASGYTF----TSYWMHWVKQRPGQGLEWIGNINPS--NGGTNYNEKFK-SKATLTVDKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-54*01': 'QVQLQQSGA-ELVRPGTSVKVSCKASGYAF----TNYLIEWVKQRPGQGLEWIGVINPG--SGGTNYNEKFK-GKATLTADKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-54*02': 'QVQLQQSGA-ELVRPGTSVKVSCKASGYAF----TNYLIEWVKQRPGQGLEWIGVINPG--SGGTNYNEKFK-GKATLTADKSSNTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-55*01': 'QVQLQQPGA-ELVKPGASVKMSCKASGYTF----TSYWITWVKQRPGQGLEWIGDIYPG--SGSTNYNEKFK-SKATLTVDTSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-56*01': 'QVQLQQSGP-ELVRPGASVKISCKAPGYTF----TSHWMQWVRQRPGQGLEWIGEIFPG--SGSTYYNEKFK-GKATLTVDTSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-58*01': 'EVQLQQSGA-ELVRPGSSVKMSCKTSGYTF----TSYGINWVKQRPGQGLEWIGYIYIG--NGYTEYNEKFK-GKATLTSDTSSSTAYMQLSSLTSEDSAIYFCAR----------------------', 'IGHV1-59*01': 'QVQLQQPGA-ELVRPGTSVKLSCKASGYTF----TSYWMHWVKQRPGQGLEWIGVIDPS--DSYTNYNQKFK-GKATLTVDTSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-61*01': 'QVQLQQPGA-ELVRPGSSVKLSCKASGYTF----TSYWMDWVKQRPGQGLEWIGNIYPS--DSETHYNQKFK-DKATLTVDKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-62-1*01': 'QVQLQQSGA-ELVRPGASVKLSCKASGYTF----TSYWMQWVKQRPGQGLEWIGEIFPG--SGSTYYNEKFK-GKATLTVDTSSSTAYMQLSSLTAENSAIYLCKE----------------------', 'IGHV1-62-2*01': 'QVQLQQSGA-ELVKPGASVKLSCKASGYTF----TEYTIHWVKQRSGQGLEWIGWFYPG--SGSIKYNEKFK-DKATLTADKSSSTVYMELSRLTSEDSAVYFCARHE--------------------', 'IGHV1-63*01': 'QVQLQQSGA-ELVRPGTSVKMSCKASGYTF----TNYWIGWAKQRPGHGLEWIGDIYPG--GGYTNYNEKFK-GKATLTADKSSSTAYMQFSSLTSEDSAIYYCAR----------------------', 'IGHV1-63*02': 'QVQLQQSGA-ELVRPGTSVKMSCKAAGYTF----TNYWIGWVKQRPGHGLEWIGDIYPG--GGYTNYNEKFK-GKATLTADTSSSTAYMQLSSLTSEDSAIYYCAR----------------------', 'IGHV1-64*01': 'QVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWMHWVKQRPGQGLEWIGMIHPN--SGSTNYNEKFK-SKATLTVDKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-66*01': 'QVQLQQSGP-ELVKPGASVKISCKASGYSF----TSYYIHWVKQRPGQGLEWIGWIYPG--SGNTKYNEKFK-GKATLTADTSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-69*01': 'QVQLQQPGA-ELVMPGASVKLSCKASGYTF----TSYWMHWVKQRPGQGLEWIGEIDPS--DSYTNYNQKFK-GKSTLTVDKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-69*02': 'QVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWMHWVKQRPGQGLEWIGEIDPS--DSYTNYNQKFK-GKATLTVDKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-7*01': 'QVQLQQSGA-ELAKPGASVKLSCKASGYTF----TSYWMHWVKQRPGQGLEWIGYINPS--SGYTKYNQKFK-DKATLTADKSSSTAYMQLSSLTYEDSAVYYCAR----------------------', 'IGHV1-71*01': 'QVQLQQSGA-ELVKPGASVKLSCKASGYTF----TEYTIHWVKQRSGQGLEWIGWFYPG--SGSIKYNEKFK-DKATLTADKSSSTVYMELSRLTSEDSAVYFCARHE--------------------', 'IGHV1-72*01': 'QVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWMHWVKQRPGRGLEWIGRIDPN--SGGTKYNEKFK-SKATLTVDKPSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1-72*04': 'QVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWMHWVKQRPGRGLEWIGRIDPN--SGGTKYNEKFK-SKATLTVDTSSSTAYMQLSSLTSEDSAVHYCAR----------------------', 'IGHV1-74*01': 'QVQLQQPGA-ELVKPGASVKVSCKASGYTF----TSYWMHWVKQRPGQGLEWIGRIHPS--DSDTNYNQKFK-GKATLTVDKSSSTAYMQLSSLTSEDSAVYYCAI----------------------', 'IGHV1-74*04': 'HVQLQQPGA-ELVKPGASVKVSCKASGYTF----TSYWMHWVKQRPGQGLEWIGRIHPS--DSDTNYNQKFK-GKATLTVDKSSSTAYMQLSSLTSEDSAVYYCAI----------------------', 'IGHV1-75*01': 'QVQLQQSGP-ELVKPGASVKISCKASGYTF----TDYYINWVKQRPGQGLEWIGWIFPG--SGSTYYNEKFK-GKATLTVDKSSSTAYMLLSSLTSEDSAVYFCAR----------------------', 'IGHV1-76*01': 'QVQLKQSGA-ELVRPGASVKLSCKASGYTF----TDYYINWVKQRPGQGLEWIARIYPG--SGNTYYNEKFK-GKATLTAEKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-77*01': 'QVQLKQSGA-ELVKPGASVKISCKASGYTF----TDYYINWVKQRPGQGLEWIGKIGPG--SGSTYYNEKFK-GKATLTADKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-78*01': 'QVQLQQSDA-ELVKPGASVKISCKVSGYTF----TDHTIHWMKQRPEQGLEWIGYIYPR--DGSTKYNEKFK-GKATLTADKSSSTAYMQLNSLTSEDSAVYFCAR----------------------', 'IGHV1-80*01': 'QVQLQQSGA-ELVKPGASVKISCKASGYAF----SSYWMNWVKQRPGKGLEWIGQIYPG--DGDTNYNGKFK-GKATLTADKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-81*01': 'QVQLQQSGA-ELARPGASVKLSCKASGYTF----TSYGISWVKQRTGQGLEWIGEIYPR--SGNTYYNEKFK-GKATLTADKSSSTAYMELRSLTSEDSAVYFCAR----------------------', 'IGHV1-82*01': 'QVQLQQSGP-ELVKPGASVKISCKASGYAF----SSSWMNWVKQRPGKGLEWIGRIYPG--DGDTNYNGKFK-GKATLTADKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-84*01': 'QIQLQQSGP-ELVKPGASVKISCKASGYTF----TDYYINWVKQRPGQGLEWIGWIYPG--SGNTKYNEKFK-GKATLTVDTSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1-85*01': 'QVQLQQSGP-ELVKPGASVKLSCKASGYTF----TSYDINWVKQRPGQGLEWIGWIYPR--DGSTKYNEKFK-GKATLTVDTSSSTAYMELHSLTSEDSAVYFCAR----------------------', 'IGHV1-87*01': 'QVQLQQSGA-ELARPGASVKMSCKASGYTF----TSYWMQWVKQRPGQGLEWIGAIYPG--DGDTRYTQKFK-GRATLTADKSSSTAYMQLSSLTSEDSAVYYCAT----------------------', 'IGHV1-9*01': 'QVQLQQSGA-ELMKPGASVKLSCKATGYTF----TGYWIEWVKQRPGHGLEWIGEILPG--SGSTNYNEKFK-GKATFTADTSSNTAYMQLSSLTTEDSAIYYCAR----------------------', 'IGHV10-1*01': 'EVQLVESGG-GLVQPKGSLKLSCAASGFSF----NTYAMNWVRQAPGKGLEWVARIRSKSNNYATYYADSVK-DRFTISRDDSESMLYLQMNNLKTEDTAMYYCVR----------------------', 'IGHV10-1*02': 'EVQLVESGG-GLVQPKGSLKLSCAASGFTF----NTYAMNWVRQAPGKGLEWVARIRSKSNNYATYYADSVK-DRFTISRDDSQSMLYLQMNNLKTEDTAMYYCVS----------------------', 'IGHV10-3*01': 'EVQLVESGG-GLVQPKGSLKLSCAASGFTF----NTYAMHWVRQAPGKGLEWVARIRSKSSNYATYYADSVK-DRFTISRDDSQSMLYLQMNNLKTEDTAMYYCVR----------------------', 'IGHV10-3*02': 'EVQLVESGG-GLVQPKGSLKLSCAASVFTF----NTYAMHWVRQAPGKGLEWVARIRSKSSNYATYYADSVK-DRFTISRDDSQSMLYLQMNNLKTEDTAMYYCVR----------------------', 'IGHV10-3*03': 'EVQLVESGG-GLVQPKGSLKLSCAASVFTF----NTYAMHWVCQAPGKGLEWVARIRSKSNNYATYYADSVK-DRFTISRDDSQSMLYLQMNNLKTEDTAMYYCVR----------------------', 'IGHV10S3*01': 'EVQLVETGG-GLVQPKGSLKLSCAASGFTF----NTNAMNWVRQAPGKGLEWVARIRSKSNNYATYYADSVK-DRFTISRDDSQSMLYLQMNNLKTEDTAMYYCVR----------------------', 'IGHV11-1*01': 'EVQLLETGE-GLVPPGGSRGLSCEGSGFTF----SGFWMSWVRQTPGKTLEWIGDINSD--GSAINYAPSIK-DRFTIFRDNDKSTLYLQMSNVRSEDTATYFCMR----------------------', 'IGHV11-1*02': 'EVQLLETGG-GLVQPGGSRGLSCEGSGFTF----SGFWMSWVRQTPGKTVEWIGDINSD--GSAINYAPSIK-DRFTIFRDNDKSTLYLQMSNVRSEDPATYFCMR----------------------', 'IGHV11-2*01': 'EVQLLETGG-GLVQPGGSRGLSCEGSGFTF----SGFWMSWVRQTPGKTLEWIGDINSD--GSAINYAPSIK-DRFTIFRDNDKSTLYLQMSNVRSEDTATYFCMR----------------------', 'IGHV11-2*02': 'EVQLLETGG-GLVQPGGSRGLSCEGSGFTF----SGFWMSWVRQTPGKTLEWIGDINSD--GSAINYAPSIK-DRFTIFRDNDKSTLYLQMSNVRSEDTATYFCMR----------------------', 'IGHV12-1-1*01': 'QIQLKESGP-AVIKPSQSLSLTCIVSGFSIT--SSSYCWHWIRQPPGKGLEWMGRICYE---GSIYYSPSIK-SRSTISRDTSLNKFFIQLSSVTNEDTAMYYCSREN--------------------', 'IGHV12-3*01': 'QMQLQESGP-GLVKPSQSLFLTCSITGFPIT---SGYYWIWIRQSPGKPLEWMGYITHS---GETFYNPSLQ-SPISITRETSKNQFFLQLNSVTTEDTAMYYCAGDR--------------------', 'IGHV12-3*02': 'QMQLQESGP-GLVKPSQSLFLACSITGFPIT---SGYYWIWIRQSPGKPLEWMGYITHS---GETFYNPSLQ-SPISITRETSKNQFFLQLNSVTTEDTAMYYCAGDR--------------------', 'IGHV13-2*01': 'QVQLVETGG-GLVRPGNSLKLSCVTSGFTF----SNYRMHWLRQPPGKRLEWIAVITVKSDNYGANYAESVK-GRFAISRDDSKSSVYLEMNRLREEDTATYFCSR----------------------', 'IGHV13-2*02': 'QVQLVETGG-GLVRPGNSLKLSCVTSGFTF----SNYRMHWLRQPPGKRLEWIAVITVKSDNYGANYAESVK-GRFTISRDDSKSSVYLQMNRLREEDTATYYCSR----------------------', 'IGHV14-1*01': 'EVQLQQSGA-ELVRPGASVKLSCTASGFNI----KDYYMHWVKQRPEQGLEWIGRIDPE--DGDTEYAPKFQ-GKATMTADTSSNTAYLQLSSLTSEDTAVYYCTT----------------------', 'IGHV14-1*02': 'EVQLQQSGA-ELVRPGALVKLSCKASGFNI----KDYYMHWVKQRPEQGLEWIGWIDPE--NGNTIYDPKFQ-GKASITADTSSNTAYLQLSSLTSEDTAVYYCAR----------------------', 'IGHV14-2*01': 'EVQLQQSGA-ELVKPGASVKLSCTASGFNI----KDYYMHWVKQRTEQGLEWIGRIDPE--DGETKYAPKFQ-GKATITADTSSNTAYLQLSSLTSEDTAVYYCAR----------------------', 'IGHV14-3*01': 'EVQLQQSVA-ELVRPGASVKLSCTASGFNI----KNTYMHWVKQRPEQGLEWIGRIDPA--NGNTKYAPKFQ-GKATITADTSSNTAYLQLSSLTSEDTAIYYCAR----------------------', 'IGHV14-3*02': 'EVQLQQSGA-ELVKPGASVKLSCTASGFNI----KDTYMHWVKQRPEQGLEWIGRIDPA--NGNTKYDPKFQ-GKATITADTSSNTAYLQLSSLTSEDTAVYYCAR----------------------', 'IGHV14-4*01': 'EVQLQQSGA-ELVRPGASVKLSCTASGFNI----KDDYMHWVKQRPEQGLEWIGWIDPE--NGDTEYASKFQ-GKATITADTSSNTAYLQLSSLTSEDTAVYYCTT----------------------', 'IGHV14-4*02': 'EVQLQQSGA-ELVRSGASVKLSCTASGFNI----KDYYMHWVKQRPEQGLEWIGWIDPE--NGDTEYAPKFQ-GKATMTADTSSNTAYLQLSSLTSEDTAVYYCNA----------------------', 'IGHV15-2*01': 'QVHLQQSGS-ELRSPGSSVKLSCKDFDSEVF---PIAYMSWVRQKPGHGFEWIGGILPS--IGRTIYGEKFE-DKATLDADTLSNTAYLELNSLTSEDSAIYYCAR----------------------', 'IGHV1S12*01': 'QVQLQQSGP-ELVKPGASVKISCKASGYTF----TSYYIHWVKQRPGQGLEWIGYIYPR--DGSTNYNEKFK-GKATLTADTSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1S14*01': 'QVQLQQPGS-VLVRPGTSVKLSCKASGYTF----TSYWMHWAKQRPGQGLEWIGEIHPN--CGNINYNEKFK-GKATLTVDTSSSTAYVDLSSLTSEDSAVYYCAR----------------------', 'IGHV1S26*01': 'QVQLQQSGA-ELVKTGASVKMSCKASGYTF----TSYTMHWVKQRPGQGLEWIGYINPS--SGYTNYNQKFK-DKATLTADKSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1S29*02': 'EVQLQQSGP-ELVKPGASVKISCKASGYTF----TDYNMHWVKQSHGKSLEWIGYIYPY--NGGTGYNQKFK-SKATLTVDNSSSTAYMELSSLTSEDSAVYYCAR----------------------', 'IGHV1S35*01': 'QVQLQQPGA-VLVRHGASVKLSCKASGYTF----TSSWMHWAKQRHGQGLEWIGEIHPN--SGNTNYNEKFK-GKATLTVDKSSSTAYVDLSSLTSEDSAVYYCAR----------------------', 'IGHV1S36*01': 'PVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWMHWVKQRPGRGLEWIGNIDPN--SGGTKYNEKFK-SKATLTVDKPSSTAYMQLSSLTSEDSAVYYCTR----------------------', 'IGHV1S40*01': 'QVQLQQSGP-ELVRPGLSVKLSCKASGYIF----ITYWMNWVKQRPGQGLEWIGQIFPA--SGSTNYNEMFE-GKATLTVDTSSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1S45*01': 'EVQLQQSGP-ELVKPGASVKMSCKASGYTF----TDYVMHWVKQSNGKSLEWIGYINPY--NDYTSYNQKFK-GKATLTVDKSSSTAYMQLNSLTSEDSAVYYCAR----------------------', 'IGHV1S49*01': 'EVQLQQSGA-ELVRPGSSVKLSCKTSGYTF----TSYGINWVKQRPGQGLEWIGYIYLG--NGYTAYNEKFK-GKATLTSDTSSSTAYMQLRSLTSEDSVIKFCAR----------------------', 'IGHV1S5*01': 'QVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWMHWVKQRPGEGLEWIGNIYPG--SSSTNYNEKFK-SKATLTVDTPSSTAYMQLSSLTSEDSAVYYCAR----------------------', 'IGHV1S50*01': 'QVQLQQSGA-ELVKPGASVRISCKTSGYTF----TSYNIHWVKERPGQGLEWIGWIYPG--DGNTKYNEKFK-GKTTLTADKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1S52*01': 'QVQLQQSGA-ELVRPGTSVKVSCKASGYVF----TNYLIEWVKQRPGQGLEWIGVINPG--SGGTNYNEKFK-GKATLTADKSSSTAYMQLSSLTSDDSAVYFCAR----------------------', 'IGHV1S53*01': 'QVQLQQSDA-ELVKPGASVKISCKASGYTF----TDHAIHWVKQKPEQGLEWIGYISPG--NGDIKYNEKFK-GKATLTADKSSSTAYMQLNSLTSEDSAVYFCKR----------------------', 'IGHV1S53*02': 'QVQLQQSDA-ELVKPGASVKISCKASGYTF----TDHAIHWVKQKPEQGLEWIGYISPG--NGDIKYNEKFK-GKATLTADKSSSTAYMQLNSLTSEDSAVYFCKR----------------------', 'IGHV1S53*03': 'QVQLQQSDT-ELVKPGASVKISCKASGYTF----TDHAIHWVKQRPEQGLEWIGYISPG--NGDIKYNEKFK-GKATLTADKSSSTAYMQLNSLTSEDSAVYFCKR----------------------', 'IGHV1S55*01': 'QVQLQQPGS-VLVRPGASVKLSCKASGYTF----TSYWMNWVKQRPGQGLEWIGGIYPN--SGSTDYNEKFK-GKATLTVDTSSSTTYMDLSSLTSKDSAVYYCAR----------------------', 'IGHV1S56*01': 'QVQLQQSGP-ELVKPGASVRISCKASGYTF----TSYNIHWVKQRPGQGLEWIGWIYPG--DGNTKYNEKFK-GKTTLTADKSSSTAYMQLSSLTSEDSAVYFCAR----------------------', 'IGHV1S61*01': 'QVQLQQPGA-ELVKPGASVKLSCKASGYTF----TSYWINWVKQRPGQGLEWIGNIYPG--SSSTNYNEKFK-SKATLTVDTSSSTAYMQLSSLTSDDSAVYYCAR----------------------', 'IGHV2-2*01': 'QVQLKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWSG---GSTDYNAAFI-SRLSISKDNSKSQVFFKMNSLQADDTAIYYCAR----------------------', 'IGHV2-2*02': 'QVQLKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWSG---GSTDYNAAFI-SRLSISKDNSKSQVFFKMNSLQANDTAIYYCAR----------------------', 'IGHV2-2*03': 'QVQLKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWSG---GSTDYNAAFI-SRLSISKDNSKSQVFFKMNSLQSNDTAIYYCAR----------------------', 'IGHV2-2-2*01': 'QVQMKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWSG---GSTDYNAAFI-SRLSISKDNSKSQVFFKMNSLQADDTAIYYCVR----------------------', 'IGHV2-3*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYGVSWVRQPPGKGLEWLGVIWGD---GSTNYHSALI-SRLSISKDNSKSQVFLKLNSLQTDDTATYYCAK----------------------', 'IGHV2-3-1*01': 'QVQLKESGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWSG---GSTDYNAPFI-SRLSISKDNSKSQIFFKMNSLQADDTAIYYCAR----------------------', 'IGHV2-4*01': 'QVQLKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQPPGKGLEWLGVIWSG---GSTDYNAAFI-SRLSISKDNSKSQVFFKMNSLQADDTAIYYCAK----------------------', 'IGHV2-4*02': 'QVQLKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQPPGKGLEWLGVIWSG---GSTDYNAAFI-SRLSISKDNSKSQVFFKMNSLQADDTAIYYCAR----------------------', 'IGHV2-4-1*01': 'QVQLKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWSG---GSTDYNAAFI-SRLSISKDNSKSQVFFKMNSLQADDTAIYYCAR----------------------', 'IGHV2-5*01': 'QVQLKQSGP-GLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWRG---GSTDYNAAFM-SRLSITKDNSKSQVFFKMNSLQADDTAIYYCAK----------------------', 'IGHV2-5-1*01': 'QVQLKQSGP-SLVQPSQSLSITCTVSGFSL----TSYGVHWVRQSPGKGLEWLGVIWRG---GSTDYNAAFM-SRLSITKDNSKSQVFFKMNSLQADDTAIYYCAK----------------------', 'IGHV2-6*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYGVDWVRQSPGKGLEWLGVIWGV---GSTNYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAS----------------------', 'IGHV2-6*02': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYGVHWVRQPPGKGLEWLVVIWSD---GSTTYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAR----------------------', 'IGHV2-6*03': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYGVHWVRQPPGKGLEWLVVIWSD---GSTTYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAR----------------------', 'IGHV2-6-1*01': 'QVQLKESGP-GLVAPSQSLSITCTISGFSL----TSYGVHWVRQPPGKGLEWLVVIWSD---GSTTYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAR----------------------', 'IGHV2-6-2*01': 'QVQLKESGP-DLVAPSQSLSITCTVSGFSL----TSYGVHWVRQPPGKGLEWLVVIWSD---GSTTYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAR----------------------', 'IGHV2-6-3*01': 'QVQ-NKSGP-GLVEPSQSLSITCTVYWFSL----TSYGVSWVRQPPGKGLKWLGVIWAG---GSTNYNSALI-SRLSISKDNSKSQVFLKMNSLQTDDTAIYYCVR----------------------', 'IGHV2-6-4*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----SRYSVHWVRQPPGKGLEWLGMIWGG---GSTDYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAR----------------------', 'IGHV2-6-5*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TDYGVSWIRQPPGKGLEWLGVIWGG---GSTYYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAK----------------------', 'IGHV2-6-6*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TNSGVHWVRQSPGKGLEWLGVIWGD---GSTNYNSAFK-SRLSISKDNSKSQVFLKMNSLQTDDTARYYCAK----------------------', 'IGHV2-6-7*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TGYGVNWVRQPPGKGLEWLGMIWGD---GSTDYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTARYYCAR----------------------', 'IGHV2-6-7*02': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TGYGVNWVRQPPGKGLEWLGMIWGD---GSTDYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTARYYCAR----------------------', 'IGHV2-6-8*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYGVDWVRQSPGKGLEWLGVIWGV---GSTNYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAS----------------------', 'IGHV2-7*01': 'QVQMKESGP-DLVQPSQTLSLTCTVSGFSL----SSYGVHWFRKPPRKGLEWLGGIWSG---GSIYYTPALS-SRLSVSRDTSKSQVFFKMSSLQSEDTAVYHCAR----------------------', 'IGHV2-9*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYGVDWVRQPPGKGLEWLGVIWGG---GSTNYNSALM-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAK----------------------', 'IGHV2-9*02': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYGVHWVRQPPGKGLEWLGVIWAG---GSTNYNSALM-SRLSISKDNSKSQVFLKMNSLQTDDTAMYYCAR----------------------', 'IGHV2-9-1*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYAISWVRQPPGKGLEWLGVIWTG---GGTNYNSALK-SRLSISKDNSKSQVFLKMNSLQTDDTARYYCAR----------------------', 'IGHV2-9-2*01': 'QVQLKESGP-GLVAPSQSLSITCTVSGFSL----TSYDISWIRQPPGKGLEWLGVIWTG---GGTNYNSAFM-SRLSISKDNSKSQVFLKMNSLQTDDTAIYYCVR----------------------', 'IGHV2S3*01': 'QVQLKQSGP-GLVAPSQSLFITCTVYGFSL----TSYEINWVRQPPGKGLEWLGVIWTG---GSTNYNSALI-SRLSISKDNSKSLVFLKMNSLQTDDTAIYYCVR----------------------', 'IGHV3-1*01': 'DVQLQESGP-GMVKPSQSLSLTCTVTGYSIT---SGYDWHWIRHFPGNKLEWMGYISYS---GSTNYNPSLK-SRISITHDTSKNHFFLKLNSVTTEDTATYYCAR----------------------', 'IGHV3-1*02': 'DVQLQESGP-DLVKPSQSLSLTCTVTGYSIT---SGYSWHWIRQFPGNKLEWMGYIHYS---GSTNYNPSLK-SRISITRDTSKNQFFLQLNSVTTEDTATYYCAR----------------------', 'IGHV3-2*02': 'DVQLQESGP-GLVKPSQSLSLTCTVTGYSIT---SDYAWNWIRQFPGNKLEWMGYISYS---GSTSYNPSLK-SRISITRDTSKNQFFLQLNSVTTEDTATYYCAR----------------------', 'IGHV3-3*01': 'DVQLQESGP-SLVRPSQTLSLTCTVTGFSIN---SDCYWIWIRQFPGNKLEYIGYTFYS---GITYYNPSLE-SRTYITRDTSKNQFSLKLSSVTTEDTATYYCAR----------------------', 'IGHV3-4*01': 'DVQLQESGP-ALVKPSQTVSLTCTVTGYSIT--NGNHWWNWIRQVSGSKLEWIGYISSS---GSTDSNPSLK-SRISITRDTSKNQLFLQLNSVTTEDIATYYCAR----------------------', 'IGHV3-4*02': 'DVQLQESGP-GLVKPSQTVSLTCTVTGYSIT--NGNHWWNWIRQVSGNKLEWMGYISSS---GSTDSNPSLK-SQISITRDTSKNQLFLQLNSVTIEDIATYYCAR----------------------', 'IGHV3-5*01': 'DVQLQESGP-GLVKPSQTVFLTCTVTGISIT--TGNYRWSWIRQFPGNKLEWIGYIYYS---GTITYNPSLT-SRTTITRDTPKNQFFLEMNSLTAEDTATYYCAR----------------------', 'IGHV3-5*02': 'DVQLQESGP-GLVKPSQTVSLTCTVTGISIT--TGNYRWSWIRQFPGNKLEWIGYIYYS---GTITYNPSLT-SRTTITRDTSKNQFFLEMNSLTAEDTATYYCAR----------------------', 'IGHV3-6*01': 'DVQLQESGP-GLVKPSQSLSLTCSVTGYSIT---SGYYWNWIRQFPGNKLEWMGYISYD---GSNNYNPSLK-NRISITRDTSKNQFFLKLNSVTTEDTATYYCAR----------------------', 'IGHV3-6*02': 'DVQLQESGP-GLVKPSQSLSLTCSVTGYSIT---SGYYWNWIRQFPGNKLEWMGYISYD---GSNNYNPSLK-NRISITRDTSKNQFFLKLNSVTTEDTATYYCAR----------------------', 'IGHV3-8*01': 'EVQLQESGP-GLAKPSQTLSLTCSVTGYSI----TSDYWNWIRKFPGNKLEYMGYISYS---GSTYYNPSLK-SRISITRDTSKNQYYLQLNSVTTEDTATYYCAR----------------------', 'IGHV3-8*02': 'EVQLQESGP-SLVKPSQTLSLTCSVTGDSI----TSGYWNWIRKFPGNKLEYMGYISYS---GSTYYNPSLK-SRISITRDTSKNQYYLQLNSVTTEDTATYYCAR----------------------', 'IGHV3S1*01': 'EVQLQESGP-SLVKPSQTLSLTCSVTGDSI----TSDYWNWIRKFPGNKLEYMGYISYS---GSTYYNPSLK-SRISITRDTSKNQYYLQLNSVTSEDTATYYCAR----------------------', 'IGHV3S1*02': 'EVQLQESGP-SLVKPSQTLSLTCSVTGDSI----TSGYWNWIRKFPGNKLEYMGYISYS---GSTYYNPSLK-SRISITRDTSKNQYYLQLNSVTTEDTPTYYCAR----------------------', 'IGHV4-1*01': 'EVKLLQSGG-GLVQPGGSLKLSCAASGIDF----SRYWMSWVRRAPGKGLEWIGEINPD--SSTINYAPSLK-DKFIISRDNAKNTLYLQMSKVRSEDTALYYCAR----------------------', 'IGHV4-1*02': 'EVKLLESGG-GLVQPGGSLKLSCAASGFDF----SRYWMSWVRQAPGKGLEWIGEINPD--SSTINYTPSLK-DKFIISRDNAKNTLYLQMSKVRSEDTALYYCAR----------------------', 'IGHV4-2*02': 'EVKLLESGG-GLVQPGGSLNLSCAASGFDF----SRYWMSWARQAPGKGQEWIGEINPG--SSTINYTPSLK-DKFIISRDNAKNTLYLQMSKVRSEDTALYYCAR----------------------', 'IGHV5-12*01': 'EVKLVESGG-GLVQPGGSLKLSCAASGFTF----SDYYMYWVRQTPEKRLEWVAYISNG--GGSTYYPDTVK-GRFTISRDNAKNTLYLQMSRLKSEDTAMYYCAR----------------------', 'IGHV5-12*02': 'EVKLVESGG-GLVQPGGSLKLSCATSGFTF----SDYYMYWVRQTPEKRLEWVAYISNG--GGSTYYPDTVK-GRFTISRDNAKNTLYLQMSRLKSEDTAMYYCAR----------------------', 'IGHV5-12-1*01': 'EVQLVESGG-GLVKPGGSLKLSCAASGFAF----SSYDMSWVRQTPEKRLEWVAYISSG--GGSTYYPDTVK-GRFTISRDNAKNTLYLQMSSLKSEDTAMYYCAR----------------------', 'IGHV5-12-2*01': 'EVKLVESGG-GLVQPGGSLKLSCAASGFTF----SSYTMSWVRQTPEKRLEWVAYISNG--GGSTYYPDTVK-GRFTISRDNAKNTLYLQMSSLKSEDTAMYYCAR----------------------', 'IGHV5-12-4*01': 'EVKLVESGG-GLVKPGQSLKLSCAASGFTF----SNYYMSWVHQTPEKRLEWVAYISSS--GVSTYYPDNVK-GRFAISRDNAKNTLYLQMTSLKSEDTALYYCAR----------------------', 'IGHV5-15*01': 'EVKLVESGG-GLVQPGGSLKLSCAASGFTF----SDYGMAWVRQAPRKGPEWVAFISNL--AYSIYYADTVT-GRFTISRENAKNTLYLEMSSLRSEDTAMYYCAR----------------------', 'IGHV5-15*02': 'EVKLVESGG-GLVQPGGSRKLSCAASGFTF----SDYGMAWVRQAPGKGPEWVAFISNL--AYSIYYADTVT-GRFTISRENAKNTLYLEMSSLRSEDTAMYYCAR----------------------', 'IGHV5-15*05': 'EVKLVESGG-ALVQPGGSLKLSCAASGFTF----SDYGMAWVRQAPRKGPEWVAFISNL--AYSIYYADTVT-GRFTISRENAKNTLYLEMSSLRSEDTAMYYCAR----------------------', 'IGHV5-16*01': 'EVKLVESEG-GLVQPGSSMKLSCTASGFTF----SDYYMAWVRQVPEKGLEWVANINYD--GSSTYYLDSLK-SRFIISRDNAKNILYLQMSSLKSEDTATYYCAR----------------------', 'IGHV5-17*01': 'EVQLVESGG-GLVKPGGSLKLSCAASGFTF----SDYGMHWVRQAPEKGLEWVAYISSG--SSTIYYADTVK-GRFTISRDNAKNTLFLQMTSLRSEDTAMYYCAR----------------------', 'IGHV5-17*02': 'DVQLVESGG-GLVQPGGSRKLSCAASGFTF----SSFGMHWVRQAPEKGLEWVAYISSG--SSTIYYADTVK-GRFTISRDNPKNTLFLQMTSLRSEDTAMYYCAR----------------------', 'IGHV5-2*01': 'EVQLVESGG-GLVQPGESLKLSCESNEYEF----PSHDMSWVRKTPEKRLELVAAINSD--GGSTYYPDTME-RRFIISRDNTKKTLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV5-2*02': 'EVQLVESGG-GLVQPRESLKLSCESNEYEF----PSHDMSWVRKTPEKRLELVAAINSD--GGSTYYPDTME-RRFIISRDNTKKTLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV5-4*01': 'EVQLVESGG-GLVKPGGSLKLSCAASGFTF----SSYAMSWVRQTPEKRLEWVATISDG--GSYTYYPDNVK-GRFTISRDNAKNNLYLQMSHLKSEDTAMYYCAR----------------------', 'IGHV5-4*02': 'EVQLVESGG-GLVKPGGSLKLSCAASGFTF----SDYYMYWVRQTPEKRLEWVATISDG--GSYTYYPDSVK-GRFTISRDNAKNNLYLQMSSLKSEDTAMYYCAR----------------------', 'IGHV5-6*01': 'EVQLVESGG-DLVKPGGSLKLSCAASGFTF----SSYGMSWVRQTPDKRLEWVATISSG--GSYTYYPDSVK-GRFTISRDNAKNTLYLQMSSLKSEDTAMYYCAR----------------------', 'IGHV5-6-1*01': 'EVQLVESGG-DLVKPGGSLKLSCAASGFTF----SSYGMSWVRQTPDKRLEWVATISSG--GSYTYYPDSVK-GRFTISRDNAKNTLYLQMSSLKSEDTAMYYCAR----------------------', 'IGHV5-6-2*01': 'DVKLVESGG-GLVKLGGSLKLSCAASGFTF----SSYYMSWVRQTPEKRLELVAAINSN--GGSTYYPDTVK-GRFTISRDNAKNTLYLQMSSLKSEDTALYYCAR----------------------', 'IGHV5-6-3*01': 'EVQLVESGG-GLVQPGGSLKLSCAASGFTF----SSYGMSWVRQTPDKRLELVATINSN--GGSTYYPDSVK-GRFTISRDNAKNTLYLQMSSLKSEDTAMYYCAR----------------------', 'IGHV5-6-4*01': 'DVKLVESGG-GLVKPGGSLKLSCAASGFTF----SSYTMSWVRQTPEKRLEWVATISSG--GSYTYYPDSVK-GRFTISRDNAKNTLYLQMSSLKSEDTAMYYCTR----------------------', 'IGHV5-6-4*02': 'EVKLVESGG-GLVKPGGSLKLSCAASGFTF----SSYTMSWVRQSPEKRLEWVATISSG--GSYTYYPDSVK-GRFTISRDNAKNTLYLQMSSLKSEDTAMYYCTR----------------------', 'IGHV5-6-5*01': 'EVKLVESGG-GLVKPGGSLKLSCAASGFTF----SSYAMSWVRQTPEKRLEWVASISSG---GSTYYPDSVK-GRFTISRDNARNILYLQMSSLRSEDTAMYYCAR----------------------', 'IGHV5-6-6*01': 'EVKLVESGG-GLVQPGGSLKLSCAASGFTF----SSYAMSWIRQTPDKRLEWVASISSG--GSYTYYPDSVK-GRFTIPRDNTKNTLYLQMSSLSSKDTALYYCAR----------------------', 'IGHV5-9*01': 'EVMLVESGG-GLVKPGGSLKLSCAASGFTF----SSYTMSWVRQTPEKRLEWVATISGG--GGNTYYPDSVK-GRFTISRDNAKNTLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV5-9*02': 'EVKLVESGG-GLVKPGGSLKLSCAASGFAF----SSYDMSWVRQTPEKRLEWVATISSG--GSYTYYPDSVK-GRFTISRDNARNTLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV5-9*03': 'EVMLVESGG-GLVKPGGSLKLSCAASGFTF----SSYTMSWVRQTPEKRLEWVATISSG--GGNTYYPDSVK-GRFTISRDNAKNNLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV5-9-1*01': 'EVMLVESGG-GLVKPGGSLKLSCAASGFTF----SSYAMSWVRQTPEKRLEWVATISSG--GSYTYYPDSVK-GRFTISRDNAKNTLYLQMSSLRSEDTAMYYCAR----------------------', 'IGHV5-9-1*02': 'DVKLVESGE-GLVKPGGSLKLSCAASGFTF----SSYAMSWVRQTPEKRLEWVAYISSG--GDYIYYADTVK-GRFTISRDNARNTLYLQMSSLKSEDTAMYYCTR----------------------', 'IGHV5-9-2*01': 'EVKLVESGG-GLVKPGGSLKLSCAASGFTF----SSYGMSWVRQTPEKRLEWVATISGG--GSYTYYPDSVK-GRFTISRDNAKNNLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV5-9-3*01': 'EVQLVESGG-GLVKPGGSLKLSCAASGFTF----SSYAMSWVRQTPEKRLEWVATISSG--GSYTYYPDSVK-GRFTISRDNAKNTLYLQMSSLRSEDTAMYYCAR----------------------', 'IGHV5-9-4*01': 'EVQLVESGG-GLVKPGGSLKLSCAASGFTF----SSYAMSWVRQSPEKRLEWVAEISSG--GSYTYYPDTVT-GRFTISRDNAKNTLYLEMSSLRSEDTAMYYCAR----------------------', 'IGHV5-9-5*01': 'EVMLVESGG-GLVKPGGSLKLSCAASGFTF----SSYTMSWVRQTPEKRLEWVATISSG--GGNTYYPDSVK-GRFTISRDNAKNNLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV5S4*01': 'EVKLVESGG-GLVKPGGSLKLSCATSGFTF----SSYGMSWVRQTPEKRLEWVATISGG--GSYTYYPDSVK-GRFTISRDNAKNNLYLQMSSLRSEDTALYYCAR----------------------', 'IGHV6-3*01': 'EVKLEESGG-GLVQPGGSMKLSCVASGFTF----SNYWMNWVRQSPEKGLEWVAQIRLKSDNYATHYAESVK-GRFTISRDDSKSSVYLQMNNLRAEDTGIYYCT-----------------------', 'IGHV6-3*02': 'EVKLEESGG-GLVQPGGSMKLSCVASGFTF----SNYWMSWVRQSPEKGLEWVAQIRLKSDNYATHYAESVK-GRFTISRDDSKSSVYLQMNNLRAEDTGIYYCT-----------------------', 'IGHV6-3*03': 'EVKLEESGG-GLVQPGGSMKLSCVASGFTF----SNYWMSWVRQSPEKGLEWVAQIRLKSDNYATHYAESVK-GRFTISRDDSKSSVYLQMNNLRAEDTGIYYCTG----------------------', 'IGHV6-4*01': 'DVNLEVSGG-GLVKPGGSMQLFCVASGFTF----VDGWMDWVRQSPEKGLEWVAEIANKANNYATYYPESVK-GRFTISRDDFKSSVYLHMNSLRAEDTGIYYCTR----------------------', 'IGHV6-6*01': 'EVKLEESGG-GLVQPGGSMKLSCAASGFTF----SDAWMDWVRQSPEKGLEWVAEIRNKANNHATYYAESVK-GRFTISRDDSKSSVYLQMNSLRAEDTGIYYCTR----------------------', 'IGHV6-6*02': 'EVKLEESGG-GLVQPGGSMKLSCVASGFTF----SNYWMNWVRQSPEKGLEWVAEIRLKSNNYATHYAESVK-GRFTISRDDSKSSVYLQMNNLRAEDTGIYYCTR----------------------', 'IGHV6-7*01': 'EEKLDESGG-GLVQPGRSMKLSCVASGFTF----TNSWMNWFCQSPEKGLEWVAQIKSKPYNYETYYSDSVK-GRFTISRDDSKSSVYLQMNNLRAEDTGIYYCTW----------------------', 'IGHV6-7*02': 'EVKLDETGG-GLVQPGRPMKLSCVASGFTF----SDYWMNWVRQSPEKGLEWVAQIRNKPYNYETYYSDSVK-GRFTISRDDSKSSVYLQMNNLRAEDMGIYYCTW----------------------', 'IGHV7-1*01': 'EVKLVESGG-GLVQSGRSLRLSCATSGFTF----SDFYMEWVRQAPGKGLEWIAASRNKANDYTTEYSASVK-GRFIVSRDTSQSILYLQMNALRAEDTAIYYCARDA--------------------', 'IGHV7-1*02': 'EVKLVESGG-GLVQPGGSLRLSCATSGFTF----SDFYMEWVRQPPGKRLEWIAASRNKANDYTTEYSASVK-GRFIVSRDTSQSILYLQMNALRAEDTAIYYCARDA--------------------', 'IGHV7-1*03': 'EVKLVESGG-GLVQSGRSLRLSCATSGFTF----SDFYMEWVRQAPGKGLEWIAASRNKANDYTTEYSASVK-GRFIVSRDTSQSILYLQMNALRAEDTAIYYCARDA--------------------', 'IGHV7-3*01': 'EVKLVESGG-GLVQPGGSLSLSCAASGFTF----TDYYMSWVRQPPGKALEWLGFIRNKANGYTTEYSASVK-GRFTISRDNSQSILYLQMNALRAEDSATYYCARY---------------------', 'IGHV7-3*02': 'EVKLVESGG-GLVQPGGSLRLSCATSGFTF----TDYYMSWVRQPPGKALEWLGFIRNKANGYTTEYSASVK-GRFTISRDNSQSILYLQMNTLRAEDSATYYCARD---------------------', 'IGHV7-3*03': 'EVKLVESGG-GLVQPGGSLSLSCAASGFTF----TDYYMSWVRQLPGKALEWLGFIRNKANGYTTEYSASVK-GRFTISRDNSQSILYLQMNALRAEDSATYYCAKD---------------------', 'IGHV7-3*04': 'EVKLVESGG-GLVQPGGSLSLSCAASGFTF----TDYYMSWVRQPPGKALEWLALIRNKANGYTTEYSASVK-GRFTISRDNSQSILYLQMNALRAEDSATYYCARD---------------------', 'IGHV7-4*01': 'EVKLMESGG-GLVQPGASLRLSCAASGFTF----TDYYMSWVRQPPGKAPEWLALIRNKANGYTTEYTASVK-GRFTISRDNSQNILYLQMNTLRAEDSATYYCVKAV--------------------', 'IGHV7-4*02': 'EVKLMESGG-GLVQPGASLRLSCEASGFTF----TDYYMSWVRQPPGKSPEWLALIRNKANGYTTEYSASVK-GRFTISRDNSQNILYLQMNTLRAEASATYYCAKDV--------------------', 'IGHV7-4*03': 'EVKLMESGG-GLVQPGASLRLSCEASGFTF----TDYYMSWVRQPPGKSPEWLALIRNKANGYTTEYSASVK-GRFTISRDNSQNILYLQMNTLRAEASATYYCAKDV--------------------', 'IGHV7-4*04': 'EVKLVESGG-GLVQPGGSLRLSCAASGFTF----TDYYMSWVRQPPGKAPEWLALIRNKANGYTTEYTASVK-GRFTISRDNSQNILYLQMNTLRAEDSATYYCVKAV--------------------', 'IGHV8-11*01': 'QITQKESGP-GILQPSQTLSLTCSFSGFSLS--TSGMGVGWIHQPSGNGLEWLAHIWWN---DNKYYNTALK-SRLTISKDTSNNQVFLKIASVDTADTATYYCARI---------------------', 'IGHV8-12*01': 'QVTLKESGP-GILQSSQTLSLTCSFSGFSLS--TSGMGVSWIRQPSGKGLEWLAHIYWD---DDKRYNPSLK-SRLTISKDTSRNQVFLKITSVDTADTATYYCARR---------------------', 'IGHV8-6*01': 'QVTLKESGP-GILQPSQTLSLTCSFSGFSLS--TFGMGVSWIRQPSGKDLEWLAHIYWD---DDKHYNPSLK-SQLRISKDTSNNQVFLKITTVDTVDTATYYCARR---------------------', 'IGHV8-8*01': 'QVTLKESGP-GILQPSQTLSLTCSFSGFSLS--TFGMGVGWIRQPSGKGLEWLAHIWWD---DDKYYNPALK-SRLTISKDTSKNQVFLKIANVDTADTATYYCARI---------------------', 'IGHV9-1*01': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TEYPMHWVKQAPGKGFKWMGMIYTD--TGEPTYAEEFK-GRFAFSLETSASTAYLQINNLKNEDTATYFCVR----------------------', 'IGHV9-1*02': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TNYGMNWVKQAPGKGLKWMGWINTY--TGEPTYADDFK-GRFAFSLETSASTAYLQINNLKNEDMATYFCAR----------------------', 'IGHV9-2*01': 'QIQFVQSGP-ELKKPGETVKISCKASVYTF----TEYPMHWVKQAPGKGFKWMGWINTY--SGEPTYADDFK-GRFAFSLETSASTAYLQINNLKNEDTATYFCAR----------------------', 'IGHV9-2*02': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TNYAMHWVKQAPGKGLKWMGWKYTN--TGEPTYGDDFK-GRFAFSLETSASTAYLQINNLKNEDMATYFCAR----------------------', 'IGHV9-2-1*01': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TDYSMHWVKQAPGKGLKWMGWINTE--TGEPTYADDFK-GRFAFSLETSASTAYLQINNLKNEDTATYFCAR----------------------', 'IGHV9-3*01': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TTYGMSWVKQAPGKGLKWMGWINTY--SGVPTYADDFK-GRFAFSLETSASTAYLQINNLKNEDTATYFCAR----------------------', 'IGHV9-3*02': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TNYGMNWVKQAPGKGLKWMGWINTN--TGEPTYAEEFK-GRFAFSLETSASTAYLQINNLKNEDTATYFCAR----------------------', 'IGHV9-3*03': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TNYGMNWVKQAPGKGLKWMGWINTN--TGEPTYAEEFK-GRFAFSLETSASTAYLQINNLKNEDTATYFC------------------------', 'IGHV9-3-1*01': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TNYGMNWVKQAPGKGLKWMGWINTY--TGEPTYADDFK-GRFAFSLETSASTAYLQINNLKNEDTATYFCAR----------------------', 'IGHV9-4*01': 'QIQLVQSGP-ELKKPGETVKISCKASGYTF----TTAGMQWVQKMPGKGFKWIGWINTH--SGEPKYAEDFK-GRFAFSLETSASTAYLQISNLKNEDTATYFCAR----------------------', 'IGHV9-4*02': 'QIQLVQSGP-ELKKPGETVRISCKASGYTF----TTAGMQWVQKMPGKGLKWIGWINTH--SGVPKYAEDFK-GRFAFSLETSASTAYLQISNLKNEDTATYFCAR----------------------'}, 'rat': {'IGHV2S13*01': 'QVQLKESGP-GLVQPSQTLSLTCTVSGFSL-----TSYNVHWVRQPPGKGLEWMGVIWSG---GNTDYNSALK-PRLSISRDTSKSQVFLTMNSLQTEDTGIY-YCNR--------------------', 'IGHV2S18*01': 'QVQLKESGP-GLVQPSETLSLTCTVSGFSL-----TSYSVHWVRQHSGKSLEWMGRMWSD---GDTSYNSAFT-SRLSISRDTSKSQVFLKMNSLQTEDTGTY-YCAR--------------------', 'IGHV2S61*01': 'QVQLKESGP-GLVQPSQTLSLTCTVSGFSL-----SSYGVIWVRQPPGKGLEWMGVIWGN---GNTNYNSALK-SRLSISRDTSKSQVFLKMNNLQTEDTAMY-FCA---------------------', 'IGHV2S63*01': 'EVQLKESGP-GLVQPSQTLSLTCTVSGFSL-----TDYSVHWVRQPPGKGLEWMGVMWSG---GSTAYNSALK-SRLSISRDTSKSQVFLKMNSLQTEDTAIY-YCTR--------------------', 'IGHV5-43*01': 'EVQLVESGG-GLVQPGSSLKVSCVASGFTF-----SSYVMHWFRQAPENGIEWLAYINTD--SSSTHYAETVK-GRFTISRDNAKNTVDMQLSSLRSEDTAMY-FCAR--------------------', 'IGHV5S10*01': 'EVQLVESGG-GLVQPGRSLKLSCAASGFTF-----SDYNMAWVRQAPKKGLEWVATIIYD--GSRTYYRDSVK-GRFTISRDNAKSTLYLQMDSLRSEDTATY-YCAT--------------------', 'IGHV5S11*01': 'EVQLVESGG-GLVQPGRSMKLSCAASGFTF-----SNYYMAWVRQAPTKGLEWVASISTG--GGNTYYRDSVK-GRFTISRDNAKSTLYLQMDSLRSEETATY-YCAR--------------------', 'IGHV5S13*01': 'EVQLVESGG-GLVQPGRSLKLSCAASGFTF-----SNYGMAWVRQAPTKGLEWVASISTG--GGNTYYRDSVK-GRFTISRDNAKNTQYLQMDSLRSEDTATY-YCAR--------------------', 'IGHV5S8*01': 'EVKLVESGG-GLVQPGRPLKLSCAASGFTF-----SSNWLNWIRQAPGKGLEWVASINPD--GSSTLYPDTVK-GRFVVSKDNAKNTRYLQMNNLRSEDTAMY-YCAR--------------------'}, 'rabbit': {'IGHV1S1*01': 'QEQLKESGG-RLVMPGGILTLTCTASGSNI----SSYGVSWFRQAPGKGLEWIRYISYG---GSAYYKSWVK-GRFTISKTSS--TVDLKMTSLTASDTATYFCAR----------------------', 'IGHV1S13*01': 'QEQLEESGG-GLVTPGGTLTLTCTVSGFSL----SSYGVSWVRQAAGKGLEWIGYISSS---GSAYYASWVN-GRFTISKTSS--TVDLKMTSLRAADTATYFCAR----------------------', 'IGHV1S17*01': 'QEQQKESGG-RLVMPGGSLTLTCTVSGFSL----SSYNMGWVRQAPGEGLEYIGWISTG---GSAYYASWVN-GRFTISKTST--TMDLKMTSLTAADTATYFCAR----------------------', 'IGHV1S24*01': 'QEQLKESGG-GLVTPGGILSLTCTASGFSI----SSYRMGWVRQAPGKGLEYIGYISYG---GSAYYKSWVK-GRFTISKTSS--TVDLKMTSLTASDKATYFCAR----------------------', 'IGHV1S25*01': 'QEQLKESGG-VLVTPGGILSLTCTASGFSI----SSYRMGWVRQAPGKGLEYIGIIYTG---GSAYYASWVN-GRFTISKTSS--TVDLKMTSLTAADMATYFCAR----------------------', 'IGHV1S26*01': 'Q-SVKESEG-GLFKPTDTLTLTCTVSGFSL----SSYAISWVRQAPGNGLEWIGIINSY---GSTYYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S28*01': 'Q-SLEESRG-GLIKPGGTLTLTCTASGFTI----SSYDMSWVRQAPGKELEWIGYISYG---GSAYYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S31*01': 'Q-SVEESRG-GLIKPTDTLTLTCTVSGFSL----SSYGVIWVRQAPGNGLEYIGTIGSS---GSAYYASWAK-SRSTITRNTNLNTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S33*01': 'QEQLEESGG-GLVKPGDTLTLTCKASGFSL----SSYDMSWVRQAPGKGLEWIGFIWSG---GSTDYASWVN-GRIIISSDNTQNTVSLLMNSLSARDTATYFCAG----------------------', 'IGHV1S34*01': 'Q-SVKESEG-GLFKPTDTLTLTCTVSGFSL----SSNAISWVRQAPGNGLEWIGAIGSS---GSAYYASWAK-SRSTITRNTNLNTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S36*01': 'QEQLEESGG-GLVTPGGTLTLTCTASGFSL----SSYGVSWVRQAAGKGLEWIGYISSS---GSAYYASWVN-GRFTISKTSS--TVDLKMTSLTASDTATYFCAR----------------------', 'IGHV1S40*01': 'Q-SLEESGG-DLVKPGASLTLTCTASGFSFS---SSYYMCWVRQAPGKGLEWIACIYAGS-SGSTYYASWAK-GRFTISKTSS-TTVTLQMTSLTAADTATYFCAR----------------------', 'IGHV1S43*01': 'QQQLEESGG-GLVKPGGTLTLTCKASGIDFS---SYYYICWVRQAPGKGLELIACIYTS--SGSTWYASWVN-GRFTISRSTSLNTVDLKMTSLTAADTATYFCAR----------------------', 'IGHV1S44*01': 'Q-SLEESGG-RLVTPGGSLTLTCTVSGIDL----TSYAMGWVRQAPGKGLEYIGIISSS---GSTYYASWAK-GRFTISKTSST-TVDLKMTSLTTEDTATYFCAG----------------------', 'IGHV1S45*01': 'QEQLEESGG-DLVKPEGSLTLTCTASGFSFS---SSYWICWVRQAPGKGLEWIACIYAGS-SGSTYYASWAK-GRFTISKTSST-TVTLQMTSLTAADTATYFCAR----------------------', 'IGHV1S47*01': 'QEQLVESGG-GLVQPEGSLTLTCKASGFDF----SSNAMCWVRQAPGKGPEWIACIYNG--DGSTYYASWVN-GRFTISRSTSLNTVTLQMTSLTAVDTATYFCAR----------------------', 'IGHV1S49*01': 'Q-SVKESEG-GLFKPADTLTLTCTASGFTI----SSYGVSWVRQAPGKGPEWIGAIDIN---GRTYYATWAK-SRATITRNVNENTVTLRVTSLTAADTATYFCAR----------------------', 'IGHV1S50*01': 'Q-SVKESEG-GLFKPTDTLTLTCTVSGFSL----SSYAISWVRQAPGNGLEYIGYISFT---NTAYYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S51*01': 'Q-SVKESEG-GLFKPTDTLTLTCTVSGFSL----SSYNIGWVRQAPGSGLEWIGIISYG---GSAYYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S52*01': 'Q-SVKESEG-GLFKPTDTLTLTCTVSGFSL----SSYNMGWVRQAPGNELEWIGIITSY---GSTYYASWAK-SRSTITRNTNENPVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S53*01': 'QEQLEESGG-GLVNPGGTLTLTCTVSGFTI----STYGVSWVRQAPGNGLEWIGTVNYD---GSTHYASWAK-SRSTITRNTNENTATLKMTSLTGADTATYFCAR----------------------', 'IGHV1S54*01': 'Q-SLGESRG-GLIKPGGTLTLTCTASGFTI----SSYDMSWVRQAPGEGLEYIGCINSY---GTTYYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCGRE---------------------', 'IGHV1S55*01': 'Q-SVKESEG-GLFKPTDTLTLSCTVSGFSL----SSYGVSWVRQAPGEGLECIGWISTD---GSTYYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCARA---------------------', 'IGHV1S56*01': 'QEQLKESGG-GLVTPGGILSLTCTASGFSL----STYNMGWVPPAPGKGLEYIGWINTG---GSPYCTSWAG-KRSTITRNTSENTVTLEMTSLTAADTATYLCAK----------------------', 'IGHV1S57*01': 'Q-TVKESEG-GLFKPTHTLTLTCTASGFSL----SSYPIIWVRQAPGNGLEWIGITNTY---GSPYYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S58*01': 'QEQLKESGG-RLVMPGGSLTITCTVSGFSL----SSNAISWVRQAPGNGLEWIGVINSG---GTAYYASWAK-GRSTISRNTKENTVTLQMTSLTAADTATYFCAR----------------------', 'IGHV1S59*01': 'Q-SVKESEG-GLFKPTETLTLTCTVSGFSL----RDYRTGWVRQAPGKELEVVAYIRGD---GVIYYASWAK-KRSTITRNTNENTVTLKMTSLTAADTATYFCGR----------------------', 'IGHV1S60*01': 'Q-SVKESEG-GLFKATETLTLTCTLSGFSL----NNNAIHWVRQAPGKGLEWIGMIYGS---GATYYASWVS-GRATITRDTNENTVTLKMTSLTDADTATYFCAR----------------------', 'IGHV1S61*01': 'Q-SVKESEG-GLFKPTDTLTLTCTVSGFSL----TSMSISWVRQAPGNGLEWIGAINRV---STTYYTTWAK-SRSTITRNTNENTVTLKMTSLTVADTATYFCTR----------------------', 'IGHV1S62*01': 'Q-SVKESEG-GLIKPTDTLTLTCTVSGFSL----TNYIIFWVRQAPGKELEWIGYIHGG---GNTYYASWAK-SRSTITRDTKENTVTLKMASLTASDTATYFCAR----------------------', 'IGHV1S63*01': 'Q-SVKESEG-GLFKPTDTLTLTCTASGFSI----SSYRMGWVRQAPGNGLEVIGYIRGD---GVTYCASWAK-SRSTITRNTNENTATLKMTSLTAADTATYFCGR----------------------', 'IGHV1S65*01': 'Q-SLEEFGG-GLIRPASTLTLTCTVSGFSL----NEVGVIWVRQAPGKELEWIGYISYR---GNAYYASWAK-SRSTITRNTKENTVTLKVTGLTAADTATYFCAR----------------------', 'IGHV1S66*01': 'Q-SVKESEG-GLFEPTDTLTLTCTVSGFSL----TKYGVMWVRQAPGNGLEWIGFIAYS---GNTYYTTWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCAR----------------------', 'IGHV1S67*01': 'Q-SVKESEG-GLFKPTDTLTLTCTVSGFSL----STYGVIWVRQAPGNGLEYIGSIASG---GSAGYASWAK-SRSTITRNTNENTVTLKMTSLTAADTATYFCAA----------------------', 'IGHV1S68*01': 'Q-SVEESRG-GLIKPADTLTLTCTVSGFSL----NNYGVIWVRQAPGSGLEYIGTIDIG---VTAFYASWAK-SRSTITKNTNENTVTLRMTSLTAADTATYFCAS----------------------', 'IGHV1S69*01': 'Q-SVEESGG-RLVTPGTPLTLTCTVSGFSL----SSYAMSWVRQAPGKGLEWIGIISSS---GSTYYASWAK-GRFTISKTST--TVDLKITSPTTEDTATYFCAR----------------------', 'IGHV1S7*01': 'Q-QLKESGG-GLVKPGGSLKLCCKASGFTF----SSYYMCWVRQAPGKGLEWIGCIYAG--SGSTHYASWVN-GRFTLSRDNAQSTVCLQLNSLTAADTATYFCAR----------------------', 'IGHV1S8*01': 'QKQLVESGG-GLDQPAGSLKLSCKDSGFTL----SSNAMCWVHQAPGKGLEWIACIDSY---GSTNYVSRVN-GRFTISSDNTQNMVDLEMNSLTAADMAIYFCAR----------------------'}, 'rhesus': {'IGHV4-2*01': 'QLQLQESGP-GLVKPSETLSLTCAVSGGSI----SSNYWSWIRQPPGKGLEWIGRISGS--GGSTDYNPSLK-SRVTISTDTSKNQFSLKLSSVTAADTAVYYCAR----------------------'}, 'pig': {'IGHV1-10*01': 'EVKLVESGG-GLVQPGGSLRLSCVGSGFTF----SSTYINWVRQAPGKGLEWLAAISTS--GGSTYYADSVK-GRFTISRDDSQNTAYLQMNSLRTEDTARYYCAT----------------------', 'IGHV1-11*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFDF----SSYGVGWVRQAPGKGLESLASIGSGSYIGSTYYADSVK-GRFTISSDDSQNTVYLQMNSLRTEDTARYYCAR----------------------', 'IGHV1-12*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFDF----SDYAFSWVRQAPGKGLEWVAAIASSDYDGSTYYADSVK-GRFTISSDDSQNMVYLQMNSLRTEDTARYYCAI----------------------', 'IGHV1-14*01': 'EVKLVESGG-GLVQPGGSLRLSCVGSGFTF----SSYEISWVRQAPGKGLEWLAAISTS--GGSTYYADSVK-GRFTISKDDSQNTAYLQMNSLRTEDTARYYCAR----------------------', 'IGHV1-15*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFTF----SSYSMSWVRQAPGKGLEWLAGIYSS--GSSTYYADSVK-GRFTISSDNSQNTAYLQMNSLRTEDTARYYCAR----------------------', 'IGHV1-2*01': 'EVKLVECGG-GLVQPGGSLRLSCVGSGYTF----SSYGMSWVRQAPGKGLEWLAGIDSGSYSGSSYYADSVK-GRFTISRDDSQNTAYLQMNSLRTEDTARYYCAT----------------------', 'IGHV1-4*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFTF----SSTYINWVRQAPGKGLEWLAAISTS--GGSTYYADSVK-GRFTISRDNSQNTAYLQMNSLRTEDTARYYCAT----------------------', 'IGHV1-4*02': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFTF----SSTYINWVRQAPGKGLEWLAAISTS--GGSTYYADSVK-GRFTISRDNSQNTAYLQMNSLRTEDTARYYCAR----------------------', 'IGHV1-5*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGITF----SSYAVSWVRQAPGKGLESLASIGSGSYIGSTDYADSVK-GRFTISSDDSQNTVYLQMNSLRTEDTARYYCAR----------------------', 'IGHV1-6*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFDF----SDNAFSWVRQAPGKGLEWVAAIASSDYDGSTYYADSVK-GRFTISSDNSQNTVYLQMNSLRTEDTARYYCAI----------------------', 'IGHV1-6*02': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFDF----SDNAFSWVRQAPGKGLEWVAAIASSDYDGSTYYADSVK-GRFTISRDNSQNTVYLQMNSLRTEDTARYYCAI----------------------', 'IGHV1-8*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGITF----SSYAVSWVRQAPGKGLEWLAGIDSGSYSGSTYYADSVK-GRFTISRDDSQNTAYLQMTSLRTEDTARYYCAG----------------------', 'IGHV1S2*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFTF----SSYEISWVRQAPGKGLEWLAGIYSS--GGSTYYADSVK-GRFTISRDNSQNTAYLQMNSLRTEDTARYYCAR----------------------', 'IGHV1S5*01': 'EEKLVESGG-GLVQPGGSLRLSCVGSGFTF----SSYAVSWVRQAPGKGLEWLAGIDSGSYSGSTYYADSVK-GRFTISRDNSQNTAYLQMNSLRTEDTARYYCAR----------------------', 'IGHV1S6*01': 'QEKLVESGG-GLVQPGGSLRLSCVGSGFDF----SSYGVGWVRQAPGKGLESLASIGSGSYIGSTDDADSVK-GRFTISSDNSQNTAYLQMNSLRTEDTARYYCAR----------------------'}, 'alpaca': {'IGHV3-1*01': 'EVQLVESGG-GLVQPGGSLRLSCAASGFTF----DDYAMSWVRQAPGKGLEWVSAISWN--GGSTYYAESMK-GRFTISRDNAKNTLYLQMNSLKSEDTAVYYCAK----------------------', 'IGHV3-3*01': 'QVQLVESGG-GLVQAGGSLRLSCAASGRTF----SSYAMGWFRQAPGKEREFVAAISWS--GGSTYYADSVK-GRFTISRDNAKNTVYLQMNSLKPEDTAVYYCAA----------------------', 'IGHV3S1*01': 'QVQLVESGG-GLVQPGGSLRLSCAASGFTF----SSYWMYWVRQAPGKGLEWVSAINTG--GGSTYYADSVK-GRFTISRDNAKNTLYLQMNSLKSEDTAVYYCAK----------------------', 'IGHV3S53*01': 'QVQLVESGG-GLVQPGGSLRLSCAASGSIF----SINAMGWYRQAPGKQRELVAAITSG---GSTNYADSVK-GRFTISRDNAKNTVYLQMNSLKPEDTAVYYCNA----------------------', 'IGHV4S1*01': 'QVQLQESGP-GLVKPSQTLSLTCTVSGGSIT--TSYYAWSWIRQPPGKGLEWMGVIAYD---GSTYYSPSLK-SRTSISRDTSKNQFSLQLSSVTPEDTAVYYCAR----------------------'}}, 'K': {'human': {'IGKV1-12*01': 'DIQMTQSPSSVSASVGDRVTITCRASQGI------SSWLAWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQAN--------------------', 'IGKV1-12*02': 'DIQMTQSPSSVSASVGDRVTITCRASQGI------SSWLAWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQAN--------------------', 'IGKV1-13*02': 'AIQLTQSPSSLSASVGDRVTITCRASQGI------SSALAWYQQKPGKAPKLLIYDA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQFN--------------------', 'IGKV1-16*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SNYLAWFQQKPGKAPKSLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQYN--------------------', 'IGKV1-17*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------RNDLGWYQQKPGKAPKRLIYAA-------SSLQSGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCLQHN--------------------', 'IGKV1-17*02': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------RNDLGWYQQKPGKAPKRLIYAA-------SSLQSGVP-SRFSGSG--SGTEFTLTISNLQPEDFATYYCLQHN--------------------', 'IGKV1-17*03': 'DIQMTQSPSAMSASVGDRVTITCRASQGI------SNYLAWFQQKPGKVPKRLIYAA-------SSLQSGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCLQHN--------------------', 'IGKV1-27*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SNYLAWYQQKPGKVPKLLIYAA-------STLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDVATYYCQKYN--------------------', 'IGKV1-33*01': 'DIQMTQSPSSLSASVGDRVTITCQASQDI------SNYLNWYQQKPGKAPKLLIYDA-------SNLETGVP-SRFSGSG--SGTDFTFTISSLQPEDIATYYCQQYD--------------------', 'IGKV1-39*01': 'DIQMTQSPSSLSASVGDRVTITCRASQSI------SSYLNWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQSY--------------------', 'IGKV1-5*01': 'DIQMTQSPSTLSASVGDRVTITCRASQSI------SSWLAWYQQKPGKAPKLLIYDA-------SSLESGVP-SRFSGSG--SGTEFTLTISSLQPDDFATYYCQQYN--------------------', 'IGKV1-5*02': 'DIQMTQSPSTLSASVGDRVTIICRASQSI------SSWLAWYQQKPGKAPKLLIYDA-------SSLESGVP-SRFSGSG--SGTEFTLTISSLQPDDFATYYCQQYN--------------------', 'IGKV1-5*03': 'DIQMTQSPSTLSASVGDRVTITCRASQSI------SSWLAWYQQKPGKAPKLLIYKA-------SSLESGVP-SRFSGSG--SGTEFTLTISSLQPDDFATYYCQQYN--------------------', 'IGKV1-6*01': 'AIQMTQSPSSLSASVGDRVTITCRASQGI------RNDLGWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCLQDY--------------------', 'IGKV1-6*02': 'AIQMTQSPSSLSASVGDRVTITCRASQGI------RNDLGWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCLQDY--------------------', 'IGKV1-8*01': 'AIRMTQSPSSFSASTGDRVTITCRASQGI------SSYLAWYQQKPGKAPKLLIYAA-------STLQSGVP-SRFSGSG--SGTDFTLTISCLQSEDFATYYCQQYY--------------------', 'IGKV1-9*01': 'DIQLTQSPSFLSASVGDRVTITCRASQGI------SSYLAWYQQKPGKAPKLLIYAA-------STLQSGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQLN--------------------', 'IGKV1-NL1*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SNSLAWYQQKPGKAPKLLLYAA-------SRLESGVP-SRFSGSG--SGTDYTLTISSLQPEDFATYYCQQYY--------------------', 'IGKV1D-12*01': 'DIQMTQSPSSVSASVGDRVTITCRASQGI------SSWLAWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQAN--------------------', 'IGKV1D-12*02': 'DIQMTQSPSSVSASVGDRVTITCRASQGI------SSWLAWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQAN--------------------', 'IGKV1D-13*01': 'AIQLTQSPSSLSASVGDRVTITCRASQGI------SSALAWYQQKPGKAPKLLIYDA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQFN--------------------', 'IGKV1D-13*02': 'AIQLTQSPSSLSASVGDRVTITCRASQGI------SSALAWYQQKPGKAPKLLIYDA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQFN--------------------', 'IGKV1D-16*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SSWLAWYQQKPEKAPKSLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQYN--------------------', 'IGKV1D-16*02': 'DIQMTQSPSSLSASVGDRVTITCRARQGI------SSWLAWYQQKPEKAPKSLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQYN--------------------', 'IGKV1D-17*01': 'NIQMTQSPSAMSASVGDRVTITCRARQGI------SNYLAWFQQKPGKVPKHLIYAA-------SSLQSGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCLQHN--------------------', 'IGKV1D-33*01': 'DIQMTQSPSSLSASVGDRVTITCQASQDI------SNYLNWYQQKPGKAPKLLIYDA-------SNLETGVP-SRFSGSG--SGTDFTFTISSLQPEDIATYYCQQYD--------------------', 'IGKV1D-39*01': 'DIQMTQSPSSLSASVGDRVTITCRASQSI------SSYLNWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQSY--------------------', 'IGKV1D-43*01': 'AIRMTQSPFSLSASVGDRVTITCWASQGI------SSYLAWYQQKPAKAPKLFIYYA-------SSLQSGVP-SRFSGSG--SGTDYTLTISSLQPEDFATYYCQQYY--------------------', 'IGKV1D-8*01': 'VIWMTQSPSLLSASTGDRVTISCRMSQGI------SSYLAWYQQKPGKAPELLIYAA-------STLQSGVP-SRFSGSG--SGTDFTLTISCLQSEDFATYYCQQYY--------------------', 'IGKV1D-8*02': 'AIWMTQSPSLLSASTGDRVTISCRMSQGI------SSYLAWYQQKPGKAPELLIYAA-------STLQSGVP-SRFSGSG--SGTDFTLTISCLQSEDFATYYCQQYY--------------------', 'IGKV1D-8*03': 'VIWMTQSPSLLSASTGDRVTISCRMSQGI------SSYLAWYQQKPGKAPELLIYAA-------STLQSGVP-SRFSGSG--SGTDFTLTISCLQSEDFATYYCQQYY--------------------', 'IGKV2-24*01': 'DIVMTQTPLSSPVTLGQPASISCRSSQSLVHS-DGNTYLSWLQQRPGQPPRLLIYKI-------SNRFSGVP-DRFSGSG--AGTDFTLKISRVEAEDVGVYYCMQAT--------------------', 'IGKV2-28*01': 'DIVMTQSPLSLPVTPGEPASISCRSSQSLLHS-NGYNYLDWYLQKPGQSPQLLIYLG-------SNRASGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQAL--------------------', 'IGKV2-29*02': 'DIVMTQTPLSLSVTPGQPASISCKSSQSLLHS-DGKTYLYWYLQKPGQSPQLLIYEV-------SSRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQGI--------------------', 'IGKV2-29*03': 'DIVMTQTPLSLSVTPGQPASISCKSSQSLLHS-DGKTYLYWYLQKPGQSPQLLIYEV-------SSRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQGI--------------------', 'IGKV2-30*01': 'DVVMTQSPLSLPVTLGQPASISCRSSQSLVYS-DGNTYLNWFQQRPGQSPRRLIYKV-------SNRDSGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQGT--------------------', 'IGKV2-40*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLDSDDGNTYLDWYLQKPGQSPQLLIYTL-------SYRASGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQRI--------------------', 'IGKV2D-26*01': 'EIVMTQTPLSLSITPGEQASISCRSSQSLLHS-DGYTYLYWFLQKARPVSTLLIYEV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDFGVYYCMQDA--------------------', 'IGKV2D-26*02': 'EIVMTQTPLSLSITPGEQASMSCRSSQSLLHS-DGYTYLYWFLQKARPVSTLLICEV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDFGVYYCMQDA--------------------', 'IGKV2D-26*03': 'EIVMTQTPLSLSITPGEQASMSCRSSQSLLHS-DGYTYLYWFLQKARPVSTLLIYEV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDFGVYYCMQDA--------------------', 'IGKV2D-28*01': 'DIVMTQSPLSLPVTPGEPASISCRSSQSLLHS-NGYNYLDWYLQKPGQSPQLLIYLG-------SNRASGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQAL--------------------', 'IGKV2D-29*01': 'DIVMTQTPLSLSVTPGQPASISCKSSQSLLHS-DGKTYLYWYLQKPGQPPQLLIYEV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQSI--------------------', 'IGKV2D-29*02': 'DIVMTQTPLSLSVTPGQPASISCKSSQSLLHS-DGKTYLYWYLQKPGQSPQLLIYEV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQSI--------------------', 'IGKV2D-30*01': 'DVVMTQSPLSLPVTLGQPASISCRSSQSLVYS-DGNTYLNWFQQRPGQSPRRLIYKV-------SNWDSGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQGT--------------------', 'IGKV2D-40*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLDSDDGNTYLDWYLQKPGQSPQLLIYTL-------SYRASGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQRI--------------------', 'IGKV3-11*01': 'EIVLTQSPATLSLSPGERATLSCRASQSV------SSYLAWYQQKPGQAPRLLIYDA-------SNRATGIP-ARFSGSG--SGTDFTLTISSLEPEDFAVYYCQQRS--------------------', 'IGKV3-11*02': 'EIVLTQSPATLSLSPGERATLSCRASQSV------SSYLAWYQQKPGQAPRLLIYDA-------SNRATGIP-ARFSGSG--SGRDFTLTISSLEPEDFAVYYCQQRS--------------------', 'IGKV3-15*01': 'EIVMTQSPATLSVSPGERATLSCRASQSV------SSNLAWYQQKPGQAPRLLIYGA-------STRATGIP-ARFSGSG--SGTEFTLTISSLQSEDFAVYYCQQYN--------------------', 'IGKV3-20*01': 'EIVLTQSPGTLSLSPGERATLSCRASQSVS-----SSYLAWYQQKPGQAPRLLIYGA-------SSRATGIP-DRFSGSG--SGTDFTLTISRLEPEDFAVYYCQQYG--------------------', 'IGKV3D-11*01': 'EIVLTQSPATLSLSPGERATLSCRASQGV------SSYLAWYQQKPGQAPRLLIYDA-------SNRATGIP-ARFSGSG--PGTDFTLTISSLEPEDFAVYYCQQRS--------------------', 'IGKV3D-11*02': 'EIVLTQSPATLSLSPGERATLSCRASQSV------SSYLAWYQQKPGQAPRLLIYDA-------SNRATGIP-ARFSGSG--PGTDFTLTISSLEPEDFAVYYCQQRS--------------------', 'IGKV3D-11*03': 'EIVLTQSPATLSLSPGERATLSCRASQGV------SSNLAWYQQKPGQAPRLLIYDA-------SNRATGIP-ARFSGSG--PGTDFTLTISSLEPEDFAVYYCQQRS--------------------', 'IGKV3D-15*01': 'EIVMTQSPATLSVSPGERATLSCRASQSV------SSNLAWYQQKPGQAPRLLIYGA-------STRATGIP-ARFSGSG--SGTEFTLTISSLQSEDFAVYYCQQYN--------------------', 'IGKV3D-15*03': 'EIVMTQSPATLSVSPGERATLSCRASQSV------SSNLAWYQQKPGQAPRLLIYGA-------SIRATGIP-ARFSGSG--SGTEFTLTISILQSEDFAVYYCQQYN--------------------', 'IGKV3D-20*01': 'EIVLTQSPATLSLSPGERATLSCGASQSVS-----SSYLAWYQQKPGLAPRLLIYDA-------SSRATGIP-DRFSGSG--SGTDFTLTISRLEPEDFAVYYCQQYG--------------------', 'IGKV3D-7*01': 'EIVMTQSPATLSLSPGERATLSCRASQSVS-----SSYLSWYQQKPGQAPRLLIYGA-------STRATGIP-ARFSGSG--SGTDFTLTISSLQPEDFAVYYCQQDY--------------------', 'IGKV4-1*01': 'DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWA-------STRESGVP-DRFSGSG--SGTDFTLTISSLQAEDVAVYYCQQYY--------------------', 'IGKV5-2*01': 'ETTLTQSPAFMSATPGDKVNISCKASQDI------DDDMNWYQQKPGEAAIFIIQEA-------TTLVPGIP-PRFSGSG--YGTDFTLTINNIESEDAAYYFCLQHD--------------------', 'IGKV6-21*01': 'EIVLTQSPDFQSVTPKEKVTITCRASQSI------GSSLHWYQQKPDQSPKLLIKYA-------SQSFSGVP-SRFSGSG--SGTDFTLTINSLEAEDAATYYCHQSS--------------------', 'IGKV6-21*02': 'EIVLTQSPDFQSVTPKEKVTITCRASQSI------GSSLHWYQQKPDQSPKLLIKYA-------SQSISGVP-SRFSGSG--SGTDFTLTINSLEAEDAATYYCHQSS--------------------', 'IGKV6D-21*01': 'EIVLTQSPDFQSVTPKEKVTITCRASQSI------GSSLHWYQQKPDQSPKLLIKYA-------SQSFSGVP-SRFSGSG--SGTDFTLTINSLEAEDAATYYCHQSS--------------------', 'IGKV6D-21*02': 'EIVLTQSPDFQSVTPKEKVTITCRASQSI------GSSLHWYQQKPDQSPKLLIKYA-------SQSISGVP-SRFSGSG--SGTDFTLTINSLEAEDAAAYYCHQSS--------------------'}, 'mouse': {'IGKV1-110*01': 'DVVMTQTPLSLPVSLGDQASISCRSSQSLVHS-NGNTYLHWYLQKPGQSPKLLIYKV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDLGVYFCSQST--------------------', 'IGKV1-110*02': 'DVVMTQTPLSLPVSLGDQASISCRSSQSLVHS-NGNTYLYWYLQKPGQSPKLLIYRV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDLGVYFCFQGT--------------------', 'IGKV1-117*01': 'DVLMTQTPLSLPVSLGDQASISCRSSQSIVHS-NGNTYLEWYLQKPGQSPKLLIYKV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDLGVYYCFQGS--------------------', 'IGKV1-117*02': 'DVVMTQTPLSLPVSLGDQASISCRSSQSIVHS-NGNTYLEWYLQKPGQSPKLLIYKV-------SNRLSGVP-DRFSGSG--SGTDFTLKISRVEAEDLGVYYCFQGS--------------------', 'IGKV1-122*01': 'DAVMTQTPLSLPVSLGDQASISCRSSQSLENS-NGNTYLNWYLQKPGQSPQLLIYRV-------SNRFSGVL-DRFSGSG--SGTDFTLKISRVEAEDLGVYFCLQVT--------------------', 'IGKV1-132*01': 'DVVMTQTPLSLSVTIGQPASISCKSSQSLLYS-NGKTYLNWLQQRPGQAPKHLMYQV-------SKLDPGIP-DRFSGSG--SETDFTLKISRVEAEDLGVYYCLQGT--------------------', 'IGKV1-133*01': 'DVVMTQTPLTLSVTIGQPASISCKSSQSLLYS-NGKTYLNWLLQRPGQSPKRLIYLV-------SKLDSGVP-DRFTGSG--SGTDFTLKISRVEAEDLGVYYCVQGT--------------------', 'IGKV1-135*01': 'DVVMTQTPLTLSVTIGQPASISCKSSQSLLDS-DGKTYLNWLLQRPGQSPKRLIYLV-------SKLDSGVP-DRFTGSG--SGTDFTLKISRVEAEDLGVYYCWQGT--------------------', 'IGKV1-88*01': 'DVVVTQTPLSLPVSFGDQVSISCRSSQSLANS-YGNTYLSWYLHKPGQSPQLLIYGI-------SNRFSGVP-DRFSGSG--SGTDFTLKISTIKPEDLGMYYCLQGT--------------------', 'IGKV1-99*01': 'DVVLTQTPLSLPVNIGDQASISCKSTKSLLNS-DGFTYLDWYLQKPGQSPQLLIYLV-------SNRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDLGVYYCFQSN--------------------', 'IGKV10-94*01': 'DIQMTQTTSSLSASLGDRVTISCRASEDI------SNYLNWYQQKPDGTVKLLIYYA-------SSLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-94*02': 'DIQMTQTTSSLSASLGDRVTISCSASQGI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-94*03': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-94*04': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-94*05': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-94*06': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQDS--------------------', 'IGKV10-94*07': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQRPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-94*08': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-95*01': 'DIQMTQTTSSLSASLGDRVTISCRASEDI------STYLNWYQQKPDGTVKLLIYYT-------SGLHSGVP-SRFSGSG--SGADYSLTISNLEPEDIATYYCQQYS--------------------', 'IGKV10-96*01': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEQEDIATYFCQQGS--------------------', 'IGKV10-96*02': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEQEDIATYFCQQGS--------------------', 'IGKV10-96*03': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEQEDIATYFCQQDS--------------------', 'IGKV10-96*04': 'DIQMTQTTSSLSASLGDRVTISCRASQDI------SNYLNWYQQKPDGTVKLLIYYT-------SRLHSGVP-SRFSGSG--SGTDYSLTISNLEQEDIATYFCQQDS--------------------', 'IGKV11-125*01': 'DVQMIQSPSSLSASLGDIVTMTCQASQGT------SINLNWFQQKPGKAPKLLIYGA-------SNLEDGVP-SRFSGSR--YGTDFTLTISSLEDEDMATYFCLQHS--------------------', 'IGKV12-38*01': 'DIQMTQSPASLAASVGETVTITCRASENI------YYSLAWYQQKQGKSPQLLIYNA-------NSLEDGVP-SRFSGSG--SGTQYSMKINSMQPEDTATYFCKQAY--------------------', 'IGKV12-41*01': 'DIQMTQSPASLSASVGETVTITCRASGNI------HNYLAWYQQKQGKSPQLLVYNA-------KTLADGVP-SRFSGSG--SGTQYSLKINSLQPEDFGSYYCQHFW--------------------', 'IGKV12-41*02': 'DIQMTQSPASLSASVGETVTITCRASGNI------HNYLAWYQQKQGKSPQLLVYNA-------KTLADGVP-SRFSGSG--SGTQYSLKINSLQPEDFGSYYCQHFW--------------------', 'IGKV12-44*01': 'DIQMTQSPASLSASVGETVTITCRASENI------YSYLAWYQQKQGKSPQLLVYNA-------KTLAEGVP-SRFSGSG--SGTQFSLKINSLQPEDFGSYYCQHHY--------------------', 'IGKV12-46*01': 'DIQMTQSPASLSVSVGETVTITCRASENI------YSNLAWYQQKQGKSPQLLVYAA-------TNLADGVP-SRFSGSG--SGTQYSLKINSLQSEDFGSYYCQHFW--------------------', 'IGKV12-89*01': 'DIQMTQSPASLSASVGETVTITCGASENI------YGALNWYQRKQGKSPQLLIYGA-------TNLADGMS-SRFSGSG--SGRQYSLKISSLHPDDVATYYCQNVL--------------------', 'IGKV12-98*01': 'DIQMTQSPASQSASLGESVTITCLASQTI------GTWLAWYQQKPGKSPQLLIYAA-------TSLADGVP-SRFSGSG--SGTKFSFKISSLQAEDFVSYYCQQLY--------------------', 'IGKV12-e*01': 'DIQMTQSPASLSVSVGETVTITCRASENI------YSNLAWLFSRNRENPPSLVYAA-------TNLADGVP-SRFSGSG--SGTQYSLKINSQQPEDFGSYYCQHFW--------------------', 'IGKV13-84*01': 'DIQMTQSSSSFSVSLGDRVTITCKASEDI------YNRLAWYQQKPGNAPRLLISGA-------TSLETGVP-SRFSGSG--SGKDYTLSITSLQTEDVATYYCQQYW--------------------', 'IGKV13-85*01': 'DIQMTQSSSYLSVSLGGRVTITCKASDHI------NNWLAWYQQKPGNAPRLLISGA-------TSLETGVP-SRFSGSG--SGKDYTLSITSLQTEDVATYYCQQYW--------------------', 'IGKV14-100*01': 'DILMTQSPSSMSVSLGDTVSITCHASQGI------SSNIGWLQQKPGKSFKGLIYHG-------TNLEDGVP-SRFSGSG--SGADYSLTISSLESEDFADYYCVQYA--------------------', 'IGKV14-111*01': 'DIKMTQSPSSMYASLGERVTITCKASQDI------NSYLSWFQQKPGKSPKTLIYRA-------NRLVDGVP-SRFSGSG--SGQDYSLTISSLEYEDMGIYYCLQYD--------------------', 'IGKV14-126*01': 'DIKMTQSPSSMYASLGERVTITCKASQDI------KSYLSWYQQKPWKSPKTLIYYA-------TSLADGVP-SRFSGSG--SGQDYSLTISSLESDDTATYYCLQHG--------------------', 'IGKV14-130*01': 'EIQMTQSPSSMSASLGDRITITCQATQDI------VKNLNWYQQKPGKPPSFLIYYA-------TELAEGVP-SRFSGSG--SGSDYSLTISNLESEDFADYYCLQFY--------------------', 'IGKV16-104*01': 'DVQITQSPSYLAASPGETITINCRASKSI------SKYLAWYQEKPGKTNKLLIYSG-------STLQSGIP-SRFSGSG--SGTDFTLTISSLEPEDFAMYYCQQHN--------------------', 'IGKV17-121*01': 'ETTVTQSPASLSMAIGEKVTIRCITSTDI------DDDMNWYQQKPGEPPKLLISEG-------NTLRPGVP-SRFSSSG--YGTDFVFTIENMLSEDVADYYCLQSD--------------------', 'IGKV17-127*01': 'ETTVTQSPASLSVATGEKVTIRCITSTDI------DDDMNWYQQKPGEPPKLLISEG-------NTLRPGVP-SRFSSSG--YGTDFVFTIENTLSEDVADYYCLQSD--------------------', 'IGKV18-36*01': 'TGETTQAPASLSFSLGETATLSCRSSESV------GSYLAWYQQKAEQVPRLLIHSA-------STRAGGVP-VRFSGTG--SGTDFTLTISSLEPEDAAVYYCQPFK--------------------', 'IGKV19-93*01': 'DIQMTQSPSSLSASLGGKVTITCKASQDI------NKYIAWYQHKPGKGPRLLIHYT-------STLQPGIP-SRFSGSG--SGRDYSFSISNLEPEDIATYYCLQYD--------------------', 'IGKV19-93*02': 'DIQMTQSPSSLSASLGGKVTITCKASQDI------NKYIAWYQHKPGKGPRLLIHYT-------STLQPGIP-SRFSGSG--SGRDYSFSISNLEPEDIATYYCLQYD--------------------', 'IGKV2-109*01': 'DIVMTQAAFSNPVTLGTSASISCRSSKSLLHS-NGITYLYWYLQKPGQSPQLLIYQM-------SNLASGVP-DRFSSSG--SGTDFTLRISRVEAEDVGVYYCAQNL--------------------', 'IGKV2-109*02': 'DIVMTQAAFSNPVTLGTSASISCRSSKSLLHS-NGITYLYWYLQKPGQSPQLLIYQM-------SNLASGVP-DRFSSSG--SGTDFTLRISRVEAEDVGVYYCAQNL--------------------', 'IGKV2-109*03': 'DIVMTQAAFSNPVTLGTSASISCSSSKSLLHS-NGITYLYWYLQRPGQSPQLLIYRM-------SNLASGVP-DRFSGSG--SGTDFTLRISRVEAEDVGVYYCAQML--------------------', 'IGKV2-109*04': 'DIVMTQAAFSNPVTLGTSASISCRSSKSLLHS-DGITYLYWYLQRPGQSPQLLIYRM-------SNLASGVP-DRFSGSG--SGTDFTLRISRVEAEDVGVYYCAQML--------------------', 'IGKV2-112*01': 'DIVITQDELSNPVTSGESVSISCRSSKSLLYK-DGKTYLNWFLQRPGQSPQLLIYLM-------STRASGVS-DRFSGSG--SGTDFTLEISRVKAEDVGVYYCQQLV--------------------', 'IGKV2-112*02': 'DIVITQDELSNPVTSGESVSISCRSSKSLLYK-DGKTYLNWFLQRPGQSPQLLVYWM-------STRASGVS-DRFSGSG--SGTDFTLEISRVKAEDVGVYYCQQVV--------------------', 'IGKV2-137*01': 'DIVMTQAAPSVPVTPGESVSISCRSSKSLLHS-NGNTYLYWFLQRPGQSPQLLIYRM-------SNLASGVP-DRFSGSG--SGTAFTLRISRVEAEDVGVYYCMQHL--------------------', 'IGKV2-a*01': 'DIVMTQAAFSNPVTLGTSASISCRSSKSLLHS-SGNTYLYWFLQKPGQSPQLLIYYI-------SNLASGVP-DRFSGSG--SGTDFTLRISRVEAEDVGVYYCMQGL--------------------', 'IGKV20-101-2*01': 'NIQVIQSPF-LSASVGERVTISCKTHQHI------NSSIAWYQQKVGKAPILLIRDA-------SFSLTDTP-SRFTGNG--FGTDFTLSISSMQSQDGATYFCQQHF--------------------', 'IGKV3-1*01': 'DIVLTQSPASLAVSLGQRATISCRASESVEY--YGTSLMQWYQQKPGQPPKLLIYAA-------SNVESGVP-ARFSGSG--SGTDFSLNIHPVEEDDIAMYFCQQSR--------------------', 'IGKV3-10*01': 'NIVLTQSPASLAVSLGQRATISCRASESVDS--YGNSFMHWYQQKPGQPPKLLIYLA-------SNLESGVP-ARFSGSG--SRTDFTLTIDPVEADDAATYYCQQNN--------------------', 'IGKV3-12*01': 'DIVLTQSPASLAVSLGQRATISCRASKSVST--SGYSYMHWYQQKPGQPPKLLIYLA-------SNLESGVP-ARFSGSG--SGTDFTLNIHPVEEEDAATYYCQHSR--------------------', 'IGKV3-2*01': 'DIVLTQSPASLAVSLGQRATISCRASESVDN--YGISFMNWFQQKPGQPPKLLIYAA-------SNQGSGVP-ARFSGSG--SGTDFSLNIHPMEEDDTAMYFCQQSK--------------------', 'IGKV3-3*01': 'DIVLTQSPASLAVSLGQRATIFCRASQSVDY--NGISYMHWFQQKPGQPPKLLIYAA-------SNLESGIP-ARFSGSG--SGTDFTLNIHPVEEEDAATYYCQQSI--------------------', 'IGKV3-4*01': 'DIVLTQSPASLAVSLGQRATISCKASQSVDY--DGDSYMNWYQQKPGQPPKLLIYAA-------SNLESGIP-ARFSGSG--SGTDFTLNIHPVEEEDAATYYCQQSN--------------------', 'IGKV3-5*01': 'DIVLTQSPASLAVSLGQRATISCRASESVDS--YGNSFMHWYQQKPGQPPKLLIYRA-------SNLESGIP-ARFSGSG--SRTDFTLTINPVEADDVATYYCQQSN--------------------', 'IGKV3-7*01': 'DIVLTQSPASLAVSLGQRATISCRASQSVST--SSYSYMHWYQQKPGQPPKLLIKYA-------SNLESGVP-ARFSGSG--SGTDFTLNIHPVEEEDTATYYCQHSW--------------------', 'IGKV3-9*01': 'DIVLTQSPASLAVSLGQRATISCQASESVSF--AGTSLMHWYQQKPGQPPKLLIYRA-------SNLESGVP-ARFSGSG--SESDFTLTIDPVEEDDAAMYYCMQSM--------------------', 'IGKV4-50*01': 'ENVLTQSPAIMSASLGEKVTMSCRASSSV-------NYMYWYQQKSDASPKLWIYYT-------SNLAPGVP-ARFSGSG--SGNSYSLTISSMEGEDAATYYCQQFT--------------------', 'IGKV4-51*01': 'ENVLTQSPAIMAASLGEKVTMTCSASSSVS-----SSYLHWYQQKSGTSPKLWIYGT-------SNLASGVP-ARFSGSG--AGISYSLTISSMEAENDATYYCQQWS--------------------', 'IGKV4-53*01': 'EIVLTQSPALMAASPGEKVTITCSVSSSIS-----SSNLHWYQQKSETSPKPWIYGT-------SNLASGVP-VRFSGSG--SGTSYSLTISSMEAEDAATYYCQQWS--------------------', 'IGKV4-55*01': 'QIVLTQSPAIMSASPGEKVTMTCSASSSV-------SYMYWYQQKPGSSPRLLIYDT-------SNLASGVP-VRFSGSG--SGTSYSLTISRMEAEDAATYYCQQWS--------------------', 'IGKV4-57*01': 'QIVLTQSPAIMSASPGEKVTITCSASSSV-------SYMHWFQQKPGTSPKLWIYST-------SNLASGVP-ARFSGSG--SGTSYSLTISRMEAEDAATYYCQQRS--------------------', 'IGKV4-57-1*01': 'ENVLTQSPAIMSASPGEKVTMTCRASSSVS-----SSYLHWYQQKSGASPKLWIYST-------SNLASGVP-ARFSGSG--SGTSYSLTISSVEAEDAATYYCQQYS--------------------', 'IGKV4-58*01': 'ENVLTQSPAIMAASLGQKVTMTCSASSSVS-----SSYLHWYQQKSGASPKPLIHRT-------SNLASGVP-ARFSGSG--SGTSYSLTISSVEAEDDATYYCQQWS--------------------', 'IGKV4-59*01': 'QIVLTQSPAIMSASPGEKVTMTCSASSSV-------SYMHWYQQKSGTSPKRWIYDT-------SKLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAATYYCQQWS--------------------', 'IGKV4-61*01': 'QIVLTQSPAIMSASPGEKVTISCSASSSV-------SYMYWYQQKPGSSPKPWIYRT-------SNLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAATYYCQQYH--------------------', 'IGKV4-63*01': 'ENVLTQSPAIMSASPGEKVTMTCSASSSV-------SYMHWYQQKSSTSPKLWIYDT-------SKLASGVP-GRFSGSG--SGNSYSLTISSMEAEDVATYYCFQGS--------------------', 'IGKV4-68*01': 'QIVLTQSPALMSASPGEKVTMTCSASSSV-------SYMYWYQQKPRSSPKPWIYLT-------SNLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAATYYCQQWS--------------------', 'IGKV4-69*01': 'QILLTQSPAIMSASPGEKVTMTCSASSSV-------SYMHWYQQKPGSSPKPWIYDT-------SNLASGFP-ARFSGSG--SGTSYSLIISSMEAEDAATYYCHQRS--------------------', 'IGKV4-70*01': 'QIVLTQSPAIMSASPGEKVTMTCSASSSI-------SYMHWYQQKPGTSPKRWIYDT-------SKLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAATYYCHQRS--------------------', 'IGKV4-71*01': 'QIVLTQSPAIMSASPGEKVTMTCSASSSV-------SYMHWYQQKPGSSPRLWIYLT-------FNLASGVP-ARFSGSG--SGTSYSLSISSMEAEDAATYYCQQWS--------------------', 'IGKV4-72*01': 'QIVLSQSPAILSASPGEKVTMTCRASSSV-------SYMHWYQQKPGSSPKPWIYAT-------SNLASGVP-ARFSGSG--SGTSYSLTISRVEAEDAATYYCQQWS--------------------', 'IGKV4-73*01': 'QIVLTQSPAIMSASPGERVTMTCSASSSVS-----SSYLYWYQQKSGSSPKLWIYSI-------SNLASGVP-ARFSGSG--SGTSYSLTINSMEAEDAATYYCQQWS--------------------', 'IGKV4-74*01': 'QIVLTQSPAIMSASLGERVTMTCTASSSVS-----SSYLHWYQQKPGSSPKLWIYST-------SNLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAATYYCHQYH--------------------', 'IGKV4-78*01': 'QIVLTQSPAIMSASPGEKVTMTCSARSSVS-----SSYLYWYQQKPGSSPKLWIYST-------SNLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAATFYCQQYS--------------------', 'IGKV4-79*01': 'QIVLTQSPAIMSASPGEKVTLTCSASSSVS-----SSYLYWYQQKPGSSPKLWIYST-------SNLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAASYFCHQWS--------------------', 'IGKV4-80*01': 'QIVLTQSPAIMSASLGEEITLTCSASSSV-------SYMHWYQQKSGTSPKLLIYST-------SNLASGVP-SRFSGSG--SGTFYSLTISSVEAEDAADYYCHQWS--------------------', 'IGKV4-81*01': 'ENVLTQSPAIMAASPGEKVTMTCSASSSVS-----SSNLHWYQQKSGTSTKFWIYRT-------SNLASEVP-APFSGSG--SGTSYSLTISSVEAEDAATYYCQQWS--------------------', 'IGKV4-86*01': 'EIVLTQSPAITAASLGQKVTITCSASSSV-------SYMHWYQQKSGTSPKPWIYEI-------SKLASGVP-ARFSGSG--SGTSYSLTISSMEAEDAAIYYCQQWN--------------------', 'IGKV4-90*01': 'EILLTQSPAIIAASPGEKVTITCSASSSV-------SYMNWYQQKPGSSPKIWIYGI-------SNLASGVP-ARFSGSG--SGTSFSFTINSMEAEDVATYYCQQRS--------------------', 'IGKV4-91*01': 'EIVLTQSPTTMAASPGEKITITCSASSSIS-----SNYLHWYQQKPGFSPKLLIYRT-------SNLASGVP-ARFSGSG--SGTSYSLTIGTMEAEDVATYYCQQGS--------------------', 'IGKV4-92*01': 'EMVLTQSPVSITASRGEKVTITCRASSSIS-----SNYLHWYQQKPGSSPKLLIYRT-------SILASGVL-DSFSGSG--SESSYTLTISCMQDEVAATYYCQQGS--------------------', 'IGKV5-37*01': 'DILLTQSPATLSVTPGETVSLSCRASQSI------YKNLHWYQQKSHRSPRLLIKYA-------SDSISGIP-SRFTGSG--SGTDYTLSINSVKPEDEGIYYCLQGY--------------------', 'IGKV5-39*01': 'DIVMTQSPATLSVTPGDRVSLSCRASQSI------SDYLHWYQQKSHESPRLLIKYA-------SQSISGIP-SRFSGSG--SGSDFTLSINSVEPEDVGVYYCQNGH--------------------', 'IGKV5-43*01': 'DIVLTQSPATLSVTPGDSVSLSCRASQSI------SNNLHWYQQKSHESPRLLIKYA-------SQSISGIP-SRFSGSG--SGTDFTLSINSVETEDFGMYFCQQSN--------------------', 'IGKV5-45*01': 'DIVLTQSPATLSVTPGDRVSLSCRASQSI------SNYLHWYQQKSHESPRLLIKYA-------SQSISGIP-SRFSGSG--SGTDFTLSINSVETEDFGMYFCQQSN--------------------', 'IGKV5-48*01': 'DILLTQSPAILSVSPGERVSFSCRASQSI------GTSIHWYQQRTNGSPRLLIKYA-------SESISGIP-SRFSGSG--SGTDFTLSINSVESEDIADYYCQQSN--------------------', 'IGKV6-14*01': 'DIVMTQSQKFMSTSVGDRVSITCKASQNV------RTAVAWYQQKPGQSPKALIYLA-------SNRHTGVP-DRFTGSG--SGTDFTLTISNVQSEDLADYFCLQHW--------------------', 'IGKV6-15*01': 'DIVMTQSQKFMSTSVGDRVSVTCKASQNV------GTNVAWYQQKPGQSPKALIYSA-------SYRYSGVP-DRFTGSG--SGTDFTLTISNVQSEDLAEYFCQQYN--------------------', 'IGKV6-17*01': 'DIVMTQSHKFMSTSVGDRVSITCKASQDV------STAVAWYQQKPGQSPKLLIYSA-------SYRYTGVP-DRFTGSG--SGTDFTFTISSVQAEDLAVYYCQQHY--------------------', 'IGKV6-20*01': 'NIVMTQSPKSMSMSVGERVTLSCKASENV------GTYVSWYQQKPEQSPKLLIYGA-------SNRYTGVP-DRFTGSG--SATDFTLTISSVQAEDLADYHCGQSY--------------------', 'IGKV6-23*01': 'DIVMTQSHKFMSTSVGDRVSITCKASQDV------GTAVAWYQQKPGQSPKLLIYWA-------STRHTGVP-DRFTGSG--SGTDFTLTISNVQSEDLADYFCQQYS--------------------', 'IGKV6-25*01': 'DIVMTQSHKFMSTSVGDRVSITCKASQDV------STAVAWYQQKPGQSPKLLIYWA-------STRHTGVP-DRFTGSG--SGTDYTLTISSVQAEDLALYYCQQHY--------------------', 'IGKV6-29*01': 'NIVMTQSPKSMSMSVGERVTLSCKASENV------GTYVSWYQQKPEQSPKLLIYGA-------SNRYPGVP-DRFTGSG--SATDFTLTISSLQAEDLADYHCGQGY--------------------', 'IGKV6-32*01': 'SIVMTQTPKFLLVSAGDRVTITCKASQSV------SNDVAWYQQKPGQSPKLLIYYA-------SNRYTGVP-DRFTGSG--YGTDFTFTISTVQAEDLAVYFCQQDY--------------------', 'IGKV6-32*02': 'SIVMTQTPKFLLVSAGERVTITCKASQSV------SNDVAWYQQKPGQSPKLLIYYA-------SNRYTGVP-DRFTGSG--YGTDFTFTISTVQAEDLAVYFCQQDY--------------------', 'IGKV6-b*01': 'SIVMTQTPKFLPVSAGDRVTMTCKASQSV------GNNVAWYQQKPGQSPKLLIYYA-------SNRYTGVP-DRFTGSG--SGTDFTFTISSVQVEDLAVYFCQQHY--------------------', 'IGKV6-c*01': 'SIVMTQTPKFLPVTAEDRVTITCKASQSV------SNEVAWYQQKPGQSPKLLIYYA-------SNRYTGVP-DRFTGSG--SGTDFTFTISSVQVEDLAVYFCQQHY--------------------', 'IGKV6-d*01': 'SIVMTQSPKSLPVSAGDRVTMTCKASQSV------SNDVAWYQQKPGQSPKLLIYYA-------SNRYTGVP-ERFTGSG--SGTDFTFTISGVQAEDLAVYFCQQHY--------------------', 'IGKV7-33*01': 'DIVMTQSPTFLAVTASKKVTISCTASESLYSSKHKVHYLAWYQKKPEQSPKLLIYGA-------SNRYIGVP-DRFTGSG--SGTDFTLTISSVQVEDLTHYYCAQFY--------------------', 'IGKV8-16*01': 'EIVLTQSIPSLTVSAGERVTISCKSNQNLLWSGNQRYCLVWHQWKPGQTPTPLITWT-------SDRYSGVP-DRFIGSG--SVTDFTLTISSVQAEDVAVYFCQQHL--------------------', 'IGKV8-19*01': 'DIVMTQSPSSLTVTAGEKVTMSCKSSQSLLNSGNQKNYLTWYQQKPGQPPKLLIYWA-------STRESGVP-DRFTGSG--SGTDFTLTISSVQAEDLAVYYCQNDY--------------------', 'IGKV8-21*01': 'DIVMSQSPSSLAVSAGEKVTMSCKSSQSLLNSRTRKNYLAWYQQKPGQSPKLLIYWA-------STRESGVP-DRFTGSG--SGTDFTLTISSVQAEDLAVYYCKQSY--------------------', 'IGKV8-24*01': 'DIVMTQSPSSLAMSVGQKVTMSCKSSQSLLNSSNQKNYLAWYQQKPGQSPKLLVYFA-------STRESGVP-DRFIGSG--SGTDFTLTISSVQAEDLADYFCQQHY--------------------', 'IGKV8-27*01': 'NIMMTQSPSSLAVSAGEKVTMSCKSSQSVLYSSNQKNYLAWYQQKPGQSPKLLIYWA-------STRESGVP-DRFTGSG--SGTDFTLTISSVQAEDLAVYYCHQYL--------------------', 'IGKV8-28*01': 'DIVMTQSPSSLSVSAGEKVTMSCKSSQSLLNSGNQKNYLAWYQQKPGQPPKLLIYGA-------STRESGVP-DRFTGSG--SGTDFTLTISSVQAEDLAVYYCQNDH--------------------', 'IGKV8-28*02': 'DIVMTQSPSSLSVSAGDKVTMSCKSSQSLLNSRNQKNYLAWYQQKPWQPPKLLIYGA-------STRESGVP-DRFTGSG--SGTDFTLTISSVQAEDLAVYYCQNDY--------------------', 'IGKV8-30*01': 'DIVMSQSPSSLAVSVGEKVTMSCKSSQSLLYSSNQKNYLAWYQQKPGQSPKLLIYWA-------STRESGVP-DRFTGSG--SGTDFTLTISSVKAEDLAVYYCQQYY--------------------', 'IGKV8-34*01': 'DILMTQSPSSLTVSAGEKVTMSCKSSQSLLASGNQNNYLAWHQQKPGRSPKMLIIWA-------STRVSGVP-DRFIGSG--SGTDFTLTINSVQAEDLAVYYCQQSY--------------------', 'IGKV9-120*01': 'DIQMTQSPSSLSASLGERVSLTCRASQDI------GSSLNWLQQEPDGTIKRLIYAT-------SSLDSGVP-KRFSGSR--SGSDYSLTISSLESEDFVDYYCLQYA--------------------', 'IGKV9-123*01': 'DIQMIQSPSSMFASLGDRVSLSCRASQGI------RGNLDWYQQKPGGTIKLLIYST-------SNLNSGVP-SRFSGSG--SGSDYSLTISSLESEDFADYYCLQRN--------------------', 'IGKV9-124*01': 'DIQMTQSPSSLSASLGERVSLTCRASQEI------SGYLSWLQQKPDGTIKRLIYAA-------STLDSGVP-KRFSGSR--SGSDYSLTISSLESEDFADYYCLQYA--------------------'}, 'rat': {'IGKV10S6*01': 'AIQVTQSPTSLSASLGDRVTLTCRASQDI------NNKMAWYQQKPGEVPQLLIYYA-------STLQSGTP-SRFSGSG--AGTDFSFTISHLQSEDFATYYCLQGY--------------------', 'IGKV12S1*01': 'DIQVTQSPASLSASPEEIVTITCQASQDI------GSSLLWYQQKPGKSPQLLIYSA-------TILADGVP-SRFSGSR--SGTQYSLKISRLQVEDIGTYYCLQVS--------------------', 'IGKV12S14*01': 'DIQMTQSPASLSASLEEIVTITCQASQDI------GNWLAWYQQKPGKSPQLLIYGA-------TSLADGVP-SRFSGSR--SGTQYSLKISRLQVEDIGIYYCQQAS--------------------', 'IGKV12S16*01': 'DIQMTQSPASLSASLEEIVTITCQASQDI------GNWLSWYQQKPGKSPQLLIYGA-------TSLADGVP-SRFSGSR--SGTQYSLKISRLQVEDIGIYYCLQAY--------------------', 'IGKV12S17*01': 'DIQMTQSPASLSASLEEIVTITCQASQDI------GNWLAWYQQKPGKSPQLLIYGA-------TSLADGVP-SRFSGSR--SGTQYSLKISRLQVEDPGIYYCLQGY--------------------', 'IGKV12S20*01': 'DIQMTQSPASLSASPEEIVTITCQASQDI------GNWLAWYQQKPGKSPQLLIYSA-------TSLADGIP-SRFSGSR--SGTQYSLKISRLQVEDTGIYYCLQRY--------------------', 'IGKV12S22*01': 'DIQMTQSPASLSASLEEIVTITCQPSQGI------GNYLSWYQQKLGKSPQLLIHSA-------TSLEDGVP-SRFSGSR--SGTQYSLKINRLQVEDTGIYYCLQIS--------------------', 'IGKV12S24*01': 'DIQMTQSPASLSASLEEIVTITCQASQDI------GNYLSWYQQKPGKSPQLLIHSA-------TSLADGVP-SRFSGSR--SGTQYSLKINRLQVEDTGIYYCLQHY--------------------', 'IGKV12S25*01': 'DIQMTQSPASLSASLDEIVTITCQASLDI------GNWLAWYQQKPGKSPQLLIYGA-------TSLADGVP-SRFSGSR--SGTQYSLKICKLQVEDTGIYYCLQHY--------------------', 'IGKV12S7*01': 'DIHVTQSPASLSASPEEIVTITCQASQDI------GSSLLWYQQKPGKSPQLLIYSA-------TILADGVP-SRFSGSR--SGTQYSLKISRLQVEDIGTYYCLQFS--------------------', 'IGKV14S1*01': 'DIQMTQSPSSMSASLGDRVTITCQASQDI------GNNLIWFQQKPGKSPRPMIYYA-------TNLANGVP-SRFSGSR--SGSDYSLTISSLESEDMADYHCLQYK--------------------', 'IGKV14S13*01': 'DIQMTQSPSSLPASLGDRVTITCRASQDI------GNYLRWFQQKPGKSPRLMIYGA-------TNLAAGVP-SRFSGSR--SGSDYSLTISSLESEDMADYYCLQSK--------------------', 'IGKV14S14*01': 'DIQMTQSPSSLSASLGDRVTITCRASQDI------GNYLTWFQQKPGKSPRRMIYGA-------TNLAAGVP-SRFSGSR--SGSDYSLTISSLESEDVADYHCLQSI--------------------', 'IGKV14S15*01': 'DIQMTQSPSSMSASLGDRVTITCRASQDI------GNYLSWFQQKPGKSPRRMIYGA-------TNLAAGVP-SRFSGSR--SGSDYSLTISSLESEDMAIYYCLQSI--------------------', 'IGKV14S16*01': 'NIQMTQSSSSMPASLIDREILACRASQDI------RNYLSWYQQKPGKSPKLMISGG-------TNLAARIP-SRFSGSR--SGSDYSLTISSLESEDEADYHCLQYD--------------------', 'IGKV14S18*01': 'DIQMTQSPSSLPASLGDRVTITCRASQDI------GNYLRWFQQKPGKSPRLMIYGA-------TNLANGVP-SRFSGSR--SGSDYSLTINSLESEDMAIYYCLQHN--------------------', 'IGKV14S19*01': 'DIQMTQSPSSMSVSLGDRVTITCRASQDI------GNYLSWYQQKPEKSPKLMIYGA-------TNLEDGVP-SRFSGSR--SGSDYSLTINSLESEDTGIYFCLQHK--------------------', 'IGKV14S2*01': 'DIQMTQAPSSLPASLGDRVTITCRASQDI------GNYLRWFQQKPGKSPRRMIYGA-------TNLAAGVP-SRFSGSR--SGSDYSLTISSLESEDMADYYCVQSK--------------------', 'IGKV14S8*01': 'DIQMTQSPSSMSASLGDTVTINCLASQDI------GNYLSWYQQKPGKSPKLMIYGA-------TNLEDGVP-SRFSGSR--SGSDYSLTINSLGYDDEGIYHCHQYY--------------------', 'IGKV14S9*01': 'DIQMTQSPSSMSVSLGDTVTITCRASQDV------GIYVNWFQQKPGKSPRHMIYRA-------TNLADGVP-SRFSGSR--SGSDYSLTISSLESEDVADYHCLQYD--------------------', 'IGKV15S4*01': 'DIQMTQSPSFLSASLGNSITITCHASQNI------KGWLAWYQQKSGNAPELLIYKA-------SSLQSGVP-SRFSGSG--SGTDYIFTISNLQPEDIATYYCQHYQ--------------------', 'IGKV17S1*01': 'ETTVTQSPASLSMAVGEKVSISCKTSTDI------DDDMNWYQQKSGEAPKLLISEG-------NTLRPGVP-SRFSSSG--YGTDFVFTINNVLLGDEGIYYCQQSD--------------------', 'IGKV1S1*01': 'DVVMTQTPVSLSLAIGQPASISCKSSQSLLGT-SGKTFLNWILQRPGQSPKRLIYQV-------SKLYSEVP-DRFSGSG--SETEFTLKISRVEAEDLGVYYCWQGT--------------------', 'IGKV1S12*01': 'DVVMTQTPPSLSVAIGQSVSISCKSSQSLVHS-DGKTYLNWLLQNPGQSPKRLIYQV-------SNLGSGVP-DRFSGTG--SEKDFTLKISRVEAEDLGVYHCVQAT--------------------', 'IGKV1S14*01': 'DVVMTQTPPSLSVAIGQSVSISCKSSQSLVYS-DGKTYLHWLLQSPGRSPKRLIYQV-------SNLGSGVP-DRFSGTG--SQKDFTLKISRVEAEDLGVYYCAQTT--------------------', 'IGKV1S18*01': 'DVVMTQTPVSLSVAIGQPASISCKSSQSLVHS-DGKTYLNWLLQRPGQSPKRLIYLV-------SKLDSGIP-DRFSGSG--SETDFTLKISRVEADDLGVYYCLQGT--------------------', 'IGKV1S5*01': 'DIVMTQTPLSLSVAIGQSAFICCKSSQSLLYS-NGKKYLNWFLQRPGQSPKCLIYLV-------SKLDFGVP-DRFTGSG--SETNFTLEISRVEAENLGVYYCMQGS--------------------', 'IGKV1S7*01': 'DIVMTQTPLSLSVAIGQSASISCKSSQSLKYS-DGKTYLNWVFQSPGQSPKRLIYQV-------SKLDSGVP-DRFSGTG--SETDFTLKISRVEAEDLGVYYCCQVH--------------------', 'IGKV1S8*01': 'DVVMTQTPVSLSLAIGQPASISCKSSQSLIHS-DGKTYLSWILQRPGQSPKRLIYLV-------SKLDSGVP-DRFSGSG--SETEFTLKISRVEAEDLGVYYCWQAT--------------------', 'IGKV20S1*01': 'DIRMTQTPASLSASLGESVTITCRASQDI------GKSLLWFQQKTGKPPKILIYTA-------SNLVSGIS-PRFSGSG--SGTQFSLKISSLKPEDTANYYCCQGY--------------------', 'IGKV22S1*01': 'DIQMTQSPSVLSASVGDRVTLNCKASQNI------NKYLNWYQQKLGEAPKLLIYNT-------NNLQTGIP-SRFSGSG--SGTDFTLTISSLQPEDFATYFCFQHN--------------------', 'IGKV22S2*01': 'DIQMTQSPSFLSASVGDRVTLSCKASQNI------NKYLAWYQQKLGEAPKLLIYNA-------NSLQTGIP-SRFSGSG--SGTDFTLTISSLQPEDVATYFCLQHN--------------------', 'IGKV22S4*01': 'DIQMTQSPSFLSASVGDRVTINCKASQNI------NRYLNWYQQKLGEAPKLLIYNA-------NSLQTGIP-SRFSGSG--SGTDFTLTISSLQPEDVATYFCLQHN--------------------', 'IGKV22S7*01': 'DIQMTQSPSFLSASVGDRVTINCKASQNI------NKYLNWYQQKLGEAPKLLIYNT-------NNLQTGIP-SRFSGSG--SGTDYTLTISSLQPEDVATYFCLQHS--------------------', 'IGKV22S9*01': 'DIQMTQSPSLLSASVGDRVTLSCKASQSI------YNSLAWYQQKLGEAPKLLIYKT-------NSLQTGIP-SSFSGSG--SGTDYTLTISSLQPEDVATYFCQKYN--------------------', 'IGKV2S3*01': 'DIMMTQSPLSVAVTPGESASISCRSSKSLLHS-NGITYLSWYLQRPEKSPQLLIYQI-------SNLASGVS-GRFSGSG--SGTDFTLKISRVETEDVGIYYCVQFL--------------------', 'IGKV3S1*01': 'DIVLTQSPA-LAVSLEQRATISCKTSQNVDN--YGISYMHWYQQKPGQQPKLLIYEG-------SNLASGIP-ARFSGSG--SGTDFTLTIDPVEADDIATYYCQQSK--------------------', 'IGKV3S18*01': 'DIVLTQSPV-LAVSLGQRATISCRASQSVSI--SSINLMHWYQQKPGQQPKLLIYRA-------SNLASGIP-ARFSGSG--SGTDFTLTIDPVQADDIAAYYCQQSR--------------------', 'IGKV3S19*01': 'DIVLTQSPA-LAVSLGQRATISCRASQSVSI--SRYNLMHWYQQKPGQQPKLLIYRA-------SNLASGIP-ARFSGSG--SGTDFTLTINPVQADDIATYYCQQSR--------------------', 'IGKV3S8*01': 'DTVLTQSPA-LAVSPGERVTISCRASESV------STLMHWYQQKPGQQPTLLIYLA-------SNLESGVP-AMFSGSG--SGTDFTLTIDPVEADDTATYFCQQSW--------------------', 'IGKV3S9*01': 'DTVLTQSPA-LAVSPGERVTISCRASESV------STLMHWYQQKPGQQPTLLIYLA-------SNLESGVP-ARFSGSG--SGTDFTLTIDPVEADDTATYYCQQSW--------------------', 'IGKV6S11*01': 'NTVMTQSPTSMFISVGDRVTMNCKASQNV------GTNVDWYQQKTGQSPKLLIYGA-------SNRYTGVP-DRFTGSG--SGTDFTLTISNMQAEDLAVYYCLQYN--------------------', 'IGKV6S8*01': 'ETVMTQSPTSMSTSIGERVTLNCKASQSV------GINVDWYQQTPGQSPKLLIYGA-------SNRHTGVP-DRFTGSG--FGRDFTLTISNMEAEDLAVYYCLQYG--------------------', 'IGKV8S5*01': 'DIVMTQSPSSLAVSAGETVTINCKSSQSLFGSVRQKNYLAWYQQKPGQSPKLLIYLA-------STRESGVP-DRFIGSG--SGTDFTLTISSVQAEDLANYYCQQYY--------------------', 'IGKV8S6*01': 'DIVMTQSPSSLAVSAGETVTINCKSSQSLLSSGNQKNYLAWYQQKPGQSPKLLIYLA-------STRESGVP-DRFIGSG--SGTDFTLTISSVQAEDLADYYCQQHY--------------------', 'IGKV9S2*01': 'QITLTQQAESLWVSPGEKVSITCRASQSLLYT-DGKHYLSWYQQRPGQTTKALIYHA-------SIRTDGVP-TRFIGSG--SGTEFTLSIEDVQPEDIALYYCLQTL--------------------'}, 'rabbit': {'IGKV1S2*01': 'AQVLTQTESPVSAPVGGTVTINCQASQSVY----DNNWLSWYQQKPGQPPKLLIYDA-------SKLASGVP-SRFSGSG--SGTQFTLTISGVQCDDAATYYCQGSY--------------------', 'IGKV1S2*02': 'AQVLTQTESPVSAPVGGTVTINCQASQSVY----DNNYLSWYQQKPGQPPKLLIYDA-------SKLASGVP-SRFSGSG--SGTQFTLTISGVQCDDAATYYCQGSY--------------------', 'IGKV1S3*01': 'AQVLTQTPASVSAAVGGTVTINCQASESI------SSYLNWYQQKLGQPPKLLIYYA-------STLASGVP-SRFKGSG--SGTEYTLTISGVQCDDAATYYCQHGY--------------------', 'IGKV1S3*02': 'AQVMTQTPASVSAAVGGTVTIICQASESI------SSYLNWYQQKLGQPPKLLIYYA-------STLASGVP-SRFKGSG--SGTEYTLTISGVQCDDAATYYCQHGY--------------------', 'IGKV1S5*01': 'DPVMTQTPSSTSAAVGGTVTINCQSSQNVY----SNNYLSWFQQKPGQPPKLLIYGA-------SKLASGVP-SRFSGSG--SGKQFTLTISGVQCDDAATYYCAGYY--------------------', 'IGKV1S6*01': 'DGVMTQTPAPVSAAVGGTVTINCQASQSI------GSDLSWYQQKPGQPPKLLIYSA-------SKLATGVP-SRFNGSG--SGTQFTLTISGVQCDDAATYYCQCTY--------------------'}, 'rhesus': {'IGKV1S1*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SSYLNWYQQKPGKAPKLLIYYA-------NRLESGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQYN--------------------', 'IGKV1S10*01': 'DIQMTQSPSSLSASVGDRVTITCRASQTI------SSYLAWYQQKPGKVPKLLIYAA-------STLQSGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S11*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SSWLAWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFAVYYCQQRN--------------------', 'IGKV1S12*01': 'DIQMTQSPSSLSASVGDRVTITCRASQTI------SSYLAWYQQKPGKVPKLLIYAA-------STLQSGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCLQYN--------------------', 'IGKV1S13*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SNYLAWYQQKPGKAPKLLIYAA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFAAYYCLQHN--------------------', 'IGKV1S14*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SNYLAWYQQKPGKAPKPLIYYA-------SNLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S15*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SNYLAWYQQKPGKAPKPLIYYA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQGN--------------------', 'IGKV1S16*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SNYLAWYQQKPGKAPKPLIYYA-------SSLESGVP-SRFSGSG--YGTDFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S17*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SNNLAWYQQKPGKAPKLLIYAA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCLQYN--------------------', 'IGKV1S19*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SNYLNWYQQKPGKAPKLLIYAA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFAAYYCLQHN--------------------', 'IGKV1S2*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SNWLAWYQQKPGKAPKLLIYRA-------SNLETGVP-SRFSGSG--SGTDFTLTISSLQPEDIATYYCQQHD--------------------', 'IGKV1S20*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------STYLNWYQQKPGKAPKLLIYAA-------SSLESGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCLQYN--------------------', 'IGKV1S21*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SNYLNWYQQKPGKAPKRLIYDA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFAAYYCLQYN--------------------', 'IGKV1S22*01': 'DIQMTQSPSSLSASVGDKVTITCRASQGI------SNALAWYQQKPGKAPKLLIYAA-------SSLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S23*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SDYLSWYQQKPGKAPKLLIYAA-------SSLESGVP-SRFSGSG--SGTEFTLTISSLQPEDFAAYYCLQGY--------------------', 'IGKV1S24*01': 'DIQMSQSPSSLSASVGDRVTITCRASQGI------SSYLNWYQQKPGKAPKLLIYYA-------NSLASGVP-SRFSGSG--SGTEFTLTISSLQPEDFAAYYCLQGY--------------------', 'IGKV1S25*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SSWLAWYQQKPGKVPKLLIYAA-------STLQSGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S26*01': 'DIQMSQSPSSLSASVGDRVTITCRASQGI------SSYLNWYQQKPGKAPKLLIYYA-------NSLASGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQGN--------------------', 'IGKV1S27*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SNWLAWYQQKPGKAPKLLIYAA-------SNLETGVP-SRFSGSG--SGTDFTLTISSLQPEDIATYYCQQHD--------------------', 'IGKV1S28*01': 'DIQMTQSPSSLSASVGDRVTITCRASQGI------SSYLNWYQQKPGKAPKLLIYYA-------NRLESGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQYN--------------------', 'IGKV1S3*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SNYLAWYQQKPGKAPKPLIYYA-------SSLESGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S4*01': 'DIQMTQSPSSLSASVGDRVTITCQASQGI------SSWLAWYQQKPGKAPKLLLYKA-------PGLQSGVP-SMFSGSG--SGTEFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S5*01': 'DIQMTQSPSSLSASVGDRVTITCRASQTI------SSYLAWYQQKPGKAPKRLIYAA-------SSLESGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S6*01': 'DIQMTQSPSSLSASVGDKVTITCRASQGI------SSWLAWYQQKPGKAPKLLIYKA-------SSLASGVP-SRFSGSG--SGTEFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV1S7*01': 'DIQMTQSPSSLSASVGDTVTITCRASQGI------SSYLNWFQQKPGKAPKLLIYAA-------TTLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFAAYYCLQHN--------------------', 'IGKV1S8*01': 'DIQMTQSPSSLSASVGDRVTITCRASQTI------SSYLAWYQQKPGKVPKLLIYAA-------STLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQYN--------------------', 'IGKV1S9*01': 'DIQMSQSPSSLSASVGDTVTITCRASQGI------SNYLNWFQQKPGKAPKLLIYAA-------TTLQSGVP-SRFSGSG--SGTDFTLTISSLQPEDFATYYCQQHN--------------------', 'IGKV2S1*01': 'DTVMTQTPLSLPVT-G-PASISCRSSQSLPYG-NGVNYLNWYLQKPDQPPQLLIYLG-------SSRFPGVP-DRFTVSR--SDTDFTLQISRVKAEDVGVYYCVQCL--------------------', 'IGKV2S10*01': 'DIVMTQTPLSLPITPGEPASISCRSSQSLLHS-NGNTYLHWYLQKPGQSPQLLIYGG-------SNRASGVP-DRFSGSG--SGTDFTLKISKVEAEDVGVYYCVQAI--------------------', 'IGKV2S11*01': 'DIVMTQTPLSLPITPGEPASISCRSSQSLLHS-NGNTYLHWYLQKPGQSPQLLIYGG-------SNRASGVP-DRFSGSG--SGTDFTLKISKVEAEDVGVYYCVHAI--------------------', 'IGKV2S12*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLHT-DGYTYLDWYLQKPGQSPQLLIYGG-------SNRASGVP-DRFSGSG--SGTDFTLKISKVEAEDVGVYYCMQHK--------------------', 'IGKV2S13*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLHT-DGYTYLDWYLQKPGQSPQLLIYGG-------SNRASGVP-DRFSGSG--SGTDFTLKISKVEAEDVGVYYCMQHK--------------------', 'IGKV2S15*03': 'DIVMTQTPLSLSVTPGQPASISCKSSQSLLHS-DGKTYLYWYLQKPGQSPQLLIYEV-------SSRFSGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCMQGI--------------------', 'IGKV2S17*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLDSEDGNTYLDWYLQKPGQSPQLLIYEV-------SNRASGVP-DRFSGSG--SDTDFTLKISRVEAEDVGVYYCMQAL--------------------', 'IGKV2S18*01': 'DIVMIQTPLSLPVTLGEPASISCRSSQSLVYS-DGKTYLYWYLQKPGQSPQLLMYLV-------SKRASGVP-DKFSGSG--SGTDFTLKISRVEAEDVGVYYCMQAL--------------------', 'IGKV2S19*01': 'DIVMIQTPLSLPVTLGEPASISCRSSQSLVYS-DGKTYLYWYLQKPGQSPQLLMYLV-------SKRASGVP-DKFSGSG--SGTDFTLKISRVEAEDVGVYYCMQAL--------------------', 'IGKV2S2*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLHS-NGNTYLDWYLQKPGQSPRLLIYKV-------TNRESGVP-DRFSGSG--SGTDFTLKISRVEPEDVGVYYCMQST--------------------', 'IGKV2S20*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLDSEDGNTYLEWYLQKPGQSPQPLIYEV-------SNRASGVP-DRFSGSG--SDTDFTLKISRVEAEDVGVYYCMQGI--------------------', 'IGKV2S3*01': 'DIVMTQTPLSLPVTPGEPASISCRSSQSLLHS-NGNTYLHWYLQKPGQSPRLLIYKV-------TNRESGVP-DRFSGSG--SGTDFTLKISRVEPEDVGVYYCMQST--------------------', 'IGKV2S4*01': 'DVVMTQSPLSLPITPGQPASISCRSSQSLVHS-DGNTYLSWYQQKPGQPPRLLIYKV-------SNRYSGVP-DRFSGSG--AGTDFTLKISRVEAEDVGVYYCGQGT--------------------', 'IGKV2S5*01': 'DVAMTQSPLSLPVTPGQPASISCRSSQSLLHS-NGNTYLSWFQQKPGQSPRRLIYKV-------SNRDSGVP-DRFSGSG--AGTDFTLKISRVEAEDVGVYYCMEGT--------------------', 'IGKV2S8*01': 'DVVMTQSPLSLPVTPGQPASISCRSSQSLVHS-DGKTYLNWLQQKPGQPPRRLIYQV-------SNRDSGVP-DRFSGSG--AGTDFTLKISRVEAEDVGVYYCGQGT--------------------', 'IGKV2S9*01': 'DVVMTQSPLSLPVTPGQPASISCRSSQSLVHS-DGKTYLNWLQQKPGQPPRRLIYQV-------SNRDSGVP-DRFSGSG--AGTDFTLKISRVEAEDVGVYYCVQGT--------------------', 'IGKV3S1*01': 'EIVMTQSPATLSLSPGETATLSCRASESV------GSYLAWYQQKPGQAPKLLVHSA-------YFRATGIP-DRFSGSG--SRTEFTLTISSLEPEDVGVYHCQQYN--------------------', 'IGKV3S10*01': 'EIVMTQSPATLSLSPRERATLSCRASQSV------SSNLAWYQQKPGQAPRLLIYYA-------SNRATGIP-DRFSGSG--SGTDFTLTISSLEPEDVGVYYCQQES--------------------', 'IGKV3S11*01': 'QVILTQSPATLSLSPGERATLSCRASQSV------GSNLAWYQQKPGQAPRLLIYGA-------SSRATGIP-DRFSGSG--SGTDFTLTISSLEPEDVAVYYCLQRS--------------------', 'IGKV3S2*01': 'EIVMTQSPATLSLSPGERATLSCRASQSV------SSSLAWYQQKPGQAPRLLIYGA-------SSRATGIP-DRFSGSG--SGTEFTLTISSLEPEDVAVYYCLQRS--------------------', 'IGKV3S3*01': 'EIVMTQSPATLSLSPGERATLSCRASQSV------GSSLAWYQQKPGQAPRLLIYGA-------SSRATGIP-DRFSGSG--SGTDFTLTISSLEPEDVAVYYCLQRS--------------------', 'IGKV3S4*01': 'EIVMTQSPATLSLSPGERATLSCRASQSV------SSRLAWYQQKPGQAPRLLIYDA-------SSRVTGIP-DRFSGSG--SGTDFTLTISSLEPEDVAVYFCQQES--------------------', 'IGKV3S5*01': 'EIVMTQSPATLSLSPGERATLSCRASQSV------SSRLAWYKQKPGQAPRLLIYDA-------SSRATGIP-DRFSGSG--SGTEFTLTISSLEPEDVAVYLCQQES--------------------', 'IGKV3S6*01': 'EIVMTQSPATLSLSPGERATLSCRASQSV------SSRLAWYQQKPGQAPRLLIYDA-------SSRATGIP-DRFSGSG--SGTEFTLTISSLEPEDVAVYFCQQES--------------------', 'IGKV3S7*01': 'EIVMTQSPATLSLSPGERATLSCRASQSV------SSSLAWYQQKPGQAPRLLIYGA-------SSRATGIP-DRFSGSG--SGTEFTLTISSLEPEDFAVYYCQQYS--------------------', 'IGKV3S8*01': 'EIVMTQSPATLSLSPRERATLSCRASQSV------SSNLAWYQQKPGQAPRLLIYYA-------SNRATGIP-DRFSGSG--SGTDFTLTISSLEPEDVGVYYCQQES--------------------', 'IGKV3S9*01': 'EIVMTQSPATLSLSPGERATLSCRASQSV------SSYVAWYQQKPGQAPRLLIYGA-------SSRATGIP-DRFSGSG--SGTEFTLTISSLEPEDFAVYYCQQYS--------------------', 'IGKV4-3*01': 'DIVMTQSPDSLAVSLGERVTINCKSSQSLLYSSNNKNYLAWYQQKPGQAPKLLIYWA-------STRESGVP-NRFSGSG--SGSDFTLTISGLQAEDVAVYYCQQYY--------------------', 'IGKV5-1*01': 'ETILTQSAAFVSATPGDKVTISCRAGQDI------DDDMNWYQQEPGEAPKLIIKDA-------TTLVSGIP-PRFSGSG--YGTDFTLTINNVESEDAAYYFCLQHD--------------------', 'IGKV5-2*01': 'EMILTQSPAFVSATPGDKVTISCRAGQDI------DDDMNWYQQEPGEAPKLIIKDA-------TTLVSGIP-PQFSGSG--YGIDFTLTINSMKSEDTAYYFCQQRD--------------------'}, 'pig': {'IGKV1-11*01': 'AIQLTQSPASLAASLGDTVSITCRASQSI------NKWLAWYQQQAGKAPKLLIYSA-------STLQSGVP-SRFKGSG--SGTDFTLTISGLQAEDVATYYCQQHH--------------------', 'IGKV1-7*01': 'AIQLTQSPASLAASLGDTVSITCRAHQTI------SSYLAWYQQQPGKPPKLLLCDA-------CTLQSGVP-CGFKGSG--SGTHFTLTISGLQAEDVATYYCQQLN--------------------', 'IGKV1-9*01': 'AIQLTQSPASLAASLGDTVSITCRASQSV------SNNLAWYQQQAGKPPKLLIYWA-------SALQSGVP-SRFKGSV--SGTDFTLTISGLQAEDVATYYCQQLN--------------------', 'IGKV2-10*01': 'AIVLTQTPLSLSVSPGEPASISCRSSQSLVDS-DGDSLLHWYLQKPGQSPQLLIYEA-------TNRASGVP-DRFSGSG--SGTDFTLKISRVEAEDVGVYYCFQAL--------------------', 'IGKV2-12*01': 'AIVLTQTPLSLSVSPGEPASISCRSTQSLRGS-YGKNYLNWYQQKPGQSPKLLIYWA-------TNRASGVP-DRFSGSR--SGTDFTLKIIRLEAEDAGVYSCLQDI--------------------', 'IGKV2-13*02': 'AIVLTQTPLSLSVSPGEPASISCRSSQSLEE--YGSNLLSWYQQKPGQSPQLLIYEA-------TNRASGVP-DRFSGSG--SGTDFTLKISRVEAEDAGVYYCQQFK--------------------', 'IGKV2-6*01': 'AIVLTQSPLSLSVSPGAPASISCRSSQSLES--YSYNFLSWYQQKPGQSPRLLIYFA-------TNKASGVP-DRFSGSG--SGTDFTLKISRVEAEDAGVYYCQQNK--------------------', 'IGKV2-8*01': 'AIVLTQTPLSLSVSPGEPASISCRSSQSLEI--YGSNFLSWYQQKPGQSPQLLIYEA-------TNRASGVP-DRFSGSG--SGTDFTLKISRVEAEDAGVYYCQQHK--------------------'}}, 'L': {'human': {'IGLV1-36*01': 'QSVLTQPPS-VSEAPRQRVTISCSGSSSNI----GNNAVNWYQQLPGKAPKLLIYYD-------DLLPSGVS-DRFSGSK--SGTSASLAISGLQSEDEADYYCAAWD--------------------', 'IGLV1-40*01': 'QSVLTQPPS-VSGAPGQRVTISCTGSSSNIG---AGYDVHWYQQLPGTAPKLLIYGN-------SNRPSGVP-DRFSGSK--SGTSASLAITGLQAEDEADYYCQSYD--------------------', 'IGLV1-40*02': 'QSVVTQPPS-VSGAPGQRVTISCTGSSSNIG---AGYDVHWYQQLPGTAPKLLIYGN-------SNRPSGVP-DRFSGSK--SGTSASLAITGLQAEDEADYYCQSYD--------------------', 'IGLV1-40*03': 'QSVVTQPPS-VSGAPGQRVTISCTGSSSNIG---AGYDVHWYQQLPGTAPKLLIYGN-------SNRPSGVP-DRFSGSK--SGASASLAITGLQAEDEADYYCQSYD--------------------', 'IGLV1-44*01': 'QSVLTQPPS-ASGTPGQRVTISCSGSSSNI----GSNTVNWYQQLPGTAPKLLIYSN-------NQRPSGVP-DRFSGSK--SGTSASLAISGLQSEDEADYYCAAWD--------------------', 'IGLV1-47*01': 'QSVLTQPPS-ASGTPGQRVTISCSGSSSNI----GSNYVYWYQQLPGTAPKLLIYRN-------NQRPSGVP-DRFSGSK--SGTSASLAISGLRSEDEADYYCAAWD--------------------', 'IGLV1-47*02': 'QSVLTQPPS-ASGTPGQRVTISCSGSSSNI----GSNYVYWYQQLPGTAPKLLIYSN-------NQRPSGVP-DRFSGSK--SGTSASLAISGLRSEDEADYYCAAWD--------------------', 'IGLV1-51*01': 'QSVLTQPPS-VSAAPGQKVTISCSGSSSNI----GNNYVSWYQQLPGTAPKLLIYDN-------NKRPSGIP-DRFSGSK--SGTSATLGITGLQTGDEADYYCGTWD--------------------', 'IGLV1-51*02': 'QSVLTQPPS-VSAAPGQKVTISCSGSSSNI----GNNYVSWYQQLPGTAPKLLIYEN-------NKRPSGIP-DRFSGSK--SGTSATLGITGLQTGDEADYYCGTWD--------------------', 'IGLV10-54*01': 'QAGLTQPPS-VSKGLRQTATLTCTGNSNNV----GNQGAAWLQQHQGHPPKLLSYRN-------NNRPSGIS-ERLSASR--SGNTASLTITGLQPEDEADYYCSAWD--------------------', 'IGLV10-54*02': 'QAGLTQPPS-VSKGLRQTATLTCTGNSNIV----GNQGAAWLQQHQGHPPKLLSYRN-------NNRPSGIS-ERFSASR--SGNTASLTITGLQPEDEADYYCSALD--------------------', 'IGLV2-11*01': 'QSALTQPRS-VSGSPGQSVTISCTGTSSDVG---GYNYVSWYQQHPGKAPKLMIYDV-------SKRPSGVP-DRFSGSK--SGNTASLTISGLQAEDEADYYCCSYA--------------------', 'IGLV2-11*02': 'QSALTQPRS-VSGSPGQSVTISCTGTSSDVG---GYNYVSWYQQHPGKAPKLMIYDV-------SKRPSGVP-DRFSGSK--SGNTASLTISGLQAEDEADYYCCSYA--------------------', 'IGLV2-14*01': 'QSALTQPAS-VSGSPGQSITISCTGTSSDVG---GYNYVSWYQQHPGKAPKLMIYEV-------SNRPSGVS-NRFSGSK--SGNTASLTISGLQAEDEADYYCSSYT--------------------', 'IGLV2-14*02': 'QSALTQPAS-VSGSPGQSITISCTGTSSDVG---SYNLVSWYQQHPGKAPKLMIYEG-------SKRPSGVS-NRFSGSK--SGNTASLTISGLQAEDEADYYCSSYT--------------------', 'IGLV2-18*01': 'QSALTQPPS-VSGSPGQSVTISCTGTSSDVG---SYNRVSWYQQPPGTAPKLMIYEV-------SNRPSGVP-DRFSGSK--SGNTASLTISGLQAEDEADYYCSLYT--------------------', 'IGLV2-18*02': 'QSALTQPPS-VSGSPGQSVTISCTGTSSDVG---SYNRVSWYQQPPGTAPKLMIYEV-------SNRPSGVP-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYT--------------------', 'IGLV2-18*03': 'QSALTQPPS-VSGSPGQSVTISCTGTSSDVG---SYNRVSWYQQPPGTAPKLMIYEV-------SNRPSGVP-DRFSGSK--SGNTASLTTSGLQAEDEADYYCSSYT--------------------', 'IGLV2-18*04': 'QSALTQPPS-VSGSPGQSVTISCTGTSSDVG---SYNRVSWYQQPPGTAPKLMIYEV-------SNRPSGVP-DRSSGSK--SGNTASLTISGLQAEDEADYYCSSYT--------------------', 'IGLV2-23*01': 'QSALTQPAS-VSGSPGQSITISCTGTSSDVG---SYNLVSWYQQHPGKAPKLMIYEG-------SKRPSGVS-NRFSGSK--SGNTASLTISGLQAEDEADYYCCSYA--------------------', 'IGLV2-23*02': 'QSALTQPAS-VSGSPGQSITISCTGTSSDVG---SYNLVSWYQQHPGKAPKLMIYEV-------SKRPSGVS-NRFSGSK--SGNTASLTISGLQAEDEADYYCCSYA--------------------', 'IGLV2-23*03': 'QSALTQPAS-VSGSPGQSITISCTGTSSDVG---SYNLVSWYQQHPGKAPKLMIYEG-------SKRPSGVS-NRFSGSK--SGNTASLTISGLQAEDEADYYCCSYA--------------------', 'IGLV2-8*01': 'QSALTQPPS-ASGSPGQSVTISCTGTSSDVG---GYNYVSWYQQHPGKAPKLMIYEV-------SKRPSGVP-DRFSGSK--SGNTASLTVSGLQAEDEADYYCSSYA--------------------', 'IGLV2-8*02': 'QSALTQPPS-ASRSPGQSVTISCTGTSSDVG---GYNYVSWYQQHPGKAPKLMIYEV-------SKRPSGVP-DRFSGSK--SGNTASLTVSGLQAEDEADYYCSSYA--------------------', 'IGLV3-1*01': 'SYELTQPPS-VSVSPGQTASITCSGDKLG------DKYACWYQQKPGQSPVLVIYQD-------SKRPSGIP-ERFSGSN--SGNTATLTISGTQAMDEADYYCQAWD--------------------', 'IGLV3-10*01': 'SYELTQPPS-VSVSPGQTARITCSGDALP------KKYAYWYQQKSGQAPVLVIYED-------SKRPSGIP-ERFSGSS--SGTMATLTISGAQVEDEADYYCYSTD--------------------', 'IGLV3-12*01': 'SYELTQPHS-VSVATAQMARITCGGNNIG------SKAVHWYQQKPGQDPVLVIYSD-------SNRPSGIP-ERFSGSN--PGNTTTLTISRIEAGDEADYYCQVWD--------------------', 'IGLV3-12*02': 'SYELTQPHS-VSVATAQMARITCGGNNIG------SKAVHWYQQKPGQDPVLVIYSD-------SNRPSGIP-ERFSGSN--PGNTATLTISRIEAGDEADYYCQVWD--------------------', 'IGLV3-16*01': 'SYELTQPPS-VSVSLGQMARITCSGEALP------KKYAYWYQQKPGQFPVLVIYKD-------SERPSGIP-ERFSGSS--SGTIVTLTISGVQAEDEADYYCLSAD--------------------', 'IGLV3-19*01': 'SSELTQDPA-VSVALGQTVRITCQGDSLR------SYYASWYQQKPGQAPVLVIYGK-------NNRPSGIP-DRFSGSS--SGNTASLTITGAQAEDEADYYCNSRD--------------------', 'IGLV3-21*01': 'SYVLTQPPS-VSVAPGKTARITCGGNNIG------SKSVHWYQQKPGQAPVLVIYYD-------SDRPSGIP-ERFSGSN--SGNTATLTISRVEAGDEADYYCQVWD--------------------', 'IGLV3-21*02': 'SYVLTQPPS-VSVAPGQTARITCGGNNIG------SKSVHWYQQKPGQAPVLVVYDD-------SDRPSGIP-ERFSGSN--SGNTATLTISRVEAGDEADYYCQVWD--------------------', 'IGLV3-21*03': 'SYVLTQPPS-VSVAPGKTARITCGGNNIG------SKSVHWYQQKPGQAPVLVVYDD-------SDRPSGIP-ERFSGSN--SGNTATLTISRVEAGDEADYYCQVWD--------------------', 'IGLV3-22*01': 'SYELTQLPS-VSVSPGQTARITCSGDVLG------ENYADWYQQKPGQAPELVIYED-------SERYPGIP-ERFSGST--SGNTTTLTISRVLTEDEADYYCLSGD--------------------', 'IGLV3-25*01': 'SYELMQPPS-VSVSPGQTARITCSGDALP------KQYAYWYQQKPGQAPVLVIYKD-------SERPSGIP-ERFSGSS--SGTTVTLTISGVQAEDEADYYCQSAD--------------------', 'IGLV3-25*02': 'SYELTQPPS-VSVSPGQTARITCSGDALP------KQYAYWYQQKPGQAPVLVIYKD-------SERPSGIP-ERFSGSS--SGTTVTLTISGVQAEDEADYYCQSAD--------------------', 'IGLV3-27*01': 'SYELTQPSS-VSVSPGQTARITCSGDVLA------KKYARWFQQKPGQAPVLVIYKD-------SERPSGIP-ERFSGSS--SGTTVTLTISGAQVEDEADYYCYSAA--------------------', 'IGLV3-9*01': 'SYELTQPLS-VSVALGQTARITCGGNNIG------SKNVHWYQQKPGQAPVLVIYRD-------SNRPSGIP-ERFSGSN--SGNTATLTISRAQAGDEADYYCQVWD--------------------', 'IGLV3-9*02': 'SYELTQPLS-VSVALGQAARITCGGNNLG------YKSVHWYQQKPGQAPVLVIYRD-------NNRPSGIP-ERFSGSN--SGNTATLTISRAQAGDEADYYCQVWD--------------------', 'IGLV4-60*01': 'QPVLTQSSS-ASASLGSSVKLTCTLSSGHS-----SYIIAWHQQQPGKAPRYLMKLEGS---GSYNKGSGVP-DRFSGSS--SGADRYLTISNLQLEDEADYYCETWD--------------------', 'IGLV4-60*02': 'QPVLTQSSS-ASASLGSSVKLTCTLSSGHS-----SYIIAWHQQQPGKAPRYLMKLEGS---GSYNKGSGVP-DRFSGSS--SGADRYLTISNLQFEDEADYYCETWD--------------------', 'IGLV4-60*03': 'QPVLTQSSS-ASASLGSSVKLTCTLSSGHS-----SYIIAWHQQQPGKAPRYLMKLEGS---GSYNKGSGVP-DRFSGSS--SGADRYLTISNLQSEDEADYYCETWD--------------------', 'IGLV4-69*01': 'QLVLTQSPS-ASASLGASVKLTCTLSSGHS-----SYAIAWHQQQPEKGPRYLMKLNSD---GSHSKGDGIP-DRFSGSS--SGAERYLTISSLQSEDEADYYCQTWG--------------------', 'IGLV4-69*02': 'QLVLTQSPS-ASASLGASVKLTCTLSSGHS-----SYAIAWHQQQPEKGPRYLMKLNSD---GSHSKGDGIP-DRFSGSS--SGAERYLTISSLQSEDEADYYCQTWG--------------------', 'IGLV5-37*01': 'QPVLTQPPS-SSASPGESARLTCTLPSDINV---GSYNIYWYQQKPGSPPRYLLYYYSD---SDKGQGSGVP-SRFSGSKDASANTGILLISGLQSEDEADYYCMIWP--------------------', 'IGLV5-39*01': 'QPVLTQPTS-LSASPGASARFTCTLRSGINV---GTYRIYWYQQKPGSLPRYLLRYKSD---SDKQQGSGVP-SRFSGSKDASTNAGLLLISGLQSEDEADYYCAIWY--------------------', 'IGLV5-39*02': 'QPVLTQPTS-LSASPGASARFTCTLRSGINV---GTYRIYWYQQNPGSLPRYLLRYKSD---SDKQQGSGVP-SRFSGSKDASTNAGLLLISGLQSEDEADYYCAIWY--------------------', 'IGLV5-45*01': 'QAVLTQPAS-LSASPGASASLTCTLRSGINV---GTYRIYWYQQKPGSPPQYLLRYKSD---SDKQQGSGVP-SRFSGSKDASANAGILLISGLQSEDEADYYCMIWH--------------------', 'IGLV5-45*02': 'QAVLTQPSS-LSASPGASASLTCTLRSGINV---GTYRIYWYQQKPGSPPQYLLRYKSD---SDKQQGSGVP-SRFSGSKDASANAGILLISGLQSEDEADYYCMIWH--------------------', 'IGLV5-45*03': 'QAVLTQPSS-LSASPGASASLTCTLRSGINV---GTYRIYWYQQKPGSPPQYLLRYKSD---SDKQQGSGVP-SRFSGSKDASANAGILLISGLQSEDEADYYCMIWH--------------------', 'IGLV5-45*04': 'QAVLTQPSS-LSASPGASASLTCTLCSGINV---GTYRIYWYQQKPGSPPQYLLRYKSD---SDKQQGSGVP-SRFSGSKDASANAGILLISGLQSEDEADYYCMIWH--------------------', 'IGLV5-52*01': 'QPVLTQPSS-HSASSGASVRLTCMLSSGFSV---GDFWIRWYQQKPGNPPRYLLYYHSD---SNKGQGSGVP-SRFSGSNDASANAGILRISGLQPEDEADYYCGTWH--------------------', 'IGLV6-57*01': 'NFMLTQPHS-VSESPGKTVTISCTRSSGSI----ASNYVQWYQQRPGSSPTTVIYED-------NQRPSGVP-DRFSGSIDSSSNSASLTISGLKTEDEADYYCQSYD--------------------', 'IGLV6-57*02': 'NFMLTQPHS-VSESPGKTVTISCTGSSGSI----ASNYVQWYQQRPGSAPTTVIYED-------NQRPSGVP-DRFSGSIDSSSNSASLTISGLKTEDEADYYCQSYD--------------------', 'IGLV7-43*01': 'QTVVTQEPS-LTVSPGGTVTLTCASSTGAVT---SGYYPNWFQQKPGQAPRALIYST-------SNKHSWTP-ARFSGSL--LGGKAALTLSGVQPEDEAEYYCLLYY--------------------', 'IGLV7-46*01': 'QAVVTQEPS-LTVSPGGTVTLTCGSSTGAVT---SGHYPYWFQQKPGQAPRTLIYDT-------SNKHSWTP-ARFSGSL--LGGKAALTLSGAQPEDEAEYYCLLSY--------------------', 'IGLV7-46*02': 'QAVVTQEPS-LTVSPGGTVTLTCGSSTGAVT---SGHYPYWFQQKPGQAPRTLIYDT-------SNKHSWTP-ARFSGSL--LGGKAALTLLGAQPEDEAEYYCLLSY--------------------', 'IGLV8-61*01': 'QTVVTQEPS-FSVSPGGTVTLTCGLSSGSVS---TSYYPSWYQQTPGQAPRTLIYST-------NTRSSGVP-DRFSGSI--LGNKAALTITGAQADDESDYYCVLYM--------------------', 'IGLV8-61*02': 'QTVVTQEPS-FSVSPGGTVTLTCGLSSGSVS---TSYYPSWYQQTPGQAPRTLIYST-------NTRSSGVP-DCFSGSI--LGNKAALTITGAQADDESDYYCVLYM--------------------'}, 'mouse': {'IGLV1*01': 'QAVVTQESA-LTTSPGETVTLTCRSSTGAVT---TSNYANWVQEKPDHLFTGLIGGT-------NNRAPGVP-ARFSGSL--IGDKAALTITGAQTEDEAIYFCALWY--------------------', 'IGLV1*02': 'QAVVTQESA-LTTSPGETVTLTCRSSTGAAT---TSNYANWVQEKPDHLFTGLIGGT-------NNRAPGVP-ARFSGSL--IGDKAALTITGAQTEDEAIYFCALWY--------------------', 'IGLV2*01': 'QAVVTQESA-LTTSPGGTVILTCRSSTGAVT---TSNYAIWVQEKTDHLFAGVIGDT-------SNRAPGVP-ARFSGSL--IGDKAALTITGAQTEDDAMYFCALWY--------------------', 'IGLV2*02': 'QAVVTQESA-LTTSPGGTVILTCRSSTGAVT---TSNYANWVQEKPDHLFTGLIGGT-------SNRAPGVP-VRFSGSL--IGDKAALTITGAQTEDDAMYFCALWY--------------------', 'IGLV3*01': 'QPVLTQSSS-ASFSLGASAKLTCTLSSEHS-----TYIIEWYQQQPLKPPKYVMQLKKD---GSHSKGDGIP-DRFSGSS--SGADRYLSISNIQPEDEAIYICGVDD--------------------'}, 'rat': {'IGLV1S1*01': 'QAVVTQESA-LTTLPGGTVTLTCHSSTGAVT---TSNYANWIQEKADHLFTGIVGDT-------SNRAPGAP-ARFSGSL--LEGKAALTITGAQIEDEATYFCSLWY--------------------', 'IGLV2S1*01': 'QFTLTQPKS-VSGSLRSTITIPCERSSGDI----GDSYVSWYQQHLGRPPINVIYAD-------DQRPSEVS-DRFSGSIDSSSNSASLTITNLQMDDEADYFCQSYD--------------------', 'IGLV3S1*01': 'QAVLTQPNS-VSTSLGSTVKLSCTLSSGNI----ENNYVHWYQQYEGRSPTTMIYND-------DKRPDGVP-DRFSGSIDSSSNSAFLTINNVEIEDEAIYFCHSYV--------------------', 'IGLV3S2*01': 'QFVLTQPNS-VSTNLGSTVKLSCKRSTGNI----GSNYVNWYQQHEGRSPTTMIYRD-------DKRPDGVP-DRFSGSIDRSSNSALLTINNVQTEDEADYFCQSYS--------------------', 'IGLV3S3*01': 'QFVLTQSNS-MSTSLGSTVKLSCKRSTGNI----GSSYVYWYQQHEGRSPTTMIYDD-------DKRPDGVP-DRFSGSIDSSSNSAFLTINNVQIEDEAIYFCQSYS--------------------', 'IGLV4S1*01': 'SYELIQPPS-ASVTLGNTVSLTCVGDELS------KRYAQWYQQKPDKTIVSVIYKD-------SERPSGIS-DRFSGS--SSGTTATLTIHGTLAEDEADYYCLSTY--------------------'}, 'rabbit': {'IGLV2S1*01': 'QPALTQPSS-AFGALGGSVTISCTGTSDDVG---YTNAVYWYRQLPGMSPTLLIYYD-------SKRPSGIP-ERFSGSK--SGNTASLTISWLQPEDEAAYYCSSYR--------------------', 'IGLV2S2*01': 'QPALTQPSS-VSGALGGSVTITCAGSNSDIG---YNSLISWYQQLPGSVPKLLMFRV-------DRLASGIP-ERFSGSK--SGTTASLTISGLQPEDEADYYCVSYT--------------------', 'IGLV3S2*01': 'SYELTQLPS-VSVSLGQTARITCGGNSIG------SKAVHWYQQKPGLAPGLLIYND-------DERPSGVP-DRFSGSN--SGDTATLTISGAQAGDEADYYCQLWD--------------------', 'IGLV3S6*01': 'SHELTKLPS-VSVSLGQTARITCGGDSIE------EYSVHWYQKRPGQAPVLLIYRD-------SNRLSGIP-DHFSGSN--SGNTATLTISGAQAGDEADYYCQVWD--------------------', 'IGLV3S9*01': 'SYELTQLPS-VSVSLGQTARITCGGDSIE------SYAVSWYQQKPGLAPVLLIYRD-------SKWPSGIP-DRFSGSN--SGNTATLTISRAQAGDEADYYCQVFN--------------------', 'IGLV4S3*01': 'QPVLTQSPS-ASAALGSSAKLTCTLSSAHK-----TYYIEWYQQQQGEAPRYLMQLKSD---GSYTKGTGVP-DRFSGSS--SGADRYLIISSVQAEDEADYICGVTG--------------------', 'IGLV4S4*01': 'QPVLTQSPS-VSAALGASAKLTCTLSSAHK-----TYTIDWYQQQQGEAPRYLMQLKSD---GSYTKGTGVP-DRFSGSS--SGADRYLIIPSVQADDEADYYCGADY--------------------', 'IGLV5S1*01': 'QPVLTQPPS-LSASLGTTARLTCTLSTGYSV---GSLGVLWLQQVPGRPPRYLLTYHTE---EFKHQGSGVP-TRFSGSKDTSENSFVLSISGLQPEDEADYYCFTAH--------------------', 'IGLV5S10*01': 'QPVLTQPPS-LSASLDTTARLTCTLSTGYSV---GEYPLVWLQQVPGRPPRYLLGYHTD---DIKHQGSGVP-SRFSGSKDDSANAGVLSISGLQPEDEADYYCAVG---------------------', 'IGLV5S2*01': 'QPVLTQPPS-LSASLGTTARLTCTLSTGYSV---GEYPLVWLQQVPGRPPRYLLGYHTD---DIKHQGSGVH-SRFSGSKDTSENAGVLSISGLQPEDEADYYCATAH--------------------', 'IGLV5S3*01': 'QPVLTQPPS-LSASLGTTARLTCTLSTGYSV---GKYPLVWLQQVPGRPPRYLLTYHTE---EFKHQGSGVH-SRFSGSKDTSENAGVLSISGLQPEDEADYYCVTAH--------------------', 'IGLV5S5*01': 'QPVLTQPPS-LSASLDTTARLTCTLSTGYSV---GSLGVLWLQQVPGRPPRYLLAYHTD---DMKHQGSGVP-SRFSGSKDTSENSFVLSISGLQPEDEADYYCATAC--------------------', 'IGLV5S6*01': 'QPVLTQPPS-LSASLGTTARLTCTLSTGYSV---GSLGVLWLQQVPGRAPRYLLSYNTD---EEKHQGSGVP-TRFSGSKDTSENSFVLSISGLQPEDEADYYCATAH--------------------', 'IGLV5S9*01': 'QPVLTQPPS-LSASLDTTARLTCTLSTGYSV---GSYVIGWYQQVPGRPPRYLLTYHTE---EIKHQGSGVH-SRFSGSKDDSANAGVLSISGLQPEDEADYYCATAH--------------------', 'IGLV6S1*01': 'SVVFTQPQA-VSGSLGETVSISCTRSSGNI----GANYVYWYQQHQGHAPSQLIYQY-------DKRPSGVP-DWFSDSKDSASNSASLTIAGLQPEDEADYYCLSGY--------------------', 'IGLV6S3*01': 'QFVLTQPQS-VSGSLGQTVSISCNRDSGNI----EDYYVHWYQQHPGKAPTTVIYND-------DQRPSGVP-DRFSGSIDSTSNSASLTITGLLAEDEADYYCLSSD--------------------', 'IGLV6S5*01': 'QFVLTQPQS-VSGSLGQTVSISCNRDSGNI----EDYYVHWYQQHPGKAPTTVIYND-------DQRPSGVP-DRFSGSIDSTSNSASLTIAGLQAEDEADYHCQSYD--------------------', 'IGLV6S6*01': 'QFVLNQPQS-VSGSLGQTVSISCNRDSGNI----EEKYVHWYQQHPGKAPTTVIYSD-------DQRPSGVP-DRFSGSINSASNSASLTITGLLAEDEADYHCQSYD--------------------', 'IGLV6S7*01': 'QFVLTQPQS-VSGSLGQTVSISCNRDSGNI----EDYYVHWYQQHPGKAPTTVIYND-------DQRPSGVP-DRFSGSIDSTSNSASLTITGLLAEDEADYYCLSSD--------------------'}, 'rhesus': {'IGLV1-10*01': 'QSVLTQPPS-ASGAPGQSVTISCSGSSSNI----GSNYVYWYQQLSGKAPKLLIYNN-------NQRPSGVP-DRFSGSK--SGTSASLAISGLQSEDEADYYCAAWD--------------------', 'IGLV1-12*01': 'QSVLTQPPS-VSGDPGQRVTISCTGSSSNI----GGYDVYWYQQLPGTAPKLLIYEN-------NKRPSGVS-DRFSGSK--SGTSASLTITGLQSEDEAEYYCETWD--------------------', 'IGLV1-13*01': 'QSVLTQPPS-VSGDPGQRVTISCTGSSSNI----GGYYVYWYQQFPGTAPKLLIYDN-------NKRPSGVS-DRFSGSK--SGTSASLTITGLQPGDEADYYCGAWD--------------------', 'IGLV1-14*01': 'QSVLTQPPS-VSGAPGQRVTISCTGSSSNI----GGYYVSWYQQLPGTTPKLLIYQD-------NKRPSGVS-DRFSGSK--SGTSASLTITGLQTEDEADYYCLSYD--------------------', 'IGLV1-15*01': 'QSVLTQPPS-ASEAARKSVTISCSGSSSNI----GSNSVSWYQQLPGTAPKLLIYYN-------DQRASGVS-DRFSGSK--SGTSASLAISGLQTEDEADYYCAAWD--------------------', 'IGLV1-6*01': 'QSVLTQPPS-VSAAPGQRVTISCSGSSFNF----RRYYVSWYQQLPGAAPKLLIYDV-------NKRPSGVS-DRFSGSQ--SGTSATLGISGLRPEDEADYYCSAWD--------------------', 'IGLV1-7*01': 'QSVLTQPPS-VSGAPGQRVTISCTGSSSNI----GGYYVQWYQQLPGTAPKLLIYEN-------NKRPSGVS-DRFSGSQ--SGTSASLTITGLQSEDEADYYCQSYD--------------------', 'IGLV1-8*01': 'QSVLTQPPS-ASGAPGQSVTISCSGSSSNI----GSNYVYWYQQLPGTAPKLLIYYS-------NQRPSGVP-DRFSGSK--SGTSASLAITGLRSEDEADYYCAAWD--------------------', 'IGLV1-9*01': 'QSVLTQPPS-ASGAPGQSVTISCSGSSSNI----RGNGVHWYQQLSGMAPKLLIYNN-------NQRPSGVP-DRFSGSK--SGTSASLAITGLQSEDEADYYCEAWD--------------------', 'IGLV10-1*01': 'QAGLTQPPS-VSKGLRQTATLTCTGNSNNV----GNQGAAWLQQHQGHPPKLLSYRN-------NNRPSGIS-ERFSASR--SGNTASLTITGLQPEDEADYYCSAWD--------------------', 'IGLV11-1*01': 'QPVLTQPPS-LSASPGASARLPCTLSSDLSV---GSKNMYWYQQKPGSAPRLFLYYYSD---SDKQLGPGVP-NRVSGSKETSSNTAFLLISGLQPEDEADYYCQVYD--------------------', 'IGLV1S1*01': 'QSVLTQPPS-VSGAPGQRVTISCTGSSSNIG---AGYYVQWYQQLPGTAPKLLIYEN-------NKRPSGVS-DRFSGSK--SGTSASLTITGLQSEDEADYYCQSYD--------------------', 'IGLV1S2*01': 'QSVLTQPPS-VSGAPGQRVTISCTGSSSNIG---AGYGVQWYQQLPGTAPKLLIYEN-------NKRPSGVS-DRFSGSQ--SGTSASLTITGLQSEDEADYYCLSYD--------------------', 'IGLV1S3*01': 'QSVLTQPPS-VSGAPGQRVTISCTGSSSNI----GGYYVSWYQQLPGTTPKLLIYQD-------NKRPSGVS-DRFSGSK--SGTSASLTITGLQTEDEADYYCLSYD--------------------', 'IGLV1S4*01': 'QSVLTQPPS-ASGAPGQSVTISCSGSSSNI----GGNNVYWYQQLPGTAPKLLIYYS-------NQRPSGVP-DRFSGSK--SGTSASLAITGLRSEDEADYYCAAWD--------------------', 'IGLV1S6*01': 'QSVLTQPPS-VSAAPGQKVTISCSGSSSNI----GRSYVSWYQQVPGTAPKLLIYDN-------NKRPSGVS-DRFSGSK--SGTSASLAITGLQTGDEADYYCGAWD--------------------', 'IGLV1S7*01': 'QSVLTQPPS-VSAAPGQKVTISCSGSSSNI----GRSYVSWYQQVPGTAPKLLIYQD-------NKRPSGVS-DRFSGSK--SGTSASLAITGLQTGDEADYYCSAWD--------------------', 'IGLV2-3*01': 'QAALTQPRS-VSGSPGQSVTISCTGTSSDIG---GYNYVSWYQQHPGTAPKLMIYAV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCCSYA--------------------', 'IGLV2-7*01': 'QAALTQPPS-MSGSPGQSVTISCTGTSSDIG---GYNRVSWYQQHPGKAPKLMIYEV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2-8*01': 'QAAPTQSPS-VSGSAGQSVTISCTGTSSDIG---YYNAVSWYQQHPGKAPKLMIYEV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2-9*01': 'QAALTQSPS-VSGSPGQSVTISCTGTSSDIG---GYNRVSWYQQHPGKAPKLMIYEV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2S1*01': 'QAALTQPPS-VSGSPGQSVTISCTGTSSDIG---GYNYVSWYQQHPGKAPKLMIYDV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2S10*01': 'QSAPIQSPS-VSGSLGQSVTISCTGTSSDIG---RYNYVSWYRQQPGTTTKLMMYKV-------NMRPSGVS-DRFSGSK--SGNTASLTISGLQAEDKADYYCSSYE--------------------', 'IGLV2S11*01': 'QSAPIQSPS-VSGSLGQSVTISCTGTSSDIG---RYNYVSWYRQQPGTTTKLMMYKV-------NMRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYE--------------------', 'IGLV2S2*01': 'QAAPTQPPS-VSGSPGQSVTISCTGTSSDIG---GYNYVSWYQQHPGKAPKLMIYDV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2S3*01': 'QAALTQPRS-VSGSPGQSVTISCTGTSSDIG---GYNYVSWYQQHPGTAPKLMIYEV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2S4*01': 'QAAPTQSPS-VSGSPGQSVTISCTGTSSDIG---GYNRVSWYQQHPGKAPKLMIYEV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2S5*01': 'QAAPTQSPS-VSGSPGQSVTISCTGTSSDIG---GYNRVSWYQQHPGKAPKLMIYEV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2S6*01': 'QAAPTQPPS-VSGSPGQSVTISCTGTSSDIG---YYNAVSWYQQHPGKAPKLMIYEV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCSSYA--------------------', 'IGLV2S7*01': 'QSAPTQPPS-VSGSPGQSVTISCTGTSSDVG---GYNYVSWYQQHPGKAPKLMIYGV-------SNRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCCSYT--------------------', 'IGLV2S9*01': 'QAALTQPPS-VSKSLGQSVTISCTGTSNDVG---GYNDVSWYQQHPGTAPRLLIYDV-------SKRPSGVS-DRFSGSK--SGNTASLTISGLQAEDEADYYCCSYR--------------------', 'IGLV3-1*01': 'SYELTQPPS-VSVSPGQTARITCGGDNLG------SKYVHWYQQKPAQAPVLVIYYD-------SDRPSGIP-ERFSGSK--SGNTATLTISGVEAGDEADYYCQVWD--------------------', 'IGLV3-10*01': 'SYELTQPPS-VSVSPGQTARITCSGDALP------KKYAYWFQQKPGQSPVLIIYED-------SKRPSGIP-ERFSGSS--SGTVATLTISGAQVEDEADYYCYSTD--------------------', 'IGLV3-2*01': 'SSELTQEPA-LSVALGHTVRMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNTGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3-3*01': 'SYELTQPPL-VSVSPGQTARITCSGDVLK------ENYADWYQQKPGQAPVLLIHED-------SKRPSGIP-ERFSGST--SGDTTTLTISSTLSEDEADYSCFSGN--------------------', 'IGLV3-4*01': 'SYDVTQPRS-VSVSPGQTARITCGGDNIG------SKVVHWYQQKPAQAPVLVIYRD-------SKRPSGIP-ERFSGSN--SGNTATLTISGVEAGDEADYYCQVWD--------------------', 'IGLV3-5*01': 'SSELTQDPA-VSVALGQTVRITCQGDSLR------SYYASWYQQKPGQAPVLVVYGN-------NNRPSGIP-ERFSGSS--SGNTASLTITGAQVEDEADYYCDSWD--------------------', 'IGLV3-6*01': 'SYELTQPPS-VSAASGQTARITCGGDNIG------SKNVHWYQQKPAQAPVLVIYAD-------SKRPSGIP-ERFSGSN--SGNTATLTISRVEAGDEADYYCQVWD--------------------', 'IGLV3S1*01': 'SSELTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNTGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3S10*01': 'SSGLTQEPA-LSVALGHTVRMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------TNRPSGIP-GRFSGSW--SGNTGSLTITGAQVEDEADYYCGSWD--------------------', 'IGLV3S11*01': 'SSGLTQEPA-LSVALGHTVRMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------TNRPSGIP-GRFSGSW--SGNTGSLTITGAQVEDEADYYCGSWD--------------------', 'IGLV3S12*01': 'SSELTQKPA-LSVALGHTVRMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------TNRPSGIP-GRFSVSW--SGNTASLTITGAQVEDEADYYCGSWD--------------------', 'IGLV3S13*01': 'SSELTQDPA-VSVALGQTVRITCQGDSLR------SYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNTGSLTITGAQVEDEADYYCGSWD--------------------', 'IGLV3S14*01': 'SYELTQPPS-VSVSPGQTAKITCSGEILA------KKYARWFQQKPGQAPVLVIYKD-------SERPSGIP-ERFSGSS--SGTTVTLTVSGAQAEDEADYYCYSGD--------------------', 'IGLV3S15*01': 'SYELTQPPS-VSVSPGQTAKITCSGEILA------KKYARWFQQKPGQAPVLVIYKD-------SERPSGIP-ERFSGSS--SGTTVTLTVSGAQAEDEADYYCYSGD--------------------', 'IGLV3S16*01': 'SYELTQPPS-VSVSLGQTAKITCSGDVLA------KYYAHWFQQKPGQAPVLVIYKD-------SERPSGIP-ERFSGSS--SGTTVTLTISGAQAEDEADYYCYSGD--------------------', 'IGLV3S2*01': 'SSELTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNTGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3S3*01': 'SSELTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIS-ERFSGSS--SGNTASLTITGAQVEDEADYYCDSWD--------------------', 'IGLV3S4*01': 'SSELTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNRGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3S5*01': 'SSELTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNTGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3S6*01': 'SSELTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNTGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3S7*01': 'SSGLTQEPA-LSVALGHTVRMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------TNRPSGIP-GRFSGSW--SGNRGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3S8*01': 'SSELTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNTGSLTITAAQVEDEADYYCNSWD--------------------', 'IGLV3S9*01': 'SSGLTQEPA-LSVALGHTVSMTCQGDSLK------TYYASWYQQKPGQVPVLVIYGN-------NYRPSGIP-GRFSVSW--SGNRGSLTITAAQVEDEADYYCGSWD--------------------', 'IGLV4-2*01': 'QPVLTQSPS-ASASLGASVKLTCTLSSGHS-----SYAIAWHQQQQGKAPRYLMRLNSD---GSHSKGDGIP-DRFSGSS--SGAERYLTISNLQSEDEADYYCQTWD--------------------', 'IGLV4S1*01': 'QPVLTQSPS-ASASLGASIKLTCTLSSGHS-----SYTIAWHQQQQGKAPRYLMWLKSD---GSHSKGDGIP-DRFSGSS--SGAERYLTISNLQSEDEADYYCQTWD--------------------', 'IGLV4S2*01': 'QPVLTQSPS-ASASLGASVKLTCTLSSGHS-----SYTIAWHQQQQGKAPRYLMWLKSD---GSHSKGDGIP-DRFSGSS--SGAERYLTISNLQSEDEADYYCQTWD--------------------', 'IGLV4S3*01': 'QPVLTQSPS-ASASLGASVKLTCTLSSGHS-----SYTIAWHQQQQGKAPRYLMWLKSD---GSHSKGDGIP-DRFSGSS--SGAERYLTISNLQSEDEADYYCQTWD--------------------', 'IGLV4S4*01': 'QPVLTQSPS-ASASLGASVKLTCTLSSGHS-----SYAIAWHQQQQGKAPRYLMRLNSV---GSHSKGDGIP-DRFSGSS--SGAERYLTISNLQSEDEADYYCQTWT--------------------', 'IGLV5-10*01': 'QPVLTQPAS-LSASPGASASLTCTFSGGTNV---GDYTIHWYQQKPGSPPRYLLKYKSD---SDKHQGSGVP-SRFSGSKDASANTGILRISGLQSEDEADYYCAIGR--------------------', 'IGLV5-10*02': 'QPVLTQPAS-VSASPGASASLTCTFSGGTNV---GDYTIHWYQQKPGSPPRYLLKYKSD---SDKHQGSGVP-SRFSGSKDASANTGILRISGLQSEDEADYYCAIGR--------------------', 'IGLV5-11*01': 'QPVLTQPTS-LSASPGASARLTCTLRSGISV---GSYRIFWYQQKPGSPPRYLLNYHTD---SDKHQGSGVP-SRFSGSKDASANAGILLISGLQSEDEADYYCMIWH--------------------', 'IGLV5-13*01': 'QPVLTQPTS-LSASPGASVRLTCTLRSGISV---GGYNIHWYQQKPGSPPRYLLYYYSD---SNKGQGSGVP-SRFSGSKDASANAGILLISGFQSEDEADYYCTTWH--------------------', 'IGLV5-14*01': 'KPMLTQPAS-LSASPGASASLTCTFSGGINV---AGYHIFWYQQKPGSPPRYLLRYKSD---SDKGQGSGVP-SRFSGSKDASANTGILRISGLQSEDEADYYCAIGH--------------------', 'IGLV5-5*01': 'QPVLTQPPS-LSASQGASARLSCTLSSGFSA---DLYWIYWYQHKPGSPPRYLLSLYQN---SLHDLGSGVP-RRISGLMEDWSNKGLLLISDLQPEDEADYYCMIEH--------------------', 'IGLV5-6*01': 'QSVLTQPPS-LSASLEALARLTCTLSSGISV---GGKIVYWYQQKPGSNPRYLLSYYSE---SSKHQGSGVP-GRFSGSKDASTNSGILHVSGLQPEDEADYYCKIWH--------------------', 'IGLV5-7*01': 'QPVLTQPSS-HSASPGAPARLTCTLSSGFSV---GDFWIRWYQQKPGSPPRYLLYYHSD---SDKHQGSGVP-SRFSGSNDASANAGILHISGLQPEDEADYYCCTWH--------------------', 'IGLV5S2*01': 'KPMLTQPAS-LSASPGASASLTCTFSGGINV---AGYHIFWYQQKPGSPPRYLLRYKSD---SDKGQGSGVP-SRFSGSKDASANAGILRISGLQSEDEADYYCAIGR--------------------', 'IGLV5S3*01': 'QPVLTQPAS-LSASPGASARLTCTLSSGISV---GSYRIFWYQQKPGSPPRYLLNYHTD---SDKHQGSGVP-SRFSGSKDASANAGILLISGLQSEDEADYYCMIWH--------------------', 'IGLV6-1*01': 'EVVFTQPHS-VSGSPGQTVTISCTRSSGSI----DSEYVQWYQQRPGSAPTTVIYKD-------NQRPSGVP-DRFSGSIDSSSNSASLAISGLKSEDEADYYCQSYD--------------------', 'IGLV6-2*01': 'EVVFTQPHS-VSGSPGQTVTISCTRSSGSI----DSEYVQWYQQRPGNAPTTVIYKD-------NQRPSGVP-DRFSGSIDSSSNSASLAISGLKSEDEADYYCQSAD--------------------', 'IGLV6-5*01': 'EVVFTQPHS-VSGSPGQTVTISCTHSSGSI----DNSYVYWYQQRPGSAPTTVIYND-------DQRPSGVP-DRSSGSIDSSSNSASLTISGLKSEDEADYYCQSYD--------------------', 'IGLV7-1*01': 'QAVVTQEPS-LTVSPGGTVTLTCGSSAGAVT---GSHYPYWFQQKPGQAPRTLIYDT-------SNKLSWTP-ARFSGSL--LGGKAALTLSGAQPEDEAEYYCWLHY--------------------', 'IGLV7-2*01': 'QAVVTQEPS-LTVSPGGTVTLTCASSTGAVT---SGHYPHWFQQKPGQAPKTLIYDT-------SNKLSWTP-ARFSGSL--AGGKAALTLSGAQPEDEAEYYCWLYY--------------------', 'IGLV7-3*01': 'QAVVTQEPS-MTVSPGGTVTLTCASSTGAVT---SGHSPHWFQQKPGQAPKTLIYNT-------NYKHSWTP-ARFSGSL--LGGKAALTLSGAQPEDKAEYYCWLYY--------------------', 'IGLV7-4*01': 'QAVVTQEPS-LTVSPGGTVTLTCASSTGAVT---SGHSPHWCQQKPGQAPRTLIYNT-------SFKHSWTP-ARFSGSL--LGGKAALILSGAQPEDEAEYYCLLHY--------------------', 'IGLV8-1*01': 'ETVVTQEPS-LSVYPGGTVTLTCGLSSGSVS---TSNYPSWYQQTPGQAPRTLIYST-------NTRPSGVP-DRFSGSI--LGNKAALTITGAQADDESDYYCTLYM--------------------'}, 'pig': {'IGLV2-6*01': 'QSALTQPPS-VSRNLKEMETISCAGTSSDI-----GGYVSWYQQHPGLAPKFLIYYV-------NTRASGIP-DGFCGSK--SGNTASLTISGLQAEDEADYYCSSPR--------------------', 'IGLV5-14*01': 'QAVLTQPPS-LSASPGPSARLPCTLSSGSSV---GSYHISWYQRKPGRPPWYLLRFHFAS-SKDQGSGVPSC-FSGDKDA--SAHAGLLLISGLQPEDKADCDCLNWQ--------------------', 'IGLV8-18*01': 'QTV-IQEPA-MSVSLGGTVTLTCAFSSGSVT---SSNYPGWFQQTPGQPPRTVIYST-------NSRPTGVP-SRFSGAI--SGNKATLTITGAQAEDEADYFCALYK--------------------', 'IGLV8-19*01': 'QTV-IQEPA-MSVSLGGTVTLTCAFSSGSVT---SSNNPGWFQQTPGQPPRTVIYQT-------NNRPTGVP-SRFSGAI--SGNKATLTITGAQAEDEADYFCALGK--------------------', 'IGLV8-19*02': 'QTV-IQEPA-MSVSLGGTVTLTCAFSSGSVT---SSNYPSWYQQTPGQPPRQLIYST-------NSRPTGVP-SRFSGAI--SGNKATLTITGAQAEDEADYFCALYK--------------------'}}, 'A': {'human': {'TRAV1-1*01': 'GQSLEQ-PSEVTAVEGAIVQINCTYQTSG------FYGLSWYQQHDGGAPTFLSYNAL----DGLEET-----GRFSSFLSRSDSYGYLLLQELQMKDSASYFCAVR---------------------', 'TRAV1-2*01': 'GQNIDQ-PTEMTATEGAIVQINCTYQTSG------FNGLFWYQQHAGEAPTFLSYNVL----DGLEEK-----GRFSSFLSRSKGYSYLLLKELQMKDSASYLCAVR---------------------', 'TRAV1-2*03': 'GQNIDQ-PTEMTATEGAIVQINCTYQTSG------FNGLFWYQQHAGEAPTFLSYNVL----DGLEEK-----GRFSSFLSRSKGYSYLLLKELQMKDSASYLCAVR---------------------', 'TRAV10*01': 'KNQVEQSPQSLIILEGKNCTLQCNYTVSP------FSNLRWYKQDTGRGPVSLTIMTFS---ENTKSN-----GRYTATLDADTKQSSLHITASQLSDSASYICVVS---------------------', 'TRAV10*02': 'KNQVEQSPQSLIILEGKNCTLQCNYTVSP------FSNLRWYKQDTGRGPVSLTIMTFS---ENTKSN-----GRYTATLDADTKQSSLHITASQLSDSASYICVVS---------------------', 'TRAV12-1*01': 'RKEVEQDPGPFNVPEGATVAFNCTYSNSA------SQSFFWYRQDCRKEPKLLMSVYS----SGN-ED-----GRFTAQLNRASQYISLLIRDSKLSDSATYLCVVN---------------------', 'TRAV12-2*01': 'QKEVEQNSGPLSVPEGAIASLNCTYSDRG------SQSFFWYRQYSGKSPELIMFIYS----NGDKED-----GRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVN---------------------', 'TRAV12-3*01': 'QKEVEQDPGPLSVPEGAIVSLNCTYSNSA------FQYFMWYRQYSRKGPELLMYTYS----SGNKED-----GRFTAQVDKSSKYISLFIRDSQPSDSATYLCAMS---------------------', 'TRAV13-1*01': 'GENVEQHPSTLSVQEGDSAVIKCTYSDSA------SNYFPWYKQELGKGPQLIIDIRSN---VGEKKD-----QRIAVTLNKTAKHFSLHITETQPEDSAVYFCAAS---------------------', 'TRAV13-2*01': 'GESVGLHLPTLSVQEGDNSIINCAYSNSA------SDYFIWYKQESGKGPQFIIDIRSN---MDKRQG-----QRVTVLLNKTVKHLSLQIAATQPGDSAVYFCAEN---------------------', 'TRAV14/DV4*01': 'AQKITQTQPGMFVQEKEAVTLDCTYDTSDP-----SYGLFWYKQPSSGEMIFLIYQGSY--DQQNATE-----GRYSLNFQKARKSANLVISASQLGDSAMYFCAMRE--------------------', 'TRAV14/DV4*02': 'AQKITQTQPGMFVQEKEAVTLDCTYDTSDQ-----SYGLFWYKQPSSGEMIFLIYQGSY--DEQNATE-----GRYSLNFQKARKSANLVISASQLGDSAMYFCAMRE--------------------', 'TRAV16*01': 'AQRVTQPEKLLSVFKGAPVELKCNYSYSG------SPELFWYVQYSRQRLQLLLRHI------SRESI-----KGFTADLNKGETSFHLKKPFAQEEDSAMYYCALS---------------------', 'TRAV17*01': 'SQQGEEDPQALSIQEGENATMNCSYKTSI-------NNLQWYRQNSGRGLVHLILIRSN---EREKHS-----GRLRVTLDTSKKSSSLLITASRAADTASYFCATD---------------------', 'TRAV18*01': 'GDSVTQTEGPVTLPERAALTLNCTYQSSY------STFLFWYVQYLNKEPELLLKSSE----NQETDS-----RGFQASPIKSDSSFHLEKPSVQLSDSAVYYCALR---------------------', 'TRAV19*01': 'AQKVTQAQTEISVVEKEDVTLDCVYETRDT-----TYYLFWYKQPPSGELVFLIRRNSF--DEQNEIS-----GRYSWNFQKSTSSFNFTITASQVVDSAVYFCALSE--------------------', 'TRAV2*01': 'KDQVFQ-PSTVASSEGAVVEIFCNHSVSN------AYNFFWYLHFPGCAPRLLVKGS------KPSQQ-----GRYNMTYER--FSSSLLILQVREADAAVYYCAVE---------------------', 'TRAV20*01': 'EDQVTQSPEALRLQEGESSSLNCSYTVSG------LRGLFWYRQDPGKGPEFLFTLYSA---GEEKEK-----ERLKATLTK--KESFLHITAPKPEDSATYLCAVQ---------------------', 'TRAV20*02': 'EDQVTQSPEALRLQEGESSSLNCSYTVSG------LRGLFWYRQDPGKGPEFLFTLYSA---GEEKEK-----ERLKATLTK--KESFLHITAPKPEDSATYLCAVQ---------------------', 'TRAV21*01': 'KQEVTQIPAALSVPEGENLVLNCSFTDSA------IYNLQWFRQDPGKGLTSLLLIQSS---QREQTS-----GRLNASLDKSSGRSTLYIAASQPGDSATYLCAVR---------------------', 'TRAV22*01': 'GIQVEQSPPDLILQEGANSTLRCNFSDSV-------NNLQWFHQNPWGQLINLFYIPS-----GTKQN-----GRLSATTVATERYSLLYISSSQTTDSGVYFCAVE---------------------', 'TRAV23/DV6*01': 'QQQVKQSPQSLIVQKGGISIINCAYENTA------FDYFPWYQQFPGKGPALLIAIRPD---VSEKKE-----GRFTISFNKSAKQFSLHIMDSQPGDSATYFCAAS---------------------', 'TRAV23/DV6*05': 'QQQVKQSPQSLIVQKGGISIINCAYENTA------FDYFPWYQQFPGKGPALLIAIRPD---VSEKKE-----GRFTISFNKSAKQFSSHIMDSQPGDSATYFCAAS---------------------', 'TRAV24*01': 'ILNVEQSPQSLHVQEGDSTNFTCSFPSSN------FYALHWYRWETAKSPEALFVMTLN---GDEKKK-----GRISATLNTKEGYSYLYIKGSQPEDSATYLCAF----------------------', 'TRAV25*01': 'GQQVMQIPQYQHVQEGEDFTTYCNSSTTL-------SNIQWYKQRPGGHPVFLIQLVKS---GEVKKQ-----KRLTFQFGEAKKNSSLHITATQTTDVGTYFCAG----------------------', 'TRAV26-1*01': 'DAKTTQ-PPSMDCAEGRAANLPCNHSTISG-----NEYVYWYRQIHSQGPQYIIHGLK-----NNETN-----EMASLIITEDRKSSTLILPHATLRDTAVYYCIVRV--------------------', 'TRAV26-1*02': 'DAKTTQ-PTSMDCAEGRAANLPCNHSTISG-----NEYVYWYRQIHSQGPQYIIHGLK-----NNETN-----EMASLIITEDRKSSTLILPHATLRDTAVYYCIVRV--------------------', 'TRAV26-2*01': 'DAKTTQ-PNSMESNEEEPVHLPCNHSTISG-----TDYIHWYRQLPSQGPEYVIHGLT-----SNVNN-----RMASLAIAEDRKSSTLILHRATLRDAAVYYCILRD--------------------', 'TRAV27*01': 'TQLLEQSPQFLSIQEGENLTVYCNSSSVF-------SSLQWYRQEPGEGPVLLVTVVTG---GEVKKL-----KRLTFQFGDARKDSSLHITAAQPGDTGLYLCAG----------------------', 'TRAV27*03': 'TQLLEQSPQFLSIQEGENLTVYCNSSSVF-------SSLQWYRQEPGEGPVLLVTVVTG---GEVKKL-----KRLTFQFGDARKDSSLHITAAQTGDTGLYLCAG----------------------', 'TRAV29/DV5*01': 'DQQVKQNSPSLSVQEGRISILNCDYTNSM------FDYFLWYKKYPAEGPTFLISISSI---KDKNED-----GRFTVFLNKSAKHLSLHIVPSQPGDSAVYFCAAS---------------------', 'TRAV29/DV5*02': 'DQQVKQNSPSLSVQEGRISILNCDYTNSM------FDYFLWYKKYPAEGPTFLISISSI---KDKNED-----GRFTVFLNKSAKHLSLDIVPSQPGDSAVYFCAAS---------------------', 'TRAV29/DV5*04': 'DQQVKQNSPSLSVQEGRISILNCDYTNSM------FDYFLWYKKYPAEGPTFLISISSI---KDKNED-----GRFTVFLNKSAKHLSLHIVPSQPGDSAVYFCAAS---------------------', 'TRAV3*01': 'AQSVAQPEDQVNVAEGNPLTVKCTYSVSG------NPYLFWYVQYPNRGLQFLLKYITG--DNLVKGS-----YGFEAEFNKSQTSFHLKKPSALVSDSALYFCAVRD--------------------', 'TRAV30*01': 'QQPV-QSPQAVILREGEDAVINCSSSKAL-------YSVHWYRQKHGEAPVFLMILLKG---GEQKGH-----EKISASFNEKKQQSSLYLTASQLSYSGTYFCGTE---------------------', 'TRAV30*05': 'QQPV-QSPQAVILREGEDAVINCSSSKAL-------YSVHWYRQKHGEAPVFLMILLKG---GEQKGH-----DKISASFNEKKQQSSLYLTASQLSYSGTYFCGTE---------------------', 'TRAV34*01': 'SQELEQSPQSLIVQEGKNLTINCTSSKTL-------YGLYWYKQKYGEGLIFLMMLQKG---GEEKSH-----EKITAKLDEKKQQSSLHITASQPSHAGIYLCGAD---------------------', 'TRAV35*01': 'GQQLNQSPQSMFIQEGEDVSMNCTSSSIF-------NTWLWYKQEPGEGPVLLIALYKA---GELTSN-----GRLTAQFGITRKDSFLNISASIPSDVGIYFCAGQ---------------------', 'TRAV36/DV7*01': 'EDKVVQSPLSLVVHEGDTVTLNCSYEVTN------FRSLLWYKQEKKAP-TFLFMLTSS---GIEKKS-----GRLSSILDKKELSSILNITATQTGDSAIYLCAVE---------------------', 'TRAV36/DV7*05': 'EDKVVQSPLSLVVHEGDTVTLNCSYEVTN------FRSLLWYKQEKKAP-TFLFMLTSS---GIEKKS-----GRLSSILDKKELFSILNITATQTGDSAIYLCAVE---------------------', 'TRAV38-1*01': 'AQTVTQSQPEMSVQEAETVTLSCTYDTSEN-----NYYLFWYKQPPSRQMILVIRQEAY--KQQNATE-----NRFSVNFQKAAKSFSLKISDSQLGDTAMYFCAFMK--------------------', 'TRAV38-2/DV8*01': 'AQTVTQSQPEMSVQEAETVTLSCTYDTSES-----DYYLFWYKQPPSRQMILVIRQEAY--KQQNATE-----NRFSVNFQKAAKSFSLKISDSQLGDAAMYFCAYRS--------------------', 'TRAV39*01': 'ELKVEQNPLFLSMQEGKNYTIYCNYSTTS-------DRLYWYRQDPGKSLESLFVLLSN---GAVKQE-----GRLMASLDTKARLSTLHITAAVHDLSATYFCAVD---------------------', 'TRAV4*01': 'LAKTTQ-PISMDSYEGQEVNITCSHNNIAT-----NDYITWYQQFPSQGPRFIIQGYK-----TKVTN-----EVASLFIPADRKSSTLSLPRVSLSDTAVYYCLVGD--------------------', 'TRAV40*01': 'SNSVKQT-GQITVSEGASVTMNCTYTSTG------YPTLFWYVEYPSKPLQLLQRET------MENSK-----NFGGGNIKD--KNSPIVKYSVQVSDSAVYYCLLG---------------------', 'TRAV41*01': 'KNEVEQSPQNLTAQEGEFITINCSYSVGI-------SALHWLQQHPGGGIVSLFMLSS-----GKKKH-----GRLIATINIQEKHSSLHITASHPRDSAVYICAVR---------------------', 'TRAV5*01': 'GEDVEQS-LFLSVREGDSSVINCTYTDSS------STYLYWYKQEPGAGLQLLTYIFSN---MDMKQD-----QRLTVLLNKKDKHLSLRIADTQTGDSAIYFCAES---------------------', 'TRAV6*01': 'SQKIEQNSEALNIQEGKTATLTCNYTNYS------PAYLQWYRQDPGRGPVFLLLIREN---EKEKRK-----ERLKVTFDTTLKQSLFHITASQPADSATYLCALD---------------------', 'TRAV6*07': 'SQKIEQNSEALNIQEGKTATLTCNYTNYS------PAYLQWYRQDPGRGPVFLLLIREN---EKEKRK-----ERLKVTFDTTLKQSLFHITASQPADSATYLCALD---------------------', 'TRAV7*01': 'ENQVEHSPHFLGPQQGDVASMSCTYSVSR------FNNLQWYRQNTGMGPKHLLSMYSA---GYEKQK-----GRLNATLLK--NGSSLYITAVQPEDSATYFCAVD---------------------', 'TRAV8-1*01': 'AQSVSQHNHHVILSEAASLELGCNYSYGG------TVNLFWYVQYPGQHLQLLLKYFSG--DPLVKGI-----KGFEAEFIKSKFSFNLRKPSVQWSDTAEYFCAVN---------------------', 'TRAV8-2*01': 'AQSVTQLDSHVSVSEGTPVLLRCNYSSSY------SPSLFWYVQHPNKGLQLLLKYTSA--ATLVKGI-----NGFEAEFKKSETSFHLTKPSAHMSDAAEYFCVVS---------------------', 'TRAV8-2*03': 'AQSVTQLDSHVSVSEGTPVLLRCNYSSSY------SPSLFWYVQHPNKGLQLLLKYTSA--ATLVKGI-----NGFEAEFKKSETSFHLTKPSAHMSDAAEYFCVVS---------------------', 'TRAV8-3*01': 'AQSVTQPDIHITVSEGASLELRCNYSYGA------TPYLFWYVQSPGQGLQLLLKYFSG--DTLVQGI-----KGFEAEFKRSQSSFNLRKPSVHWSDAAEYFCAVG---------------------', 'TRAV8-4*01': 'AQSVTQLGSHVSVSEGALVLLRCNYSSSV------PPYLFWYVQYPNQGLQLLLKYTSA--ATLVKGI-----NGFEAEFKKSETSFHLTKPSAHMSDAAEYFCAVS---------------------', 'TRAV8-6*01': 'AQSVTQLDSQVPVFEEAPVELRCNYSSSV------SVYLFWYVQYPNQGLQLLLKYLSG--STLVESI-----NGFEAEFNKSQTSFHLRKPSVHISDTAEYFCAVS---------------------', 'TRAV8-6*02': 'AQSVTQLDSQVPVFEEAPVELRCNYSSSV------SVYLFWYVQYPNQGLQLLLKYLSG--STLVKGI-----NGFEAEFNKSQTSFHLRKPSVHISDTAEYFCAVS---------------------', 'TRAV9-1*01': 'GDSVVQTEGQVLPSEGDSLIVNCSYETTQ------YPSLFWYVQYPGEGPQLHLKAMKA---NDKGRN-----KGFEAMYRKETTSFHLEKDSVQESDSAVYFCALS---------------------', 'TRAV9-2*01': 'GNSVTQMEGPVTLSEEAFLTINCTYTATG------YPSLFWYVQYPGEGLQLLLKATKA---DDKGSN-----KGFEATYRKETTSFHLEKGSVQVSDSAVYFCALS---------------------', 'TRAV9-2*02': 'GDSVTQMEGPVTLSEEAFLTINCTYTATG------YPSLFWYVQYPGEGLQLLLKATKA---DDKGSN-----KGFEATYRKETTSFHLEKGSVQVSDSAVYFCALS---------------------'}, 'mouse': {'TRAV1*01': 'GQGVEQ-PDNLMSVEGTFARVNCTYSTSG------FNGLSWYQQREGHAPVFLSYVVL----DGLKDS-----GHFSTFLSRSNGYSYLLLTELQIKDSASYLCAVR---------------------', 'TRAV10*01': 'GEKVEQHESTLSVREGDSAVINCTYTDTA------SSYFPWYKQEAGKGLHFVIDIRSN---VDRKQS-----QRLIVLLDKKAKRFSLHITATQPEDSAIYFCAAS---------------------', 'TRAV10D*01': 'GEKVEQHQSTLSVREGDSAVINCTYTDTA------SSYFPWYKQEAGKSLHFVIDIRSN---VDRKQS-----QRLTVLLDKKAKRFSLHITATQPEDSAIYFCAAS---------------------', 'TRAV10N*01': 'GEKVEQHESTLSVREGDSAVINCTYTDTA------SSYFPWYKQEAGKSLHFVIDIRSN---VDRKQS-----QRLTVLLDKKAKRFSLHITATQPEDSAIYFCAAS---------------------', 'TRAV11*01': 'KTQVEQSPQSLVVRQGENCVLQCNYSVTP------DNHLRWFKQDTGKGLVSLTVLVHE---NDKTSN-----GRYSATLDKDAKHSTLHITATLLDDTATYICVVG---------------------', 'TRAV11*02': 'KTQVEQSPQSLVVRQGENCVLQCNYSVTP------DNHLRWFKQDTGKGLVSLTVLVDQ---KDKTSN-----GRYSATLDKDAKHSTLHITATLLDDTATYICVVG---------------------', 'TRAV11D*01': 'KTQVEQSPQSLVVRQGENCVLQCNYSVTP------DNHLRWFKQDTGKGLVSLTVLVHE---NDKTSN-----GRYSATLDKDAKHSTLHITATLLDDTATYICVVG---------------------', 'TRAV12-1*01': 'GDSVTQTEGLVTVTEGLPVMLNCTYQTAYS-----DVAFFWYVQYLNEAPKLLLRSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSVQLSDSALYYCALS---------------------', 'TRAV12-1*02': 'GDSVTQTEGLVTVTEGLPVMLNCTYQTTYS-----DVAFFWYVQYLNEAPKLLLRSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSYAQLSDSGLYYCALS---------------------', 'TRAV12-2*01': 'GDSVTQTEGLVTLTEGLPVMLNCTYQTAY------STFLFWYVQHLNEAPKLLLKSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSAQLSDSALYYCALS---------------------', 'TRAV12-3*01': 'GDSVTQTEGLVTLTEGLPVMLNCTYQTIYS-----NPFLFWYVHYLNESPRLLLKSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSAQLSDSALYYCALS---------------------', 'TRAV12D-1*01': 'GDSVTQTEGLVTVTEGLPVKLNCTYQTTYL-----TIAFFWYVQYLNEAPQVLLKSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSAQLSDSALYYCALS---------------------', 'TRAV12D-1*02': 'GDSVTQTEGLVTVTEGLPVKLNCTYQTTYL-----TIAFFWYVQYLNEAPQVLLRSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSAQLSDSALYYCALS---------------------', 'TRAV12D-2*01': 'GDSVTQTEGLVTLTEGLPVMLNCTYQSTY------SPFLFWYVQHLNEAPKLLLKSFTD---NKRPEH-----QGFHATLHKSSSSFHLQKSSAQLSDSALYYCALS---------------------', 'TRAV12D-2*02': 'GDSVTQTEGLVTLTKGLPVMLNCTYQTTY------SPFLFWYVQHLNEAPKLLLKSSTD---NKRTEH-----QGFYATLHKSSSSFHLQKSSVQLSDSALYFCALS---------------------', 'TRAV12D-3*02': 'GDSVTQTEGLVTLTEGLPVMLNCTYQTIYS-----NAFLFWYVHYLNESPWLLLRSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSAQLSDSALYYCALS---------------------', 'TRAV12N-1*01': 'GDSVTQTEGLVTVTEGLPVMLNCTYQTAYS-----DVAFFWYVQYLNEAPKLLLRSSTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSVQLSDSALYYCALS---------------------', 'TRAV12N-2*01': 'GDSVTQTEGLVTLTKGLPVMLNCTYQTTY------SPFLFWYVQHLNEAPKLLLKSSTD---NKRTEH-----QGFYATLHKSSSSFHLQKSSVQLSDSALYFCALS---------------------', 'TRAV12N-3*01': 'GDSVTQTEGLVTLTEGLPVMLNCTYQTIYS-----NPFLFWYVQHLNESPRLLLKSFTD---NKRTEH-----QGFHATLHKSSSSFHLQKSSAQLSDSALYYCALS---------------------', 'TRAV13-1*01': 'GQQVQQSPASLVLQEGENAELQCNFSTSL-------NSMQWFYQRPEGSLVSLFYNPS-----GTKQS-----GRLTSTTVIKERRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV13-2*01': 'GQQVQQSPSSLVLQEGENAELQCNFSSTA-------TQLQWFYQSPGGSLVSLLSNPS-----GTKHT-----GRLTSTTVTKERRSSLHISSSQTTDSGTYLCAID---------------------', 'TRAV13-3*01': 'GQQVQQSPASLVLQEGENAELQCTYSTTL-------NSMQWFYQRPGGRLVSLLYSPS----WAEQRG-----GRLTSSAASNESRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV13-4/DV7*01': 'GQQVQQSPASLVLQEGENAELQCNFSSTA-------TQLQWFYQRPGGSLVSLLYNPS-----GTKHT-----GRLTSTTVTKERRSSLHISSSQITDSGTYFCAME---------------------', 'TRAV13-5*01': 'GQQVQQSPASLVLQEGENAELQCSFSIST-------NQVQWFYQRPGGRLIGLSYIP------GMKPT-----GKQTSSTVTKGRHSSLTISSSQTTDSGTYFCVLS---------------------', 'TRAV13D-1*01': 'GQQVQQSPASLVLQEGENAELQCNFSTSL-------NSMQWFYQRPGGSLVSLFYNPS-----GTKQS-----GRLTSTTVIKERRSSLHISSSQTTDSGTYLCAME---------------------', 'TRAV13D-2*01': 'GQQVQQSPASLVLQEGENAELQCNFSSTA-------TRLQWFYQHPGGRLVSLFYNPS-----GTKHT-----GRLTSTTVTNERRGSLHISSSQTTDSGTYFCAID---------------------', 'TRAV13D-3*01': 'GQQVEQSPASLVLQEGENAELQCTYSTTL-------NSMQWFYQRPGGRLVSLLYSPS----WAEQRG-----GRLTSSAASNESRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV13D-3*02': 'GQQVQQSPASLVLQEGENAELQCTYSTTL-------NSMQWFYQRPGGRLVSLLYSPS----WAEQRG-----GRLTSSAASNESRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV13D-4*01': 'EQQVQQSPASLVLQEGENAELQCSFSIFT-------NQVQWFYQRPGGRLVSLLYNPS-----GTKQS-----GRLTSTTVIKERRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV13D-4*03': 'GQQVQQSPASLVLQEGENAELQCNFSSTA-------TRLQWFYQRPGGSLVSLLYNPS-----GTKHT-----GRLTSTTVTKERRSSLHISSSQTTDSGTYFCAME---------------------', 'TRAV13N-1*01': 'GQQVQQSPTSLVLQEGENAELQCNFSTSL-------NSMQWFYQRPGGSLISVFYNPS-----GTKQS-----GRLTSTTVIKERRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV13N-2*01': 'GQQVQQSPASLVLQEGENAELQCNFSSTA-------TQLQWFYQSPGGSLVSLLSNPS-----GTKHT-----GRLTSTTVTKERRSSLHISSSQTTDSGTYLCAID---------------------', 'TRAV13N-3*01': 'GQQVQQSPASLVLQEGENAELQCTYSTTL-------NSMQWFYQRPGGRLVSLLYSPS----WAEQRG-----GRLTSSAASNESRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV13N-4*01': 'EQQVQQSPASLVLQEAENAELQCSFSIFT-------NQVQWFYQRPGGRLVSLLYNPS-----GTKQS-----GRLTSTTVIKERRSSLHISSSQITDSGTYLCAME---------------------', 'TRAV14-1*01': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FNYFPWYQQFPGEGPALLISIRSV---SDKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14-1*02': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FNYFPWYQQFPGEGPALLISISSV---SDKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14-2*01': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FDYFPWYWQFPRESPALLIAIRPV---SNKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14-2*02': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FDYFPWYHQFPGESPALLIAIRPV---SNKKED-----GRFTIFFNKREKKFSLHIADSQPGDSATYFCAAS---------------------', 'TRAV14-2*03': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FDYFPWYRLFPGESPALLIAIRPV---SNKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14-3*01': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLISILSV---SDKKED-----GRFTIFFNKREKKLSLHIADSQPGDSATYFCAAS---------------------', 'TRAV14-3*02': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAILSV---SNKKED-----GRFTIFFNKREKKLSLHIADSQPGDSATYFCAAS---------------------', 'TRAV14D-1*01': 'QQQVRQSPQSLTVWEGGTTVLTCSYEDST------FNYFPWYQQFPGEGPALLISILSV---SDKKED-----GRFTTFFNKREKKLSLHIIDSQPGDSATYFCAAS---------------------', 'TRAV14D-2*01': 'QQQVRQSPQSLTVWEGETTILNCSYEDST------FDYFPWYRQFPGKSPALLIAISLV---SNKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14D-2*03': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FDYFPWYWQFPRESPALLIAIRPV---SNKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14D-3/DV8*01': 'QQQVRQSSQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAIRSV---SDKKED-----GRFTIFFNKREKNLSLHIKDSQPGDSATYFCAAS---------------------', 'TRAV14D-3/DV8*02': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAIRSV---SDKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14D-3/DV8*03': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAIRSV---SDKKED-----GGFTIFFNKREKNLSLHIKDSQPGDSATYFCAAS---------------------', 'TRAV14D-3/DV8*08': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLISILSV---SDKKED-----GRFTIFFNKREKKLSLHIADSQPGDSATYFCAAS---------------------', 'TRAV14N-1*01': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FNYFPWYQQFPGEGPALLISIRSV---SDKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14N-2*01': 'QQQVRQSPQSLTVWEGETAILNCSYEDST------FDYFPWYWQFPRESPALLIAIRPV---SNKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14N-3*01': 'QQQVRQSSQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAIRSV---SDKKED-----GRFTIFFNKREKNLSLHIKDSQPGDSATYFCAAS---------------------', 'TRAV15-1/DV6-1*01': 'AQKVIQVWSTTSRQEGEKLTLDCSYKTSQV-----LYHLFWYKHLLSGEMVLLIRQMPS--TIAIERS-----GRYSVVFQKSRKSISLVISTLQPDDSGKYFCALWE--------------------', 'TRAV15-2/DV6-2*01': 'AQKVTQVQSTGSSQWG-EVTLHCSYETSEY-----FYVILWYKQLFSGEMVFLIYQTSF--DTQNQRN-----SRYSVVFQKSLKSISLVISASQPEDSGTYFCALSE--------------------', 'TRAV15D-1/DV6D-1*01': 'AQKVIQVWSTPSRQEGEELTLDCSYETSQV-----LYHLFWYKHLLSGEMVFLIRQMSS--STAKERS-----GRYSVVFQKSLKSISLVISALQPDDSGKYFCALWE--------------------', 'TRAV15D-1/DV6D-1*02': 'AQKVIQVWSTASRQEGEELTLDCSYETSQV-----LYHLFWYKHLLSGEMVFLIRQMSS--STAKERS-----GRYSVVFQKSLKSISLVISALQPDDSGKYFCALWE--------------------', 'TRAV15D-2/DV6D-2*01': 'AQRVTQVQPTGSSQWGEEVTLDCSYETSEY-----FYCIIWYRQLFSGEMVFLIYQTSF--DTQNQRN-----GRYSVVFQKSLKSISLVISASQPEDSGTYFCALSE--------------------', 'TRAV15D-2/DV6D-2*03': 'AQRVTQVQPTGSSQWGEEVTLDCSYETSEY-----FYRIFWYRQLFSGEMVFLIYQPSF--DTQNQRS-----GRYSVVFQKSFKSISLVISASQPEDSGTYFCALSE--------------------', 'TRAV15N-1*01': 'AEKVIQVWSTASRQEGEELTLDCSYETSQV-----LYHLFWYKHLLSGEMVFLIRQTSS--STAKERS-----GRYSVVFQKSLKSISLIISALQPDDSGKYFCALWE--------------------', 'TRAV15N-2*01': 'AQRVTQVQPTGSSQWGEEVTLDCSYETSEY-----FYRIFWYRQLFSGEMVFLIYQPSF--DTQNQRS-----GRYSVVFQKSFKSISLVISASQPEDSGTYFCALSE--------------------', 'TRAV16*01': 'AQKVTQTQTSISVMEKTTVTMDCVYETQDS-----SYFLFWYKQTASGEIVFLIRQDSY--KKENATV-----GHYSLNFQKPKSSIGLIITATQIEDSAVYFCAMRE--------------------', 'TRAV16D/DV11*01': 'AQKVTQTQTSISVMEKTTVTMDCVYETQDS-----SYFLFWYKQTASGEIVFLIRQDSY--KKENATV-----GHYSLNFQKPKSSIGLIITATQIEDSAVYFCAMRE--------------------', 'TRAV16D/DV11*02': 'AQKVTQTQTSISVMEKTTVTMDCVYETQDS-----SYFLFWYKQTASGEIVFLIRQDSY--KKENATV-----GHYSLNFQKPKSSIGLIITATQIEDSAVYFCAMRE--------------------', 'TRAV16N*01': 'AQKVTQTQTSISVVEKTTVTMDCVYETRDS-----SYFLFWYKQTASGEIVFLIRQDSY--KKENATV-----GHYSLNFQKPKSSIGLIITATQIEDSAVYFCAMRE--------------------', 'TRAV17*01': 'AQSVDQPDAHVTLYEGASLELRCSYSYSA------APYLFWYVQYPGQSLQFLLKYITG--DAVVKGT-----KGFEAEFRKSNSSFNLKKSPAHWSDSAKYFCALE---------------------', 'TRAV19*01': 'GQQVKQSSPSLTVQEGGILILNCDYENDM------FDYFAWYKKYPDNSPTLLISVRSN---VDKRED-----GRFTVFLNKSGKHFSLHITASQPEDTAVYLCAAG---------------------', 'TRAV2*01': 'LAKTTQ-PPSMEAYEGQEVNVSCSHTNIAT-----SEYIYWYRQVPHQGPQFIIQGYK-----DYVVN-----EVASLFISADRKLSTLSLPWVSLRDAAVYYCIVTD--------------------', 'TRAV21/DV12*01': 'DAKTTQ-PDSMESTEGETVHLPCSHATISG-----NEYIYWYRQVPLQGPEYVTHGLQ-----QNTTN-----SMAFLAIASDRKSSTLILTHVSLRDAAVYHCILRV--------------------', 'TRAV3-1*01': 'GEQVEQRPPHLSVREGDSAIIICTYTDSA------TAYFSWYKQEAGAGLQLLMSVLSN---VDRKEE-----QGLTVLLNKKDKRLSLNLTAAHPGDSAVYFCAVS---------------------', 'TRAV3-3*01': 'GEQVEQRPPHLSVREGDSAVITCTYTDPN------SYYFFWYKQEPGASLQLLMKVFSS---TEINEG-----QGFTVLLNKKDKRLSLNLTAAHPGDSAAYFCAVS---------------------', 'TRAV3-4*01': 'GEQVEQRPPHLSVPEGDSAVIICTYTDSA------TAYFYWYKQEPGAGLQLLMSVFSN---VDRKEE-----QGLTVLLNKKDKQLSLNLTAAHPGDSAVYFCAVS---------------------', 'TRAV3D-3*01': 'GEQVEQRPPHLSVREGDSAFITCTYTDPN------SYYFFWYKQEPGASLQLLMKVFSS---TEINEG-----QGFTVLLNKKDKRLSLNLTAAHPGDSAAYFCAVS---------------------', 'TRAV3D-3*02': 'GEQVEQRPPHLSVREGDSAVIICTYTDPN------SYYFFWYKQEPGAGLQLLMKVFSS---TEINEG-----QGFTVLLNKKDKQLSLNLTAAHPGDSAVYFCAVS---------------------', 'TRAV3N-3*01': 'GEQVEQRPPHLSVREGDSAVIICTYTDPN------SYYFFWYKQEPGAGLQLLMKVFSS---TEINEG-----QGFTVLLNKKDKQLSLNLTAAHPGDSAVYFCAVS---------------------', 'TRAV4-2*01': 'GMPVEQNPPALSLYEGADSGLRCNFSTTM-------KSVQWFQQNHRGRLITLFYLAQ-----GTKEN-----GRLKSTFNSKERYSTLHIKDAQLEDSGTYFCAAE---------------------', 'TRAV4-2*02': 'GMPVEQNPPALSLYEGAESGLRCNFSTTM-------KGVQWFQQNHRGRLITLFYLAQ-----GTKEN-----GRLKSTFNSKERYSTLHIKDAQLEDSGTYFCAVE---------------------', 'TRAV4-3*01': 'GDQVKQSPSALSLQEGTNSALRCNFSIAT-------TTVQWFLQNPRGSLMNLFYLVP-----GTKEN-----GRLKSTFNSKESYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV4-4/DV10*01': 'GDQVEQSPSALSLHEGTDSALRCNFTTTM-------RSVQWFRQNSRGSLISLFYLAS-----GTKEN-----GRLKSAFDSKERYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV4D-3*01': 'GDKVKQSPSALSLQEGTNSALRCNFSIAA-------TTVQWFLQNPRGSLINLFYLVP-----GTKEN-----GRLKSTFNSKESYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV4D-4*01': 'GDQVEQSPSALSLHEGTSSALRCNFTTTT-------RSVQWFRQNSRGSLINLFYLAS-----GTKEN-----GRLKSAFDSKELYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV4D-4*04': 'GDQVEQSPSALSLHKGTGSALRCNFTTTT-------RAVQWFRQNSRGSLINLFYLAS-----GTKEN-----GRLKSAFDSKERYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV4N-3*01': 'GDKVKQSPSALSLQEGTNSALRCNFSIAA-------TTVQWFLQNPRGSLMNLFYLVP-----GTKEN-----GRLKSAFDSKESYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV4N-4*01': 'GDQVEQSPSALSLHEGTGSALRCNFTTTM-------RAVQWFRKNSRGSLINLFYLAS-----GTKEN-----GRLKSAFDSKERYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV5-1*01': 'GEQVEQLPSSLIVQEGASVLINCSYTDSA------SVYFPWYKQEPGKRLQFIIDIRSN---MERKQN-----QRLTLLFDKKTKHLSLHITATQPGDSAIYFCSAS---------------------', 'TRAV5D-4*05': 'GEQVEQLPSILRVQEGSSASINCTYENSA------SNYFPWYKQEPGENPKLIIDIRSN---MERKQT-----QGLIVLLDKKAKRFSLHITDTQPGDSAMYFCAAS---------------------', 'TRAV5N-4*01': 'GEQVEQLPSILRVQEGSSASINCTYENSA------SNYFPWYKQEPGENPKLIIDIRSN---MERKQT-----QGLIVLLDKKAKRFSLHITDTQPGDSAMYFCAAS---------------------', 'TRAV6-1*01': 'GDSVTQMQGQVTLSEDDFLFINCTYSTTW------YPTLFWYVQYPGEGPQLLLKVTTA---NNKGIS-----RGFEATYDKRTTSFHLQKASVQESDSAVYYCVLG---------------------', 'TRAV6-2*01': 'GNSVTQMQGQVTLSEEEFLFINCTYSTTG------YPTLFWYVQYPGEGPQLLLKVTTA---NNKGSS-----RGFEATYDKGTTSFHLQKASVQESDSAVYYCVLG---------------------', 'TRAV6-3*01': 'GDSVIQMQGQVTLSENDFLFINCTYSTTG------YPTLFWYVQYSGEGPQLLLQVTTA---NNKGSS-----RGFEATYDKGTTSFHLQKTSVQEIDSAVYYCAMR---------------------', 'TRAV6-3*02': 'GDSVIQMQGQVTLSENDFLFINCTYSTTG------YPTLFWYVQYSGEGPQLLLQVTTA---NNKGSS-----RGFEATYDKGTTSFHLQKTSVQEIDSAVYYCAMR---------------------', 'TRAV6-4*01': 'GDSVTQKQGQVTLSEDDFLFINCTYSTTT------YPTLLWYVQYPGQGPQLLLKVTTA---NNKGIS-----RGFEATYDKGTTSFHLQKASVQESDSAVYFCALV---------------------', 'TRAV6-4*03': 'GDSVTQKQGQVTLSEDDFLFINCTYSTTT------YPTLLWYVQYLGQGPQLLLKVTTA---NNKGIS-----RGFEATYDKGTTSFHLQKASVQESDSAVYFCALV---------------------', 'TRAV6-5*01': 'GDSVTQTEGPVTLSEGTSLTVNCSYETKQ------YPTLFWYVQYPGEGPQLLFKVPKA---NEKGSS-----RGFEATYNKEATSFHLQKASVQESDSAVYYCALS---------------------', 'TRAV6-5*04': 'GDSVTQTEGPVTLSEGTSLTVNCSYETKQ------YPTLFWYVQYPGEGPQLLFKVPKA---NEKGSN-----RGFEATYNKEATSFHLQKASVQESDSAVYYCALS---------------------', 'TRAV6-6*01': 'GDSVTQTEGQVTVSESKSLIINCTYSTTSI----AYPNLFWYVRYPGEGLQLLLKVITA---GQKGSS-----RGFEATYNKETTSFHLQKASVQESDSAVYYCALG---------------------', 'TRAV6-7/DV9*01': 'GDSVTQTEGQVALSEEDFLTIHCNYSASG------YPALFWYVQYPGEGPQFLFRASRD---KEKGSS-----RGFEATYNKETTSFHLQKASVQESDSAVYYCALG---------------------', 'TRAV6D-3*01': 'GDSVIQMQGQVTLSENDFLFINCTYSTTG------YPTLFWYVQYSGEGPQLLLQVTTA---NNKGSS-----RGFEATYDKGTTSFHLQKTSVQEIDSAVYYCAMR---------------------', 'TRAV6D-4*01': 'GDSVTQKQGQVTLSEDDFLFINCTYSTTT------YPTLFWYVQYPGQGPQLLLKVTTA---NNKGIS-----RGFEATYDKGTTSFHLQKASVQESDSAVYFCALV---------------------', 'TRAV6D-5*01': 'GDSVTQTEGPVTLSEGTSLTVNCSYETKQ------YPTLFWYVQYPGEGPQLLFKVPKA---NEKGSN-----RGFEATYNKEATSFHLQKASVQESDSAVYYCALG---------------------', 'TRAV6D-6*01': 'GDSVTQTEGQVTVSESKSLIINCTYSATSI----AYPNLFWYVRYPGEGLQLLLKVITA---GQKGSS-----RGFEATYNKETTSFHLQKASVQESDSAVYYCALS---------------------', 'TRAV6D-6*02': 'GDSVTQTEGPVTVSESESLIINCTYSATSI----AYPNLFWYVRYPGEGLQLLLKVITA---GQKGSS-----RGFEATYNKETTSFHLQKASVQESDSAVYYCALG---------------------', 'TRAV6D-6*05': 'GDSVTQTEGQVTVSESKSLIINCTYSATSI----GYPNLFWYVRYPGEGLQLLLKVITA---GQKGSS-----RGFEATYNKEATSFHLQKASVQESDSAVYYCALS---------------------', 'TRAV6D-7*01': 'GDSVTQTEGQVALSEEDFLTIHCNYSASG------YPALFWYVQYPGEGPQFLFRASRD---KEKGSS-----RGFEATYNKEATSFHLQKASVQESDSAVYYCALS---------------------', 'TRAV6D-7*04': 'GDSVTQTEGQVALSEEDFLTIHCNYSASG------YPALFWYVQYPGEGPQFLFRASRD---KEKGSS-----RGFEATYDKGTTSFHLRKASVQESDSAVYYCALG---------------------', 'TRAV6N-5*01': 'GDSVTQTEGPVTLSEGTSLTVNCSYETKQ------YPTLFWYVQYPGEGPQLLFKVPKA---NEKGSN-----RGFEATYNKEATSFHLQKASVQESDSAVYYCALG---------------------', 'TRAV6N-6*01': 'GDSVTQTEGQVTVSESKSLIINCTYSATSI----GYPNLFWYVRYPGEGLQLLLKVITA---GQKGSS-----RGFEATYNKEATSFHLQKASVQESDSAVYYCALS---------------------', 'TRAV6N-7*01': 'GDSVTQTEGQVALSEEDFLTIHCNYSASG------YPALFWYVQYPGEGPQFLFRASRD---KEKGSS-----RGFEATYDKGTTSFHLRKASVQESDSAVYYCALG---------------------', 'TRAV7-1*01': 'QQKVQQSPESLIVPEGGMASLNCTFSDRN------SQYFWWYRQHSGEGPKALMSIFS----NGDKKE-----GRFTAHLNKASLYVSLHIKDSQPSDSALYFCAVS---------------------', 'TRAV7-2*01': 'QQKVQQSPESLIVPEGGMASLNCTSSDRN------VDYFWWYRQRSGKSPKMLMAIFS----NGEKEE-----GRFTVHLNKASLHTSLHIRDSQPSDSALYFCAVS---------------------', 'TRAV7-2*02': 'QQKVQQSPESLIVPEGGMASLNCTSSDRN------VDYFWWYRQHSGKSPKMLMSIFS----NGEKEE-----GRFTVHLNKASLHTSLHIRDSQPSDSALYLCAAS---------------------', 'TRAV7-3*01': 'QQNVQQSPESLIVPEGARTSLNCTFSDSA------SQYFWWYRQHSGKAPKALMSIFS----NGEKEE-----GRFTIHLNKASLHFSLHIRDSQPSDSALYLCAVS---------------------', 'TRAV7-3*04': 'QQKVQQSPESLIVPEGAMTSLNCTFSDSA------SQYFAWYRQHSGKAPKALMSIFS----NGEKEE-----GRFTIHLNKASLHFSLHIRDSQPSDSALYLCAVS---------------------', 'TRAV7-4*01': 'QQKVQQSPESLSVPEGGMASFNCTSSDRN------FQYFWWYRQHSGEGPKALMSIFS----DGDKKE-----GRFTAHLNKASLHVSLHIRDSQPSDSALYFCAASE--------------------', 'TRAV7-4*02': 'QQKVQQSPESLSVPEGGMASLNCTSSDRN------FQYFWWYRQHSGEGPKALMSIFS----DGDKKE-----GRFTAHLNKASLHVSLHIRDSQPSDSALYFCAASE--------------------', 'TRAV7-5*01': 'QQKVQQSPESLTVSEGAMASLNCTFSDGT------SDNFRWYRQHSGKGLEVLVSIFS----DGEKEE-----GRFTAHLNRASLHVSLHIREPQPSDSAVYLCAMS---------------------', 'TRAV7-6*01': 'QEKVQQSPESLIVPEGAMASLNCTFSNSA------SQSIWWYQQHPGKGPEALISIFS----NGNKKE-----GRLTVYLNRASLHVSLHIRDSQPTDSAIYLCAVS---------------------', 'TRAV7D-2*01': 'QQKVQQSPESLIVPEGGMASLNCTSSDRN------VDYFWWYRQHSGKSPKMLMSIFS----NGEKEE-----GRFTVHLNKASLHTSLHIRDSQPSDSALYLCAAS---------------------', 'TRAV7D-3*01': 'QQKVQQSPESLIVPEGAMTSLNCTFSDSA------SQYFAWYRQHSGKAPKALMSIFS----NGEKEE-----GRFTIHLNKASLHFSLHIRDSQPSDSALYLCAVS---------------------', 'TRAV7D-4*01': 'QQKVQQSPESLSVPEGGMASLNCTSSDRN------FQYFWWYRQHSGEGPKALMSIFS----DGDKKE-----GRFTAHLNKASLHVSLHIRDSQPSDSALYFCAASE--------------------', 'TRAV7D-4*02': 'QQKVQQSPESLSVPEGGMASLNCTSSDRN------FQYFWWYRQHSGEGPKALMSIFS----DGDKKE-----GRFTAHLNKASLHVSLHIRDSQPSDSALYFCAASE--------------------', 'TRAV7D-5*01': 'QQKVQQSPESLTVSEGAMASLNCTFSDGT------SDNFRWYRQHSGKGLEMLVSIFS----DGEKEE-----GRFTAHLNRASLHVSLHIREPQPSDSAVYLCAVS---------------------', 'TRAV7D-6*01': 'QEKVQQSPESLTVPEGAMASLNCTISDSA------SQSIWWYQQNPGKGPKALISIFS----NGNKKE-----GRLTVYLNRASLHVSLHIRDSHPSDSAVYLCAAS---------------------', 'TRAV7D-6*02': 'QEKVQQSPESLIVPEGAMSSLNCTFSNSA------SQSIWWYQQHPGKGPEALISIFS----NGNKKE-----GRLTVYLNRASLHVSLHIRDSQPSDSAVYLCAVS---------------------', 'TRAV7N-4*01': 'QQKVQQSPESLSVPEGGMASLNCTSSDRN------FQYFWWYRQHSGEGPKALMSIFS----DGDKKE-----GRFTAHLNKASLHVSLHIRDSQPSDSALYFCAVSE--------------------', 'TRAV7N-5*01': 'QQKVQQSPESLTVSEGAMASLNCTFSDRS------SDNFRWYRQHSGKGLEVLVSIFS----DGEKEE-----GSFTAHLNRASLHVFLHIREPQPSDSALYLCAVS---------------------', 'TRAV7N-6*01': 'QEKVQQSPESLIVPEGAMSSLNCTFSNSA------SQSIWWYQQHPGKGPEALISIFS----NGNKKE-----GRLTVYLNRASLHVSLHIRDSQPSDSAVYLCAVS---------------------', 'TRAV8-1*01': 'SQLAEENPWALSVHEGESVTVNCSYKTSI-------TALQWYRQKSGEGPAQLILIRSN---EREKRN-----GRLRATLDTSSQSSSLSITATRCEDTAVYFCATD---------------------', 'TRAV8-1*03': 'SQLAEENSWALSVHEGESVTVNCSYKTSI-------TALQWYRQKSGKGPAQLILIRSN---EREKRN-----GRLRATLDTSSQSSSLSITATRCEDTAVYFCATD---------------------', 'TRAV8-2*01': 'SQWGEENLQALSIQEGEDVTMNCSYKTYT-------TVVHWYRQDSGRGPALIILIRSN---EREKRS-----GRLRATLDTSSQSSSLSITAAQCEDTAVYFCATD---------------------', 'TRAV8D-1*01': 'SQLAEENLWALSVHEGESVTVNCSYKTSI-------TALQWYRQKSGEGPAQLILIRSN---EREKRN-----GRLRATLDTSSQSSSLSITATRCEDTAVYFCATD---------------------', 'TRAV8D-2*01': 'SQWGEENLQALSIQEGEDVTMNCSYKTYT-------TVVQWYRQKSGKGPALIILIRSN---EREKRS-----GRLRATLDTSSQSSSLSITGTLATDTAVYFCATD---------------------', 'TRAV8D-2*03': 'SQWGEENLQALSIQEGEDVTMNCSYKTYT-------TVVHWYRQDSGRGPALIILIRSN---EREKRS-----GRLRATLDTSSQSSSLSITAAQCEDTAVYFCATD---------------------', 'TRAV8N-2*01': 'SQWGEENLQALSIQEGEDVTMNCSYKTYT-------TVVQWYRQKSGKGPAQLILIRSN---EREKRS-----GRLRATLDTSSQSSSLSITGTLATDTAVYFCATD---------------------', 'TRAV9-1*01': 'TQTVSQSDAHVTVFEGDSVELRCNYSYGG------SIYLSWYIQHHGRGLQFLLKYYSG--NPVVQGV-----NGFEAEFSKSDSSFHLRKASVHWSDSAVYFCAVS---------------------', 'TRAV9-1*02': 'TQTVSQSDAHVTVFEGDSVELRCNYSYGG------SIYLSWYIQHHGHGLQFLLKYYSG--NPVVQGV-----NGFEAEFSKSDSSFHLRKASVHWSDSAVYFCAVS---------------------', 'TRAV9-2*01': 'AQSVTQPDARVTVSEGASLQLRCKYSYSG------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NGFEAEFSKSDSSFHLRKASVHWSDSAVYFCAAS---------------------', 'TRAV9-3*01': 'AQSVTQPDARVTVSEGASLQLRCKYSSSV------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NGFEAEFSKSNSSFHLRKASVHWSDSAVYFCAVS---------------------', 'TRAV9-4*01': 'AQSVTQPDARVTVSEGASLQLRCKYSYSA------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NGFEAEFSKSNSSFHLRKASVHWSDSAVYFCAVS---------------------', 'TRAV9D-1*01': 'TQTVSQSDAHVTVFEGDSVELRCNYSYGG------SIYLSWYIQHHGRGLQFLLKYYSG--NPVVQGV-----NGFEAEFSKSDSSFHLRKASVHWSDSAVYFCAAS---------------------', 'TRAV9D-2*01': 'AQSVTQPDARVTVSQGASLQLRCKYSYSG------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NGFEAEFSKSNSSFHPRKASVHWSDSAVYFCAVS---------------------', 'TRAV9D-2*02': 'AQSVTQPDARVTVSQGASLQLRCKYSYSG------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NGFEAEFSKSNSSFHPRKASVHWSDSAVYFCAVS---------------------', 'TRAV9D-3*01': 'AQSVTQPDARVTVSEGASLQLRCKYSYSA------TPYLFWYVQYPRQGLQMLLKYYSG--DPVVQGV-----NGFEAEFSKSDSSFHLRKASVHWSDSAVYFCAVS---------------------', 'TRAV9D-4*01': 'AQSVTQPDARVTVSEGASLQLRCKYSYSG------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NGFEAEFSKSNSSFHLRKASVHWSDSAVYFCALS---------------------', 'TRAV9N-2*01': 'AQSVTQPDARVTVSEGASLQLRCKYSSSG------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NGFEAEFSKSNSSFHLRKASVHWSDSAVYFCVLS---------------------', 'TRAV9N-3*01': 'AQSVTQPDARVTVSEGASLQLRCKYSYFG------TPYLFWYVQYPRQGLQLLLKYYPG--DPVVQGV-----NGFEAEFSKSNSSFHLRKASVHWSDWAVYFCAVS---------------------', 'TRAV9N-4*01': 'AQSVTQPDARVTVSEGASLQLRCKYSYSA------TPYLFWYVQYPRQGLQLLLKYYSG--DPVVQGV-----NSFEAEFSKSNSSFHLQKASVHWSDSAVYFCALS---------------------'}}, 'B': {'human': {'TRBV10-1*01': 'DAEITQSPRHKITETGRQVTLACHQTWNH-------NNMFWYRQDLGHGLRLIHYSYG----VQDTNKGEVS-DGYSVSRS-NTEDLPLTLESAASSQTSVYFCASSE--------------------', 'TRBV10-1*02': 'DAEITQSPRHKITETGRQVTLACHQTWNH-------NNMFWYRQDLGHGLRLIHYSYG----VHDTNKGEVS-DGYSVSRS-NTEDLPLTLESAASSQTSVYFCASSE--------------------', 'TRBV10-2*01': 'DAGITQSPRYKITETGRQVTLMCHQTWSH-------SYMFWYRQDLGHGLRLIYYSAA----ADITDKGEVP-DGYVVSRS-KTENFPLTLESATRSQTSVYFCASSE--------------------', 'TRBV10-2*02': 'DAGITQSPRYKITETGRQVTLMCHQTWSH-------SYMFWYRQDLGHGLRLIYYSAA----ADITDKGEVP-DGYVVSRS-KTENFPLTLESATRSQTSVYFCASSE--------------------', 'TRBV10-3*01': 'DAGITQSPRHKVTETGTPVTLRCHQTENH-------RYMYWYRQDPGHGLRLIHYSYG----VKDTDKGEVS-DGYSVSRS-KTEDFLLTLESATSSQTSVYFCAISE--------------------', 'TRBV10-3*02': 'DAGITQSPRHKVTETGTPVTLRCHQTENH-------RYMYWYRQDPGHGLRLIHYSYG----VKDTDKGEVS-DGYSVSRS-KTEDFLLTLESATSSQTSVYFCAISE--------------------', 'TRBV11-1*01': 'EAEVAQSPRYKITEKSQAVAFWCDPISGH-------ATLYWYRQILGQGPELLVQFQD----ESVVDDSQLPKDRFSAERL-KGVDSTLKIQPAELGDSAMYLCASSL--------------------', 'TRBV11-2*01': 'EAGVAQSPRYKIIEKRQSVAFWCNPISGH-------ATLYWYQQILGQGPKLLIQFQN----NGVVDDSQLPKDRFSAERL-KGVDSTLKIQPAKLEDSAVYLCASSL--------------------', 'TRBV11-2*03': 'EAGVAQSPRYKIIEKRQSVAFWCNPISGH-------ATLYWYQQILGQGPKLLIQFQN----NGVVDDSQLPKDRFSAERL-KGVDSTLKIQPAKLEDSAVYLCASSL--------------------', 'TRBV11-3*01': 'EAGVVQSPRYKIIEKKQPVAFWCNPISGH-------NTLYWYLQNLGQGPELLIRYEN----EEAVDDSQLPKDRFSAERL-KGVDSTLKIQPAELGDSAVYLCASSL--------------------', 'TRBV12-3*01': 'DAGVIQSPRHEVTEMGQEVTLRCKPISGH-------NSLFWYRQTMMRGLELLIYFNN----NVPIDDSGMPEDRFSAKMP-NASFSTLKIQPSEPRDSAVYFCASSL--------------------', 'TRBV12-4*01': 'DAGVIQSPRHEVTEMGQEVTLRCKPISGH-------DYLFWYRQTMMRGLELLIYFNN----NVPIDDSGMPEDRFSAKMP-NASFSTLKIQPSEPRDSAVYFCASSL--------------------', 'TRBV12-5*01': 'DARVTQTPRHKVTEMGQEVTMRCQPILGH-------NTVFWYRQTMMQGLELLAYFRN----RAPLDDSGMPKDRFSAEMP-DATLATLKIQPSEPRDSAVYFCASGL--------------------', 'TRBV13*01': 'AAGVIQSPRHLIKEKRETATLKCYPIPRH-------DTVYWYQQGPGQDPQFLISFYE----KMQSDKGSIP-DRFSAQQF-SDYHSELNMSSLELGDSALYFCASSL--------------------', 'TRBV14*01': 'EAGVTQFPSHSVIEKGQTVTLRCDPISGH-------DNLYWYRRVMGKEIKFLLHFVK----ESKQDESGMPNNRFLAERT-GGTYSTLKVQPAELEDSGVYFCASSQ--------------------', 'TRBV15*01': 'DAMVIQNPRYQVTQFGKPVTLSCSQTLNH-------NVMYWYQQKSSQAPKLLFHYYD----KDFNNEADTP-DNFQSRRP-NTSFCFLDIRSPGLGDTAMYLCATSR--------------------', 'TRBV15*02': 'DAMVIQNPRYQVTQFGKPVTLSCSQTLNH-------NVMYWYQQKSSQAPKLLFHYYD----KDFNNEADTP-DNFQSRRP-NTSFCFLDIRSPGLGDAAMYLCATSR--------------------', 'TRBV16*01': 'GEEVAQTPKHLVRGEGQKAKLYCAPIKGH-------SYVFWYQQVLKNEFKFLISFQN----ENVFDETGMPKERFSAKCL-PNSPCSLEIQATKLEDSAVYFCASSQ--------------------', 'TRBV18*01': 'NAGVMQNPRHLVRRRGQEARLRCSPMKGH-------SHVYWYRQLPEEGLKFMVYLQK----ENIIDESGMPKERFSAEFP-KEGPSILRIQQVVRGDSAAYFCASSP--------------------', 'TRBV19*01': 'DGGITQSPKYLFRKEGQNVTLSCEQNLNH-------DAMYWYRQDPGQGLRLIYYSQI----VNDFQKGDIA-EGYSVSRE-KKESFPLTVTSAQKNPTAFYLCASSI--------------------', 'TRBV19*02': 'DGGITQSPKYLFRKEGQNVTLSCEQNLNH-------DAMYWYRQVPGQGLRLIYYSHI----VNDFQKGDIA-EGYSVSRE-KKESFPLTVTSAQKNPTAFYLCASSI--------------------', 'TRBV2*01': 'EPEVTQTPSHQVTQMGQEVILRCVPISNH-------LYFYWYRQILGQKVEFLVSFYN----NEISEKSEIFDDQFSVERP-DGSNFTLKIRSTKLEDSAMYFCASSE--------------------', 'TRBV20-1*01': 'GAVVSQHPSWVICKSGTSVKIECRSLDFQ------ATTMFWYRQFPKQSLMLMATSNEG---SKATYEQGVEKDKFLINHA-SLTLSTLTVTSAHPEDSSFYICSAR---------------------', 'TRBV24-1*01': 'DADVTQTPRNRITKTGKRIMLECSQTKGH-------DRMYWYRQDPGLGLRLIYYSFD----VKDINKGEIS-DGYSVSRQ-AQAKFSLSLESAIPNQTALYFCATSD--------------------', 'TRBV24-1*02': 'DADVTQTPRNRITKTGKRIMLECSQTKGH-------DRMYWYRQDPGLGLQLIYYSFD----VKDINKGEIS-DGYSVSRQ-AQAKFSLSLESAIPNQTALYFCATSD--------------------', 'TRBV25-1*01': 'EADIYQTPRYLVIGTGKKITLECSQTMGH-------DKMYWYQQDPGMELHLIHYSYG----VNSTEKGDLS-SESTVSRI-RTEHFPLTLESARPSHTSQYLCASSE--------------------', 'TRBV27*01': 'EAQVTQNPRYLITVTGKKLTVTCSQNMNH-------EYMSWYRQDPGLGLRQIYYSMN----VEVTDKGDVP-EGYKVSRK-EKRNFPLILESPSPNQTSLYFCASSL--------------------', 'TRBV28*01': 'DVKVTQSSRYLVKRTGEKVFLECVQDMDH-------ENMFWYRQDPGLGLRLIYFSYD----VKMKEKGDIP-EGYSVSRE-KKERFSLILESASTNQTSMYLCASSL--------------------', 'TRBV29-1*01': 'SAVISQKPSRDICQRGTSLTIQCQVDSQV-------TMMFWYRQQPGQSLTLIATANQG---SEATYESGFVIDKFPISRP-NLTFSTLTVSNMSPEDSSIYLCSVE---------------------', 'TRBV3-1*01': 'DTAVSQTPKYLVTQMGNDKSIKCEQNLGH-------DTMYWYKQDSKKFLKIMFSYNN----KELIINETVP-NRFSPKSP-DKAHLNLHINSLELGDSAVYFCASSQ--------------------', 'TRBV30*01': 'SQTIHQWPATLVQPVGSPLSLECTVEGTS------NPNLYWYRQAAGRGLQLLFYSVG-----IGQISSEVP-QNLSASRP-QDRQFILSSKKLLLSDSGFYLCAWS---------------------', 'TRBV30*02': 'SQTIHQWPATLVQPVGSPLSLECTVEGTS------NPNLYWYRQAAGRGLQLLFYSVG-----IGQISSEVP-QNLSASRP-QDRQFILSSKKLLLSDSGFYLCAWS---------------------', 'TRBV4-1*01': 'DTEVTQTPKHLVMGMTNKKSLKCEQHMGH-------RAMYWYKQKAKKPPELMFVYSY----EKLSINESVP-SRFSPECP-NSSLLNLHLHALQPEDSALYLCASSQ--------------------', 'TRBV4-2*01': 'ETGVTQTPRHLVMGMTNKKSLKCEQHLGH-------NAMYWYKQSAKKPLELMFVYNF----KEQTENNSVP-SRFSPECP-NSSHLFLHLHTLQPEDSALYLCASSQ--------------------', 'TRBV4-3*01': 'ETGVTQTPRHLVMGMTNKKSLKCEQHLGH-------NAMYWYKQSAKKPLELMFVYSL----EERVENNSVP-SRFSPECP-NSSHLFLHLHTLQPEDSALYLCASSQ--------------------', 'TRBV5-1*01': 'KAGVTQTPRYLIKTRGQQVTLSCSPISGH-------RSVSWYQQTPGQGLQFLFEYFS----ETQRNKGNFP-GRFSGRQF-SNSRSEMNVSTLELGDSALYLCASSL--------------------', 'TRBV5-4*01': 'ETGVTQSPTHLIKTRGQQVTLRCSSQSGH-------NTVSWYQQALGQGPQFIFQYYR----EEENGRGNFP-PRFSGLQF-PNYSSELNVNALELDDSALYLCASSL--------------------', 'TRBV5-5*01': 'DAGVTQSPTHLIKTRGQQVTLRCSPISGH-------KSVSWYQQVLGQGPQFIFQYYE----KEERGRGNFP-DRFSARQF-PNYSSELNVNALLLGDSALYLCASSL--------------------', 'TRBV5-6*01': 'DAGVTQSPTHLIKTRGQQVTLRCSPKSGH-------DTVSWYQQALGQGPQFIFQYYE----EEERQRGNFP-DRFSGHQF-PNYSSELNVNALLLGDSALYLCASSL--------------------', 'TRBV5-8*01': 'EAGVTQSPTHLIKTRGQQATLRCSPISGH-------TSVYWYQQALGLGLQFLLWYDE----GEERNRGNFP-PRFSGRQF-PNYSSELNVNALELEDSALYLCASSL--------------------', 'TRBV6-1*01': 'NAGVTQTPKFQVLKTGQSMTLQCAQDMNH-------NSMYWYRQDPGMGLRLIYYSAS----EGTTDKGEVP-NGYNVSRL-NKREFSLRLESAAPSQTSVYFCASSE--------------------', 'TRBV6-2*01': 'NAGVTQTPKFRVLKTGQSMTLLCAQDMNH-------EYMYWYRQDPGMGLRLIHYSVG----EGTTAKGEVP-DGYNVSRL-KKQNFLLGLESAAPSQTSVYFCASSY--------------------', 'TRBV6-3*01': 'NAGVTQTPKFRVLKTGQSMTLLCAQDMNH-------EYMYWYRQDPGMGLRLIHYSVG----EGTTAKGEVP-DGYNVSRL-KKQNFLLGLESAAPSQTSVYFCASSY--------------------', 'TRBV6-4*01': 'IAGITQAPTSQILAAGRRMTLRCTQDMRH-------NAMYWYRQDLGLGLRLIHYSNT----AGTTGKGEVP-DGYSVSRA-NTDDFPLTLASAVPSQTSVYFCASSD--------------------', 'TRBV6-4*02': 'TAGITQAPTSQILAAGRSMTLRCTQDMRH-------NAMYWYRQDLGLGLRLIHYSNT----AGTTGKGEVP-DGYSVSRA-NTDDFPLTLASAVPSQTSVYFCASSD--------------------', 'TRBV6-5*01': 'NAGVTQTPKFQVLKTGQSMTLQCAQDMNH-------EYMSWYRQDPGMGLRLIHYSVG----AGITDQGEVP-NGYNVSRS-TTEDFPLRLLSAAPSQTSVYFCASSY--------------------', 'TRBV6-6*01': 'NAGVTQTPKFRILKIGQSMTLQCTQDMNH-------NYMYWYRQDPGMGLKLIYYSVG----AGITDKGEVP-NGYNVSRS-TTEDFPLRLELAAPSQTSVYFCASSY--------------------', 'TRBV6-6*02': 'NAGVTQTPKFRILKIGQSMTLQCAQDMNH-------NYMYWYRQDPGMGLKLIYYSVG----AGITDKGEVP-NGYNVSRS-TTEDFPLRLELAAPSQTSVYFCASSY--------------------', 'TRBV6-8*01': 'NAGVTQTPKFHILKTGQSMTLQCAQDMNH-------GYMSWYRQDPGMGLRLIYYSAA----AGTTDK-EVP-NGYNVSRL-NTEDFPLRLVSAAPSQTSVYLCASSY--------------------', 'TRBV6-9*01': 'NAGVTQTPKFHILKTGQSMTLQCAQDMNH-------GYLSWYRQDPGMGLRRIHYSVA----AGITDKGEVP-DGYNVSRS-NTEDFPLRLESAAPSQTSVYFCASSY--------------------', 'TRBV7-2*01': 'GAGVSQSPSNKVTEKGKDVELRCDPISGH-------TALYWYRQSLGQGLEFLIYFQG----NSAPDKSGLPSDRFSAERT-GGSVSTLTIQRTQQEDSAVYLCASSL--------------------', 'TRBV7-2*02': 'GAGVSQSPSNKVTEKGKDVELRCDPISGH-------TALYWYRQRLGQGLEFLIYFQG----NSAPDKSGLPSDRFSAERT-GESVSTLTIQRTQQEDSAVYLCASSL--------------------', 'TRBV7-2*03': 'GAGVSQSPSNKVTEKGKDVELRCDPISGH-------TALYWYRQRLGQGLEFLIYFQG----NSAPDKSGLPSDRFSAERT-GESVSTLTIQRTQQEDSAVYLCTSSL--------------------', 'TRBV7-3*01': 'GAGVSQTPSNKVTEKGKYVELRCDPISGH-------TALYWYRQSLGQGPEFLIYFQG----TGAADDSGLPNDRFFAVRP-EGSVSTLKIQRTERGDSAVYLCASSL--------------------', 'TRBV7-4*01': 'GAGVSQSPRYKVAKRGRDVALRCDSISGH-------VTLYWYRQTLGQGSEVLTYSQS----DAQRDKSGRPSGRFSAERP-ERSVSTLKIQRTEQGDSAVYLCASSL--------------------', 'TRBV7-6*01': 'GAGVSQSPRYKVTKRGQDVALRCDPISGH-------VSLYWYRQALGQGPEFLTYFNY----EAQQDKSGLPNDRFSAERP-EGSISTLTIQRTEQRDSAMYRCASSL--------------------', 'TRBV7-7*01': 'GAGVSQSPRYKVTKRGQDVTLRCDPISSH-------ATLYWYQQALGQGPEFLTYFNY----EAQPDKSGLPSDRFSAERP-EGSISTLTIQRTEQRDSAMYRCASSL--------------------', 'TRBV7-8*01': 'GAGVSQSPRYKVAKRGQDVALRCDPISGH-------VSLFWYQQALGQGPEFLTYFQN----EAQLDKSGLPSDRFFAERP-EGSVSTLKIQRTQQEDSAVYLCASSL--------------------', 'TRBV7-8*02': 'GAGVSQSPRYKVAKRGQDVALRCDPISGH-------VSLFWYQQALGQGPEFLTYFQN----EAQLDKSGLPSDRFFAERP-EGSVSTLKIQRTQKEDSAVYLCASSL--------------------', 'TRBV7-9*01': 'DTGVSQNPRHKITKRGQNVTFRCDPISEH-------NRLYWYRQTLGQGPEFLTYFQN----EAQLEKSRLLSDRFSAERP-KGSFSTLEIQRTEQGDSAMYLCASSL--------------------', 'TRBV7-9*03': 'DTGVSQDPRHKITKRGQNVTFRCDPISEH-------NRLYWYRQTLGQGPEFLTYFQN----EAQLEKSRLLSDRFSAERP-KGSFSTLEIQRTEQGDSAMYLCASSL--------------------', 'TRBV9*01': 'DSGVTQTPKHLITATGQRVTLRCSPRSGD-------LSVYWYQQSLDQGLQFLIQYYN----GEERAKGNIL-ERFSAQQF-PDLHSELNLSSLELGDSALYFCASSV--------------------', 'TRBV9*02': 'DSGVTQTPKHLITATGQRVTLRCSPRSGD-------LSVYWYQQSLDQGLQFLIHYYN----GEERAKGNIL-ERFSAQQF-PDLHSELNLSSLELGDSALYFCASSV--------------------'}, 'mouse': {'TRBV1*01': 'VTLLEQNPRWRLVPRGQAVNLRCILKNSQ------YPWMSWYQQDLQKQLQWLFTLRS----PGDKEVKSLPGADYLATRV-TDTELRLQVANMS--QGRTLYCTCSA--------------------', 'TRBV12-1*01': 'DAGVIQTPRHKVTKMGQEVTLKCQPISGH-------EGLFWYSQTSVQGPKLLISFNN----EAPIDDSGMPKEWFSAEIS-NKSLSTLKIKSTEPGDSATYLCASSI--------------------', 'TRBV12-2*01': 'DAGVIQTPRHNVTKMGQEVTLKCQPISGH-------AALYWYSQTSVQGPKLLIYFNN----QAPIDDSGMPKERFSAEIS-NKSLSTLKIKPTEPGDSATYLCASSV--------------------', 'TRBV13-1*01': 'EAAVTQSPRNKVTVTGGNVTLSCRQTNSH-------NYMYWYRQDTGHGLRLIHYSYG----AGNLRIGDVP-DGYKATRT-TQEDFFLLLELASPSQTSLYFCASSD--------------------', 'TRBV13-1*02': 'EAAVTQSPRNKVTVTGGNVTLSCRQTNSH-------NYMYWYRQDTGHGLRLIHYSYG----AGNLQIGDVP-DGYKATRT-TQEDFFLLLELASPSQTSLYFCASSD--------------------', 'TRBV13-2*01': 'EAAVTQSPRNKVAVTGGKVTLSCNQTNNH-------NNMYWYRQDTGHGLRLIHYSYG----AGSTEKGDIP-DGYKASRP-SQENFSLILELATPSQTSVYFCASGD--------------------', 'TRBV13-3*01': 'EAAVTQSPRSKVAVTGGKVTLSCHQTNNH-------DYMYWYRQDTGHGLRLIHYSYV----ADSTEKGDIP-DGYKASRP-SQENFSLILELASLSQTAVYFCASSD--------------------', 'TRBV14*01': 'EAGVTQSPRYAVLQEGQAVSFWCDPISGH-------DTLYWYQQPRDQGPQLLVYFRD----EAVIDNSQLPSDRFSAVRP-KGTNSTLKIQSAKQGDTATYLCASSF--------------------', 'TRBV15*01': 'GAMVIQSSRYQVTRVGKPVNLSCSQNMNH-------DSMYWYQQKLNQAPKLLLYYYD----KNFNREAETS-DNFQASRP-NTSFCSLNILSPGLGDSGVYLCASSK--------------------', 'TRBV16*01': 'HAEVTQTPGHLVKGKGQKAKMHCVPIKGH-------SYVDWYQQIPAKEFKFFISFQN----DAIFDKTGMPEKRFSAACP-QNASCSLEIKPTELQDSAVYFCASSE--------------------', 'TRBV17*01': 'DTTVKQNPRYKLARVGKPVNLICSQTMNH-------DTMYWYQKKPNQAPKLLLFYYD----KILNREADTF-EKFQSSRP-NNSFCSLYIGSAGLEYSAMYLCASSR--------------------', 'TRBV19*01': 'DGGITQTPKYLLREEGREVTLKCEQDFNH-------DYMYWYRQDPGQGPRLIYYSPL----KDDVQRGDIP-EGYFGSRG-KKTIFSLAVTSTRKNHTALYLCASSR--------------------', 'TRBV2*01': 'DPKIIQKPKYLVAVTGSEKILICEQYLGH-------NAMYWYRQSAKKPLEFMFSYSY----QKLMDNQTAS-SRFQPQSS-KKNHLDLQITALKPDDSATYFCASSQ--------------------', 'TRBV20*01': 'GALVYQYPRRTICKSGTSMRMECQAVGFQ------ATSVAWYRQSPQKTFELIALSTVN---SAIKYEQNFTQEKFPISHP-NLSFSSMTVLNAYLEDRGLYLCGAR---------------------', 'TRBV23*01': 'DAAVTQKPRYLIKMKGQEAEMKCIPEKGH-------TAVFWYQQKQSKELKFLIYFQN----QQPLDQIDMVKERFSAVCP-SSSLCSLGIRTCEAEDSALYLCSSSQ--------------------', 'TRBV24*01': 'VAGVTQTPRYLVKEKGQKAHMSCSPEKGH-------TAFYWYQQNQKQELTFLISFRN----EEIMEQTDLVKKRFSAKCS-SNSRCILEILSSEEDDSALYLCASSL--------------------', 'TRBV26*01': 'GAVVTQFPKHRIVGPGKELTLQCLQDMNY-------VLMYWYRQDPGFGLQLIYYSTG----ARNFEKGDVP-QGYRVSRK-DLQSFSLTLESASTNQTSVYLCTSSE--------------------', 'TRBV29*01': 'STLLSQEPRRDICQCGTSMTIQCETDTQV-------SLMYWYRQLPGQSLILIATANQG---MEATYESGFTKEKFPISRP-TLTFSSLTVNNMSFEDSSFYLCSAE---------------------', 'TRBV3*01': 'DTAVSQTPKHLIAQTGNKKFLKCEQKLGH-------DTMYWYKQDSQKLLKVMFIYNN----KDLILNETVP-RCFLPESP-DKTRLNLHINFLETGDTAVYFCASSL--------------------', 'TRBV30*01': 'SVLLYQKPNRDICQSGTSLKIQCVADSQV-------VSMFWYQQFQEQSLMLMATANEG---SEATYESGFTKDKFPISRP-NLTFSTLTVNNARPGDSSIYFCSSR---------------------', 'TRBV31*01': 'AQTIHQWPVAEIKAVGSPLSLGCTIKGKS------SPNLYWYWQATGGTLQQLFYSIT-----VGQVESVVQ-LNLSASRP-KDDQFILSTEKLLLSHSGFYLCAWS---------------------', 'TRBV5*01': 'NTKITQSPRYLIL-GRANKSLECEQHLGH-------NAMYWYKQSAEKPPELMFLYNL----KQLIRNETVP-SRFIPECP-DSSKLLLHISAVDPEDSAVYFCASSQ--------------------', 'TRBV18*01': 'NAGVTQNPRHLVIRTGQEATLKCSPEKGH-------VYFYWYQQPQGESLKFMVYLQN----AQTVDDSGMSKERFSTNIS-KEGQSILKIQPAELGDSAVYFCASSL--------------------', 'TRBV21*01': 'DPNVTQTPGYLVKGKGQKAKMECVPIKGH-------SYVFWYHRKLEEEFKFLVYLQD----KDIVDKIEGFDNQFSAECP-KNSPCTLEINSTEPGDSALYMCASNK--------------------', 'TRBV22*01': 'HAEIYQMPAFLLTRAGRAVTLECKQNLRY-------NAMYWYWQDPGQSLRLIYYSTV----EKDVQRGDIT-EGYNVSRE-EKGLFPLTMNLAHTNQTGVYLCSGSA--------------------', 'TRBV25*01': 'DAGITQTPRHCVSGTGKKIVLECSQTMGY-------ENMYWYRQDPGKALQLIHYSYG----VNNTENEELS-SGSTVSRL-RKEVFPLTLESASPSQTSLYLCASSE--------------------', 'TRBV27*01': 'EAGVTQTPRHFITRTGRQLTLYCSQDMDH-------DVMFWYRQDPGVGLKLIYFSRN----VKFIEVGDVP-DGYSVLRK-EKKDFPLTLKSTGTNQTSLYLCASSV--------------------', 'TRBV4-1*01': 'GNGVTQTPKHLVSGTGRSVTLKCKQHLGH-------NAMYWYKQSVQKPPKLMLAYSY----KDLLQNETAS-SRFLPDRS-NNSQLDLQIKALEPEDSALYLCASSK--------------------', 'TRBV4-2*01': 'DPGVIQTPKHLVTGTTRSVTLKCKQHLGH-------NAMYWYKQSVQKPPKLMLAYNY----KNLFENATAS-SRFLPDRS-NNSQLDLQIKALEPEDSALYLCASSK--------------------', 'TRBV5-2*01': 'ASGVTQTPRHLIKTRGQKVTLRCSFVSGH-------LSVYWYQQVLGQGPRFLIQYYD----RKERDKGDMP-ERFSARQF-SNSSSQLDLDLLELEDSALYLCASSE--------------------', 'TRBV7-1*01': 'GPGVSQFSRHRVTERGQNVTLGCDPISGH-------AVLYWYRQTIGKGPEFLVYFQN----ADALDKSGMTNDRLSAKRS-EGTNSTLQIQRAEQGDSAVHLCASSP--------------------', 'TRBV7-2*01': 'GPGVSQFPRHRVTERGQNVTLGCDPISGH-------TVLYWYRQTTGKGLEFLVYFQN----TDAVDKSGMTNDRLSVQRS-KGTNSTLQIQRAEQGDSAVYLCASSP--------------------', 'TRBV8*01': 'EAEITQTPRYSIIQIGTKKTLECSQDMNH-------FAMFWYRQDPGQGLRLIYYSSD----SPSTTKGDVA-EGYSVSRK-EQKFSPLTLESASTNQTSVYLCASSE--------------------'}}, 'G': {'human': {'TRGV2*01': 'SSNLEGRTKSVIRQTGSSAEITCDLAEGS------NGYIHWYLHQEGKAPQRLQYYDSY--NSKVVLESGVSPGKYYTYAS-TRNNLRLILRNLIENDSGVYYCATWD--------------------', 'TRGV2*03': 'SSNLEGRTKSVIRQTGSSAEITCDLAEGS------NGYIHWYLHQEGKAPQRLQYYDSY--NSKVVLESGVSPGKYYTYAS-TRNNLRLILRNLIENDFGVYYCATWD--------------------', 'TRGV3*01': 'SSNLEGRTKSVTRQTGSSAEITCDLTVTN------TFYIHWYLHQEGKAPQRLLYYDVS--TARDVLESGLSPGKYYTHTP-RRWSWILRLQNLIENDSGVYYCATWD--------------------', 'TRGV3*02': 'SSNLEGRTKSVTRQTGSSAEITCDLTVTN------TFYIHWYLHQEGKAPQRLLYYDVS--TARDVLESGLSPGKYYTHTP-RRWSWILRLQNLIENDSGVYYCATWD--------------------', 'TRGV4*01': 'SSNLEGRTKSVIRQTGSSAEITCDLAEGS------TGYIHWYLHQEGKAPQRLLYYDSY--TSSVVLESGISPGKYDTYGS-TRKNLRMILRNLIENDSGVYYCATWD--------------------', 'TRGV4*02': 'SSNLEGRTKSVIRQTGSSAEITCDLAEGS------TGYIHWYLHQEGKAPQRLLYYDSY--TSSVVLESGISPGKYDTYGS-TRKNLRMILRNLIENDSGVYYCATWD--------------------', 'TRGV5*01': 'SSNLEGGTKSVTRPTRSSAEITCDLTVIN------AFYIHWYLHQEGKAPQRLLYYDVS--NSKDVLESGLSPGKYYTHTP-RRWSWILILRNLIENDSGVYYCATWD--------------------', 'TRGV8*01': 'SSNLEGRTKSVTRPTGSSAVITCDLPVEN------AVYTHWYLHQEGKAPQRLLYYDSY--NSRVVLESGISREKYHTYAS-TGKSLKFILENLIERDSGVYYCATWD--------------------', 'TRGV9*01': 'AGHLEQPQISSTKTLSKTARLECVVSGITI----SATSVYWYRERPGEVIQFLVSISYD---GTVRKESGIPSGKFEVDRIPETSTSTLTIHNVEKQDIATYYCALWE--------------------', 'TRGV9*02': 'AGHLEQPQISSTKTLSKTARLECVVSGIKI----SATSVYWYRERPGEVIQFLVSISYD---GTVRKESGIPSGKFEVDRIPETSTSTLTIHNVEKQDIATYYCALWE--------------------'}, 'mouse': {'TRGV1*01': 'LGQLEQTELSVTRETDESAQISCIVSLPYF----SNTAIHWYRQKAKK-FEYLIYVST----NYNQRPLGGKNKKIEASKDFQTSTSTLKINYLKKEDEATYYCAVWI--------------------', 'TRGV1*02': 'LGQLEQTELSVTRETDESAQISCIVSLPYF----SNTAIHWYRQKAKK-FEYLIYVST----NYNQRPLGGKNKKIEASKDFQTSTSTLKINYLKKEDEATYYCAVWI--------------------', 'TRGV2*01': 'LGQLEQTELSVTRETDENVQISCIVYLPYF----SNTAIHWYRQKTNQQFEYLIYVAT----NYNQRPLGGKHKKIEASKDFKSSTSTLEINYLKKEDEATYYCAVWM--------------------', 'TRGV3*01': 'LGQLEQTELSVTRATDESAQISCIVSLPCF----SNTAIHWYRQKPNQQFEYLIYVET----NYNQQPLGGKNKKIEASKDFQTSTSTLKINYLKKEDEATYYCAVWI--------------------', 'TRGV5*01': 'DSWISQDQLSFTRRPNKTVHISCKLSGVPL----HNTIVHWYQLKEGEPLRRIFYGS------VKTYKQDKSHSRLEIDEK-DDGTFYLIINNVVTSDEATYYCACWD--------------------', 'TRGV6*01': 'TSLTSPLGSYVIKRKGNTAFLKCQIKTSVQK---PDAYIHWYQEKPGQRLQRMLCSSSK---ENIVYEKDFSDERYEARTWQSDLSSVLTIHQVREEDTGTYYCACWD--------------------', 'TRGV6*02': 'SSLTSPLGSYVIKRKGNTAFLKCQIKTSVQK---PDAYIHWYQEKPGQRLQRMLCSSSK---ENIVYEKDFSDERYEARTWQSDLSSVLTIHQVTEEDTGTYYCACWD--------------------', 'TRGV7*01': 'SSNLEERIMSITKLEGSSAIMTCDTHRTG-------TYIHWYRFQKGRAPEHLLYYNFV--SSTTVVDSRFNSEKYHVYEG-PDKRYKFVLRNVEESDSALYYCASWA--------------------', 'TRGV7*02': 'SSNLEERIMSITKLEGSSAIMTCDTHRTG-------TYIHWYRFQKGRAPEHLLYYNFV--SSTTVVDSRFNLEKYHVYEG-PDKRYKFVLRNVEESDSALYYCASWA--------------------'}}, 'D': {'human': {'TRAV14/DV4*01': 'AQKITQTQPGMFVQEKEAVTLDCTYDTSDP-----SYGLFWYKQPSSGEMIFLIYQGSY--DQQNATE-----GRYSLNFQKARKSANLVISASQLGDSAMYFCAMRE--------------------', 'TRAV14/DV4*02': 'AQKITQTQPGMFVQEKEAVTLDCTYDTSDQ-----SYGLFWYKQPSSGEMIFLIYQGSY--DEQNATE-----GRYSLNFQKARKSANLVISASQLGDSAMYFCAMRE--------------------', 'TRAV23/DV6*01': 'QQQVKQSPQSLIVQKGGISIINCAYENTA------FDYFPWYQQFPGKGPALLIAIRPD---VSEKKE-----GRFTISFNKSAKQFSLHIMDSQPGDSATYFCAAS---------------------', 'TRAV23/DV6*05': 'QQQVKQSPQSLIVQKGGISIINCAYENTA------FDYFPWYQQFPGKGPALLIAIRPD---VSEKKE-----GRFTISFNKSAKQFSSHIMDSQPGDSATYFCAAS---------------------', 'TRAV29/DV5*01': 'DQQVKQNSPSLSVQEGRISILNCDYTNSM------FDYFLWYKKYPAEGPTFLISISSI---KDKNED-----GRFTVFLNKSAKHLSLHIVPSQPGDSAVYFCAAS---------------------', 'TRAV29/DV5*02': 'DQQVKQNSPSLSVQEGRISILNCDYTNSM------FDYFLWYKKYPAEGPTFLISISSI---KDKNED-----GRFTVFLNKSAKHLSLDIVPSQPGDSAVYFCAAS---------------------', 'TRAV29/DV5*04': 'DQQVKQNSPSLSVQEGRISILNCDYTNSM------FDYFLWYKKYPAEGPTFLISISSI---KDKNED-----GRFTVFLNKSAKHLSLHIVPSQPGDSAVYFCAAS---------------------', 'TRAV36/DV7*01': 'EDKVVQSPLSLVVHEGDTVTLNCSYEVTN------FRSLLWYKQEKKAP-TFLFMLTSS---GIEKKS-----GRLSSILDKKELSSILNITATQTGDSAIYLCAVE---------------------', 'TRAV36/DV7*05': 'EDKVVQSPLSLVVHEGDTVTLNCSYEVTN------FRSLLWYKQEKKAP-TFLFMLTSS---GIEKKS-----GRLSSILDKKELFSILNITATQTGDSAIYLCAVE---------------------', 'TRAV38-2/DV8*01': 'AQTVTQSQPEMSVQEAETVTLSCTYDTSES-----DYYLFWYKQPPSRQMILVIRQEAY--KQQNATE-----NRFSVNFQKAAKSFSLKISDSQLGDAAMYFCAYRS--------------------', 'TRDV1*01': 'AQKVTQAQSSVSMPVRKAVTLNCLYETSWW-----SYYIFWYKQLPSKEMIFLIRQG-------SDEQNAKS-GRYSVNFKKAAKSVALTISALQLEDSAKYFCALGE--------------------', 'TRDV2*01': 'AIELVPEHQTVPVSIGVPATLRCSMKGEAI----GNYYINWYRKTQGNTITFIYREK-------DIYGPGFK-DNFQGDIDIAKNLAVLKILAPSERDEGSYYCACDT--------------------', 'TRDV2*03': 'AIELVPEHQTVPVSIGVPATLRCSMKGEAI----GNYYINWYRKTQGNTMTFIYREK-------DIYGPGFK-DNFQGDIDIAKNLAVLKILAPSERDEGSYYCACDT--------------------', 'TRDV3*01': 'CDKVTQSSPDQTVASGSEVVLLCTYDTVYS-----NPDLFWYRIRPDYSFQFVFYGDN----SRSEGADFTQ-GRFSVKHILTQKAFHLVISPVRTEDSATYYCAF----------------------', 'TRDV3*02': 'CDKVTQSSPDQTVASGSEVVLLCTYDTVYS-----NPDLFWYWIRPDYSFQFVFYGDN----SRSEGADFTQ-GRFSVKHILTQKAFHLVISPVRTEDSATYYCAF----------------------'}, 'mouse': {'TRAV13-4/DV7*01': 'GQQVQQSPASLVLQEGENAELQCNFSSTA-------TQLQWFYQRPGGSLVSLLYNPS-----GTKHT-----GRLTSTTVTKERRSSLHISSSQITDSGTYFCAME---------------------', 'TRAV14D-3/DV8*01': 'QQQVRQSSQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAIRSV---SDKKED-----GRFTIFFNKREKNLSLHIKDSQPGDSATYFCAAS---------------------', 'TRAV14D-3/DV8*02': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAIRSV---SDKKED-----GRFTIFFNKREKKLSLHITDSQPGDSATYFCAAS---------------------', 'TRAV14D-3/DV8*03': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLIAIRSV---SDKKED-----GGFTIFFNKREKNLSLHIKDSQPGDSATYFCAAS---------------------', 'TRAV14D-3/DV8*08': 'QQQVRQSPQSLTVWEGETAILNCSYENSA------FDYFPWYQQFPGEGPALLISILSV---SDKKED-----GRFTIFFNKREKKLSLHIADSQPGDSATYFCAAS---------------------', 'TRAV15-1/DV6-1*01': 'AQKVIQVWSTTSRQEGEKLTLDCSYKTSQV-----LYHLFWYKHLLSGEMVLLIRQMPS--TIAIERS-----GRYSVVFQKSRKSISLVISTLQPDDSGKYFCALWE--------------------', 'TRAV15-2/DV6-2*01': 'AQKVTQVQSTGSSQWG-EVTLHCSYETSEY-----FYVILWYKQLFSGEMVFLIYQTSF--DTQNQRN-----SRYSVVFQKSLKSISLVISASQPEDSGTYFCALSE--------------------', 'TRAV15D-1/DV6D-1*01': 'AQKVIQVWSTPSRQEGEELTLDCSYETSQV-----LYHLFWYKHLLSGEMVFLIRQMSS--STAKERS-----GRYSVVFQKSLKSISLVISALQPDDSGKYFCALWE--------------------', 'TRAV15D-1/DV6D-1*02': 'AQKVIQVWSTASRQEGEELTLDCSYETSQV-----LYHLFWYKHLLSGEMVFLIRQMSS--STAKERS-----GRYSVVFQKSLKSISLVISALQPDDSGKYFCALWE--------------------', 'TRAV15D-2/DV6D-2*01': 'AQRVTQVQPTGSSQWGEEVTLDCSYETSEY-----FYCIIWYRQLFSGEMVFLIYQTSF--DTQNQRN-----GRYSVVFQKSLKSISLVISASQPEDSGTYFCALSE--------------------', 'TRAV15D-2/DV6D-2*03': 'AQRVTQVQPTGSSQWGEEVTLDCSYETSEY-----FYRIFWYRQLFSGEMVFLIYQPSF--DTQNQRS-----GRYSVVFQKSFKSISLVISASQPEDSGTYFCALSE--------------------', 'TRAV16D/DV11*01': 'AQKVTQTQTSISVMEKTTVTMDCVYETQDS-----SYFLFWYKQTASGEIVFLIRQDSY--KKENATV-----GHYSLNFQKPKSSIGLIITATQIEDSAVYFCAMRE--------------------', 'TRAV16D/DV11*02': 'AQKVTQTQTSISVMEKTTVTMDCVYETQDS-----SYFLFWYKQTASGEIVFLIRQDSY--KKENATV-----GHYSLNFQKPKSSIGLIITATQIEDSAVYFCAMRE--------------------', 'TRAV21/DV12*01': 'DAKTTQ-PDSMESTEGETVHLPCSHATISG-----NEYIYWYRQVPLQGPEYVTHGLQ-----QNTTN-----SMAFLAIASDRKSSTLILTHVSLRDAAVYHCILRV--------------------', 'TRAV4-4/DV10*01': 'GDQVEQSPSALSLHEGTDSALRCNFTTTM-------RSVQWFRQNSRGSLISLFYLAS-----GTKEN-----GRLKSAFDSKERYSTLHIRDAQLEDSGTYFCAAE---------------------', 'TRAV6-7/DV9*01': 'GDSVTQTEGQVALSEEDFLTIHCNYSASG------YPALFWYVQYPGEGPQFLFRASRD---KEKGSS-----RGFEATYNKETTSFHLQKASVQESDSAVYYCALG---------------------', 'TRDV1*01': 'TQMLHQSPQSLTIQEGDEVTMSCNLSTSL-------YALLWYRQGDDGSLVSLVTLQ-------KGGDEKSK-DKITAKLDKKMQQSSLQIQASQPSHSGTYLCGGK---------------------', 'TRDV2-1*01': 'AQTVSQHQQEKSVQVAESATLDCTYDTSDT-----NYLLFWYKQQGGQVTLVIRQEA-------YKQYNAME-NRFSVNFQKAAKSFSLEISDSQLGDAATYFCALRG--------------------', 'TRDV2-2*01': 'AQTVSQPQKKKSVQVAESATLDCTYDTSDT-----NYLLFWYKQQGGQVTLVILQEA-------YKQYNATL-NRFSVNFQKAAKSFSLEISDSQLGDAATYFCALME--------------------', 'TRDV4*01': 'DVYLEPVAKTFTVVAGDPASFYCTVTGGDM----KNYHMSWYKKNGTNALFLVYKLN-------SNSTDGGK-SNLKGKINISKNQFILDIQKATMKDAGTYYCGSDI--------------------', 'TRDV5*01': 'CITLTQSSTDQTVASGTEVTLLCTYNADSP-----NPDLFWYRKRPDRSFQFILYRDD----TSSHDADFVQ-GRFSVKHSKANRTFHLVISPVSLEDSATYYCASGY--------------------', 'TRDV5*02': 'CITLTQSSTDQTVASGTEVTLLCTYNADSP-----NPDLFWYRKRPDRSFQFILYRDD----TSSHDADFVQ-GRFSVKHSKANRTFHLVISPVSLEDSATYYCASGY--------------------', 'TRDV5*03': 'CITLTQSSTDQTVASGTEVTLLCTYNADSP-----NPDLFWYRKRPDRSFQFILYRDD----TSSHDADFVQ-GRFSVKHSKANRTFHLVISPVSLEDSATYYCASGY--------------------'}}}}

In [10]:
all_species = list(all_germlines['V']['H'].keys())

amino_acids = sorted(list("QWERTYIPASDFGHKLCVNM"))
set_amino_acids = set(amino_acids)
anarci_path  = os.path.split(".")[0]

scheme_short_to_long = { "m":"martin", "c":"chothia", "k":"kabat","imgt":"imgt", "kabat":"kabat", "chothia":"chothia", "martin":"martin", "i":"imgt", "a":"aho","aho":"aho","wolfguy":"wolfguy", "w":"wolfguy"}

scheme_names = list(scheme_short_to_long.keys())
chain_type_to_class = {"H":"H", "K":"L", "L":"L", "A":"A", "B":"B", "G":"G", "D":"D"}

HMM_path =  os.path.join( anarci_path, "dat", "HMMs" )

all_reference_states = list(range( 1, 129)) # These are the IMGT reference states (matches)

## Functions in `schemes.py`

In [242]:
# Alphabet used for insertion (last (-1th) is a blank space for no insertion)
alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "BB", "CC", "DD", "EE", "FF", "GG", "HH", "II", "JJ", "KK", "LL", "MM", "NN", "OO", "PP", "QQ", "RR", "SS", "TT", "UU", "VV", "WW", "XX", "YY", "ZZ", " "]

# Blosum62 matrix. Used in some annotation methods to recognise pre-defined motifs
blosum62 = {('B', 'N'): 3, ('W', 'L'): -2, ('G', 'G'): 6, ('X', 'S'): 0, ('X', 'D'): -1, ('K', 'G'): -2, ('S', 'E'): 0, ('X', 'M'): -1, ('Y', 'E'): -2, ('W', 'R'): -3, ('I', 'R'): -3, ('X', 'Z'): -1, ('H', 'E'): 0, ('V', 'M'): 1, ('N', 'R'): 0, ('I', 'D'): -3, ('F', 'D'): -3, ('W', 'C'): -2, ('N', 'A'): -2, ('W', 'Q'): -2, ('L', 'Q'): -2, ('S', 'N'): 1, ('Z', 'K'): 1, ('V', 'N'): -3, ('Q', 'N'): 0, ('M', 'K'): -1, ('V', 'H'): -3, ('G', 'E'): -2, ('S', 'L'): -2, ('P', 'R'): -2, ('D', 'A'): -2, ('S', 'C'): -1, ('E', 'D'): 2, ('Y', 'G'): -3, ('W', 'P'): -4, ('X', 'X'): -1, ('Z', 'L'): -3, ('Q', 'A'): -1, ('V', 'Y'): -1, ('W', 'A'): -3, ('G', 'D'): -1, ('X', 'P'): -2, ('K', 'D'): -1, ('T', 'N'): 0, ('Y', 'F'): 3, ('W', 'W'): 11, ('Z', 'M'): -1, ('L', 'D'): -4, ('M', 'R'): -1, ('Y', 'K'): -2, ('F', 'E'): -3, ('M', 'E'): -2, ('S', 'S'): 4, ('X', 'C'): -2, ('Y', 'L'): -1, ('H', 'R'): 0, ('P', 'P'): 7, ('K', 'C'): -3, ('S', 'A'): 1, ('P', 'I'): -3, ('Q', 'Q'): 5, ('L', 'I'): 2, ('P', 'F'): -4, ('B', 'A'): -2, ('Z', 'N'): 0, ('M', 'Q'): 0, ('V', 'I'): 3, ('Q', 'C'): -3, ('I', 'H'): -3, ('Z', 'D'): 1, ('Z', 'P'): -1, ('Y', 'W'): 2, ('T', 'G'): -2, ('B', 'P'): -2, ('P', 'A'): -1, ('C', 'D'): -3, ('Y', 'H'): 2, ('X', 'V'): -1, ('B', 'B'): 4, ('Z', 'F'): -3, ('M', 'L'): 2, ('F', 'G'): -3, ('S', 'M'): -1, ('M', 'G'): -3, ('Z', 'Q'): 3, ('S', 'Q'): 0, ('X', 'A'): 0, ('V', 'T'): 0, ('W', 'F'): 1, ('S', 'H'): -1, ('X', 'N'): -1, ('B', 'Q'): 0, ('K', 'A'): -1, ('I', 'Q'): -3, ('X', 'W'): -2, ('N', 'N'): 6, ('W', 'T'): -2, ('P', 'D'): -1, ('B', 'C'): -3, ('I', 'C'): -1, ('V', 'K'): -2, ('X', 'Y'): -1, ('K', 'R'): 2, ('Z', 'R'): 0, ('W', 'E'): -3, ('T', 'E'): -1, ('B', 'R'): -1, ('L', 'R'): -2, ('Q', 'R'): 1, ('X', 'F'): -1, ('T', 'S'): 1, ('B', 'D'): 4, ('Z', 'A'): -1, ('M', 'N'): -2, ('V', 'D'): -3, ('F', 'A'): -2, ('X', 'E'): -1, ('F', 'H'): -1, ('M', 'A'): -1, ('K', 'Q'): 1, ('Z', 'S'): 0, ('X', 'G'): -1, ('V', 'V'): 4, ('W', 'D'): -4, ('X', 'H'): -1, ('S', 'F'): -2, ('X', 'L'): -1, ('B', 'S'): 0, ('S', 'G'): 0, ('P', 'M'): -2, ('Y', 'M'): -1, ('H', 'D'): -1, ('B', 'E'): 1, ('Z', 'B'): 1, ('I', 'E'): -3, ('V', 'E'): -2, ('X', 'T'): 0, ('X', 'R'): -1, ('R', 'R'): 5, ('Z', 'T'): -1, ('Y', 'D'): -3, ('V', 'W'): -3, ('F', 'L'): 0, ('T', 'C'): -1, ('X', 'Q'): -1, ('B', 'T'): -1, ('K', 'N'): 0, ('T', 'H'): -2, ('Y', 'I'): -1, ('F', 'Q'): -3, ('T', 'I'): -1, ('T', 'Q'): -1, ('P', 'L'): -3, ('R', 'A'): -1, ('B', 'F'): -3, ('Z', 'C'): -3, ('M', 'H'): -2, ('V', 'F'): -1, ('F', 'C'): -2, ('L', 'L'): 4, ('M', 'C'): -1, ('C', 'R'): -3, ('D', 'D'): 6, ('E', 'R'): 0, ('V', 'P'): -2, ('S', 'D'): 0, ('E', 'E'): 5, ('W', 'G'): -2, ('P', 'C'): -3, ('F', 'R'): -3, ('B', 'G'): -1, ('C', 'C'): 9, ('I', 'G'): -4, ('V', 'G'): -3, ('W', 'K'): -3, ('G', 'N'): 0, ('I', 'N'): -3, ('Z', 'V'): -2, ('A', 'A'): 4, ('V', 'Q'): -2, ('F', 'K'): -3, ('T', 'A'): 0, ('B', 'V'): -3, ('K', 'L'): -2, ('L', 'N'): -3, ('Y', 'N'): -2, ('F', 'F'): 6, ('L', 'G'): -4, ('B', 'H'): 0, ('Z', 'E'): 4, ('Q', 'D'): 0, ('X', 'B'): -1, ('Z', 'W'): -3, ('S', 'K'): 0, ('X', 'K'): -1, ('V', 'R'): -3, ('K', 'E'): 1, ('I', 'A'): -1, ('P', 'H'): -2, ('B', 'W'): -4, ('K', 'K'): 5, ('H', 'C'): -3, ('E', 'N'): 0, ('Y', 'Q'): -1, ('H', 'H'): 8, ('B', 'I'): -3, ('C', 'A'): 0, ('I', 'I'): 4, ('V', 'A'): 0, ('W', 'I'): -3, ('T', 'F'): -2, ('V', 'S'): -2, ('T', 'T'): 5, ('F', 'M'): 0, ('L', 'E'): -3, ('M', 'M'): 5, ('Z', 'G'): -2, ('D', 'R'): -2, ('M', 'D'): -3, ('W', 'H'): -2, ('G', 'C'): -3, ('S', 'R'): -1, ('S', 'I'): -2, ('P', 'Q'): -1, ('Y', 'A'): -2, ('X', 'I'): -1, ('E', 'A'): -1, ('B', 'Y'): -3, ('K', 'I'): -3, ('H', 'A'): -2, ('P', 'G'): -2, ('F', 'N'): -3, ('H', 'N'): 1, ('B', 'K'): 0, ('V', 'C'): -1, ('T', 'L'): -1, ('P', 'K'): -1, ('W', 'S'): -3, ('T', 'D'): -1, ('T', 'M'): -1, ('P', 'N'): -2, ('K', 'H'): -1, ('T', 'R'): -1, ('Y', 'R'): -2, ('L', 'C'): -1, ('B', 'L'): -4, ('Z', 'Y'): -2, ('W', 'N'): -4, ('G', 'A'): 0, ('S', 'P'): -1, ('E', 'Q'): 2, ('C', 'N'): -3, ('H', 'Q'): 0, ('D', 'N'): 1, ('Y', 'C'): -2, ('L', 'H'): -3, ('E', 'C'): -4, ('Z', 'H'): 0, ('H', 'G'): -2, ('P', 'E'): -1, ('Y', 'S'): -2, ('G', 'R'): -2, ('B', 'M'): -3, ('Z', 'Z'): 4, ('W', 'M'): -1, ('Y', 'T'): -2, ('Y', 'P'): -3, ('Y', 'Y'): 7, ('T', 'K'): -1, ('Z', 'I'): -3, ('T', 'P'): -1, ('V', 'L'): 1, ('F', 'I'): 0, ('G', 'Q'): -2, ('L', 'A'): -1, ('M', 'I'): 1}


def smooth_insertions(state_vector):
    '''
    The function aims to correct to the expected imgt alignment. Renumbering functions then translate from the imgt scheme to the
    appropriate scheme.        

    Handle insertions made by HMMER that we suspect may be in the wrong position.
    Edge cases include:
        - Insertions at the C terminal of fw1, fw3 and fw3 regions. Can occur when 'conserved' residues have been mutated and the 
          same amino acid appears in the the following CDR (e.g. mutate cysteine at 104 but the CDR3 has one or more cysteines)
        - Same as above possible (but not observed in structure seqs) for N terminal of fw2, fw3 and fw4... TODO
        - Heavily mutated N terminal regions that are partially recognised (e.g. 3gk8 chain H). Insertions should not be allowed 
          before N terminal deletions have been used. Preserve deletion locations that are not N terminal (e.g. 10 in IMGT H) if 
          the gap has been placed by the alignment.

    '''
    # Small overhead doing these corrections but worth it for reducing edge cases.
    
    # Enforce insertion patterns as below. The CDRs are renumbered in each case so that insertions are placed accoring to the scheme    
#  '11111111111111111111111111222222222222333333333333333334444444444555555555555555555555555555555555555555666666666666677777777777'
#  '                        mmmi                         mmmi                                             mmmi                      '
#  '                        mmmi        immm             mmmi      immm                                   mmmi         immm         '

    # Enforce any insertions at the end and beginning of framework regions to be moved into the CDR region for renumbering. 
    enforced_patterns = [ [(25,'m'),(26,'m'),( 27,'m'),( 28,'i')],
                          [(38,'i'),(38,'m'),(39,'m'),(40,'m')],
                          [(54,'m'),(55,'m'),(56,'m'),(57,'i')],
                          [(65,'i'),(65,'m'),(66,'m'),(67,'m')],
                          [(103,'m'),(104,'m'),(105,'m'),(106,'i')],
                          [(117,'i'),(117,'m'),(118,'m'),(119,'m')] ]

    # Insertions in FW1 are only allowed if there are a fewer number of n-terminal deletions made. 

    state_buffer = [] 
    sv = []
    for (state_id, state_type ), si in state_vector:
        if state_id < 23: # Everything before the cysteine at 23.
            state_buffer.append( ((state_id, state_type ), si) )
            reg = -1   
        elif 25 <= state_id < 28: # Add to the buffer 
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 0
        elif 37 < state_id <= 40: # Add to the buffer 
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 1
        elif 54 <= state_id < 57: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 2
        elif 64 < state_id <= 67: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 3
        elif 103 <= state_id < 106: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 4
        elif 116 < state_id <= 119: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 5
        elif len(state_buffer) != 0: # Add the buffer and reset

            # Find the number of insertions in the buffer
            nins = sum( 1 for s in state_buffer if s[0][1] == 'i' ) 

            # If there are insertions, adjust the alignment
            if nins > 0: # We have insertions

                if reg == -1: # FW1, only adjust if there are the same or more N terminal deletions than insertions
                    nt_dels = state_buffer[0][0][0] - 1 # Missing states
                    for (_id, _type ), _si in state_buffer: # Explicit deletion states.
                        if _type == 'd' or _si == None:
                            nt_dels +=1 
                        else: # First residue found
                            break
                    if nt_dels >= nins: # More n terminal deletions than insertions found. Likely misalignment.
                        
                        # Preserve the deleted states structure by using the same match annotations
                        new_states = [ s for s, _ in state_buffer if s[1] == 'm'] 
                        _first = new_states[0][0]

                        # Remove the deletions so that only residue positions are included
                        state_buffer = [ s for s in state_buffer if s[0][1] != 'd' ]

                        # Extend N terminal states backwards from the first match states
                        _add = len( state_buffer ) - len( new_states ) 
                        assert _add >= 0, 'Implementation logic error' # Should be adding a positive number of positions
                        new_states = [ (_,'m') for _ in range( _first - _add, _first ) ] + new_states
                        assert len(new_states)==len(state_buffer), 'Implementation logic error' # Should have the same length

                        # Assign them preserving the order of the sequence. 
                        for i in range( len(state_buffer ) ):
                            sv.append( ( new_states[i], state_buffer[i][1]) )
                    else:
                        sv += state_buffer # The insertions may be incorrect but unknown what to do. Let the alignment place.
                else:
                    # Remove any deletions in the buffer. Unlikely to happen but do anyway
                    state_buffer = [ s for s in state_buffer if s[0][1] != 'd' ]
        
                    # Define the new states defined by the enforced pattern and the length of the buffer
                    if reg % 2: # nterm fw
                        new_states = [enforced_patterns[reg][0]]*max( 0, len(state_buffer)-3) + enforced_patterns[reg][ max( 4-len(state_buffer), 1):]
                    else: # cterm fw
                        new_states = enforced_patterns[reg][:3] + [enforced_patterns[reg][2]]*max( 0, len(state_buffer)-3)
                    # Assign them preserving the order of the sequence. 
                    for i in range( len(state_buffer ) ):
                        sv.append( ( new_states[i], state_buffer[i][1]) )
                                
            else: # Nothing to do - either all match or deletion states.
                sv += state_buffer

            # Add the current state
            sv.append( ((state_id, state_type ), si) )

            # Reset state buffer
            state_buffer = [] 
            
        else: # Simply append 
            sv.append( ((state_id, state_type ), si) )
    

    return sv


# General function to give annotations for regions that have direct mappings onto the hmm alignment (imgt states)
def _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions):
    """
    General function to number a sequence and divide it into different regions  
    
    @param sequence: The sequence string
    @param state_vector: The list of states from the aligned hmm
    @param state_string: A string of states for the scheme relative to IMGT (this is X for a direct equivalence, I if needs to be treated as insertion)
    @param region_string: A string of characters that indicate which hmm states are in each regions for this scheme (i.e. how should the sequence be divided up)
    @param region_index_dict: A dictionary converting the characters in region string to an index of the regions. 
    @param rels: The difference of the numbering integer at the *start* of each region
    @param n_regions: The number of regions
    @param exclude_deletions: A list of region indices for which deletion states should not be included. Typically the CDRs. 
                              These will be reannotated in the scheme function. Also allows the reset of insertions. 
    
    @return: A list of lists where each region has been numbered according to the scheme. Some regions will need renumbering. This should be taken care of after the function called.
    
    """

    state_vector = smooth_insertions( state_vector )
    
    _regions = [ [] for _ in range(n_regions) ]
    
    # Initialise the insertion index (-1 is a blank space) and the previous state.
    insertion = -1
    previous_state_id = 1
    previous_state_type = 'd'
    start_index, end_index  = None, None
    
    region = None

    # Iterate over the aligned state vector
    for (state_id, state_type ), si in state_vector:
       
        # Retrieve the region index
        if state_type != "i" or region is None: # BUG_FIX - JD 9/4/15 - do not allow a new region to start as an insertion.
            region = region_index_dict[region_string[state_id-1]] 

       
        # Check the state_types
        if state_type == "m": # It is a match
            
            # Check whether this position is in the scheme as an independent state
            if state_string[state_id-1]=="I": # No, it should be treated as an insertion
                if previous_state_type != 'd': # Unless there was a deletion beforehand in which case this should be a real pos.
                    insertion +=1 # Increment the insertion annotation index
                rels[region] -= 1 # Update the relative numbering from the imgt states
                #print (region,state_id,rels[region])
            else: # Yes 
                insertion = -1 # Reset the insertions 
            
            # Add the numbering annotation to the appropriate region list            
            _regions[region].append( ( (state_id + rels[region], alphabet[insertion] ), sequence[si]  ) )
            previous_state_id = state_id # Record the previous state ID
            if start_index is None:
                start_index = si
            end_index = si

            previous_state_type = state_type
            
        elif state_type == "i": # It is an insertion
            insertion +=1 # Increment the insertion annotation index
            
            # Add the numbering annotation to the appropriate region list
            _regions[region].append( ( (previous_state_id + rels[region], alphabet[insertion]), sequence[si]  ) )
            if start_index is None:
                start_index = si
            end_index = si

            previous_state_type = state_type

        else: # It is a deletion
            previous_state_type = state_type
            
            # Check whether this position is in the scheme as an independent state
            if state_string[state_id-1]=="I": # No, therefore irrelevant to the scheme.
                rels[region] -= 1 # Update the relative numbering from the imgt states
                continue 
            
            insertion = -1 # Reset the insertions
            previous_state_id = state_id # Record the previous state ID, should not be needed (no delete to insert state transition)


        # Reset the inssertion index if necessary and allowed. (Means the insertion code is meaningless and will be reannotated)
        if insertion >= 25 and region in exclude_deletions:
            insertion = 0 
        
        assert insertion < 25, "Too many insertions for numbering scheme to handle" # We ran out of letters.
            
    return _regions, start_index, end_index


# Functions to perform the numbering and the corrections for each of the implemented schemes.
# These have been written fairly verbosely so that the template of how to generate a function for a new scheme is more clear.
# They have two stages: Perform the mapping between imgt and the scheme; Renumber those regions that do not map nicely onto imgt (e.g. CDR insertions)
    

    
########
# IMGT # 
########
# - Renumbering of the CDR 1 and 2 regions in IMGT has now been implemented to ensure consistency with the gapping rules of the 
# scheme. Previously gaps were defined using the HMM alignment as the underlying model was already based on the IMGT scheme. This 
# worked well in original test cases but appears to give inaccurate annotations in a significant number of cases in NGS size 
# sequence sets. We therefore now explicitly renumber the CDR 1 and 2 as with all the other schemes. 

def number_imgt(state_vector, sequence):
    """    
    Apply the IMGT numbering scheme for heavy or light chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in IMGT scheme, I is an insertion. (All X's for IMGT)
    XXXXXXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXX XXXXXXXXXXXXXXXXX XXXXXXXXXX XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXX
    11111111111111111111111111 222222222222 33333333333333333 4444444444 555555555555555555555555555555555555555 6666666666666 77777777777

    Regions - (N.B These do not match up with any particular definition of CDR)
    1. All positions before CDR1
    2. CDR1 positions
    3. Positions between CDR1/2
    4. CDR2 positions
    5. Positions between CDR2/3
    6. CDR positions 105 (inc) to 118 (exc)
    7. Positions after CDR3    
    
    """
    
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111111111111111111111222222222222333333333333333334444444444555555555555555555555555555555555555555666666666666677777777777'

    region_index_dict = {
                         "1":0,
                         "2":1,
                         "3":2,
                         "4":3,
                         "5":4,
                         "6":5,
                         "7":6
                         }
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                          1:0,
                          2:0,
                          3:0,
                          4:0,
                          5:0,
                          6:0,
                          7:0
                          }
    
    n_regions = 7

    exclude_deletions = [1,3,5]    
    
    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    ###############
    # Renumbering #
    ###############

    _numbering = [ _regions[0], # Fw1
                   [],          # CDR1
                   _regions[2], # Fw2
                   [],          # CDR2
                   _regions[4], # Fw3
                   [],          # CDR3
                   _regions[6], # Fw4

                 ]

    # The alignment from HMMER should be correct for CDRs 1 and 2. Testing has shown not always the case and 'manual' renumbering 
    # is required as with the other schemes.
    
    # CDR1
    # CDR1 has a range from 27 (inc.) to 39 (exc.) and has a theoretical maximum length of 12.
    cdr1seq    = "".join([ x[1] for x in _regions[1] if x[1] != "-" ])
    cdr1length = len(cdr1seq) 
    si = 0
    prev_state = 26
    for ann in get_imgt_cdr(cdr1length, 12, 27, 39):
        if not ann:
            _numbering[1].append( ((prev_state+1, ' '), '-') )
            prev_state += 1
        else:
            _numbering[1].append( (ann, cdr1seq[si]) )
            prev_state = ann[0]
            si += 1

    # CDR2 
    # CDR2 has a range from 56 (inc.) to 66 (exc.) and has a theoretical length of 10.
    cdr2seq    = "".join([ x[1] for x in _regions[3] if x[1] != "-" ])
    cdr2length = len(cdr2seq)
    si = 0
    prev_state = 55
    for ann in get_imgt_cdr(cdr2length, 10, 56, 66):
        if not ann:
            _numbering[3].append( ((prev_state+1, ' '), '-') )
            prev_state += 1
        else:
            _numbering[3].append( (ann, cdr2seq[si]) )
            prev_state = ann[0]
            si += 1

    # FW3. We allow the HMM to place insertions. Technically all insertion points are taken care of but in reality insertions can
    # and do occur. No specification of where the insertions should be placed.


    # CDR3 
    # CDR3 has a range from 105 (inc.) to 118 (exc.). Insertions are placed on 112 and 111 symetrically. IMGT has a technical 
    # maximum length of 65 (13 positions, 26*2 insertions) . In practice ANARCI will not recognise CDR3s of this length.
    cdr3seq    = "".join([ x[1] for x in _regions[5] if x[1] != "-" ])
    cdr3length = len(cdr3seq)
    if cdr3length > 117: return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    si = 0
    previous_state_id = 104
    for ann in get_imgt_cdr(cdr3length, 13, 105, 118):
        if ann is None:
            _numbering[5].append( ((previous_state_id+1, " "), "-"   ) )
            previous_state_id+=1
        else:
            _numbering[5].append( (ann, cdr3seq[si] ) )
            previous_state_id = ann[0]
            si+=1
  
    # Return the full vector and the start and end indices of the numbered region of the sequence
    return gap_missing( _numbering ), startindex, endindex
  
def get_imgt_cdr(length, maxlength, start, end):
    """
    Symmetrically number a CDR loop (e.g. CDRL1/CDRH2 for IMGT)
    @param length:      Define the length of target CDR
    @param maxlength:   Define the theoretical limit (e.g. L1 = 12 for the IMGT scheme)
    @param start, end:  Start and end position numbers
    """
    annotations = [ None for _ in range(max(length, maxlength)) ]
    if length == 0:
        return annotations
    elif length == 1:
        annotations[0] = (start, ' ')
        return annotations

    front, back = 0, -1
    #az = "ABCDEFGHIJKLMNOPQRSTUVWXYZ" 
    #za = "ZYXWVUTSRQPONMLKJIHGFEDCBA"

    az = alphabet[:-1]
    za = az[::-1]

    for i in range(min(length, maxlength)):
        if i % 2:
            annotations[back] = (end + back, " ")
            back -= 1
        else:
            annotations[front] = (start + front, " ")
            front += 1

    # Add insertions around the centre point
    centrepoint = [ i for i,v in enumerate(annotations) if v == None ]
    if not centrepoint:
        return annotations

    centre_left  = annotations[min(centrepoint)-1][0] # Get the index right before the first None
    centre_right = annotations[max(centrepoint)+1][0] # Get the index right after  the first None

    # For cases with an even max length
    if not maxlength % 2:
        frontfactor, backfactor = maxlength//2, maxlength//2
    # For cases with an odd max length
    else:
        frontfactor, backfactor = (maxlength//2)+1, maxlength//2

    for i in range(max(0, length-maxlength)):
        if not i % 2:
            annotations[back] = (centre_right, za[back + backfactor])
            back -= 1
        else:
            annotations[front] = (centre_left, az[front - frontfactor])
            front += 1
    
    return annotations


#######
# Aho # 
#######
# Heuristic regapping based on the AHo specification as detailed on AAAAA website. Gap order depends on the chain type
def number_aho(state_vector, sequence, chain_type):
    """    
    Apply the Aho numbering scheme
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in IMGT scheme, I is an insertion. (All X's for IMGT)

    XXXXXXX XXX XXXXXXXXXXXXXX XXXXXXXXXXXXXXXX XXXXXXXXXXXXXXX XXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXX
    AAAAAAA BBB CCCCCCCCCCCCCC DDDDDDDDDDDDDDDD EEEEEEEEEEEEEEE FFFFFFFFFFFFFFFFFFFF HHHHHHHHHHHHHHHH IIIIIIIIIIIII JJJJJJJJJJJJJ KKKKKKKKKKK


    Regions - (N.B These do not match up with any particular definition of CDR)
    A. EMPTY (now included in B)
    B. 1-10 inclusive. Indel occurs at 8
    C. 11-24 inclusive.
    D. 25-42 inclusive (deletion surround 28) 32-42 inclusive (deletions surround 36)
    E. 43-57 inclusive 
    F. 58-77 inclusive (deletions surround 63). Alpha chains have deletions at 74,75
    G. EMPTY (now included in H)
    H. 78-93 inclusive  gaps on 86 then 85, insertions on 85 linearly
    I. 94-106 inclusive
    J. 107-138 inclusive gaps on 123 symetrically.
    K. 139-149 inclusive.
    
    """
    
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string =  'BBBBBBBBBBCCCCCCCCCCCCCCDDDDDDDDDDDDDDDDEEEEEEEEEEEEEEEFFFFFFFFFFFFFFFFFFFFHHHHHHHHHHHHHHHHIIIIIIIIIIIIIJJJJJJJJJJJJJKKKKKKKKKKK'
#                     1         2             3               4              5                   7               8            9            10


    region_index_dict = dict( list(zip( "ABCDEFGHIJK", list(range(11)) )) )
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                         1:0,
                         2:0,
                         3:0,
                         4:2,
                         5:2,
                         6:2,
                         7:2,
                         8:2,
                         9:2,
                         10:21}

    n_regions = 11
    
    exclude_deletions = [1,3,4,5,7,9]    
    
    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    ###############
    # Renumbering #
    ###############

    _numbering = [ _regions[0], _regions[1], _regions[2],[], _regions[4], [], _regions[6], [], _regions[8],_regions[9],_regions[10] ]

    ##################################
    # Move the indel in fw 1 onto 8  #
    ##################################

    # Place indels on 8 
    # Find the first recognised residue and change the expected length of the stretch given the starting point. 
    # This prevents n terminal deletions being placed at 8 incorrectly.
    length = len( _regions[1] )
    if length > 0:
        start = _regions[1][0][0][0] 
        stretch_len = 10 - (start -1)
        if length > stretch_len: # Insertions are present. Place on 8
            annotations = [ (_," ") for _ in range(start,9) ] + [ (8,alphabet[_]) for _ in range( length - stretch_len ) ] + [(9," "),(10," ")]
        else:
            ordered_deletions = [(8," ")] + [(_," ") for _ in range(start, 11) if _ != 8]
            annotations = sorted( ordered_deletions[max(stretch_len-length, 0):] )
        _numbering[1] = [ (annotations[i], _regions[1][i][1]) for i in range(length) ]

    #########
    # CDR 1 # - divided in two parts in the Aho scheme.
    ######### - gaps at 28 depending on the chain type.

    # "VH domains, as well as the majority of the VA domains, have a one-residue gap in position 28, VK and VB domains a two-residue
    # gap in position 27 and 28."

    # We use the link below as the reference for the scheme.
    # https://www.bioc.uzh.ch/plueckthun/antibody/Numbering/Alignment.html

    # Some of the header lines in these images are offset by one (VH)! The gaps really are centered at 28 and 36
    # https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VK.html
    # https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VL.html
    # https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VH.html
    # https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VA.html
    # https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VB.html
    # https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VG.html
    # https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VD.html

    # We gap the CDR1 in a heuristic way using the gaps. 
    # This means that CDR1 gapping will not always be correct. For example if one grafts a Kappa CDR1 loop onto a Lambda framework
    # the gapping patter might now be incorrect. 
    # Not a fan of being so prescriptive.

    # The CDR1 region included here ranges from AHo 25 to AHo 42 inclusive
    
    # The order in which the two loops are gapped is dependent on the chain type (see alignments in URLs above).
    # Not all lengths are defined as not all lengths were crystallised in 2001 (or today). Where no example of the length was 
    # available the rule followed is to continue gapping the C terminal 'loop', then the N terminal 'loop', then 31 then the fw. 
    # In all cases I have commented where the gapping is undefined. Note that for alpha chains the gapping rules are inconsistent.

    _L = 28,36,35,37,34,38,27,29,33,39,32,40,26,30,25,31,41,42
    #                           |-> undefined by AHo. Gapping C terminal loop then N terminal then 31, then fw.                              
    _K = 28,27,36,35,37,34,38,33,39,32,40,29,26,30,25,31,41,42
    #                                 |-> undefined by AHo. Gapping C terminal loop then N terminal then fw.                              
    _H = 28,36,35,37,34,38,27,33,39,32,40,29,26,30,25,31,41,42 
    #                        |-> undefined by AHo. Gapping C terminal loop then N terminal then fw.                              
    #                            N.B. The header on the alignment image for PDB_VH is offset by 1!
    _A = 28,36,35,37,34,38,33,39,27,32,40,29,26,30,25,31,41,42    
    #                              |-> undefined by AHo. Gapping C terminal loop then N terminal then fw.     
    #                            N.B The gapping is inconsistent for alpha chains. I follow the paper's statement that most VA have
    #                                one gap at 28 and remove 28 and 27 before removing 40. 
    _B = 28,36,35,37,34,38,33,39,27,32,40,29,26,30,25,31,41,42
    #                              |-> undefined by AHo. Gapping C terminal loop then N terminal then 31, then fw.
    _D = 28,36,35,37,34,38,27,33,39,32,40,29,26,30,25,31,41,42
    #                         |-> undefined by AHo. Gapping C terminal loop then N terminal then 31, then fw.
    #                         N.B only two sequence patterns available.                                
    _G = 28,36,35,37,34,38,27,33,39,32,40,29,26,30,25,31,41,42
    #                         |-> undefined by AHo. Gapping C terminal loop then N terminal then 31, then fw.
    #                         N.B only one sequence patterns available. Delta copied.
    
    ordered_deletions = { 'L':_L,'K':_K, 'H':_H, 'A':_A, 'B':_B, 'D':_D, 'G':_G } 

    length = len( _regions[3] )

    annotations = [ (i, ' ') for i in sorted( ordered_deletions[chain_type][ max(18-length, 0): ] ) ]

    # Insertions are not described in the AHo scheme but must be included as there is a significant number of CDRH1s that are 
    # longer than the number of positions.
    insertions = max( length-18 , 0 ) 
    if insertions > 26: 
        return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    elif insertions > 0:
        # They are placed on residue 36 alphabetically.
        insertat = annotations.index( (36, ' ') )+1 # Always 12 
        assert insertat == 12, 'AHo numbering failed'  
        annotations = annotations[:insertat] + [ (36, alphabet[a]) for a in range( insertions ) ] + annotations[insertat:]

    _numbering[3] = [ (annotations[i], _regions[3][i][1]) for i in range(length) ]

    #########
    # CDR 2 #
    #########
    # Gaps are placed symetically at 63.
    # For VA a second gap is placed at 74 and 75 according to the text in the paper. However, all the reference sequences show a
    # gap at 73 and 74 see:
    #      https://www.bioc.uzh.ch/plueckthun/antibody/Sequences/Rearranged/PDB_VA.html 
    # and 
    #      https://www.bioc.uzh.ch/plueckthun/antibody/Numbering/Alignment.html
    # Either I am mis-interpreting the text in the paper or there is something a little inconsistent here...
    # Given that *all* the numbered examples show the VA gap at 73 and 74 on the AAAAA website I have decided to implement this. 
    #
    
    # This region describes 58 to 77 inclusive
    
    if chain_type == 'A':
        ordered_deletions = [74,73,63,62,64,61,65,60,66,59,67,58,68,69,70,71,72,75,76,77]
    else:
        ordered_deletions = [63,62,64,61,65,60,66,59,67,58,68,69,70,71,72,73,74,75,76,77]

    length = len(_regions[5])

    annotations = [ (i, ' ') for i in sorted( ordered_deletions[ max(20-length, 0): ] ) ]

    # Insertions are not described in the AHo scheme but must be included.
    insertions = max( length-20 , 0 ) 
    if insertions > 26: 
        return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    elif insertions > 0:
        # They are placed on residue 63 alphabetically.
        insertat = annotations.index( (63, ' ') )+1 # Always 6
        assert insertat == 6, 'AHo numbering failed'  
        annotations = annotations[:insertat] + [ (63, alphabet[a]) for a in range( insertions ) ] + annotations[insertat:]

    _numbering[5] = [ (annotations[i], _regions[5][i][1]) for i in range(length) ]

    #########
    # FW3   ############################################
    # Move deletions onto 86 then 85. Insertions on 85 #
    ####################################################
    ordered_deletions = [86,85,87,84,88,83,89,82,90,81,91,80,92,79,93,78]
    length=len( _regions[7] )

    annotations = [ (i, ' ') for i in sorted( ordered_deletions[ max(16-length, 0): ] ) ]

    # Insertions are not described in the AHo scheme but must be included.
    insertions = max( length-16 , 0 ) 
    if insertions > 26: 
        return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    elif insertions > 0:
        # They are placed on residue 85 alphabetically.
        insertat = annotations.index( (85, ' ') )+1 # Always 8
        assert insertat == 8, 'AHo numbering failed'  
        annotations = annotations[:insertat] + [ (85, alphabet[a]) for a in range( insertions ) ] + annotations[insertat:]

    _numbering[7] = [ (annotations[i], _regions[7][i][1]) for i in range(length) ]


    #########
    # CDR 3 #
    #########
    # Deletions on 123. 
    # Point of the Aho scheme is that they have accounted for all possible positions.
    # Assumption is that no more insertions will occur.... 
    # We'll put insertions on 123 linearly.(i.e.ABCDEF...) if they ever do.

    ordered_deletions = [123,124,122,125,121,126,120,127,119,128,118,129,117,130,116,131,115,132,114,133,113,134,112,135,111,
                         136,110,137,109,138,108,107]
    
    length=len( _regions[9] )

    annotations = [ (i, ' ') for i in sorted( ordered_deletions[ max(32-length, 0): ] ) ]

    # Insertions are not described in the AHo scheme but must be included.
    insertions = max( length-32 , 0 ) 
    if insertions > 26: 
        return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    elif insertions > 0:
        # They are placed on residue 123 alphabetically.
        insertat = annotations.index( (123, ' ') )+1 # Always 17
        assert insertat == 17, 'AHo numbering failed'  
        annotations = annotations[:insertat] + [ (123, alphabet[a]) for a in range( insertions ) ] + annotations[insertat:]

    _numbering[9] = [ (annotations[i], _regions[9][i][1]) for i in range(length) ]

    # AHo includes one extra position than IMGT in what it considers the variable domain for light chains. 
    #If the last state is 148 and there is at least one more residue left, then add the residue to the numbering. 
    numbering = gap_missing( _numbering )
    if len(numbering) > 0:
        if numbering[-1][0] == (148, ' ') and numbering[-1][1] != '-' and endindex+1 < len(sequence):
            numbering.append( ( (149, ' '), sequence[endindex+1]) )
            endindex +=1

    return numbering, startindex, endindex


###########
# Chothia #
###########

# Heavy chains
def number_chothia_heavy(state_vector, sequence):
    """
    Apply the Chothia numbering scheme for heavy chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in Chothia scheme, I is an insertion.

    XXXXXXXXXI XXXXXXXXXXXXX XXXXXXXIIIIXX XXXXXXXXXXXXXXXXXX XXXIXIIXXXX XXXXXXXIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXX XXXXXXXXIIIXX XXXXXXXXXXX'
    1111111111 2222222222222 3333333333333 444444444444444444 55555555555 666666666666666666666666666666666666666 7777777777777 88888888888'     
    
    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Put the insertions at Chothia position 6
     2  -  Simple mapping (treat "I" states as inserts and not own match states)
     3  -  CDRH1 - 30 (inc) to 34 (exc) put insertions on 31
     4  -  Simple mapping (treat "I" states as inserts and not own match states)
     5  -  CDRH2 - 52 (inc) 58 (exc) put insertions on 52 
     6  -  Simple mapping (treat "I" states as inserts and not own match states)
     7  -  CDRH3 93 (inc) to 103 (exc) put insertion on 100
     8  -  Simple mapping (treat "I" states as inserts and not own match states)


    Regions 1,3,5 and 7 are renumbered
    
    """

    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXXIXIIXXXXXXXXXXXIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111112222222222222333333333333333444444444444444455555555555666666666666666666666666666666666666666777777777777788888888888'

    region_index_dict = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6,"8":7}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                         1:-1,
                         2:-1,
                         3:-5,
                         4:-5,
                         5:-8,
                         6:-12,
                         7:-15}    
    
    n_regions = 8
    
    exclude_deletions = [0,2,4,6] # Don't put deletions in these regions

    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    
    ###############
    # Renumbering #
    ###############

    _numbering = [ [], _regions[1] , [], _regions[3] , [], _regions[5], [], _regions[7] ]
    
    # Chothia H region 1 (index 0)
    # Insertions are placed at Chothia position 6.
    # Count how many we recognised as insertion by the hmm
    insertions = len( [ 1 for _ in _regions[0] if _[0][1] != " " ] ) 
    # We will place all insertion in this region at Chothia position 6.
    if insertions:
        start = _regions[0][0][0][0] # The starting Chothia number as found by the HMM (could easily start from 2 for example)
        # I have a feeling this may be a source of a bug in very unusual cases. Can't break for now. Will catch mistakes in a validate function. 
        length = len( _regions[0] )
        annotations = [ (_, " ") for _ in range(start, 7) ] + [ (6, alphabet[_]) for _ in range(insertions) ] + [(7," "),(8," "),(9," ")]
        _numbering[0] =  [ (annotations[i], _regions[0][i][1]) for i in range(length) ]
    else:
        _numbering[0] = _regions[0]

    
    # CDR1 
    # Chothia H region 3 (index 2)
    # put insertions onto 31
    length = len( _regions[2] )
    insertions = max(length - 11, 0) # Pulled back to the cysteine as heavily engineered cdr1's are not playing nicely

    if insertions:
        annotations = [(_, " ") for _ in range(23,32)] + [(31, alphabet[i]) for i in range(insertions) ] + [(32," "),(33," ")]
    else:
        annotations = [(_, " ") for _ in range(23,32)][:length-2] + [(32," "),(33," ")][:length]

    _numbering[2] = [ (annotations[i], _regions[2][i][1]) for i in range(length) ]
 
    # CDR2
    # Chothia H region 5 (index 4) 
    # put insertions onto 52
    length = len( _regions[4] )
    # 50 to 57 inclusive
    insertions = max(length - 8, 0) # Eight positions can be accounted for, the remainder are insertions
    # Delete in the order, 52, 51, 50,53, 54 ,55, 56, 57
    annotations  =  [(50, " "),(51, " "), (52, " ")][:max(0,length-5)]
    annotations += [(52, alphabet[i]) for i in range(insertions) ]
    annotations += [(53, " "),(54, " "),(55, " "),(56, " "),(57, " ")][ abs( min(0,length-5) ):]
    _numbering[4] = [ (annotations[i], _regions[4][i][1]) for i in range(length) ]

    # FW3 - insertions are annotated on 82. The first three are normal positions and annotated automatically. 
    # Additional insertions do not occur with the kabat or the chothia numbering scheme.
    # It does not make sense to place more than A, B, C on 82 as Martin and AHo work show that this is not a place that accepts 
    # additional insertions. 
    # The decision here is to allow the alignment to place additional insertions. This is in contrast to Martin where the region
    # is renumbered to place insertions on 72. 
     
    # CDR3
    # Chothia H region 7 (index 6) 
    # put insertions onto 100
    length = len( _regions[6] )    
    if length > 36: return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    annotations = get_cdr3_annotations(length, scheme="chothia", chain_type="heavy")
    _numbering[6]  = [ (annotations[i], _regions[6][i][1]) for i in range(length)  ]

    # Return the full vector and the start and end indices of the numbered region of the sequence
    return gap_missing( _numbering ), startindex, endindex                                     

# Light chains
def number_chothia_light(state_vector, sequence):
    """
    Apply the Chothia numbering scheme for light chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in Chothia scheme, I is an insertion.
    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX IIIIIIX XXXXXXXXXXXXXXXXXXXX XIIIIIIIXXX XXXXXIXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXX XXXXXIIIIXX XXXXXXXXXXXXX
    11111111111111111111111111111 2222222 33333333333333333333 44444444444 5555555555555555555555555555555555555 66666666666 7777777777777
     
    
    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Simple mapping (treat "I" states as inserts and not own match states)
     2  -  CDRL1 - 24 (inc) to 35 (exc) put insertions on 30
     3  -  Simple mapping (treat "I" states as inserts and not own match states)
     4  -  CDRL2 - 51 (inc) 55 (exc) put insertions on 52
     5  -  Simple mapping (treat "I" states as inserts and not own match states)
     6  -  CDRL3 89 (inc) to 98 (exc) put insertion on 95
     7  -  Simple mapping (treat "I" states as inserts and not own match states)
    
    Region 2, 3 and 5 are renumbered
    
    """
    
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIXXXXXXXXXXXXXXXXXXXXXXIIIIIIIXXXXXXXXIXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111111111111111111222222222222222223333333333333333444444444445555555555555555555555555555555555555666666666666677777777777'

    region_index_dict = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                         1: 0,
                         2:-6,
                         3:-6,
                         4:-13,
                         5:-16,
                         6:-20,
                         }    

    
    n_regions = 7
    
    exclude_deletions = [1,3,4,5]

    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    _numbering = [ _regions[0], [], _regions[2], [], _regions[4], [], _regions[6] ]
    
    
    ###############
    # Renumbering #
    ###############

    # CDR1 
    # Chothia L region 2 (index 1)
    # put insertions onto 30
    length = len( _regions[1] )
    insertions = max(length - 11, 0) # Eleven positions can be accounted for, the remainder are insertions
    # Delete forward from 31 
    annotations  =  [(24, " "),(25, " "), (26, " "), (27, " "), (28, " "),(29, " "),(30, " ")][:max(0,length)] 
    annotations += [(30, alphabet[i]) for i in range(insertions) ]
    annotations += [(31, " "),(32, " "),(33, " "),(34, " ")][ abs( min(0,length-11) ):] 
    _numbering[1] = [ (annotations[i], _regions[1][i][1]) for i in range(length) ]


    # CDR2
    # Chothia L region 4 (index 3) 
    # put insertions onto 52. 
    length = len( _regions[3] )
    insertions = max( length - 4, 0 )
    if insertions > 0:
        annotations  = [(51, " "),(52, " ")] + [(52, alphabet[i]) for i in range(insertions) ] + [(53, " "),(54, " ")]
        _numbering[3] = [ (annotations[i], _regions[3][i][1]) for i in range(length) ]
    else: # How to gap L2 in Chothia/Kabat/Martin is unclear so we let the alignment do it.
        _numbering[3] = _regions[3]
    
    # FW3
    # Insertions on 68. First deletion 68. Otherwise default to alignment
    length = len( _regions[4] )
    insertions = max(length - 34, 0)
    if insertions > 0: # Insertions on 68
        annotations = [(i," ") for i in range(55,69)]+[(68, alphabet[i]) for i in range(insertions) ]+[(i," ") for i in range(69,89)]
        _numbering[4] = [ (annotations[i], _regions[4][i][1]) for i in range(length) ]
    elif length == 33: # First deletion on 68
        annotations = [(i," ") for i in range(55,68)]+[(i," ") for i in range(69,89)]            
        _numbering[4] = [ (annotations[i], _regions[4][i][1]) for i in range(length) ]
    else: # More deletions - allow alignment to place them 
        _numbering[4] = _regions[4]


    # CDR3
    # Chothia L region 6 (index 5) 
    # put insertions onto 95
    length = len( _regions[5] )    

    if length > 35: return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    annotations = get_cdr3_annotations(length, scheme="chothia", chain_type="light")
    _numbering[5]  = [ (annotations[i], _regions[5][i][1]) for i in range(length)  ]

    # Return the full vector and the start and end indices of the numbered region of the sequence

    return gap_missing( _numbering ), startindex, endindex    


#########
# Kabat #
#########

# Heavy chains
def number_kabat_heavy(state_vector, sequence):
    """
    Apply the Kabat numbering scheme for heavy chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in Kabat scheme, I is an insertion.
    XXXXXXXXXI XXXXXXXXXXXXXXXXXXXX IIIIXXXXXX XXXXXXXXXXXXXXXX XIXII XXXXXXXXXXXIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXX XXXXXXIII XXXXXXXXXXXXX
    1111111111 22222222222222222222 3333333333 4444444444444444 55555 666666666666666666666666666666666666666666666 777777777 8888888888888
     
    
    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Put the insertions at Chothia position 6
     2  -  Simple mapping (treat "I" states as inserts and not own match states)
     3  -  CDRH1 - 30 (inc) to 36 (exc) put insertions on 35
     4  -  Simple mapping (treat "I" states as inserts and not own match states)
     5  -  CDRH2 - 52 (inc) 58 (exc) put insertions on 52 
     6  -  Simple mapping (treat "I" states as inserts and not own match states)
     7  -  CDRH3 93 (inc) to 103 (exc) put insertion on 100
     8  -  Simple mapping (treat "I" states as inserts and not own match states)

    """
 
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXXIXIIXXXXXXXXXXXIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111112222222222222333333333333333334444444444444455555555555666666666666666666666666666666666666666777777777777788888888888'

    region_index_dict = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6,"8":7}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                         1:-1,
                         2:-1,
                         3:-5,
                         4:-5,
                         5:-8,
                         6:-12,
                         7:-15}    
    
    n_regions = 8
    
    exclude_deletions = [2,4,6]

    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    

    ###############
    # Renumbering #
    ###############
        
    # Renumbering required for 0, 2, 4, 6 regions in Chothia heavy
    
    _numbering = [ [], _regions[1] , [], _regions[3] , [], _regions[5], [], _regions[7] ]
    return _numbering


    # Kabat H region 1 (index 0)
    # Insertions are placed at Kabat position 6.
    # Count how many we recognised as insertion by the hmm
    insertions = len( [ 1 for _ in _regions[0] if _[0][1] != " " ] ) 
    # We will place all insertion in this region at Kabat position 6.
    if insertions:
        start = _regions[0][0][0][0] # The starting Kabat number as found by the HMM (could easily start from 2 for example)
        # I have a feeling this may be a source of a bug in very unusual cases. Can't break for now. Will catch mistakes in a validate function. 
        length = len( _regions[0] )
        annotations = [ (_, " ") for _ in range(start, 7) ] + [ (6, alphabet[_]) for _ in range(insertions) ] + [(7," "),(8," "),(9," ")]
        _numbering[0] =  [ (annotations[i], _regions[0][i][1]) for i in range(length) ]
    else:
        _numbering[0] = _regions[0]
    
    
    # CDR1 
    # Kabat H region 3 (index 2)
    # Put insertions onto 35. Delete from 35 backwards
    length = len( _regions[2] )
    insertions = max(0,length - 13)
    annotations = [(_,' ') for _ in range(23, 36)][:length] 
    annotations += [(35, alphabet[i]) for i in range(insertions) ]
    _numbering[2] = [ (annotations[i], _regions[2][i][1]) for i in range(length) ]
 
    # CDR2
    # Chothia H region 5 (index 4) 
    # put insertions onto 52
    length = len( _regions[4] )
    # 50 to 57 inclusive
    insertions = max(length - 8, 0) # Eight positions can be accounted for, the remainder are insertions
    # Delete in the order, 52, 51, 50,53, 54 ,55, 56, 57
    annotations  =  [(50, " "),(51, " "), (52, " ")][:max(0,length-5)]
    annotations += [(52, alphabet[i]) for i in range(insertions) ]
    annotations += [(53, " "),(54, " "),(55, " "),(56, " "),(57, " ")][ abs( min(0,length-5) ):]
    _numbering[4] = [ (annotations[i], _regions[4][i][1]) for i in range(length) ]

    # FW3 - insertions are annotated on 82. The first three are normal positions and annotated automatically. 
    # Additional insertions do not occur with the kabat or the chothia numbering scheme.
    # It does not make sense to place more than A, B, C on 82 as Martin and AHo work show that this is not a place that accepts 
    # additional insertions. 
    # The decision here is to allow the alignment to place additional insertions. This is in contrast to Martin where the region
    # is renumbered to place insertions on 72. 
     
    # CDR3
    # Chothia H region 7 (index 6) 
    # put insertions onto 100
    length = len( _regions[6] )    
    if length > 36: return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    annotations = get_cdr3_annotations(length, scheme="kabat", chain_type="heavy") #  Chothia and Kabat the same here
    _numbering[6]  = [ (annotations[i], _regions[6][i][1]) for i in range(length)  ]

    # Return the full vector and the start and end indices of the numbered region of the sequence
    return gap_missing( _numbering ), startindex, endindex            
           
# Light chains    
def number_kabat_light(state_vector, sequence):
    """
    Apply the Kabat numbering scheme for light chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in Kabat scheme, I is an insertion.
    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX IIIIIIX XXXXXXXXXXXXXXXXXXXX XIIIIIIIXXX XXXXXIXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXX XXXXXIIIIXX XXXXXXXXXXXXX
    11111111111111111111111111111 2222222 33333333333333333333 44444444444 5555555555555555555555555555555555555 66666666666 7777777777777
     
    
    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Simple mapping (treat "I" states as inserts and not own match states)
     2  -  CDRL1 - 24 (inc) to 35 (exc) put insertions on 27
     3  -  Simple mapping (treat "I" states as inserts and not own match states)
     4  -  CDRL2 - 51 (inc) 55 (exc) put insertions on 52
     5  -  Simple mapping (treat "I" states as inserts and not own match states)
     6  -  CDRL3 89 (inc) to 96 (exc) put insertion on 95
     7  -  Simple mapping (treat "I" states as inserts and not own match states)
    
    """
    
    # Set up the numbering 
 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIXXXXXXXXXXXXXXXXXXXXXXIIIIIIIXXXXXXXXIXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111111111111111111222222222222222223333333333333333444444444445555555555555555555555555555555555555666666666666677777777777'
    
    region_index_dict = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                         1: 0,
                         2:-6,
                         3:-6,
                         4:-13,
                         5:-16,
                         6:-20,
                         }    
    
    n_regions = 7
    
    exclude_deletions = [1,3,5]

    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    _numbering = [ _regions[0], [], _regions[2], [], _regions[4], [], _regions[6] ]
    
    
    ###############
    # Renumbering #
    ###############
    
    # CDR1 
    # Kabat L region 2 (index 1)
    # put insertions onto 27
    length = len( _regions[1] )
    insertions = max(length - 11, 0) # Eleven positions can be accounted for, the remainder are insertions
    # Delete forward from 28 
    annotations  =  [(24, " "),(25, " "), (26, " "), (27, " ")][:max(0,length)] 
    annotations += [(27, alphabet[i]) for i in range(insertions) ]
    annotations += [(28, " "),(29, " "),(30, " "),(31, " "),(32, " "),(33, " "),(34, " ")][ abs( min(0,length-11) ):] 
    _numbering[1] = [ (annotations[i], _regions[1][i][1]) for i in range(length) ]
  
    # CDR2
    # Chothia L region 4 (index 3) 
    # put insertions onto 52. 
    length = len( _regions[3] )
    insertions = max( length - 4, 0 )
    if insertions > 0:
        annotations  = [(51, " "),(52, " ")] + [(52, alphabet[i]) for i in range(insertions) ] + [(53, " "),(54, " ")]
        _numbering[3] = [ (annotations[i], _regions[3][i][1]) for i in range(length) ]
    else: # How to gap L2 in Chothia/Kabat/Martin is unclear so we let the alignment do it.
        _numbering[3] = _regions[3]


    # FW3
    # All insertions are placed by alignment. This is in contrast to Martin (and Chothia) where they are placed on 68. 
    # The kabat scheme was defined using a sequence alignment alone. In keeping with this, insertions in FW3 are also only placed
    # with respect to the sequence alignment (the HMM).

    # CDR3
    # Chothia L region 6 (index 5) 
    # put insertions onto 95
    length = len( _regions[5] )    

    if length > 35: return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    annotations = get_cdr3_annotations(length, scheme="kabat", chain_type="light")
    _numbering[5]  = [ (annotations[i], _regions[5][i][1]) for i in range(length)  ]

    return gap_missing( _numbering ), startindex, endindex    




#############################
# Martin (extended Chothia) #
#############################

# Heavy chains
def number_martin_heavy(state_vector, sequence):
    """
    Apply the Martin (extended Chothia) numbering scheme for heavy chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in Martin scheme, I is an insertion.
    XXXXXXXXXI XXXXXXXXXXXXXXXXXXXX IIIIXX XXXXXXXXXXXXXXXXXXXX XIXII XXXXXXXXXXXIXXXXXXXXIIIXXXXXXXXXXXXXXXXXXXXXX XXXXXXIII XXXXXXXXXXXXX
    1111111111 22222222222222222222 333333 44444444444444444444 55555 666666666666666666666666666666666666666666666 777777777 8888888888888
     
    
    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Put the insertions at Chothia position 8
     2  -  Simple mapping (treat "I" states as inserts and not own match states)
     3  -  CDRH1 - 30 (inc) to 34 (exc) put insertions on 31
     4  -  Simple mapping (treat "I" states as inserts and not own match states)
     5  -  CDRH2 - 52 (inc) 58 (exc) put insertions on 52 
     6  -  Simple mapping (treat "I" states as inserts and not own match states)
     7  -  CDRH3 93 (inc) to 103 (exc) put insertion on 100
     8  -  Simple mapping (treat "I" states as inserts and not own match states)


    Regions 1,3,5 and 7 are renumbered
    
    """
 
    # Set up the numbering 
 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXXIXIIXXXXXXXXXXXIXXXXXXXXIIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111112222222222222333333333333333444444444444444455555555555666666666666666666666666666666666666666777777777777788888888888'
    
    region_index_dict = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6,"8":7}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                         1:-1,
                         2:-1,
                         3:-5,
                         4:-5,
                         5:-8,
                         6:-12,
                         7:-15}    
    
    n_regions = 8
    
    exclude_deletions = [2,4,5,6]

    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    

    ###############
    # Renumbering #
    ###############
        
    # Renumbering required for 0, 2, 4, 6 regions in Chothia heavy
    
    _numbering = [ [], _regions[1] , [], _regions[3] , [], _regions[5], [], _regions[7] ]
    
    # Chothia H region 1 (index 0)
    # Insertions are placed at Chothia position 8.
    # Count how many we recognised as insertion by the hmm
    insertions = len( [ 1 for _ in _regions[0] if _[0][1] != " " ] ) 
    # We will place all insertion in this region at Chothia position 8.
    if insertions:
        start = _regions[0][0][0][0] # The starting Chothia number as found by the HMM (could easily start from 2 for example)
        # I have a feeling this may be a source of a bug in very unusual cases. Can't break for now. Will catch mistakes in a validate function. 
        length = len( _regions[0] )
        annotations = [ (_, " ") for _ in range(start, 9) ] + [ (8, alphabet[_]) for _ in range(insertions) ] + [(9," ")]
        _numbering[0] =  [ (annotations[i], _regions[0][i][1]) for i in range(length) ]
    else:
        _numbering[0] = _regions[0]

    
    # CDR1 
    # Chothia H region 3 (index 2)
    # put insertions onto 31
    length = len( _regions[2] )
    insertions = max(length - 11, 0) # Pulled back to the cysteine as heavily engineered cdr1's are not playing nicely
    if insertions:
        annotations = [(_, " ") for _ in range(23,32)] + [(31, alphabet[i]) for i in range(insertions) ] + [(32," "),(33," ")]
    else:
        annotations = [(_, " ") for _ in range(23,32)][:length-2] + [(32," "),(33," ")][:length]
    _numbering[2] = [ (annotations[i], _regions[2][i][1]) for i in range(length) ]
 
    # CDR2
    # Chothia H region 5 (index 4) 
    # put insertions onto 52
    length = len( _regions[4] )
    # 50 to 57 inclusive
    insertions = max(length - 8, 0) # Eight positions can be accounted for, the remainder are insertions
    # Delete in the order, 52, 51, 50,53, 54 ,55, 56, 57
    annotations  =  [(50, " "),(51, " "), (52, " ")][:max(0,length-5)]
    annotations += [(52, alphabet[i]) for i in range(insertions) ]
    annotations += [(53, " "),(54, " "),(55, " "),(56, " "),(57, " ")][ abs( min(0,length-5) ):]
    _numbering[4] = [ (annotations[i], _regions[4][i][1]) for i in range(length) ]

    # FW3 
    # Place all insertions on 72 explicitly.
    # This is in contrast to Chothia implementation where 3 insertions are on 82 and then further insertions are placed by the 
    # alignment
    # Gaps are placed according to the alignment. 
    length = len( _regions[5] )
    insertions = max(length - 35, 0)
    if insertions > 0: # Insertions on 72
        annotations = [(i,' ') for i in range(58,73)]+[(72, alphabet[i]) for i in range(insertions) ]+[(i,' ') for i in range(73,93)]
        _numbering[5] = [ (annotations[i], _regions[5][i][1]) for i in range(length) ]
    else: # Deletions - all alignment to place them. 
        _numbering[4] = _regions[4]

     
    # CDR3
    # Chothia H region 7 (index 6) 
    # put insertions onto 100
    length = len( _regions[6] )    
    if length > 36: return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    annotations = get_cdr3_annotations(length, scheme="chothia", chain_type="heavy")
    _numbering[6]  = [ (annotations[i], _regions[6][i][1]) for i in range(length)  ]

    # Return the full vector and the start and end indices of the numbered region of the sequence
    return gap_missing( _numbering ), startindex, endindex                                    

# Light chains
def number_martin_light(state_vector, sequence):
    """
    Apply the Martin numbering scheme for light chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in Martin scheme, I is an insertion.
    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX IIIIIIX XXXXXXXXXXXXXXXXXXXX XIIIIIIIXXX XXXXXIXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXX XXXXXIIIIXX XXXXXXXXXXXXX
    11111111111111111111111111111 2222222 33333333333333333333 44444444444 5555555555555555555555555555555555555 66666666666 7777777777777
     
    
    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Simple mapping (treat "I" states as inserts and not own match states)
     2  -  CDRL1 - 30 (inc) to 31 (exc) put insertions on 30
     3  -  Simple mapping (treat "I" states as inserts and not own match states)
     4  -  CDRL2 - 51 (inc) 55 (exc) put insertions on 52 
     5  -  Simple mapping (treat "I" states as inserts and not own match states)
     6  -  CDRL3 89 (inc) to 96 (exc) put insertion on 95
     7  -  Simple mapping (treat "I" states as inserts and not own match states)
    
    Region 2, 3 and 5 are renumbered
    
    """
    
    # The Martin and Chothia specification for light chains are very similar. Martin is more explicit in the location of indels 
    # but unlike the heavy chain these are additional instead of changes to the Chothia scheme. Thus, Chothia light is implemented
    # as martin light.
    return number_chothia_light(state_vector,sequence)


###########
# Wolfguy #
###########
# The Wolfguy numbering scheme is an in-house scheme used at Roche. It has been described publicly in the paper:
# Prediction of VH-VL domain orientation for antibody variable domain modeling. Bujotzek A. et al. Protein 2015 83(4) 681-95
#
# It is similar in gapping as IMGT and is defined only for heavy and light antibody chains. 
# Unlike other schemes the numbering denotes both the chain (heavy 101-499, light 501-799) and the region (less than -50 framework
# greater than -50 CDR). All CDRs of length less than 50 can be handled without the need for insertion codes. Numbering of the 
# framework behaves similarly to IMGT in that all positions are assumed to be accounted for. Framework insertions are placed by 
# the alignment. 
#
# Numbering of all CDRs is performed symmetrically with the exception of CDRL1. In this case the CDR is numbered according to a 
# pattern specific to the canonical class. This is recognised by length and by sequence similarity to a consensus sequence. If a 
# length has not been observed it is numbered symmetrically. 


def number_wolfguy_heavy(state_vector, sequence):
    """
    Apply the wolfguy numbering scheme for heavy chains 

    The scheme numbers the sequence using different segments so that the numbering tells you
    where in the antibody the sequence is describing. 

    XXXXXXXXXIXXXXXXXXXXXXXXXX XXXXXXXXXXXXXX XXXXXXXXXXXXXX XXXXXXXXXXXXXXXXXXIX XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXX
    11111111111111111111111111 22222222222222 33333333333333 44444444444444444444 555555555555555555555555555555 6666666666666 77777777777'
    
    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Simple mapping (treat "I" states as inserts and not own match states)
     2  -  CDRH1 - 155-199 (inc). Gap symmetrically about 175-176.
     3  -  Simple mapping (treat "I" states as inserts and not own match states)
     4  -  CDRH2 - 251-299 (inc). Gap symmetrically about 271-272, then gap back from 294.
     5  -  Simple mapping (treat "I" states as inserts and not own match states)
     6  -  CDRH3 331,332 and 351-399 (inc). Gap according to the  
     7  -  Simple mapping (treat "I" states as inserts and not own match states)
    
     Start gaps on rhs each time.
    """
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111111111111111111111222222222222223333333333333344444444444444444444555555555555555555555555555555666666666666677777777777'

    region_index_dict = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:100, 
                         1:124,
                         2:160,
                         3:196,
                         4:226,
                         5:244,
                         6:283}    
    
    n_regions = 7
    
    exclude_deletions = [1,3,5]

    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    ###############
    # Renumbering #
    ###############
        
    # Renumbering required for 1, 3, 5 regions in wolfguy heavy    
    _numbering = [ _regions[0], [] , _regions[2], [], _regions[4] , [], _regions[6] ]

    # CDRH1
    # Delete symmetrically about 177. Delete right first.
    # May have to change this to reflect where the point of symmetry is
    ordered_deletions = [151]
    for p1,p2 in zip( list(range(152,176)), list(range(199, 175,-1))): ordered_deletions += [ p1,p2 ]
    length = len( _regions[1] )
    annotations = sorted(ordered_deletions[:length])
    _numbering[1]  = [ ((annotations[i]," "), _regions[1][i][1]) for i in range(length)  ]
    
    # CDRH2
    # Delete symmetrically about 271. Delete right first.
    # Then delete right from 288
    ordered_deletions = [251]
    for p1,p2 in zip( list(range(252,271)), list(range(290, 271,-1))): ordered_deletions += [ p1,p2 ]
    ordered_deletions.append( 271 )
    ordered_deletions = list(range( 299, 290, -1)) + ordered_deletions
    length = len( _regions[3] )
    annotations = sorted(ordered_deletions[:length])
    _numbering[3]  = [ ((annotations[i]," "), _regions[3][i][1]) for i in range(length)  ]

    # CDRH3	
    # Delete symmetrically about 374. Delete right first. 
    # Scheme changes at length 8
    # Scheme changes at length 12
    ordered_deletions = []
    for p1,p2 in zip( list(range(356,374)), list(range(391, 373,-1))): ordered_deletions += [ p1,p2 ]
    ordered_deletions = [ 354, 394, 355, 393, 392 ] + ordered_deletions 
    ordered_deletions = [331,332] + [ 399, 398, 351, 352, 397, 353, 396, 395 ] + ordered_deletions
    length = len( _regions[5] )

    if length > len(ordered_deletions): return [], startindex, endindex # Too many insertions. Do not apply numbering.     
    annotations = sorted(ordered_deletions[:length])
    _numbering[5]  = [ ((annotations[i]," "), _regions[5][i][1]) for i in range(length)  ]    
  
    # Return the full vector and the start and end indices of the numbered region of the sequence
    return sum( _numbering, [] ), startindex, endindex   
            

def number_wolfguy_light(state_vector, sequence):
    """
    Apply the wolfguy numbering scheme for light chains 

    The scheme numbers the sequence using different segments so that the numbering tells you
    where in the antibody the sequence is describing. 

    XXXXXXX XXX XXXXXXXXXXXXX XXXXXXXXXXXXXXXXX XXXXXXXXXXXXXXX XXXXXXXXXXXXXX XXXIXXXXXXX XXXX XXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXX
    1111111 AAA BBBBBBBBBBBBB 22222222222222222 333333333333333 44444444444444 55555555555 6666 77777777777777777777 8888888888888 99999999999

    Regions - (N.B These do not match up with any particular definition of CDR)
     1  -  Simple mapping (treat "I" states as inserts and not own match states)
     A  -  Move indels onto 508
     B  -  Simple mapping (treat "I" states as inserts and not own match states)
     2  -  CDRL1 - 551-599 (inc). Assign via the matching consensus sequence and length.
     3  -  Simple mapping (treat "I" states as inserts and not own match states)
     4  -  CDRL2 - 651-699 (inc). Gap about 673 then right from 694
     5  -  Simple mapping (treat "I" states as inserts and not own match states)
     6  -  Move indels onto 713 and 714
     7  -  Simple mapping (treat "I" states as inserts and not own match states)
     8  -  CDRL3 751-799 (inc). Gap symmetrically about 374-375 
     9  -  Simple mapping (treat "I" states as inserts and not own match states)
    
    """
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
                    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '1111111AAABBBBBBBBBBBBB222222222222222223333333333333334444444444444455555555555666677777777777777777777888888888888899999999999'

    region_index_dict = {"1":0,"A":1,"B":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:500,
                         1:500,
                         2:500,    
                         3:527,
                         4:560,
                         5:595,
                         6:631,
                         7:630,
                         8:630,                                                  
                         9:646,
                         10:683}    
    
    n_regions = 11
    
    exclude_deletions = [1,3,5,7,9]

    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    ###############
    # Renumbering #
    ###############
        
    # Renumbering required for 1, 3, 5 regions in wolfguy heavy    
    _numbering = [ _regions[0], [], _regions[2], [] , _regions[4], [], _regions[6], [], _regions[8], [], _regions[10] ]


    # Gaps in the first section go 508 instead of the imgt 510 equivalent
    length = len(_regions[1] )
    annotations = sorted([ (510,' '), (509, ' '), (508, ' ')][ :length ] + [(508,a) for a in alphabet[:max(0, length-3)]])  
    _numbering[1]  = [ (annotations[i], _regions[1][i][1]) for i in range(length)  ]
    
    # CDRL1
    # Number by predicting the canonical 
    length = len(_regions[3] )
    annotations = _get_wolfguy_L1( _regions[3], length)
    _numbering[3]  = [ ((annotations[i]," "), _regions[3][i][1]) for i in range(length)  ]
    
    # CDRL2
    # Delete about 673. Finally delete right from 694. Maintain 651 as the last deletion
    ordered_deletions = []
    for p1,p2 in zip( list(range(652,673)), list(range(694, 672,-1))): ordered_deletions += [ p2,p1 ]
    ordered_deletions = [651] + list(range( 699, 694, -1)) + ordered_deletions + [673]

    length = len( _regions[5] )
    annotations = sorted(ordered_deletions[:length])
    _numbering[5]  = [ ((annotations[i]," "), _regions[5][i][1]) for i in range(length)  ]


    # The placement of the indel in wolfguy is different to that in imgt
    length = len( _regions[7] )
    insertions = max( 0, length - 4 )
    annotations = [(711, ' '), (712, ' '), (713, ' '), (714, ' ')][:length] + [ (714, a) for a in alphabet[:insertions] ]    
    _numbering[7]  = [ (annotations[i], _regions[7][i][1]) for i in range(length)  ]  
    
    # CDRL3
    # Delete symmetrically about 775. Delete right first. Finally delete 798 and 799
    ordered_deletions = []
    for p1,p2 in zip( list(range(751,775)), list(range(799, 775,-1))): ordered_deletions += [ p1,p2 ]
    ordered_deletions.append( 775 )
  
    length = len( _regions[9] )
    if length > len(ordered_deletions): return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    annotations = sorted(ordered_deletions[:length])
    _numbering[9]  = [ ((annotations[i]," "), _regions[9][i][1]) for i in range(length)  ]  
  
    # Return the full vector and the start and end indices of the numbered region of the sequence
    return sum( _numbering, [] ), startindex, endindex  


def _get_wolfguy_L1(seq, length):
    """
    Wolfguy's L1 annotation is based on recognising the length and the sequence pattern defined
    by a set of rules. If the length has not been characterised, we number symmetrically about the
    middle of the loop.
    """
    
    # These are the annotations for different lengths of L1 according to the wolfguy definitions.
    L1_sequences = {
    9: [['9',     'XXXXXXXXX', [551, 552, 554, 556, 563, 572, 597, 598, 599]]], 
    10: [['10',   'XXXXXXXXXX', [551, 552, 553, 556, 561, 562, 571, 597, 598, 599]]], 
    11: [['11a',  'RASQDISSYLA', [551, 552, 553, 556, 561, 562, 571, 596, 597, 598, 599]], 
         ['11b',  'GGNNIGSKSVH', [551, 552, 554, 556, 561, 562, 571, 572, 597, 598, 599]], 
         ['11b.2','SGDQLPKKYAY', [551, 552, 554, 556, 561, 562, 571, 572, 597, 598, 599]]], 
    12: [['12a',  'TLSSQHSTYTIE', [551, 552, 553, 554, 555, 556, 561, 563, 572, 597, 598, 599]], 
         ['12b',  'TASSSVSSSYLH', [551, 552, 553, 556, 561, 562, 571, 595, 596, 597, 598, 599]], 
         ['12c',  'RASQSVxNNYLA', [551, 552, 553, 556, 561, 562, 571, 581, 596, 597, 598, 599]], 
         ['12d',  'rSShSIrSrrVh', [551, 552, 553, 556, 561, 562, 571, 581, 596, 597, 598, 599]]], 
    13: [['13a',  'SGSSSNIGNNYVS', [551, 552, 554, 555, 556, 557, 561, 562, 571, 572, 597, 598, 599]], 
         ['13b',  'TRSSGSLANYYVQ', [551, 552, 553, 554, 556, 561, 562, 563, 571, 572, 597, 598, 599]]], 
    14: [['14a',  'RSSTGAVTTSNYAN', [551, 552, 553, 554, 555, 561, 562, 563, 564, 571, 572, 597, 598, 599]], 
         ['14b',  'TGTSSDVGGYNYVS', [551, 552, 554, 555, 556, 557, 561, 562, 571, 572, 596, 597, 598, 599]]], 
    15: [['15',   'XXXXXXXXXXXXXXX', [551, 552, 553, 556, 561, 562, 563, 581, 582, 594, 595, 596, 597, 598, 599]]], 
    16: [['16',   'XXXXXXXXXXXXXXXX', [551, 552, 553, 556, 561, 562, 563, 581, 582, 583, 594, 595, 596, 597, 598, 599]]], 
    17: [['17',   'XXXXXXXXXXXXXXXXX', [551, 552, 553, 556, 561, 562, 563, 581, 582, 583, 584, 594, 595, 596, 597, 598, 599]]]
    }    

    if length in L1_sequences: # Use the pre-defined motif 
        # Find the maximum scoring canonical form for this length. 
        curr_max = None, -10000
        for canonical in L1_sequences[length]:
            sub_score = 0
            for i in range( length ):
                try:
                    sub_score += blosum62[ (seq[i][1].upper(), canonical[1][i].upper() ) ]
                except KeyError:
                    sub_score += blosum62[ (canonical[1][i].upper(), seq[i][1].upper() ) ]
            if sub_score > curr_max[1]:
                curr_max = canonical, sub_score

        # return the annotations
        return curr_max[0][2]
    else: # Use a symmetric numbering about the anchors.
        ordered_deletions = []
        for p1,p2 in zip( list(range(551,575)), list(range(599, 575,-1))): ordered_deletions += [ p2,p1 ]
        ordered_deletions.append(575)
        return sorted( ordered_deletions[:length] )

def gap_missing( numbering ):
    '''
    Place gaps when a number is missing. All except wolfguy are continuously numbered
    '''
    # Gaps placed where a number is not present
    num = [ ((0,' '),'-') ]
    for p, a in sum( numbering, [] ):  
        if p[0] > num[-1][0][0]+1:
            for _i in range( num[-1][0][0]+1, p[0] ):
                num.append( ((_i, ' '), '-' ) )
        num.append( (p,a) )
    return num[1:]


######################
# Annotation of CDR3 #
######################
    
def get_cdr3_annotations(length, scheme="imgt", chain_type=""):
    """
    Given a length of a cdr3 give back a list of the annotations that should be applied to the sequence.
    
    This function should be depreciated
    """
    az = "ABCDEFGHIJKLMNOPQRSTUVWXYZ" 
    za = "ZYXWVUTSRQPONMLKJIHGFEDCBA"
    
    if scheme=="imgt":
        start, end = 105, 118 # start (inclusive) end (exclusive)
        annotations = [None for _ in range(max(length,13))]
        front = 0
        back  = -1
        assert (length-13) < 50, "Too many insertions for numbering scheme to handle" # We ran out of letters.
        for i in range(min(length,13)):
            if i%2:
                annotations[back] = (end+back, " ")
                back -= 1
            else:
                annotations[front] = (start+front, " ")
                front += 1
        for i in range(max(0,length-13)): # add insertions onto 111 and 112 in turn
            if i%2:
                annotations[back] = (112, za[back+6])
                back-=1
            else:
                annotations[front] = (111, az[front-7])
                front +=1        
        return annotations

    elif scheme in [ "chothia", "kabat"] and chain_type=="heavy": # For chothia and kabat
        # Number forwards from 93
        insertions = max(length - 10, 0)
        assert insertions < 27, "Too many insertions for numbering scheme to handle" # We ran out of letters.
        ordered_deletions = [ (100, ' '), (99,' '), (98,' '), (97,' '), (96,' '), (95,' '), (101,' '),(102,' '),(94,' '), (93,' ') ]
        annotations = sorted( ordered_deletions[ max(0, 10-length): ] + [ (100,a) for a in az[:insertions ] ] )
        return annotations

    elif scheme in [ "chothia", "kabat"] and chain_type=="light":
        # Number forwards from 89
        insertions = max(length - 9, 0)
        assert insertions < 27, "Too many insertions for numbering scheme to handle" # We ran out of letters.
        ordered_deletions = [ (95,' '),(94,' '),(93,' '),( 92,' '),(91,' '),(96,' '),(97,' '),(90,' '),(89,' ') ]
        annotations = sorted( ordered_deletions[ max(0, 9-length): ] + [ (95,a) for a in az[:insertions ] ] )
        return annotations

    else:
        raise AssertionError("Unimplemented scheme")



## Functions in `anarci.py`

In [209]:
class HMMscanError(Exception):
    def __init__(self, message):
        # Call the base class constructor with the parameters it needs
        super(HMMscanError, self).__init__(message)

## Utility functions ##
def read_fasta(filename):
    """
    Read a sequence file and parse as description, string
    """
    return [ r for r in fasta_iter(filename) ]

def fasta_iter(fasta_name):
    """
    Given a fasta file. yield tuples of header, sequence
    https://www.biostars.org/p/710/
    """
    if fasta_name.endswith( '.gz' ): # IOError raised upon iteration if not a real gzip file.
        fh = gzip.open(fasta_name)
    else:
        fh = open(fasta_name)
    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))
    for header in faiter:
        header = next(header)[1:].strip()
        #header = header.next()[1:].strip()
        seq = "".join(s.strip() for s in next(faiter))
        yield header, seq


def write_fasta(sequences, f):
    """
    Write a list of sequences to file.

    should be a list of name, sequence tuples

    f should be an open file
    """
    for name, sequence in sequences:
        print(">%s"%name, file=f)
        print('\n'.join(['\n'.join(wrap(block, width=80)) for block in sequence.splitlines()]), file=f)


def validate_sequence(sequence):
    """
    Check whether a sequence is a protein sequence or if someone has submitted something nasty.
    """
    assert len(sequence) < 10000, "Sequence too long."
    assert not (set( sequence.upper() ) - set_amino_acids ), "Unknown amino acid letter found in sequence: %s"% ", ".join(list((set( sequence.upper() ) - set_amino_acids )))
    return True

def validate_numbering(xxx_todo_changeme, name_seq=[]):
    """
    Wrapper to do some basic validation of the numbering.

    Further validation could be done but at the moment we just check that the numbering indices are incremental (they should be)
    """
    (numbering, start, end) = xxx_todo_changeme
    name, seq = name_seq
    last = -1
    nseq=""

    for (index, _), a in numbering:
        assert index >= last, "Numbering was found to decrease along the sequence %s. Please report."%name
        last = index
        nseq += a.replace("-","")

    assert nseq in seq.replace("-",""), "The algorithm did not number a contiguous segment for sequence %s. Please report"%name

    return numbering, start, end

def grouper(n, iterable):
    '''
    Group entries of an iterable by n
    '''
    it = iter(iterable)
    def take():
        while 1:
            yield list( islice(it,n) )
    return iter(take().__next__, [] )

def anarci_output(numbered, sequences, alignment_details, outfile, sequence_id=None, domain_id=None):
    """
    Outputs to open file

    If sequence_id is specified as an integer then only this sequence will be printed.
    Otherwise all sequences will be printed.

    If domain_id is specified as an integer then only this domain will be printed.
    Otherwise all domains will be printed.

    If domain_id is specified then sequence_id must also be specified.
    """
    assert (sequence_id is not None) or (sequence_id is None and domain_id is None), "If domain_id is specified, sequence_id must also be specified."
    for i in range(len(numbered)):
        if sequence_id is None:
            print("# %s"%sequences[i][0], file=outfile) # print the name
        if numbered[i] is not None:
            if sequence_id is not None:
                if i != sequence_id: continue
            print("# ANARCI numbered", file=outfile)
            for j in range( len(numbered[i])): # Iterate over domains
                if domain_id is not None:
                    if j != domain_id: continue
                print("# Domain %d of %d"%(j+1, len(numbered[i]) ), file=outfile)
                print("# Most significant HMM hit", file=outfile)
                print("#|species|chain_type|e-value|score|seqstart_index|seqend_index|", file=outfile)
                alignment_details[i][j]["evalue"] = str( alignment_details[i][j]["evalue"] )
                print("#|%s|%s|%s|%.1f|%d|%d|"%tuple( [alignment_details[i][j][field] for field in
                                                                     ["species","chain_type","evalue","bitscore"]]
                                                                   +[ numbered[i][j][1], numbered[i][j][2]] ), file=outfile)

                if 'germlines' in alignment_details[i][j]:
                    print('# Most sequence-identical germlines', file=outfile)
                    print('#|species|v_gene|v_identity|j_gene|j_identity|', file=outfile)
                    (species, vgene), vid =alignment_details[i][j]['germlines'].get('v_gene', [['','unknown'],0])
                    if vgene is None:
                        vgene, vid = 'unknown', 0
                    (_,jgene), jid =alignment_details[i][j]['germlines'].get('j_gene', [['','unknown'],0])
                    if jgene is None:
                        jgene, jid = 'unknown', 0
                    print('#|%s|%s|%.2f|%s|%.2f|'%(species, vgene, vid, jgene, jid ), file=outfile)
                chain_type = chain_type_to_class[  alignment_details[i][j]["chain_type"] ]
                print("# Scheme = %s"%alignment_details[i][j]["scheme"], file=outfile)
                if len( numbered[i][j][0] ) == 0:
                    print("# Warning: %s scheme could not be applied to this sequence."%alignment_details[i][j]["scheme"], file=outfile)
                for (index, insertion), aa in numbered[i][j][0]:
                    print(chain_type, ("%d"%index).ljust(5), insertion, aa, file=outfile)
        print("//", file=outfile)

def csv_output(sequences, numbered, details, outfileroot):
    '''
    Write numbered sequences to csv files. A csv file is written for each chain type.

    Kappa and Lambda chains are written to the same file

    The sequences will written aligned to the numbering scheme. Gaps in the sequences with respect to the alignment are written
    as a '-'

    @param sequences: List of name, sequence tuples
    @param numbered: Numbered sequences in the same order as the sequences list.
    @param details: List of alignment details in the same order as the sequences list.
    @param outfileroot: The file path for csv files to write. _<chain_type>.csv will be appended to this.
    '''

    chain_types = {}
    pos_ranks = {}
    all_pos = {}
    _lc = {'K':'KL','L':'KL'}


    # Divide the set into chain types and find how to order the numbering for each type.
    for i in range( len(sequences) ): # Iterate over entries
        if numbered[i] is None: continue

        for j in range(len(numbered[i])): # Iterate over domains.
            # Record the chain type index
            c = details[i][j]['chain_type']
            c = _lc.get(c, c) # Consider lambda and kappa together.
            chain_types.setdefault( c, [] ).append( (i,j) )
            if c not in pos_ranks:
                pos_ranks[c] = {}
                all_pos[c] = set()

            # Update the insertion order for the scheme. i.e. is it A B C or C B A (e.g. imgt 111 and 112 repectively)
            l = -1
            r = 0
            for p, _ in numbered[i][j][0]:
                if p[0] != l:
                    l = p[0]
                    r = 0
                else:
                    r +=1
                pos_ranks[c][p] = max( r, pos_ranks[c].get( p, r ) )
                all_pos[c].add( p )

    # Write a new file for each chain type. Kappa and lambda are written together as light chains.
    for cts in ['H','KL','A','B','G','D']:
        if cts in chain_types:
            with open( outfileroot + '_%s.csv'%cts, 'w' ) as out:

                # Sort the positions by index and insertion order
                positions = sorted( all_pos[cts], key = lambda p: (p[0], pos_ranks[cts][p]) )

                # Header line
                fields = ['Id','domain_no','hmm_species','chain_type','e-value','score','seqstart_index','seqend_index',
                          'identity_species','v_gene','v_identity','j_gene','j_identity']
                fields += [ ('%d%s'%(p)).strip() for p in positions ]
                print(','.join( fields ), file=out)

                # Iterate over the domains identified
                for i,j in chain_types[cts]:
                    line = [ sequences[i][0].replace(',',' '),
                             str(j),
                             details[i][j].get('species',''),
                             details[i][j].get('chain_type',''),
                             str(details[i][j].get('evalue','')),
                             str(details[i][j].get('bitscore','')),
                             str(numbered[i][j][1]),
                             str(numbered[i][j][2]),
                             details[i][j].get('germlines',{}).get( 'v_gene',[['',''],0] )[0][0],
                             details[i][j].get('germlines',{}).get( 'v_gene',[['',''],0] )[0][1],
                             '%.2f'%details[i][j].get('germlines',{}).get( 'v_gene',[['',''],0] )[1],
                             details[i][j].get('germlines',{}).get( 'j_gene',[['',''],0] )[0][1],
                             '%.2f'%details[i][j].get('germlines',{}).get( 'j_gene',[['',''],0] )[1] ]

                    # Hash the numbering. Insertion order has been preserved in the positions sort.
                    d = dict( numbered[i][j][0] )
                    line += [ d.get(p,'-') for p in positions ]

                    assert len( line ) == len( fields )
                    print(','.join( line ), file=out)



## Parsing and recognising domain hits from hmmscan ##
def _domains_are_same(dom1, dom2):
    """
    Check to see if the domains are overlapping.
    @param dom1:
    @param dom2:

    @return: True or False
    """
    dom1, dom2 = sorted( [dom1, dom2], key=lambda x: x.query_start  )
    if dom2.query_start >= dom1.query_end:
        return False
    return True


def _parse_hmmer_query(query, bit_score_threshold=80, hmmer_species=None):
    """

    @param query: hmmer query object from Biopython
    @param bit_score_threshold: the threshold for which to consider a hit a hit.

    The function will identify multiple domains if they have been found and provide the details for the best alignment for each domain.
    This allows the ability to identify single chain fvs and engineered antibody sequences as well as the capability in the future for identifying constant domains.

    """
    hit_table = [ ['id', 'description', 'evalue', 'bitscore', 'bias',
                    'query_start', 'query_end' ] ]

    # Find the best hit for each domain in the sequence.

    top_descriptions, domains,state_vectors = [], [], []

    if query.hsps: # We have some hits
        # If we have specified a species, check to see we have hits for that species
        # Otherwise revert back to using any species
        if hmmer_species:
            #hit_correct_species = [hsp for hsp in query.hsps if hsp.hit_id.startswith(hmmer_species) and hsp.bitscore >= bit_score_threshold]
            hit_correct_species = []
            for hsp in query.hsps:
                if hsp.bitscore >= bit_score_threshold:
                    for species in hmmer_species:
                        if hsp.hit_id.startswith(species):
                            hit_correct_species.append(hsp)

            if hit_correct_species:
                hsp_list = hit_correct_species
            else:
                print("Limiting hmmer search to species %s was requested but hits did not achieve a high enough bitscore. Reverting to using any species" %(hmmer_species))
                hsp_list = query.hsps
        else:
            hsp_list = query.hsps

        for hsp in sorted(hsp_list, key=lambda x: x.evalue): # Iterate over the matches of the domains in order of their e-value (most significant first)
            new=True
            if hsp.bitscore >= bit_score_threshold: # Only look at those with hits that are over the threshold bit-score.
                for i in range( len(domains) ): # Check to see if we already have seen the domain
                    if _domains_are_same( domains[i], hsp ):
                        new = False
                        break
                hit_table.append( [ hsp.hit_id, hsp.hit_description, hsp.evalue, hsp.bitscore, hsp.bias, hsp.query_start, hsp.query_end] )
                if new: # It is a new domain and this is the best hit. Add it for further processing.
                    domains.append( hsp )
                    top_descriptions.append(  dict( list(zip(hit_table[0], hit_table[-1])) ) ) # Add the last added to the descriptions list.

        # Reorder the domains according to the order they appear in the sequence.
        ordering = sorted( list(range(len(domains))), key=lambda x: domains[x].query_start)
        domains = [ domains[_] for _ in ordering ]
        top_descriptions = [ top_descriptions[_] for _ in ordering ]

    ndomains = len( domains )
    for i in range(ndomains): # If any significant hits were identified parse and align them to the reference state.
        domains[i].order = i
        
        info_lst=top_descriptions[i]["id"].split("_")
        species=info_lst[0]
        chain = "_".join(info_lst[1:]) 
        
        state_vectors.append( _hmm_alignment_to_states(domains[i], ndomains, query.seq_len) ) # Alignment to the reference states.
        top_descriptions[i][ "species"] = species # Reparse
        top_descriptions[i][ "chain_type"] = chain
        top_descriptions[i][ "query_start"] = state_vectors[-1][0][-1] # Make sure the query_start agree if it was changed

    return hit_table, state_vectors, top_descriptions


def _hmm_alignment_to_states(hsp, n, seq_length):
    """
    Take a hit hsp and turn the alignment into a state vector with sequence indices
    """

    # Extract the strings for the reference states and the posterior probability strings
    reference_string = hsp.aln_annotation["RF"]
    state_string = hsp.aln_annotation["PP"]

    assert len(reference_string) == len(state_string), "Aligned reference and state strings had different lengths. Don't know how to handle"

    # Extract the start an end points of the hmm states and the sequence
    # These are python indices i.e list[ start:end ] and therefore start will be one less than in the text file
    _hmm_start = hsp.hit_start
    _hmm_end = hsp.hit_end

    _seq_start = hsp.query_start
    _seq_end = hsp.query_end

    # Extact the full length of the HMM hit
    info_lst=hsp.hit_id.split('_')
    species=info_lst[0]
    ctype = "_".join(info_lst[1:])
    
    _hmm_length = get_hmm_length( species, ctype )

    # Handle cases where there are n terminal modifications.
    # In most cases the user is going to want these included in the numbered domain even though they are not 'antibody like' and
    # not matched to the germline. Only allow up to a maximum of 5 unmatched states at the start of the domain
    # Adds a bug here if there is a very short linker between a scfv domains with a modified n-term second domain
    # Thus this is only done for the first identified domain ( hence order attribute on hsp )
    if hsp.order == 0 and _hmm_start and _hmm_start < 5:
        n_extend = _hmm_start
        if _hmm_start > _seq_start:
            n_extend = min( _seq_start , _hmm_start - _seq_start )
        state_string = '8'*n_extend + state_string
        reference_string = 'x'*n_extend + reference_string
        _seq_start = _seq_start - n_extend
        _hmm_start = _hmm_start - n_extend

    # Handle cases where the alignment should be extended to the end of the j-element
    # This occurs when there a c-terminal modifications of the variable domain that are significantly different to germline
    # Extension is only made when half of framework 4 has been recognised and there is only one domain recognised.
    if n==1 and _seq_end < seq_length and (123 < _hmm_end < _hmm_length): # Extend forwards
        n_extend = min( _hmm_length - _hmm_end, seq_length - _seq_end )
        state_string = state_string + '8'*n_extend
        reference_string = reference_string + 'x'*n_extend
        _seq_end = _seq_end + n_extend
        _hmm_end = _hmm_end + n_extend



    # Generate lists for the states and the sequence indices that are included in this alignment
    hmm_states = all_reference_states[ _hmm_start : _hmm_end ]
    sequence_indices = list(range(_seq_start,  _seq_end))
    h, s = 0, 0 # initialise the current index in the hmm and the sequence

    state_vector = []
    # iterate over the state string (or the reference string)
    for i in range( len(state_string) ):
        if reference_string[i] == "x": # match state
            state_type = "m"
        else: # insert state
            state_type = "i"

        if state_string[i] == ".": # overloading if deleted relative to reference. delete_state
            state_type = "d"
            sequence_index = None
        else:
            sequence_index = sequence_indices[s]
        # Store the alignment as the state identifier (uncorrected IMGT annotation) and the index of the sequence

        state_vector.append(  ((hmm_states[h], state_type),  sequence_index )  )

        # Updates to the indices
        if state_type == "m":
            h+=1
            s+=1
        elif state_type == "i":
            s+=1
        else: # delete state
            h+=1

    return state_vector


def parse_hmmer_output(filedescriptor="", bit_score_threshold=80, hmmer_species=None):
    """
    Parse the output of HMMscan and return top alignment and the score table for each input sequence.
    """
    results  = []
    if type(filedescriptor) is str:
        openfile = open
    elif type(filedescriptor) is int:
        openfile = os.fdopen

    with openfile(filedescriptor) as inputfile:
        p = HMMERParser( inputfile )
        for query in p:
            results.append(_parse_hmmer_query(query,bit_score_threshold=bit_score_threshold,hmmer_species=hmmer_species ))

    return results


def run_hmmer(sequence_list,hmm_database="ALL",hmmerpath="", ncpu=None, bit_score_threshold=80, hmmer_species=None):
    """
    Run the sequences in sequence list against a precompiled hmm_database.

    Those sequence that have a significant hit with a bit score over a threshold will
    be recognised and an alignment given. The alignment will be used to number the
    sequence.

    @param sequence_list: a list of (name, sequence) tuples. Both are strings
    @param hmm_database: The hmm database to use. Currently, all hmms are in the ALL database.
                         The code to develop new models is in build_pipeline in the git repo.
    @param hmmerpath: The path to hmmer binaries if not in the path
    @param ncpu: The number of cpu's to allow hmmer to use.
    """

    # Check that hmm_database is available

    assert hmm_database in ["ALL","ALL_AND_C","C_ONLY"], "Unknown HMM database %s"%hmm_database
    HMM = os.path.join( HMM_path, "%s.hmm"%hmm_database )


    # Create a fasta file for all the sequences. Label them with their sequence index
    # This will go to a temp file
    fasta_filehandle, fasta_filename =  tempfile.mkstemp( ".fasta", text=True )
    with os.fdopen(fasta_filehandle,'w') as outfile:
        write_fasta(sequence_list, outfile)

    output_filehandle, output_filename =  tempfile.mkstemp( ".txt", text=True )

    # Run hmmer as a subprocess
    if hmmerpath:
        hmmscan = os.path.join(hmmerpath,"hmmscan")
    else:
        hmmscan = "hmmscan"
    try:
        if ncpu is None:
            command = [ hmmscan, "-o", output_filename, HMM,  fasta_filename]
        else:
            command = [ hmmscan, "-o", output_filename, "--cpu", str(ncpu), HMM,  fasta_filename]
        process = Popen( command, stdout=PIPE, stderr=PIPE  )
        _, pr_stderr = process.communicate()

        if pr_stderr:
            _f = os.fdopen(output_filehandle) # This is to remove the filedescriptor from the os. I have had problems with it before.
            _f.close()

            raise HMMscanError(pr_stderr)
        results = parse_hmmer_output(output_filehandle, bit_score_threshold=bit_score_threshold, hmmer_species=hmmer_species)

    finally:
        # clear up
        os.remove(fasta_filename)
        os.remove(output_filename)

    return results

def get_hmm_length( species, ctype ):
    '''
    Get the length of an hmm given a species and chain type.
    This tells us how many non-insertion positions there could possibly be in a domain (127 or 128 positions under imgt)
    '''
    try:
        return len(list(all_germlines['J'][ctype][species].values())[0].rstrip('-'))
    except KeyError:
        return 128


def number_sequence_from_alignment(state_vector, sequence, scheme="imgt", chain_type=None):
    """
    Given you have an alignment. Give back the numbering

    @param state_vector: List of states from the hmm. Effectively these are imgt columns but CDR3 has not been redone.
    @param sequence: The original sequence string or list.
    @param scheme: The numbering scheme to apply
    @param chain_type: The type of chain to apply numbering for. Some schemes do not require this (IMGT). Others (e.g. Chothia/Wolfguy) do.

    @return: A list of numbering identifier / amino acids tuples over the domain that has been numbered. The indices of the start (inclusive) and end point (exclusive) in the sequence for the numbering
    """
    scheme=scheme.lower()
    if scheme == "imgt":
        return number_imgt(state_vector, sequence)
    elif scheme == "chothia":
        if chain_type == "H":
            return number_chothia_heavy(state_vector, sequence)
        elif chain_type in "KL":
            return number_chothia_light(state_vector, sequence)
        else:
            raise AssertionError("Unimplemented numbering scheme %s for chain %s"%( scheme, chain_type))
    elif scheme == "kabat":
        if chain_type == "H":
            return number_kabat_heavy(state_vector, sequence)
        elif chain_type in "KL":
            return number_kabat_light(state_vector, sequence)
        else:
            raise AssertionError("Unimplemented numbering scheme %s for chain %s"%( scheme, chain_type))
    elif scheme == "martin":
        if chain_type == "H":
            return number_martin_heavy(state_vector, sequence)
        elif chain_type in "KL":
            return number_martin_light(state_vector, sequence)
        else:
            raise AssertionError("Unimplemented numbering scheme %s for chain %s"%( scheme, chain_type))
    elif scheme == "aho":
        return number_aho(state_vector, sequence, chain_type) # requires the chain type to heuristically put the CDR1 gap in position.
    elif scheme == "wolfguy":
        if chain_type == "H":
            return number_wolfguy_heavy( state_vector, sequence )
        elif chain_type in "KL":
            return number_wolfguy_light( state_vector, sequence )
        else:
            raise AssertionError("Unimplemented numbering scheme %s for chain %s"%( scheme, chain_type))
    else:
        raise AssertionError("Unimplemented numbering scheme %s for chain %s"%( scheme, chain_type))

def number_sequences_from_alignment(sequences, alignments, scheme="imgt", allow=set(["H","K","L","A","B","G","D"]),
                                    assign_germline=False, allowed_species=None):
    '''
    Given a list of sequences and a corresponding list of alignments from run_hmmer apply a numbering scheme.
    '''

    # Iteration over the sequence alignments performing the desired numbering
    numbered = []
    alignment_details = []
    hit_tables = []
    for i in range(len(sequences)):

        # Unpack
        hit_table, state_vectors, detailss = alignments[i] # We may have multiple domains per sequence (e.g. single chain fvs).

        # Iterate over all the domains in the sequence that have been recognised (typcially only 1 with the current hmms available)
        hit_numbered, hit_details = [], []
        for di in range( len( state_vectors ) ):
            state_vector = state_vectors[di]
            details      = detailss[di]
            details["scheme"]=scheme
            details["query_name"]=sequences[i][0]

            # Only number things that are allowed. We still keep the alignment details and hit_table
            if state_vector and details["chain_type"] in allow:
                try:
                    # Do the numbering and validate (for development purposes)
                    hit_numbered.append( validate_numbering(number_sequence_from_alignment(state_vector, sequences[i][1],
                                                            scheme=scheme, chain_type=details["chain_type"]), sequences[i] ) )
                    if assign_germline:
                        details["germlines"] = run_germline_assignment( state_vector, sequences[i][1],
                                                                        details["chain_type"], allowed_species=allowed_species)
                    hit_details.append( details )
                except AssertionError as e: # Handle errors. Those I have implemented should be assertion.
                    print(str(e), file=sys.stderr)
                    raise e # Validation went wrong. Error message will go to stderr. Want this to be fatal during development.
                except Exception as e:
                    print("Error: Something really went wrong that has not been handled", file=sys.stderr)
                    print(str(e), file=sys.stderr)
                    raise e

        if hit_numbered:
            numbered.append( hit_numbered )
            alignment_details.append( hit_details )
        else:
            numbered.append( None )
            alignment_details.append( None )
        hit_tables.append(hit_table)

    return numbered, alignment_details, hit_tables

def get_identity( state_sequence, germline_sequence ):
    """
    Get the partially matched sequence identity between two aligned sequences.
    Partial in the sense that gaps can be in the state_sequence.
    """
    # Ensure that the sequences are the expected length
    assert len( state_sequence) == len(germline_sequence ) == 128
    n, m = 0, 0
    for i in range( 128 ):
        if germline_sequence[i] == "-":continue
        if state_sequence[i].upper() == germline_sequence[i]: m+=1
        n+=1

    if not n:
        return 0
    return float(m)/n


def run_germline_assignment(state_vector, sequence, chain_type, allowed_species=None ):
    """
    Find the closest sequence identity match.
    """
    genes={'v_gene': [None,None],
           'j_gene': [None,None],
         }


    # Extract the positions that correspond to match (germline) states.
    state_dict = dict( ((i, 'm'),None) for i in range(1,129))
    state_dict.update(dict(state_vector))
    state_sequence = "".join([ sequence[state_dict[(i, 'm')]] if state_dict[(i,'m')] is not None else "-" for i in range(1,129) ])

    # Iterate over the v-germline sequences of the chain type of interest.
    # The maximum sequence identity is used to assign the germline
    if chain_type in all_germlines["V"]:
        if allowed_species is not None:
            if not all( [ sp in all_germlines['V'][chain_type] for sp in allowed_species ] ): # Made non-fatal
                return {}
        else:
            allowed_species = all_species
        seq_ids = {}
        for species in allowed_species:
            if species not in all_germlines["V"][ chain_type ]: continue # Previously bug.
            for gene, germline_sequence in all_germlines["V"][ chain_type ][ species ].items():
                seq_ids[ (species, gene) ] = get_identity( state_sequence , germline_sequence )
        genes['v_gene' ][0] = max( seq_ids, key=lambda x: seq_ids[x] )
        genes['v_gene' ][1] = seq_ids[ genes['v_gene' ][0] ]

        # Use the assigned species for the v-gene for the j-gene.
        # This assumption may affect exotically engineered abs but in general is fair.
        species = genes['v_gene' ][0][0]
        if chain_type in all_germlines["J"]:
            if species in all_germlines["J"][chain_type]:
                seq_ids = {}
                for gene, germline_sequence in all_germlines["J"][ chain_type ][ species ].items():
                    seq_ids[ (species, gene) ] = get_identity( state_sequence , germline_sequence )
                genes['j_gene' ][0] = max( seq_ids, key=lambda x: seq_ids[x] )
                genes['j_gene' ][1] = seq_ids[ genes['j_gene' ][0] ]

    return genes

def check_for_j( sequences, alignments, scheme ):
    '''
    As the length of CDR3 gets long (over 30ish) an alignment that does not include the J region becomes more favourable.
    This leads to really long CDR3s not being numberable.

    To overcome this problem, when no J region is detected we try without the v region.
    '''
    for i in range( len( sequences ) ):
        # Check the alignment for J region
        if len(alignments[i][1]) ==1: # Only do for single domain chains.

            # Check whether a J region has been identified. If not check whether there is still a considerable amount of sequence
            # remaining.
            ali = alignments[i][1][0]

            # Find the last match position.
            last_state  = ali[-1][0][0]
            last_si     = ali[-1][1]
            if last_state < 120: # No or very little J region
                if last_si + 30 < len( sequences[i][1] ): # Considerable amount of sequence left...suspicious of a long CDR3
                    # Find the position of the conserved cysteine (imgt 104).
                    cys_si = dict( ali ).get( (104,'m'), None )
                    if cys_si is not None: # 104 found.

                        # Find the corresponding index in the alignment.
                        cys_ai = ali.index( ((104, 'm'), cys_si) )

                        # Try to identify a J region in the remaining sequence after the 104. A low bit score threshold is used.
                        _, re_states, re_details  = run_hmmer( [(sequences[i][0], sequences[i][1][cys_si+1:])],
                                                               bit_score_threshold=10 )[0]

                        # Check if a J region was detected in the remaining sequence.
                        if re_states and re_states[0][-1][0][0] >= 126 and re_states[0][0][0][0] <= 117:

                            # Sandwich the presumed CDR3 region between the V and J regions.

                            vRegion   = ali[:cys_ai+1]
                            jRegion   = [ (state, index+cys_si+1) for state, index in re_states[0] if state[0] >= 117 ]
                            cdrRegion = []
                            next = 105
                            for si in range( cys_si+1, jRegion[0][1] ):
                                if next >= 116:
                                    cdrRegion.append( ( (116, 'i'), si ) )
                                else:
                                    cdrRegion.append( ( (next, 'm'), si ) )
                                    next +=1

                            # Update the alignment entry.
                            alignments[i][1][0] = vRegion + cdrRegion + jRegion
                            alignments[i][2][0]['query_end'] = jRegion[-1][1] + 1



##################################
# High level numbering functions #
##################################

# Main function for ANARCI
# Name conflict with function, module and package is kept for legacy unless issues are reported in future.
def anarci(sequences, scheme="imgt", database="ALL", output=False, outfile=None, csv=False, allow=set(["H","K","L","A","B","G","D"]),
           hmmerpath="", ncpu=None, assign_germline=False, allowed_species=['human','mouse'], bit_score_threshold=80):
    """
    The main function for anarci. Identify antibody and TCR domains, number them and annotate their germline and species.

    It is advised to use one of the wrapper functions:
        o run_anarci   - fasta file or sequence list in. Automated multiprocessing for large jobs. Sequences, numbering, details
                         and hit tables out.
        o number       - single sequence in, numbering out


    @param sequences: A list or tuple of (Id, Sequence) pairs
                              e.g. [ ("seq1","EVQLQQSGAEVVRSG ..."),
                                     ("seq2","DIVMTQSQKFMSTSV ...") ]
    @param scheme:    The numbering scheme that should be applied. Choose from imgt, chothia, kabat or martin
    @param output:    Boolean flag to say whether the result should be output.
    @param outfile:   The name of the file to output to. If output is True and outfile is None then output is printed
                      to stdout.
    @param csv:       Boolean flag to say whether the csv output alignment format or the vertical anarci format should be used.
    @param allow:     A set containing the chain types that should be recognised. If chothia, kabat or martin is used
                      as the scheme, anarci will ignore tcr chains. Choose a subset of ["H","K","L","A","B","G","D"]
    @param assign_germline: Using highest sequence identity assign the germline to the chain. Can be more accurate at identifying
                      species than the best HMM hit alone. (Bool)
    @param allowed_species: If assign_germline is true, limit the species that can be assigned to a limited set. Useful when the
                      animal species is known or when performing closest germline experiments. Choose a subset of ['human',
                      'mouse','rat','rabbit','rhesus','pig','alpaca'].


    @param bit_score_threshold: The threshold score from HMMER at which an alignment should be numbered. Lowering the threshold
                      means domain recognition is more permissive and can be useful for numbering heavily engineered molecules.
                      However, too low and false positive recognition of other ig-like molecules will occur.
    @param hmmerpath: The path to hmmscan. If left unspecified then the PATH will be searched.
    @param ncpu:      The number of cpu's that hmmer should be allowed to use. If not specified then the hmmscan
                      default is used. N.B. hmmscan must be compiled with multithreading enabled for this option to have effect.
                      Please consider using the run_anarci function for native multiprocessing with anarci.
    @param database:  The HMMER database that should be used. Normally not changed unless a custom db is created.


    @return: Three lists. Numbered, Alignment_details and Hit_tables.
             Each list is in the same order as the input sequences list.
             A description of each entry in the three lists is as followed.
               o Numbered: will be None if no domain was found for that sequence or a list of domains with their
                           numbering, start and finish indices.
               o Alignment_details: will be None if no domain was found for that sequence or a dictionary for each
                           domain identified containing the details of the alignment (chain type, e-value, species etc).
               o Hit_tables: None if no domain was found for that sequence or a nested list for each domain containing
                           the hit table from hmmscan.

    """

    # Validate the input scheme
    try:
        scheme = scheme_short_to_long[scheme.lower()]
    except KeyError:
        raise AssertionError("Unrecognised or unimplemented scheme: %s"%scheme)

    # Check we have arguments for output before doing work.
    if csv:
        assert outfile, 'If csv output is True then an outfile must be specified'
        _path, _ = os.path.split(outfile)
        assert (not _path) or os.path.exists(_path), 'Output directory %s does not exist'%_path


    # Perform the alignments of the sequences to the hmm database
    alignments = run_hmmer(sequences,hmm_database=database,hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species )

    # Check the numbering for likely very long CDR3s that will have been missed by the first pass.
    # Modify alignments in-place
    check_for_j( sequences, alignments, scheme )

    # Apply the desired numbering scheme to all sequences
    numbered, alignment_details, hit_tables = number_sequences_from_alignment(sequences, alignments, scheme=scheme, allow=allow,
                                                                              assign_germline=assign_germline,
                                                                              allowed_species=allowed_species)

    # Output if necessary
    if output:
        if csv:
            csv_output(sequences, numbered, details, outfile)
        else:
            outto, close=sys.stdout, False
            if outfile:
                outto, close = open(outfile,'w'), True
            anarci_output(numbered, sequences, alignment_details, outto)
            if close:
                outto.close()


    return numbered, alignment_details, hit_tables

# Wrapper to run anarci using multiple processes and automate fasta file reading.
def run_anarci( seq, ncpu=1, **kwargs):
    '''
    Run the anarci numbering protocol for single or multiple sequences.

    @param sequences: A list or tuple of (Id, Sequence) pairs
                              e.g. [ ("seq1","EVQLQQSGAEVVRSG ..."),
                                     ("seq2","DIVMTQSQKFMSTSV ...") ]
    @param scheme:    The numbering scheme that should be applied. Choose from imgt, chothia, kabat or martin
    @param output:    Boolean flag to say whether the result should be output.
    @param outfile:   The name of the file to output to. If output is True and outfile is None then output is printed
                      to stdout.
    @param allow:     A set containing the chain types that should be recognised. If chothia, kabat or martin is used
                      as the scheme, anarci will ignore tcr chains. Choose a subset of ["H","K","L","A","B","G","D"]
    @param assign_germline: Using highest sequence identity assign the germline to the chain. Can be more accurate at identifying
                      species than the best HMM hit alone. (Bool)
    @param allowed_species: If assign_germline is true, limit the species that can be assigned to a limited set. Useful when the
                      animal species is known or when performing closest germline experiments. Choose a subset of ['human',
                      'mouse','rat','rabbit','rhesus','pig','alpaca'].

    @param bit_score_threshold: The threshold score from HMMER at which an alignment should be numbered. Lowering the threshold
                      means domain recognition is more permissive and can be useful for numbering heavily engineered molecules.
                      However, too low and false positive recognition of other ig-like molecules will occur.
    @param hmmerpath: The path to hmmscan. If left unspecified then the PATH will be searched.
    @param ncpu:      The number of cpu's that hmmer should be allowed to use. If not specified then the hmmscan
                      default is used. N.B. hmmscan must be compiled with multithreading enabled for this option to have effect.
                      Please consider using the run_anarci function for native multiprocessing with anarci.
    @param database:  The HMMER database that should be used. Normally not changed unless a custom db is created.

    @return: Four lists. Sequences, Numbered, Alignment_details and Hit_tables.
             Each list is in the same order.
             A description of each entry in the four lists is as followed.
               o Sequences: The list of sequences formatted as [(Id,sequence), ...].
               o Numbered: will be None if no domain was found for that sequence or a list of domains with their
                           numbering, start and finish indices.
               o Alignment_details: will be None if no domain was found for that sequence or a dictionary for each
                           domain identified containing the details of the alignment (chain type, e-value, species etc).
               o Hit_tables: None if no domain was found for that sequence or a nested list for each domain containing
                           the hit table from hmmscan.

    '''
    # Parse the input sequence or fasta file.
    if isinstance(seq, list) or isinstance(seq,tuple): # A list (or tuple) of (name,sequence) sequences
        assert all( len(_) == 2 for _ in seq ), "If list or tuple supplied as input format must be [ ('ID1','seq1'), ('ID2', 'seq2'), ... ]"
        sequences = seq
    elif os.path.isfile( seq ): # Fasta file.
        # Read the sequences. All are read into memory currently...
        sequences = read_fasta( seq )
        ncpu = int(max(1, ncpu ))
    elif isinstance(seq, str): # Single sequence
        validate_sequence( seq )
        ncpu=1
        sequences = [ ["Input sequence", seq ]]

    # Handle the arguments to anarci.
    output  = kwargs.get('output', False )
    outfile = kwargs.get('outfile', False )
    csv = kwargs.get( 'csv', False )
    if csv: # Check output arguments before doing work.
        assert outfile, 'If csv output is True then an outfile must be specified'
        _path, _ = os.path.split(outfile)
        assert (not _path) or os.path.exists(_path), 'Output directory %s does not exist'%_path

    kwargs['ncpu'] = 1 # Set hmmscan ncpu to 1. HMMER has to be compiled appropriately for this to have an effect.
    kwargs['output'] = False # Overide and write the compiled results here.

    anarci_partial = partial( anarci, **kwargs )
    chunksize = math.ceil( float( len(sequences) )/ncpu )

    # Run the anarci function using a pool of workers. Using the map_async to get over the KeyboardInterrupt bug in python2.7
    if ncpu > 1:
        pool = Pool( ncpu )
        results = pool.map_async( anarci_partial, grouper( chunksize, sequences ) ).get()
        pool.close()
    else:
        results = list(map( anarci_partial, grouper( chunksize, sequences ) ))

    # Reformat the results to flat lists.
    numbered = sum( (_[0] for _ in results), [] )
    alignment_details = sum( (_[1] for _ in results ), [] )
    hit_tables = sum( (_[2] for _ in results), [] )

    # Output if necessary
    if output:
        if csv:
            csv_output(sequences, numbered, alignment_details, outfile)
        else:
            outto, close=sys.stdout, False
            if outfile:
                outto, close = open(outfile,'w'), True
            anarci_output(numbered, sequences, alignment_details, outto)
            if close:
                outto.close()

    # Return the results
    return sequences, numbered, alignment_details, hit_tables



# Wrapper function for simple sequence in numbering and chain type out behaviour.
def number(sequence, scheme="imgt", database="ALL", allow=set(["H","K","L","A","B","G","D"]), allowed_species=['human','mouse']):
    """
    Given a sequence string, use anarci to number it using the scheme of choice.
    Only the first domain will be recognised and numbered

    For multiple sequences it is advised to use run_anarci instead of iterative use of this function.

    @param sequence: An amino acid sequence string
    @param scheme: The numbering scheme that should be applied. Choose from imgt, chothia, kabat or martin
    @param database: The HMMER database that should be used. Normally not changed unless a custom db is created.
    @param allow: A set containing the chain types that should be recognised. If chothia, kabat or martin is used
                  as the scheme, anarci will ignore tcr chains.

    @return: If the sequence can be numbered, a list containing the numbering and sequence; and the chain type.
             Otherwise both are False.

    """

    try:
        validate_sequence( sequence )
        scheme = scheme_short_to_long[scheme.lower()]
    except KeyError:
        raise AssertionError("Unrecognised to unimplemented scheme: %s"%scheme)

    if len(sequence) < 70: # Length check. ANARCI can number fragments of chains well. Encourage full domain numbering.
        return False, False

    try:
        if not allowed_species:
            numbered, alignment_details, _ = anarci( [("sequence_0", sequence)], scheme=scheme, database=database, output=False, allow=allow )
        else:
            numbered, alignment_details, _ = anarci( [("sequence_0", sequence)], scheme=scheme, database=database, output=False, allow=allow, allowed_species = allowed_species )
    except AssertionError: # Catch where the user has tried to number a TCR with an antibody scheme
        return False, False


    # We return the numbering list and the chain type where kappa and lambda chains are both "L" for light
    if numbered[0]:
        return numbered[0][0][0], chain_type_to_class[alignment_details[0][0]["chain_type"]]
    else:
        return False, False


## Figure out the process for numbering

In [3]:
sequences = [ ("12e8:H","EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLEWIGWIDPEIGDTEYVPKFQGKATMTADTSSNTAYLQLSSLTSEDTAVYYCNAGHDYDRGRFPYWGQGTLVTVSAAKTTPPSVYPLAP"),
                  ("12e8:L","DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKLMIYSASNRYTGVPDRFTGSGSGTDFTLTISNMQSEDLADYFCQQYSSYPLTFGAGTKLELKRADAAPTVSIFPPSSEQLTSGGASV"),
                  ("scfv:A","DIQMTQSPSSLSASVGDRVTITCRTSGNIHNYLTWYQQKPGKAPQLLIYNAKTLADGVPSRFSGSGSGTQFTLTISSLQPEDFANYYCQHFWSLPFTFGQGTKVEIKRTGGGGSGGGGSGGGGSGGGGSEVQLVESGGGLVQPGGSLRLSCAASGFDFSRYDMSWVRQAPGKRLEWVAYISSGGGSTYFPDTVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCARQNKKLTWFDYWGQGTLVTVSSHHHHHH"),
                  ("lysozyme:A","KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL")]

In [11]:
results = anarci(sequences, scheme="imgt", output=True)
numbering, alignment_details, hit_tables = results

# 12e8:H
# ANARCI numbered
# Domain 1 of 1
# Most significant HMM hit
#|species|chain_type|e-value|score|seqstart_index|seqend_index|
#|mouse|H|3e-54|173.4|0|119|
# Scheme = imgt
H 1       E
H 2       V
H 3       Q
H 4       L
H 5       Q
H 6       Q
H 7       S
H 8       G
H 9       A
H 10      -
H 11      E
H 12      V
H 13      V
H 14      R
H 15      S
H 16      G
H 17      A
H 18      S
H 19      V
H 20      K
H 21      L
H 22      S
H 23      C
H 24      T
H 25      A
H 26      S
H 27      G
H 28      F
H 29      N
H 30      I
H 31      -
H 32      -
H 33      -
H 34      -
H 35      K
H 36      D
H 37      Y
H 38      Y
H 39      I
H 40      H
H 41      W
H 42      V
H 43      K
H 44      Q
H 45      R
H 46      P
H 47      E
H 48      K
H 49      G
H 50      L
H 51      E
H 52      W
H 53      I
H 54      G
H 55      W
H 56      I
H 57      D
H 58      P
H 59      E
H 60      -
H 61      -
H 62      I
H 63      G
H 64      D
H 65      T
H 66      E
H 67      Y
H 68      V
H 69 

In [12]:
expect_one_VH_domain_numbering, expect_one_VL_domain_numbering, expect_VH_then_VL_numbering, expect_None = numbering

In [18]:
alignment_details

[[{'id': 'mouse_H',
   'description': '',
   'evalue': '3e-54',
   'bitscore': 173.4,
   'bias': 0.3,
   'query_start': 0,
   'query_end': 120,
   'species': 'mouse',
   'chain_type': 'H',
   'scheme': 'imgt',
   'query_name': '12e8:H'}],
 [{'id': 'mouse_K',
   'description': '',
   'evalue': '1.1e-53',
   'bitscore': 171.6,
   'bias': 1.1,
   'query_start': 0,
   'query_end': 107,
   'species': 'mouse',
   'chain_type': 'K',
   'scheme': 'imgt',
   'query_name': '12e8:L'}],
 [{'id': 'human_K',
   'description': '',
   'evalue': '2.7e-53',
   'bitscore': 170.2,
   'bias': 0.6,
   'query_start': 0,
   'query_end': 107,
   'species': 'human',
   'chain_type': 'K',
   'scheme': 'imgt',
   'query_name': 'scfv:A'},
  {'id': 'human_H',
   'description': '',
   'evalue': '3.3e-58',
   'bitscore': 186.4,
   'bias': 0.4,
   'query_start': 129,
   'query_end': 248,
   'species': 'human',
   'chain_type': 'H',
   'scheme': 'imgt',
   'query_name': 'scfv:A'}],
 None]

In [19]:
hit_tables

[[['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end'],
  ['mouse_H', '', 3e-54, 173.4, 0.3, 0, 120],
  ['human_H', '', 5.6e-51, 163.0, 0.3, 0, 119]],
 [['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end'],
  ['mouse_K', '', 1.1e-53, 171.6, 1.1, 0, 107],
  ['human_K', '', 2.2e-50, 160.8, 1.8, 0, 107],
  ['human_L', '', 4.6e-36, 114.7, 1.6, 1, 104],
  ['mouse_L', '', 3.6e-26, 82.6, 0.7, 2, 104]],
 [['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end'],
  ['human_H', '', 3.3e-58, 186.4, 0.4, 129, 248],
  ['mouse_H', '', 4.5e-56, 179.3, 0.1, 129, 248],
  ['human_K', '', 2.7e-53, 170.2, 0.6, 0, 107],
  ['mouse_K', '', 1.2e-51, 165.0, 0.2, 0, 107],
  ['human_L', '', 9.3e-36, 113.7, 0.5, 2, 105],
  ['mouse_L', '', 8.9e-27, 84.5, 0.2, 3, 105]],
 [['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end']]]

## Investigate the process to generate HMM results
### run_hmmer:

In [28]:
scheme="imgt"
database="ALL"
output=False
outfile=None
csv=False
allow=set(["H","K","L","A","B","G","D"])
hmmerpath=""
ncpu=None
assign_germline=False
allowed_species=['human','mouse']
bit_score_threshold=80

In [29]:
if hmmerpath:
    print (hmmerpath)
else:
    print ("not")

not


In [22]:
alignments = run_hmmer(sequences,hmm_database=database,hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species )

In [23]:
alignments

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['mouse_H', '', 3e-54, 173.4, 0.3, 0, 120],
   ['human_H', '', 5.6e-51, 163.0, 0.3, 0, 119]],
  [[((1, 'm'), 0),
    ((2, 'm'), 1),
    ((3, 'm'), 2),
    ((4, 'm'), 3),
    ((5, 'm'), 4),
    ((6, 'm'), 5),
    ((7, 'm'), 6),
    ((8, 'm'), 7),
    ((9, 'm'), 8),
    ((10, 'd'), None),
    ((11, 'm'), 9),
    ((12, 'm'), 10),
    ((13, 'm'), 11),
    ((14, 'm'), 12),
    ((15, 'm'), 13),
    ((16, 'm'), 14),
    ((17, 'm'), 15),
    ((18, 'm'), 16),
    ((19, 'm'), 17),
    ((20, 'm'), 18),
    ((21, 'm'), 19),
    ((22, 'm'), 20),
    ((23, 'm'), 21),
    ((24, 'm'), 22),
    ((25, 'm'), 23),
    ((26, 'm'), 24),
    ((27, 'm'), 25),
    ((28, 'm'), 26),
    ((29, 'm'), 27),
    ((30, 'm'), 28),
    ((31, 'd'), None),
    ((32, 'd'), None),
    ((33, 'd'), None),
    ((34, 'd'), None),
    ((35, 'm'), 29),
    ((36, 'm'), 30),
    ((37, 'm'), 31),
    ((38, 'm'), 32),
    ((

#### Modify run_hmmer function to store the  hmmscan results (instead of deleting it)

In [30]:
fasta_filehandle, fasta_filename =  tempfile.mkstemp( ".fasta", text=True )

In [37]:
fd = os.open( "test.fasta", os.O_RDWR|os.O_CREAT )
with os.fdopen(fd,'w') as outfile:
        write_fasta(sequences, outfile)

In [226]:
def new_run_hmmer(sequence_list,hmm_database="ALL",hmmerpath="", ncpu=None, bit_score_threshold=80, hmmer_species=None,hmmscan_out_fn=""):
    """
    Run the sequences in sequence list against a precompiled hmm_database.

    Those sequence that have a significant hit with a bit score over a threshold will
    be recognised and an alignment given. The alignment will be used to number the
    sequence.

    @param sequence_list: a list of (name, sequence) tuples. Both are strings
    @param hmm_database: The hmm database to use. Currently, all hmms are in the ALL database.
                         The code to develop new models is in build_pipeline in the git repo.
    @param hmmerpath: The path to hmmer binaries if not in the path
    @param ncpu: The number of cpu's to allow hmmer to use.
    """

    # Check that hmm_database is available

    assert hmm_database in ["ALL","ALL_AND_C","C_ONLY"], "Unknown HMM database %s"%hmm_database
    HMM = os.path.join( HMM_path, "%s.hmm"%hmm_database )


    # Create a fasta file for all the sequences. Label them with their sequence index
    # This will go to a temp file
    fasta_filehandle, fasta_filename =  tempfile.mkstemp( ".fasta", text=True )
    with os.fdopen(fasta_filehandle,'w') as outfile:
        write_fasta(sequence_list, outfile)

    #output_filehandle, output_filename =  tempfile.mkstemp( ".txt", text=True )
    output_filename=f"test_hmmscan_result/test_hmmscan_out{hmmscan_out_fn}.txt"
    output_filehandle=os.open( output_filename, os.O_RDWR|os.O_CREAT )

    # Run hmmer as a subprocess
    if hmmerpath:
        hmmscan = os.path.join(hmmerpath,"hmmscan")
    else:
        hmmscan = "hmmscan"
    try:
        if ncpu is None:
            command = [ hmmscan, "-o", output_filename, HMM,  fasta_filename]
        else:
            command = [ hmmscan, "-o", output_filename, "--cpu", str(ncpu), HMM,  fasta_filename]
        process = Popen( command, stdout=PIPE, stderr=PIPE  )
        _, pr_stderr = process.communicate()

        if pr_stderr:
            _f = os.fdopen(output_filehandle) # This is to remove the filedescriptor from the os. I have had problems with it before.
            _f.close()

            raise HMMscanError(pr_stderr)
        results = parse_hmmer_output(output_filehandle, bit_score_threshold=bit_score_threshold, hmmer_species=hmmer_species)

    finally:
        # clear up
        os.remove(fasta_filename)
        #os.remove(output_filename)

    return results

##### This new function outputs the hmmscan results into `test_hmmscan_result/` folder:
##### The interpretation of the hmmscan file: https://hmmer-web-docs.readthedocs.io/en/latest/result.html#alignments
##### RF line: 
The #=GC RF line is Stockholm-format reference coordinate annotation,
with an x marking each column that the profile considered to be
consensus.
##### PP line:
The bottom line represents the posterior probability (essentially
the expected accuracy) of each aligned residue. A 0 means 0-5%, 1
means 5-15%, and so on; 9 means 85-95%, and a * means 95-100%
posterior probability. You can use these posterior probabilities to
decide which parts of the alignment are well-determined or not.
You’ll often observe, for example, that expected alignment accuracy degrades around locations of insertion and deletion, which you’d
intuitively expect.
<br/><br/>
You’ll also see expected alignment accuracy degrade at the ends of
an alignment – this is because “alignment accuracy” posterior probabilities currently not only includes whether the residue is aligned to
one profile position versus others, but also confounded with whether
a residue should be considered to be homologous (aligned to the
profile somewhere) versus not homologous at all.

In [227]:
alignments = new_run_hmmer(sequences,hmm_database=database,hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species )

In [44]:
type(alignments)

list

In [40]:
alignments

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['mouse_H', '', 3e-54, 173.4, 0.3, 0, 120],
   ['human_H', '', 5.6e-51, 163.0, 0.3, 0, 119]],
  [[((1, 'm'), 0),
    ((2, 'm'), 1),
    ((3, 'm'), 2),
    ((4, 'm'), 3),
    ((5, 'm'), 4),
    ((6, 'm'), 5),
    ((7, 'm'), 6),
    ((8, 'm'), 7),
    ((9, 'm'), 8),
    ((10, 'd'), None),
    ((11, 'm'), 9),
    ((12, 'm'), 10),
    ((13, 'm'), 11),
    ((14, 'm'), 12),
    ((15, 'm'), 13),
    ((16, 'm'), 14),
    ((17, 'm'), 15),
    ((18, 'm'), 16),
    ((19, 'm'), 17),
    ((20, 'm'), 18),
    ((21, 'm'), 19),
    ((22, 'm'), 20),
    ((23, 'm'), 21),
    ((24, 'm'), 22),
    ((25, 'm'), 23),
    ((26, 'm'), 24),
    ((27, 'm'), 25),
    ((28, 'm'), 26),
    ((29, 'm'), 27),
    ((30, 'm'), 28),
    ((31, 'd'), None),
    ((32, 'd'), None),
    ((33, 'd'), None),
    ((34, 'd'), None),
    ((35, 'm'), 29),
    ((36, 'm'), 30),
    ((37, 'm'), 31),
    ((38, 'm'), 32),
    ((

In [45]:
alignments2=alignments.copy()

## Investigate the process to generate the state vector

In [60]:
def investigate_HMMERParser():
    output_filename="test_hmmscan_result/test_hmmscan_out.txt"
    output_filehandle=os.open( output_filename, os.O_RDWR|os.O_CREAT )
    with os.fdopen(output_filehandle) as inputfile:
        p = HMMERParser( inputfile )
        return [query for query in p]

In [62]:
x=investigate_HMMERParser()

In [63]:
x

[QueryResult(id='12e8:H', 26 hits),
 QueryResult(id='12e8:L', 24 hits),
 QueryResult(id='scfv:A', 26 hits),
 QueryResult(id='lysozyme:A', 0 hits)]

In [64]:
[list(i) for i in x]

[[Hit(id='mouse_H', query_id='12e8:H', 1 hsps),
  Hit(id='human_H', query_id='12e8:H', 1 hsps),
  Hit(id='rat_H', query_id='12e8:H', 1 hsps),
  Hit(id='rabbit_H', query_id='12e8:H', 1 hsps),
  Hit(id='pig_H', query_id='12e8:H', 1 hsps),
  Hit(id='rhesus_H', query_id='12e8:H', 1 hsps),
  Hit(id='mouse_A', query_id='12e8:H', 1 hsps),
  Hit(id='rhesus_L', query_id='12e8:H', 1 hsps),
  Hit(id='human_A', query_id='12e8:H', 1 hsps),
  Hit(id='human_L', query_id='12e8:H', 1 hsps),
  Hit(id='mouse_D', query_id='12e8:H', 1 hsps),
  Hit(id='rabbit_L', query_id='12e8:H', 1 hsps),
  Hit(id='human_D', query_id='12e8:H', 1 hsps),
  Hit(id='mouse_B', query_id='12e8:H', 1 hsps),
  Hit(id='human_B', query_id='12e8:H', 1 hsps),
  Hit(id='rat_L', query_id='12e8:H', 1 hsps),
  Hit(id='mouse_L', query_id='12e8:H', 1 hsps),
  Hit(id='mouse_G', query_id='12e8:H', 1 hsps),
  Hit(id='rhesus_K', query_id='12e8:H', 1 hsps),
  Hit(id='mouse_K', query_id='12e8:H', 1 hsps),
  Hit(id='pig_L', query_id='12e8:H', 1 hs

In [69]:
x[0]

QueryResult(id='12e8:H', 26 hits)

### Function \_parse_hmmer_query

In [70]:
x[0].hsps

[HSP(hit_id='mouse_H', query_id='12e8:H', 1 fragments),
 HSP(hit_id='human_H', query_id='12e8:H', 1 fragments),
 HSP(hit_id='rat_H', query_id='12e8:H', 1 fragments),
 HSP(hit_id='rabbit_H', query_id='12e8:H', 1 fragments),
 HSP(hit_id='pig_H', query_id='12e8:H', 1 fragments),
 HSP(hit_id='rhesus_H', query_id='12e8:H', 1 fragments),
 HSP(hit_id='mouse_A', query_id='12e8:H', 1 fragments),
 HSP(hit_id='rhesus_L', query_id='12e8:H', 1 fragments),
 HSP(hit_id='human_A', query_id='12e8:H', 1 fragments),
 HSP(hit_id='human_L', query_id='12e8:H', 1 fragments),
 HSP(hit_id='mouse_D', query_id='12e8:H', 1 fragments),
 HSP(hit_id='rabbit_L', query_id='12e8:H', 1 fragments),
 HSP(hit_id='human_D', query_id='12e8:H', 1 fragments),
 HSP(hit_id='mouse_B', query_id='12e8:H', 1 fragments),
 HSP(hit_id='human_B', query_id='12e8:H', 1 fragments),
 HSP(hit_id='rat_L', query_id='12e8:H', 1 fragments),
 HSP(hit_id='mouse_L', query_id='12e8:H', 1 fragments),
 HSP(hit_id='mouse_G', query_id='12e8:H', 1 fragme

A function to order/extract/modify the hits (logic similar to my BLAST function)

### Function: _hmm_alignment_to_states: convert aln to state vector

In [72]:
x[0].hsps[0].aln_annotation

{'RF': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
 'PP': '79*******.********************....9************************..***********.*********************************999999***************7'}

In [73]:
x[0].hsps[0].aln_annotation["RF"] # reference string

'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [74]:
x[0].hsps[0].aln_annotation["PP"] # state string

'79*******.********************....9************************..***********.*********************************999999***************7'

In [75]:
x[0].hsps[0].hit_start # _hmm_start: reference start

0

In [76]:
x[0].hsps[0].hit_end # _hmm_end: reference end

128

In [77]:
x[0].hsps[0].query_start # _seq_start

0

In [78]:
x[0].hsps[0].query_end # _seq_end

120

In [79]:
t_hsps=x[0].hsps[0]

In [81]:
t_hsps.hit_id.split('_') #species, ctype

['mouse', 'H']

In [82]:
get_hmm_length( 'mouse', 'H' )

128

#### Meaning of some variables
##### hsps.hit_start: _hmm_start: reference start
##### hsps.hit_end: _hmm_end: reference end
##### hsp.query_start:_seq_start: start position for the query sequence
##### hsp.query_end:_seq_end: end position for the query sequence

##### RF: the state of the reference sequence: x: consensus: match, not x (".","-" or "_"): not consensus, usually generated gaps in the reference sequence
##### PP: the state of the query sequence: ".": generated gaps in the query sequence

 The posterior probability is encoded as 11 possible
characters 0-9 * +: 0.0 ≤ p < 0.05 is coded as 0, 0.05 ≤ p < 0.15 is
coded as 1, (... and so on ...), 0.85 ≤ p < 0.95 is coded as 9, and
0.95 ≤ p ≤ 1.0 is coded as ’ * ’. Gap characters appear in the PP line
where no residue has been assigned.
<br/><br/>

PP: the posterior probability (essentially
the expected accuracy) of each aligned residue. A 0 means 0-5%, 1
means 5-15%, and so on; 9 means 85-95%, and a * means 95-100%
posterior probability. You can use these posterior probabilities to
decide which parts of the alignment are well-determined or not.
You’ll often observe, for example, that expected alignment accuracy degrades around locations of insertion and deletion, which you’d
intuitively expect.
<br/><br/>
You’ll also see expected alignment accuracy degrade at the ends of
an alignment – this is because “alignment accuracy” posterior probabilities currently not only includes whether the residue is aligned to
one profile position versus others, but also confounded with whether
a residue should be considered to be homologous (aligned to the
profile somewhere) versus not homologous at all.

#### Add extensions in numbering:
line number refers to the github line number:
##### line 408-415: add extension before: add numbering for the N-terminal modification: could be modified to apply it to numbering 1.x in C region
##### line 420-425: add extension after: add numbering for the extended J regions: ususally inside framework 4: could be modified to apply it for the end of C domains

### The actural step to assign state (github line 436-458)
#### Each position has a state: "m" (match), "i" (insersion), "d" (deletion)
##### "m": the alignment of the query to the reference doesn't generate gaps in reference, thus could assign a numbering (interger)
##### "i": the alignment generate gaps in the reference sequence, thus these residues are "extra" ones compared with reference, shoud be assigned with value x.y
##### "d": the alignment generate gaps in the query sequence, meaning the query sequence misses the corresponding residues in the reference. Numbering should jumps here.

#### For C region: how do we adjust the code here? Unlike V region, where the reference sequence numbering doesn't include x.y, the C region reference already includes a lot x.y

#### The meaning of indices:
h: the numbering of the reference sequence within the hits
<br/>
s: the numbering of the query sequence within the hit range
<br/>
sequence_indix=sequence_indices[s]: the numbering of the query sequence from the query sequence start. (sometimes hit doesn't start at the beginning of the query sequence) (similar case for <b> hmm_states[h]<b/>)

### Returns a list of tuples: ((hmm_states[h], state_type),  sequence_index )
#### hmm_states[h]: the position corresponding to the reference sequence, numbering starts at 1
#### state_type: "m","i", or "d"
#### sequence_index: the position corresponding to the query sequence

## Continue to investigate anarci function (where run_hmmer is used)

In [46]:
check_for_j( sequences, alignments2, scheme )

In [48]:
alignments2==alignments

True

In [49]:
numbered, alignment_details, hit_tables = number_sequences_from_alignment(sequences, alignments, scheme=scheme, allow=allow,
                                                                              assign_germline=assign_germline,
                                                                              allowed_species=allowed_species)

In [52]:
hit_tables

[[['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end'],
  ['mouse_H', '', 3e-54, 173.4, 0.3, 0, 120],
  ['human_H', '', 5.6e-51, 163.0, 0.3, 0, 119]],
 [['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end'],
  ['mouse_K', '', 1.1e-53, 171.6, 1.1, 0, 107],
  ['human_K', '', 2.2e-50, 160.8, 1.8, 0, 107],
  ['human_L', '', 4.6e-36, 114.7, 1.6, 1, 104],
  ['mouse_L', '', 3.6e-26, 82.6, 0.7, 2, 104]],
 [['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end'],
  ['human_H', '', 3.3e-58, 186.4, 0.4, 129, 248],
  ['mouse_H', '', 4.5e-56, 179.3, 0.1, 129, 248],
  ['human_K', '', 2.7e-53, 170.2, 0.6, 0, 107],
  ['mouse_K', '', 1.2e-51, 165.0, 0.2, 0, 107],
  ['human_L', '', 9.3e-36, 113.7, 0.5, 2, 105],
  ['mouse_L', '', 8.9e-27, 84.5, 0.2, 3, 105]],
 [['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end']]]

## Function: number_sequences_from_alignment

In [85]:
alignments # the result from run_hmmer

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['mouse_H', '', 3e-54, 173.4, 0.3, 0, 120],
   ['human_H', '', 5.6e-51, 163.0, 0.3, 0, 119]],
  [[((1, 'm'), 0),
    ((2, 'm'), 1),
    ((3, 'm'), 2),
    ((4, 'm'), 3),
    ((5, 'm'), 4),
    ((6, 'm'), 5),
    ((7, 'm'), 6),
    ((8, 'm'), 7),
    ((9, 'm'), 8),
    ((10, 'd'), None),
    ((11, 'm'), 9),
    ((12, 'm'), 10),
    ((13, 'm'), 11),
    ((14, 'm'), 12),
    ((15, 'm'), 13),
    ((16, 'm'), 14),
    ((17, 'm'), 15),
    ((18, 'm'), 16),
    ((19, 'm'), 17),
    ((20, 'm'), 18),
    ((21, 'm'), 19),
    ((22, 'm'), 20),
    ((23, 'm'), 21),
    ((24, 'm'), 22),
    ((25, 'm'), 23),
    ((26, 'm'), 24),
    ((27, 'm'), 25),
    ((28, 'm'), 26),
    ((29, 'm'), 27),
    ((30, 'm'), 28),
    ((31, 'd'), None),
    ((32, 'd'), None),
    ((33, 'd'), None),
    ((34, 'd'), None),
    ((35, 'm'), 29),
    ((36, 'm'), 30),
    ((37, 'm'), 31),
    ((38, 'm'), 32),
    ((

In [88]:
len(alignments[0])

3

In [89]:
alignments[0][0] # hit_table

[['id',
  'description',
  'evalue',
  'bitscore',
  'bias',
  'query_start',
  'query_end'],
 ['mouse_H', '', 3e-54, 173.4, 0.3, 0, 120],
 ['human_H', '', 5.6e-51, 163.0, 0.3, 0, 119]]

In [90]:
alignments[0][1] # state_vectors

[[((1, 'm'), 0),
  ((2, 'm'), 1),
  ((3, 'm'), 2),
  ((4, 'm'), 3),
  ((5, 'm'), 4),
  ((6, 'm'), 5),
  ((7, 'm'), 6),
  ((8, 'm'), 7),
  ((9, 'm'), 8),
  ((10, 'd'), None),
  ((11, 'm'), 9),
  ((12, 'm'), 10),
  ((13, 'm'), 11),
  ((14, 'm'), 12),
  ((15, 'm'), 13),
  ((16, 'm'), 14),
  ((17, 'm'), 15),
  ((18, 'm'), 16),
  ((19, 'm'), 17),
  ((20, 'm'), 18),
  ((21, 'm'), 19),
  ((22, 'm'), 20),
  ((23, 'm'), 21),
  ((24, 'm'), 22),
  ((25, 'm'), 23),
  ((26, 'm'), 24),
  ((27, 'm'), 25),
  ((28, 'm'), 26),
  ((29, 'm'), 27),
  ((30, 'm'), 28),
  ((31, 'd'), None),
  ((32, 'd'), None),
  ((33, 'd'), None),
  ((34, 'd'), None),
  ((35, 'm'), 29),
  ((36, 'm'), 30),
  ((37, 'm'), 31),
  ((38, 'm'), 32),
  ((39, 'm'), 33),
  ((40, 'm'), 34),
  ((41, 'm'), 35),
  ((42, 'm'), 36),
  ((43, 'm'), 37),
  ((44, 'm'), 38),
  ((45, 'm'), 39),
  ((46, 'm'), 40),
  ((47, 'm'), 41),
  ((48, 'm'), 42),
  ((49, 'm'), 43),
  ((50, 'm'), 44),
  ((51, 'm'), 45),
  ((52, 'm'), 46),
  ((53, 'm'), 47),
  

In [91]:
alignments[0][2] # detailss

[{'id': 'mouse_H',
  'description': '',
  'evalue': 3e-54,
  'bitscore': 173.4,
  'bias': 0.3,
  'query_start': 0,
  'query_end': 120,
  'species': 'mouse',
  'chain_type': 'H',
  'scheme': 'imgt',
  'query_name': '12e8:H'}]

In [92]:
t_state_vectors=alignments[0][1]

In [94]:
len(t_state_vectors) # Acturally is the number of the domains

1

In [93]:
t_state_vectors[0]

[((1, 'm'), 0),
 ((2, 'm'), 1),
 ((3, 'm'), 2),
 ((4, 'm'), 3),
 ((5, 'm'), 4),
 ((6, 'm'), 5),
 ((7, 'm'), 6),
 ((8, 'm'), 7),
 ((9, 'm'), 8),
 ((10, 'd'), None),
 ((11, 'm'), 9),
 ((12, 'm'), 10),
 ((13, 'm'), 11),
 ((14, 'm'), 12),
 ((15, 'm'), 13),
 ((16, 'm'), 14),
 ((17, 'm'), 15),
 ((18, 'm'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'm'), 23),
 ((26, 'm'), 24),
 ((27, 'm'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 

In [98]:
alignments[0][2][0]

{'id': 'mouse_H',
 'description': '',
 'evalue': 3e-54,
 'bitscore': 173.4,
 'bias': 0.3,
 'query_start': 0,
 'query_end': 120,
 'species': 'mouse',
 'chain_type': 'H',
 'scheme': 'imgt',
 'query_name': '12e8:H'}

### number_sequence_from_alignment:
#### number_imgt: in schemes.py

##### _number_regions: in schemes.py

line 240:
<br/>
state_id: hmmer_index, start from 1. Thus, region/state_string[state_id-1] is the state at this position.
<br/><br/>
line 259:
<br/>
alphabet[insertion]: when insertion=-1, this is " ".

In [102]:
t_state_vectors[0]

[((1, 'm'), 0),
 ((2, 'm'), 1),
 ((3, 'm'), 2),
 ((4, 'm'), 3),
 ((5, 'm'), 4),
 ((6, 'm'), 5),
 ((7, 'm'), 6),
 ((8, 'm'), 7),
 ((9, 'm'), 8),
 ((10, 'd'), None),
 ((11, 'm'), 9),
 ((12, 'm'), 10),
 ((13, 'm'), 11),
 ((14, 'm'), 12),
 ((15, 'm'), 13),
 ((16, 'm'), 14),
 ((17, 'm'), 15),
 ((18, 'm'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'm'), 23),
 ((26, 'm'), 24),
 ((27, 'm'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 

In [103]:
sequence=sequences[0][1]
state_vector=t_state_vectors[0]
state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
                    
# Region string - regions that should be treated separately in putting the numbering together 
region_string = '11111111111111111111111111222222222222333333333333333334444444444555555555555555555555555555555555555555666666666666677777777777'

region_index_dict = {
                         "1":0,
                         "2":1,
                         "3":2,
                         "4":3,
                         "5":4,
                         "6":5,
                         "7":6
                         }
    
# Define how the scheme's numbering differs from IMGT at the start of each region. 
# This is updated in the loop below
rels              =  {0:0, 
                          1:0,
                          2:0,
                          3:0,
                          4:0,
                          5:0,
                          6:0,
                          7:0
                          }

In [105]:
sequence

'EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLEWIGWIDPEIGDTEYVPKFQGKATMTADTSSNTAYLQLSSLTSEDTAVYYCNAGHDYDRGRFPYWGQGTLVTVSAAKTTPPSVYPLAP'

In [104]:
_number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, 7, [1,3,5])

([[((1, ' '), 'E'),
   ((2, ' '), 'V'),
   ((3, ' '), 'Q'),
   ((4, ' '), 'L'),
   ((5, ' '), 'Q'),
   ((6, ' '), 'Q'),
   ((7, ' '), 'S'),
   ((8, ' '), 'G'),
   ((9, ' '), 'A'),
   ((11, ' '), 'E'),
   ((12, ' '), 'V'),
   ((13, ' '), 'V'),
   ((14, ' '), 'R'),
   ((15, ' '), 'S'),
   ((16, ' '), 'G'),
   ((17, ' '), 'A'),
   ((18, ' '), 'S'),
   ((19, ' '), 'V'),
   ((20, ' '), 'K'),
   ((21, ' '), 'L'),
   ((22, ' '), 'S'),
   ((23, ' '), 'C'),
   ((24, ' '), 'T'),
   ((25, ' '), 'A'),
   ((26, ' '), 'S')],
  [((27, ' '), 'G'),
   ((28, ' '), 'F'),
   ((29, ' '), 'N'),
   ((30, ' '), 'I'),
   ((35, ' '), 'K'),
   ((36, ' '), 'D'),
   ((37, ' '), 'Y'),
   ((38, ' '), 'Y')],
  [((39, ' '), 'I'),
   ((40, ' '), 'H'),
   ((41, ' '), 'W'),
   ((42, ' '), 'V'),
   ((43, ' '), 'K'),
   ((44, ' '), 'Q'),
   ((45, ' '), 'R'),
   ((46, ' '), 'P'),
   ((47, ' '), 'E'),
   ((48, ' '), 'K'),
   ((49, ' '), 'G'),
   ((50, ' '), 'L'),
   ((51, ' '), 'E'),
   ((52, ' '), 'W'),
   ((53, ' '), 'I'),

In [107]:
region_index_dict[region_string[1-1]]

0

In [111]:
rels[0]

0

In [110]:
state_string[9-1]

'X'

In [109]:
state_vector

[((1, 'm'), 0),
 ((2, 'm'), 1),
 ((3, 'm'), 2),
 ((4, 'm'), 3),
 ((5, 'm'), 4),
 ((6, 'm'), 5),
 ((7, 'm'), 6),
 ((8, 'm'), 7),
 ((9, 'm'), 8),
 ((10, 'd'), None),
 ((11, 'm'), 9),
 ((12, 'm'), 10),
 ((13, 'm'), 11),
 ((14, 'm'), 12),
 ((15, 'm'), 13),
 ((16, 'm'), 14),
 ((17, 'm'), 15),
 ((18, 'm'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'm'), 23),
 ((26, 'm'), 24),
 ((27, 'm'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 

# test string with insersion

In [113]:
t_seqs2=[('7c2l:A',
  'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYTLEDYKDDDDK'),
 ('7c2l:B',
  'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYTLEDYKDDDDK'),
 ('7c2l:C',
  'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYTLEDYKDDDDK'),
 ('7c2l:H',
  'EVQLVESGAEVKKPGASVKVSCKVSGYTLTELSMHWVRQAPGKGLEWMGGFDPEDGETMYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYYCATSTAVAGTPDLFDYYYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK'),
 ('7c2l:I',
  'EVQLVESGAEVKKPGASVKVSCKVSGYTLTELSMHWVRQAPGKGLEWMGGFDPEDGETMYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYYCATSTAVAGTPDLFDYYYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK'),
 ('7c2l:J',
  'EVQLVESGAEVKKPGASVKVSCKVSGYTLTELSMHWVRQAPGKGLEWMGGFDPEDGETMYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYYCATSTAVAGTPDLFDYYYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK'),
 ('7c2l:L',
  'EIVMTQSPLSSPVTLGQPASISCRSSQSLVHSDGNTYLSWLQQRPGQPPRLLIYKISNRFSGVPDRFSGSGAGTDFTLKISRVEAEDVGVYYCTQATQFPYTFGQGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC'),
 ('7c2l:M',
  'EIVMTQSPLSSPVTLGQPASISCRSSQSLVHSDGNTYLSWLQQRPGQPPRLLIYKISNRFSGVPDRFSGSGAGTDFTLKISRVEAEDVGVYYCTQATQFPYTFGQGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC'),
 ('7c2l:N',
  'EIVMTQSPLSSPVTLGQPASISCRSSQSLVHSDGNTYLSWLQQRPGQPPRLLIYKISNRFSGVPDRFSGSGAGTDFTLKISRVEAEDVGVYYCTQATQFPYTFGQGTKVDIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGEC')]

In [114]:
t2_results = anarci(t_seqs2, scheme="imgt", output=True)
t2_numbering, t2_alignment_details, t2_hit_tables = results

# 7c2l:A
//
# 7c2l:B
//
# 7c2l:C
//
# 7c2l:H
# ANARCI numbered
# Domain 1 of 1
# Most significant HMM hit
#|species|chain_type|e-value|score|seqstart_index|seqend_index|
#|human|H|1.1e-53|171.7|0|127|
# Scheme = imgt
H 1       E
H 2       V
H 3       Q
H 4       L
H 5       V
H 6       E
H 7       S
H 8       G
H 9       A
H 10      -
H 11      E
H 12      V
H 13      K
H 14      K
H 15      P
H 16      G
H 17      A
H 18      S
H 19      V
H 20      K
H 21      V
H 22      S
H 23      C
H 24      K
H 25      V
H 26      S
H 27      G
H 28      Y
H 29      T
H 30      L
H 31      -
H 32      -
H 33      -
H 34      -
H 35      T
H 36      E
H 37      L
H 38      S
H 39      M
H 40      H
H 41      W
H 42      V
H 43      R
H 44      Q
H 45      A
H 46      P
H 47      G
H 48      K
H 49      G
H 50      L
H 51      E
H 52      W
H 53      M
H 54      G
H 55      G
H 56      F
H 57      D
H 58      P
H 59      E
H 60      -
H 61      -
H 62      D
H 63      G
H 64      E
H 65      T
H 6

In [115]:
t2_alignments = run_hmmer(t_seqs2,hmm_database=database,hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species )

In [116]:
t2_alignments

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end']],
  [],
  []),
 ([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end']],
  [],
  []),
 ([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end']],
  [],
  []),
 ([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['human_H', '', 1.1e-53, 171.7, 1.2, 0, 128],
   ['mouse_H', '', 4.8e-50, 159.8, 0.7, 0, 128]],
  [[((1, 'm'), 0),
    ((2, 'm'), 1),
    ((3, 'm'), 2),
    ((4, 'm'), 3),
    ((5, 'm'), 4),
    ((6, 'm'), 5),
    ((7, 'm'), 6),
    ((8, 'm'), 7),
    ((9, 'm'), 8),
    ((10, 'd'), None),
    ((11, 'm'), 9),
    ((12, 'm'), 10),
    ((13, 'm'), 11),
    ((14, 'm'), 12),
    ((15, 'm'), 13),
    ((16, 'm'), 14),
    ((17, 'm'), 15),
    ((18, 'm'), 16),
    ((19, 'm'), 17),
    ((20, 'm'), 18),
    ((21, 'm'), 19),
   

In [117]:
t2_alignments[3]

([['id',
   'description',
   'evalue',
   'bitscore',
   'bias',
   'query_start',
   'query_end'],
  ['human_H', '', 1.1e-53, 171.7, 1.2, 0, 128],
  ['mouse_H', '', 4.8e-50, 159.8, 0.7, 0, 128]],
 [[((1, 'm'), 0),
   ((2, 'm'), 1),
   ((3, 'm'), 2),
   ((4, 'm'), 3),
   ((5, 'm'), 4),
   ((6, 'm'), 5),
   ((7, 'm'), 6),
   ((8, 'm'), 7),
   ((9, 'm'), 8),
   ((10, 'd'), None),
   ((11, 'm'), 9),
   ((12, 'm'), 10),
   ((13, 'm'), 11),
   ((14, 'm'), 12),
   ((15, 'm'), 13),
   ((16, 'm'), 14),
   ((17, 'm'), 15),
   ((18, 'm'), 16),
   ((19, 'm'), 17),
   ((20, 'm'), 18),
   ((21, 'm'), 19),
   ((22, 'm'), 20),
   ((23, 'm'), 21),
   ((24, 'm'), 22),
   ((25, 'm'), 23),
   ((26, 'm'), 24),
   ((27, 'm'), 25),
   ((28, 'm'), 26),
   ((29, 'm'), 27),
   ((30, 'm'), 28),
   ((31, 'd'), None),
   ((32, 'd'), None),
   ((33, 'd'), None),
   ((34, 'd'), None),
   ((35, 'm'), 29),
   ((36, 'm'), 30),
   ((37, 'm'), 31),
   ((38, 'm'), 32),
   ((39, 'm'), 33),
   ((40, 'm'), 34),
   ((41, 'm

In [122]:
t2_alignments[3][1]

[[((1, 'm'), 0),
  ((2, 'm'), 1),
  ((3, 'm'), 2),
  ((4, 'm'), 3),
  ((5, 'm'), 4),
  ((6, 'm'), 5),
  ((7, 'm'), 6),
  ((8, 'm'), 7),
  ((9, 'm'), 8),
  ((10, 'd'), None),
  ((11, 'm'), 9),
  ((12, 'm'), 10),
  ((13, 'm'), 11),
  ((14, 'm'), 12),
  ((15, 'm'), 13),
  ((16, 'm'), 14),
  ((17, 'm'), 15),
  ((18, 'm'), 16),
  ((19, 'm'), 17),
  ((20, 'm'), 18),
  ((21, 'm'), 19),
  ((22, 'm'), 20),
  ((23, 'm'), 21),
  ((24, 'm'), 22),
  ((25, 'm'), 23),
  ((26, 'm'), 24),
  ((27, 'm'), 25),
  ((28, 'm'), 26),
  ((29, 'm'), 27),
  ((30, 'm'), 28),
  ((31, 'd'), None),
  ((32, 'd'), None),
  ((33, 'd'), None),
  ((34, 'd'), None),
  ((35, 'm'), 29),
  ((36, 'm'), 30),
  ((37, 'm'), 31),
  ((38, 'm'), 32),
  ((39, 'm'), 33),
  ((40, 'm'), 34),
  ((41, 'm'), 35),
  ((42, 'm'), 36),
  ((43, 'm'), 37),
  ((44, 'm'), 38),
  ((45, 'm'), 39),
  ((46, 'm'), 40),
  ((47, 'm'), 41),
  ((48, 'm'), 42),
  ((49, 'm'), 43),
  ((50, 'm'), 44),
  ((51, 'm'), 45),
  ((52, 'm'), 46),
  ((53, 'm'), 47),
  

In [119]:
t_seqs2[3]

('7c2l:H',
 'EVQLVESGAEVKKPGASVKVSCKVSGYTLTELSMHWVRQAPGKGLEWMGGFDPEDGETMYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYYCATSTAVAGTPDLFDYYYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK')

In [125]:
t2_sequence=t_seqs2[3][1]
t2_state_vector=t2_alignments[3][1][0]

In [126]:
_number_regions(t2_sequence, t2_state_vector, state_string , region_string,  region_index_dict, rels, 7, [1,3,5])

([[((1, ' '), 'E'),
   ((2, ' '), 'V'),
   ((3, ' '), 'Q'),
   ((4, ' '), 'L'),
   ((5, ' '), 'V'),
   ((6, ' '), 'E'),
   ((7, ' '), 'S'),
   ((8, ' '), 'G'),
   ((9, ' '), 'A'),
   ((11, ' '), 'E'),
   ((12, ' '), 'V'),
   ((13, ' '), 'K'),
   ((14, ' '), 'K'),
   ((15, ' '), 'P'),
   ((16, ' '), 'G'),
   ((17, ' '), 'A'),
   ((18, ' '), 'S'),
   ((19, ' '), 'V'),
   ((20, ' '), 'K'),
   ((21, ' '), 'V'),
   ((22, ' '), 'S'),
   ((23, ' '), 'C'),
   ((24, ' '), 'K'),
   ((25, ' '), 'V'),
   ((26, ' '), 'S')],
  [((27, ' '), 'G'),
   ((28, ' '), 'Y'),
   ((29, ' '), 'T'),
   ((30, ' '), 'L'),
   ((35, ' '), 'T'),
   ((36, ' '), 'E'),
   ((37, ' '), 'L'),
   ((38, ' '), 'S')],
  [((39, ' '), 'M'),
   ((40, ' '), 'H'),
   ((41, ' '), 'W'),
   ((42, ' '), 'V'),
   ((43, ' '), 'R'),
   ((44, ' '), 'Q'),
   ((45, ' '), 'A'),
   ((46, ' '), 'P'),
   ((47, ' '), 'G'),
   ((48, ' '), 'K'),
   ((49, ' '), 'G'),
   ((50, ' '), 'L'),
   ((51, ' '), 'E'),
   ((52, ' '), 'W'),
   ((53, ' '), 'M'),

In [137]:
k_state_string='XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXXIXIIXXXXXXXXXXXIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXX'
k_region_string='11111111112222222222222333333333333333334444444444444455555555555666666666666666666666666666666666666666777777777777788888888888'
k_region_index_dict = {"1":0,"2":1,"3":2,"4":3,"5":4,"6":5,"7":6,"8":7}
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
k_rels              =  {0:0, 
                         1:-1,
                         2:-1,
                         3:-5,
                         4:-5,
                         5:-8,
                         6:-12,
                         7:-15}    
    
k_n_regions = 8
    
k_exclude_deletions = [2,4,6]


In [142]:
_number_regions(t2_sequence, t2_state_vector, k_state_string , k_region_string,  k_region_index_dict, k_rels, 8, [2,4,6])

4 58 -12
5 92 -18
5 93 -19
5 94 -20
6 113 -19
6 114 -20
6 115 -21


([[((-1, ' '), 'E'),
   ((0, ' '), 'V'),
   ((1, ' '), 'Q'),
   ((2, ' '), 'L'),
   ((3, ' '), 'V'),
   ((4, ' '), 'E'),
   ((5, ' '), 'S'),
   ((6, ' '), 'G'),
   ((7, ' '), 'A')],
  [((10, ' '), 'E'),
   ((11, ' '), 'V'),
   ((12, ' '), 'K'),
   ((13, ' '), 'K'),
   ((14, ' '), 'P'),
   ((15, ' '), 'G'),
   ((16, ' '), 'A'),
   ((17, ' '), 'S'),
   ((18, ' '), 'V'),
   ((19, ' '), 'K'),
   ((20, ' '), 'V'),
   ((21, ' '), 'S'),
   ((22, ' '), 'C')],
  [((15, ' '), 'K'),
   ((16, ' '), 'V'),
   ((17, ' '), 'S'),
   ((18, ' '), 'G'),
   ((19, ' '), 'Y'),
   ((20, ' '), 'T'),
   ((21, ' '), 'L'),
   ((22, ' '), 'T'),
   ((23, ' '), 'E'),
   ((24, ' '), 'L'),
   ((25, ' '), 'S'),
   ((26, ' '), 'M'),
   ((27, ' '), 'H')],
  [((36, ' '), 'W'),
   ((37, ' '), 'V'),
   ((38, ' '), 'R'),
   ((39, ' '), 'Q'),
   ((40, ' '), 'A'),
   ((41, ' '), 'P'),
   ((42, ' '), 'G'),
   ((43, ' '), 'K'),
   ((44, ' '), 'G'),
   ((45, ' '), 'L'),
   ((46, ' '), 'E'),
   ((47, ' '), 'W'),
   ((48, ' '), 'M'

In [130]:
len("XXXXXXXXXIXXXXXXXXXXXXXXXXXXXX")

30

# Figure out the meaning of "I" in the state_string in the numbering function
## The state_string in function `number_kabat_heavy` is not all "X"
### Comparison between IMGT and Kabat: https://www.imgt.org/IMGTScientificChart/Numbering/IMGT-Kabat_part1.html
### Comparison between different numbering schemes: https://plueckthun.bioc.uzh.ch/antibody/Numbering/NumFrame.html

In [ ]:
state_string =  'XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXXIXIIXXXXXXXXXXXIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXX'

In [134]:
len("XXXXXXXXX")

9

In [133]:
len("XXXXXXXXXIXXXXXXXXXXXXXXXXXXXX")

30

In [135]:
len("XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXX")

57

#### rels:
The difference between the start position of regions in different numbering schemes. Check comparison between IMGT and Kabat:https://www.imgt.org/IMGTScientificChart/Numbering/IMGT-Kabat_part1.html

In [131]:
len("11111111112222222222222")

23

In [132]:
len("1111111111222222222222233333333333333333")

40

In [136]:
len("XXXXXXXXXI")

10

In [144]:
number_kabat_heavy(t2_state_vector, t2_sequence)

4 58 -6
5 92 -10
5 93 -11
5 94 -12
6 113 -13
6 114 -14
6 115 -15


[[],
 [((10, ' '), 'E'),
  ((11, ' '), 'V'),
  ((12, ' '), 'K'),
  ((13, ' '), 'K'),
  ((14, ' '), 'P'),
  ((15, ' '), 'G'),
  ((16, ' '), 'A'),
  ((17, ' '), 'S'),
  ((18, ' '), 'V'),
  ((19, ' '), 'K'),
  ((20, ' '), 'V'),
  ((21, ' '), 'S'),
  ((22, ' '), 'C')],
 [],
 [((36, ' '), 'W'),
  ((37, ' '), 'V'),
  ((38, ' '), 'R'),
  ((39, ' '), 'Q'),
  ((40, ' '), 'A'),
  ((41, ' '), 'P'),
  ((42, ' '), 'G'),
  ((43, ' '), 'K'),
  ((44, ' '), 'G'),
  ((45, ' '), 'L'),
  ((46, ' '), 'E'),
  ((47, ' '), 'W'),
  ((48, ' '), 'M'),
  ((49, ' '), 'G')],
 [],
 [((58, ' '), 'M'),
  ((59, ' '), 'Y'),
  ((60, ' '), 'A'),
  ((61, ' '), 'Q'),
  ((62, ' '), 'K'),
  ((63, ' '), 'F'),
  ((64, ' '), 'Q'),
  ((65, ' '), 'G'),
  ((66, ' '), 'R'),
  ((67, ' '), 'V'),
  ((68, ' '), 'T'),
  ((69, ' '), 'M'),
  ((70, ' '), 'T'),
  ((71, ' '), 'E'),
  ((72, ' '), 'D'),
  ((73, ' '), 'T'),
  ((74, ' '), 'S'),
  ((75, ' '), 'T'),
  ((76, ' '), 'D'),
  ((77, ' '), 'T'),
  ((78, ' '), 'A'),
  ((79, ' '), 'Y'),
  (

In [145]:
len("XXXXXXXXXIXXXXXXXXXXXXXXXXXXXX")

30

In [146]:
len("XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXXIXIIXXXXXXXXXXXIXXXXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXX")

112

# test

In [147]:
t_seqs2

[('7c2l:A',
  'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLD

In [149]:
anarci(t_seqs2, scheme="imgt", database="ALL_AND_C", output=False, outfile=None, csv=False, allow=set(["H","K","L","A","B","G","D"]),
           hmmerpath="", ncpu=None, assign_germline=False, allowed_species=['human','mouse'], bit_score_threshold=80)

HMMscanError: b'\nError: File existence/permissions problem in trying to open HMM file dat/HMMs/ALL_AND_C.hmm.\nHMM file dat/HMMs/ALL_AND_C.hmm not found (nor an .h3m binary of it); also looked in PFAMDB\n\n'

In [150]:
len("XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXX")

57

In [151]:
len("XXXXXXXXXIXXXXXXXXXXXXXXXXXXXXIIIIXXXXXXXXXXXXXXXXXXXXXXXIXIIXXXXXXXXXXX")

72

In [152]:
len("XXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

29

In [153]:
len("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIXXXXXXXXXXXXXXXXXXXXXX")

57

In [154]:
len("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIXXXXXXXXXXXXXXXXXXXXXXIIIIIIIXXXXXXXX")

72

In [155]:
len("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIXXXXXXXXXXXXXXXXXXXXXXIIIIIIIXXXXXXXXIXXXXXXX")

80

In [158]:
"I"*8+"X"*15+"I"*3+"X"*(26-15)+"X"*(38-26-2)+"X"*(45-38)+"I"*7+"X"*(84-76)+"I"*7*2+"X"*(96-84)+"I"*2+"X"*(104-96)+"X"*(117-104)+"X"*(128-117)

'IIIIIIIIXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIIXXXXXXXXIIIIIIIIIIIIIIXXXXXXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [159]:
len('IIIIIIIIXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIIXXXXXXXXIIIIIIIIIIIIIIXXXXXXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

129

In [160]:
len("|.........")

10

In [161]:
len("87654321|........|....|123|......|..| |........| |.|...|1234567|......|12345677654321|...|......|12|......| |...........| |.........".replace(" ",""))

128

In [164]:
"1"*8+"1"*15+"2"*3+"3"*(26-15)+"4"*(38-26-2)+"5"*(45-38)+"6"*7+"7"*(84-76)+"8"*7*2+"9"*(96-84)+"a"*2+"b"*(104-96)+"c"*(117-104)+"d"*(128-117)

'11111111111111111111111222333333333334444444444555555566666667777777788888888888888999999999999aabbbbbbbbcccccccccccccddddddddddd'

In [165]:
len("11111111111111111111111222333333333334444444444555555566666667777777788888888888888999999999999aabbbbbbbbcccccccccccccddddddddddd")

129

In [166]:
len("----ASPTSPKVFPLSLCSTQP----DGNVVIACLVQGFFPQ-EPLSVTWSESGQGV----TARNFPPSQDASG-----DLYTTSSQLTLPATQC--LAGKSVTCHVKHY---TNPSQDVTVPCP------------------------")

149

In [186]:
[t_seqs2[3]]

[('7c2l:H',
  'EVQLVESGAEVKKPGASVKVSCKVSGYTLTELSMHWVRQAPGKGLEWMGGFDPEDGETMYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYYCATSTAVAGTPDLFDYYYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK')]

In [229]:
t_alns2_all_and_c=new_run_hmmer([t_seqs2[3]],hmm_database="ALL_AND_C",hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species,hmmscan_out_fn="7c2l_H_V+C_allandc" )

In [222]:
t_alns2_all_and_c

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['human_H', '', 1.7e-53, 171.9, 1.2, 0, 128],
   ['mouse_H', '', 5.5e-50, 160.4, 0.7, 0, 128],
   ['human_H_C1', '', 1.1e-45, 147.0, 9.7, 127, 232],
   ['mouse_H_C1', '', 3.4e-36, 116.0, 6.5, 127, 228]],
  [[((1, 'm'), 0),
    ((2, 'm'), 1),
    ((3, 'm'), 2),
    ((4, 'm'), 3),
    ((5, 'm'), 4),
    ((6, 'm'), 5),
    ((7, 'm'), 6),
    ((8, 'm'), 7),
    ((9, 'm'), 8),
    ((10, 'd'), None),
    ((11, 'm'), 9),
    ((12, 'm'), 10),
    ((13, 'm'), 11),
    ((14, 'm'), 12),
    ((15, 'm'), 13),
    ((16, 'm'), 14),
    ((17, 'm'), 15),
    ((18, 'm'), 16),
    ((19, 'm'), 17),
    ((20, 'm'), 18),
    ((21, 'm'), 19),
    ((22, 'm'), 20),
    ((23, 'm'), 21),
    ((24, 'm'), 22),
    ((25, 'm'), 23),
    ((26, 'm'), 24),
    ((27, 'm'), 25),
    ((28, 'm'), 26),
    ((29, 'm'), 27),
    ((30, 'm'), 28),
    ((31, 'd'), None),
    ((32, 'd'), None),
    ((33, 'd'), None),
   

In [218]:
seq_7c2l_HC=[("7c2l:HC",
              "ASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK"
             )]

In [221]:
"ASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYRVVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFFLYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK"[0:103]

'ASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSC'

In [228]:
t_alns2_c_only=new_run_hmmer(seq_7c2l_HC,hmm_database="C_ONLY",hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species,hmmscan_out_fn="7c2l_H_C_conly" )

In [220]:
t_alns2_c_only

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['human_H_C1', '', 4.7e-46, 146.9, 9.2, 0, 103],
   ['mouse_H_C1', '', 1.5e-36, 115.8, 6.2, 0, 100]],
  [[((5, 'm'), 0),
    ((6, 'm'), 1),
    ((7, 'm'), 2),
    ((8, 'm'), 3),
    ((9, 'm'), 4),
    ((10, 'm'), 5),
    ((11, 'm'), 6),
    ((12, 'm'), 7),
    ((13, 'm'), 8),
    ((14, 'm'), 9),
    ((15, 'm'), 10),
    ((16, 'm'), 11),
    ((17, 'm'), 12),
    ((18, 'm'), 13),
    ((19, 'm'), 14),
    ((20, 'm'), 15),
    ((21, 'm'), 16),
    ((22, 'm'), 17),
    ((23, 'm'), 18),
    ((24, 'd'), None),
    ((25, 'd'), None),
    ((26, 'd'), None),
    ((27, 'm'), 19),
    ((28, 'm'), 20),
    ((29, 'm'), 21),
    ((30, 'm'), 22),
    ((31, 'm'), 23),
    ((32, 'm'), 24),
    ((33, 'm'), 25),
    ((34, 'm'), 26),
    ((35, 'm'), 27),
    ((36, 'm'), 28),
    ((37, 'm'), 29),
    ((38, 'm'), 30),
    ((39, 'm'), 31),
    ((40, 'm'), 32),
    ((41, 'm'), 33),
    ((42, 'd'), Non

In [191]:
t_alns2_all_and_c

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['human_H', '', 1.7e-53, 171.9, 1.2, 0, 128],
   ['mouse_H', '', 5.5e-50, 160.4, 0.7, 0, 128],
   ['human_H_C1', '', 1.1e-45, 147.0, 9.7, 127, 232],
   ['mouse_H_C1', '', 3.4e-36, 116.0, 6.5, 127, 228]],
  [[((1, 'm'), 0),
    ((2, 'm'), 1),
    ((3, 'm'), 2),
    ((4, 'm'), 3),
    ((5, 'm'), 4),
    ((6, 'm'), 5),
    ((7, 'm'), 6),
    ((8, 'm'), 7),
    ((9, 'm'), 8),
    ((10, 'd'), None),
    ((11, 'm'), 9),
    ((12, 'm'), 10),
    ((13, 'm'), 11),
    ((14, 'm'), 12),
    ((15, 'm'), 13),
    ((16, 'm'), 14),
    ((17, 'm'), 15),
    ((18, 'm'), 16),
    ((19, 'm'), 17),
    ((20, 'm'), 18),
    ((21, 'm'), 19),
    ((22, 'm'), 20),
    ((23, 'm'), 21),
    ((24, 'm'), 22),
    ((25, 'm'), 23),
    ((26, 'm'), 24),
    ((27, 'm'), 25),
    ((28, 'm'), 26),
    ((29, 'm'), 27),
    ((30, 'm'), 28),
    ((31, 'd'), None),
    ((32, 'd'), None),
    ((33, 'd'), None),
   

In [193]:
len(t_alns2_all_and_c[0])

3

In [197]:
len(t_alns2_all_and_c[0][1])

1

In [238]:
def number_imgt_c(state_vector, sequence):
    """    
    Apply the IMGT numbering scheme for heavy or light chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in IMGT scheme, I is an insertion. (All X's for IMGT)
    IIIIIIIIXXXXXXXXXXXXXXX III XXXXXXXXXXX XXXXXXXXXX XXXXXXX IIIIIII XXXXXXXX IIIIIIIIIIIIII XXXXXXXXXXXX II XXXXXXXX XXXXXXXXXXXXX XXXXXXXXXX
    11111111111111111111111 222 33333333333 4444444444 5555555 6666666 77777777 88888888888888 999999999999 aa bbbbbbbb ccccccccccccc dddddddddd

    Regions:
    1: A strand
    2: AB turn
    3: B strand
    4: BC loop
    5: C strand
    6: CD turn
    7: D strand
    8: DE turn
    9: E strand
    a: EF turn
    b: F strand
    c: FG loop
    d: G strand
    
    """
    
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'IIIIIIIIXXXXXXXXXXXXXXXIIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXIIIIIIIXXXXXXXXIIIIIIIIIIIIIIXXXXXXXXXXXXIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    # Or could consider to change the first character into X, and the numbering starts at 0, and then do some modification after that.
    # length:128
    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111111111111111111222333333333334444444444555555566666667777777788888888888888999999999999aabbbbbbbbcccccccccccccdddddddddd'

    region_index_dict = {
        "1":0,
        "2":1,
        "3":2,
        "4":3,
        "5":4,
        "6":5,
        "7":6,
        "8":7,
        "9":8,
        "a":9,
        "b":10,
        "c":11,
        "d":12
                         }
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                          1:0,
                          2:0,
                          3:0,
                          4:0,
                          5:0,
                          6:0,
                          7:0,
                          8:0,
                          9:0,
                          10:0,
                          11:0,
                          12:0
                          }
    
    n_regions = 13

    #exclude_deletions = [1,3,5]  
    exclude_deletions=[]
    
    # Needs to write another function for the C region: _number_regions_c
    # Since the current function won't allow the region to start with I? -Check this!
    # Or consider to re-write the region string: needs to check the "smooth" function to see if we can do this
    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    return _regions
    ###############
    # Renumbering #
    ###############

    _numbering = [ [],          # A strand
                   _regions[1], # AB turn
                   _regions[2], # B strand
                   [],          # BC
                   _regions[4], # C strand
                   _regions[5], # CD
                   _regions[6], # D strand
                   [],          # DE
                   _regions[8], # E strand
                   _regions[9], # EF
                   _regions[10],# F strand
                   [],          # FG
                   _regions[12] # G strand
    
                 ]

    # The alignment from HMMER should be correct for CDRs 1 and 2. Testing has shown not always the case and 'manual' renumbering 
    # is required as with the other schemes.
    
    # A strand: deal with the insersions at the beginning of the sequence: 
    # _regions[0]
    
    # BC loop: re-numbering: change the number 32& 33 into 34&35, and all the follwing numbering so that BC loop ends at 38
    # _regions[3]
    
    # DE loop: symmetric insersion
    # _regions[7]
    
    # FG loop: similar case as CDR3 loop
    # _regions[11]
    
    

    
    return gap_missing( _numbering ), startindex, endindex

In [233]:
t_alns2_c_only

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['human_H_C1', '', 4.7e-46, 146.9, 9.2, 0, 103],
   ['mouse_H_C1', '', 1.5e-36, 115.8, 6.2, 0, 100]],
  [[((5, 'm'), 0),
    ((6, 'm'), 1),
    ((7, 'm'), 2),
    ((8, 'm'), 3),
    ((9, 'm'), 4),
    ((10, 'm'), 5),
    ((11, 'm'), 6),
    ((12, 'm'), 7),
    ((13, 'm'), 8),
    ((14, 'm'), 9),
    ((15, 'm'), 10),
    ((16, 'm'), 11),
    ((17, 'm'), 12),
    ((18, 'm'), 13),
    ((19, 'm'), 14),
    ((20, 'm'), 15),
    ((21, 'm'), 16),
    ((22, 'm'), 17),
    ((23, 'm'), 18),
    ((24, 'd'), None),
    ((25, 'd'), None),
    ((26, 'd'), None),
    ((27, 'm'), 19),
    ((28, 'm'), 20),
    ((29, 'm'), 21),
    ((30, 'm'), 22),
    ((31, 'm'), 23),
    ((32, 'm'), 24),
    ((33, 'm'), 25),
    ((34, 'm'), 26),
    ((35, 'm'), 27),
    ((36, 'm'), 28),
    ((37, 'm'), 29),
    ((38, 'm'), 30),
    ((39, 'm'), 31),
    ((40, 'm'), 32),
    ((41, 'm'), 33),
    ((42, 'd'), Non

In [240]:
t_alns2_c_only[0][1][0]

[((5, 'm'), 0),
 ((6, 'm'), 1),
 ((7, 'm'), 2),
 ((8, 'm'), 3),
 ((9, 'm'), 4),
 ((10, 'm'), 5),
 ((11, 'm'), 6),
 ((12, 'm'), 7),
 ((13, 'm'), 8),
 ((14, 'm'), 9),
 ((15, 'm'), 10),
 ((16, 'm'), 11),
 ((17, 'm'), 12),
 ((18, 'm'), 13),
 ((19, 'm'), 14),
 ((20, 'm'), 15),
 ((21, 'm'), 16),
 ((22, 'm'), 17),
 ((23, 'm'), 18),
 ((24, 'd'), None),
 ((25, 'd'), None),
 ((26, 'd'), None),
 ((27, 'm'), 19),
 ((28, 'm'), 20),
 ((29, 'm'), 21),
 ((30, 'm'), 22),
 ((31, 'm'), 23),
 ((32, 'm'), 24),
 ((33, 'm'), 25),
 ((34, 'm'), 26),
 ((35, 'm'), 27),
 ((36, 'm'), 28),
 ((37, 'm'), 29),
 ((38, 'm'), 30),
 ((39, 'm'), 31),
 ((40, 'm'), 32),
 ((41, 'm'), 33),
 ((42, 'd'), None),
 ((43, 'd'), None),
 ((44, 'm'), 34),
 ((45, 'm'), 35),
 ((46, 'm'), 36),
 ((47, 'm'), 37),
 ((48, 'm'), 38),
 ((49, 'm'), 39),
 ((50, 'm'), 40),
 ((51, 'm'), 41),
 ((52, 'm'), 42),
 ((53, 'm'), 43),
 ((54, 'm'), 44),
 ((55, 'm'), 45),
 ((56, 'm'), 46),
 ((57, 'm'), 47),
 ((58, 'd'), None),
 ((59, 'd'), None),
 ((60, 'd')

In [243]:
number_imgt_c(t_alns2_c_only[0][1][0], seq_7c2l_HC)

IndexError: list index out of range

## Check the smooth_insertions function:

In [168]:
seq_3gk8_H=[("3gk8:H",
            "AVHLQGTELVKPGASAGVKLSCKASGYTFTNYDMNWVRQRPEQGLEWIGWIFPGDGSTRYNEKFKGKATLTTDKSSSTAYQLNRLTSEDSAVYFCARRGFHGSYSFAYWGQGTLVTVSGAKTTAPSVYPLAPAAGAAGAGSSVTLGCLVKGYFPEPVTLTWNSGSLSSGVHTFPAVLADLYTLSSSVTVTSSTWPAESITCNVAHPASSTKVDKKIEPRG"
            )]

In [169]:
t_alns3=run_hmmer(seq_3gk8_H,hmm_database=database,hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species )

In [244]:
t_alns3=new_run_hmmer(seq_3gk8_H,hmm_database=database,hmmerpath=hmmerpath,ncpu=ncpu,bit_score_threshold=bit_score_threshold,hmmer_species=allowed_species, hmmscan_out_fn="_3gk8" )

In [170]:
t_alns3

[([['id',
    'description',
    'evalue',
    'bitscore',
    'bias',
    'query_start',
    'query_end'],
   ['mouse_H', '', 1.5e-49, 158.2, 0.3, 5, 118],
   ['human_H', '', 1.5e-43, 139.0, 0.5, 5, 118]],
  [[((8, 'm'), 5),
    ((9, 'm'), 6),
    ((10, 'd'), None),
    ((11, 'm'), 7),
    ((12, 'm'), 8),
    ((13, 'm'), 9),
    ((14, 'm'), 10),
    ((15, 'm'), 11),
    ((16, 'm'), 12),
    ((17, 'm'), 13),
    ((18, 'm'), 14),
    ((19, 'i'), 15),
    ((19, 'i'), 16),
    ((19, 'm'), 17),
    ((20, 'm'), 18),
    ((21, 'm'), 19),
    ((22, 'm'), 20),
    ((23, 'm'), 21),
    ((24, 'm'), 22),
    ((25, 'm'), 23),
    ((26, 'm'), 24),
    ((27, 'm'), 25),
    ((28, 'm'), 26),
    ((29, 'm'), 27),
    ((30, 'm'), 28),
    ((31, 'd'), None),
    ((32, 'd'), None),
    ((33, 'd'), None),
    ((34, 'd'), None),
    ((35, 'm'), 29),
    ((36, 'm'), 30),
    ((37, 'm'), 31),
    ((38, 'm'), 32),
    ((39, 'm'), 33),
    ((40, 'm'), 34),
    ((41, 'm'), 35),
    ((42, 'm'), 36),
    ((43, 'm'

In [176]:
len(t_alns3),len(t_alns3[0]),len(t_alns3[0][1])

(1, 3, 1)

In [177]:
t_alns3[0][1][0]

[((8, 'm'), 5),
 ((9, 'm'), 6),
 ((10, 'd'), None),
 ((11, 'm'), 7),
 ((12, 'm'), 8),
 ((13, 'm'), 9),
 ((14, 'm'), 10),
 ((15, 'm'), 11),
 ((16, 'm'), 12),
 ((17, 'm'), 13),
 ((18, 'm'), 14),
 ((19, 'i'), 15),
 ((19, 'i'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'm'), 23),
 ((26, 'm'), 24),
 ((27, 'm'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 ((57, 'm'), 51),
 ((58, 'm'), 52),
 ((59, 'm'), 53),
 ((60, 'd'), None),
 ((61

In [178]:
t_aln3_states_before_smoothing=t_alns3[0][1][0]

In [179]:
t_aln3_states_after_smoothing=smooth_insertions( t_aln3_states_before_smoothing )

In [180]:
t_aln3_states_before_smoothing

[((8, 'm'), 5),
 ((9, 'm'), 6),
 ((10, 'd'), None),
 ((11, 'm'), 7),
 ((12, 'm'), 8),
 ((13, 'm'), 9),
 ((14, 'm'), 10),
 ((15, 'm'), 11),
 ((16, 'm'), 12),
 ((17, 'm'), 13),
 ((18, 'm'), 14),
 ((19, 'i'), 15),
 ((19, 'i'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'm'), 23),
 ((26, 'm'), 24),
 ((27, 'm'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 ((57, 'm'), 51),
 ((58, 'm'), 52),
 ((59, 'm'), 53),
 ((60, 'd'), None),
 ((61

In [181]:
t_aln3_states_after_smoothing

[((6, 'm'), 5),
 ((7, 'm'), 6),
 ((8, 'm'), 7),
 ((9, 'm'), 8),
 ((11, 'm'), 9),
 ((12, 'm'), 10),
 ((13, 'm'), 11),
 ((14, 'm'), 12),
 ((15, 'm'), 13),
 ((16, 'm'), 14),
 ((17, 'm'), 15),
 ((18, 'm'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'm'), 23),
 ((26, 'm'), 24),
 ((27, 'm'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 ((57, 'm'), 51),
 ((58, 'm'), 52),
 ((59, 'm'), 53),
 ((60, 'd'), None),
 ((61, 'd'), None),
 ((62, 

In [245]:
fw1_before=[((8, 'm'), 5),
 ((9, 'm'), 6),
 ((10, 'd'), None),
 ((11, 'm'), 7),
 ((12, 'm'), 8),
 ((13, 'm'), 9),
 ((14, 'm'), 10),
 ((15, 'm'), 11),
 ((16, 'm'), 12),
 ((17, 'm'), 13),
 ((18, 'm'), 14),
 ((19, 'i'), 15),
 ((19, 'i'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20)]

In [263]:
def test_fw1(state_buffer):
    sv=[]
    nins = sum( 1 for s in state_buffer if s[0][1] == 'i' )
    nt_dels = state_buffer[0][0][0] - 1 # Missing states at the beginning (N-terminal missing states)
    
    for (_id, _type ), _si in state_buffer: # Explicit deletion states.
        if _type == 'd' or _si == None:
            nt_dels +=1
        else: # First residue found
            break
    if nt_dels >= nins: # More n terminal deletions than insertions found. Likely misalignment.
        # Preserve the deleted states structure by using the same match annotations
        new_states = [ s for s, _ in state_buffer if s[1] == 'm'] # DG: states of m only
        _first = new_states[0][0]

        # Remove the deletions so that only residue positions are included
        state_buffer = [ s for s in state_buffer if s[0][1] != 'd' ] # DG: states of m and i

        # Extend N terminal states backwards from the first match states
        _add = len( state_buffer ) - len( new_states ) # DG: the length of i
        assert _add >= 0, 'Implementation logic error' # Should be adding a positive number of positions
        new_states = [ (_,'m') for _ in range( _first - _add, _first ) ] + new_states 
        return new_states
        assert len(new_states)==len(state_buffer), 'Implementation logic error' # Should have the same length

        # Assign them preserving the order of the sequence.
        for i in range( len(state_buffer ) ):
            sv.append( ( new_states[i], state_buffer[i][1]) )
    else:
        sv += state_buffer # The insertions may be incorrect but unknown what to do. Let the alignment place.
    return sv

In [261]:
test_fw1(fw1_before)

([((8, 'm'), 5),
  ((9, 'm'), 6),
  ((10, 'd'), None),
  ((11, 'm'), 7),
  ((12, 'm'), 8),
  ((13, 'm'), 9),
  ((14, 'm'), 10),
  ((15, 'm'), 11),
  ((16, 'm'), 12),
  ((17, 'm'), 13),
  ((18, 'm'), 14),
  ((19, 'i'), 15),
  ((19, 'i'), 16),
  ((19, 'm'), 17),
  ((20, 'm'), 18),
  ((21, 'm'), 19),
  ((22, 'm'), 20)],
 [(8, 'm'),
  (9, 'm'),
  (11, 'm'),
  (12, 'm'),
  (13, 'm'),
  (14, 'm'),
  (15, 'm'),
  (16, 'm'),
  (17, 'm'),
  (18, 'm'),
  (19, 'm'),
  (20, 'm'),
  (21, 'm'),
  (22, 'm')])

In [264]:
test_fw1(fw1_before)

[(6, 'm'),
 (7, 'm'),
 (8, 'm'),
 (9, 'm'),
 (11, 'm'),
 (12, 'm'),
 (13, 'm'),
 (14, 'm'),
 (15, 'm'),
 (16, 'm'),
 (17, 'm'),
 (18, 'm'),
 (19, 'm'),
 (20, 'm'),
 (21, 'm'),
 (22, 'm')]

In [296]:
def test_smooth_insertions(state_vector):
    '''
    The function aims to correct to the expected imgt alignment. Renumbering functions then translate from the imgt scheme to the
    appropriate scheme.

    Handle insertions made by HMMER that we suspect may be in the wrong position.
    Edge cases include:
        - Insertions at the C terminal of fw1, fw3 and fw3 regions. Can occur when 'conserved' residues have been mutated and the
          same amino acid appears in the the following CDR (e.g. mutate cysteine at 104 but the CDR3 has one or more cysteines)
        - Same as above possible (but not observed in structure seqs) for N terminal of fw2, fw3 and fw4... TODO
        - Heavily mutated N terminal regions that are partially recognised (e.g. 3gk8 chain H). Insertions should not be allowed
          before N terminal deletions have been used. Preserve deletion locations that are not N terminal (e.g. 10 in IMGT H) if
          the gap has been placed by the alignment.

    '''
    # Small overhead doing these corrections but worth it for reducing edge cases.

    # Enforce insertion patterns as below. The CDRs are renumbered in each case so that insertions are placed accoring to the scheme
#  '11111111111111111111111111222222222222333333333333333334444444444555555555555555555555555555555555555555666666666666677777777777'
#  '                        mmmi                         mmmi                                             mmmi                      '
#  '                        mmmi        immm             mmmi      immm                                   mmmi         immm         '

    # Enforce any insertions at the end and beginning of framework regions to be moved into the CDR region for renumbering.
    enforced_patterns = [ [(25,'m'),(26,'m'),( 27,'m'),( 28,'i')],
                          [(38,'i'),(38,'m'),(39,'m'),(40,'m')],
                          [(54,'m'),(55,'m'),(56,'m'),(57,'i')],
                          [(65,'i'),(65,'m'),(66,'m'),(67,'m')],
                          [(103,'m'),(104,'m'),(105,'m'),(106,'i')],
                          [(117,'i'),(117,'m'),(118,'m'),(119,'m')] ]

    # Insertions in FW1 are only allowed if there are a fewer number of n-terminal deletions made.

    state_buffer = []
    sv = []
    for (state_id, state_type ), si in state_vector:
        #print (state_id-1,state_buffer)
        if state_id < 23: # Everything before the cysteine at 23.
            state_buffer.append( ((state_id, state_type ), si) )
            reg = -1
        elif 25 <= state_id < 28: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 0
        elif 37 < state_id <= 40: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 1
        elif 54 <= state_id < 57: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 2
        elif 64 < state_id <= 67: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 3
        elif 103 <= state_id < 106: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 4
        elif 116 < state_id <= 119: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 5
        elif len(state_buffer) != 0: # Add the buffer and reset
            # Find the number of insertions in the buffer
            nins = sum( 1 for s in state_buffer if s[0][1] == 'i' )

            # If there are insertions, adjust the alignment
            if nins > 0: # We have insertions

                if reg == -1: # FW1, only adjust if there are the same or more N terminal deletions than insertions
                    nt_dels = state_buffer[0][0][0] - 1 # Missing states at the beginning
                    for (_id, _type ), _si in state_buffer: # Explicit deletion states.
                        if _type == 'd' or _si == None:
                            nt_dels +=1
                        else: # First residue found
                            break
                    if nt_dels >= nins: # More n terminal deletions than insertions found. Likely misalignment.

                        # Preserve the deleted states structure by using the same match annotations
                        new_states = [ s for s, _ in state_buffer if s[1] == 'm'] # DG: states of m only
                        _first = new_states[0][0]

                        # Remove the deletions so that only residue positions are included
                        state_buffer = [ s for s in state_buffer if s[0][1] != 'd' ] # DG: states of m and i

                        # Extend N terminal states backwards from the first match states
                        _add = len( state_buffer ) - len( new_states ) # DG: the length of i
                        assert _add >= 0, 'Implementation logic error' # Should be adding a positive number of positions
                        new_states = [ (_,'m') for _ in range( _first - _add, _first ) ] + new_states 
                        assert len(new_states)==len(state_buffer), 'Implementation logic error' # Should have the same length

                        # Assign them preserving the order of the sequence.
                        for i in range( len(state_buffer ) ):
                            sv.append( ( new_states[i], state_buffer[i][1]) )
                    else:
                        sv += state_buffer # The insertions may be incorrect but unknown what to do. Let the alignment place.
                else:
                    # Remove any deletions in the buffer. Unlikely to happen but do anyway
                    #print ("here")
                    
                    state_buffer = [ s for s in state_buffer if s[0][1] != 'd' ]
                    print (state_id,state_buffer,enforced_patterns[reg])

                    # Define the new states defined by the enforced pattern and the length of the buffer
                    if reg % 2: # nterm fw
        
                        new_states = [enforced_patterns[reg][0]]* + enforced_patterns[reg][ max( 4-len(state_buffer), 1):]
                    else: # cterm fw
                        new_states = enforced_patterns[reg][:3] + [enforced_patterns[reg][2]]*max( 0, len(state_buffer)-3)
                    # Assign them preserving the order of the sequence.
                    for i in range( len(state_buffer ) ):
                        sv.append( ( new_states[i], state_buffer[i][1]) )

            else: # Nothing to do - either all match or deletion states.
                sv += state_buffer

            # Add the current state
            sv.append( ((state_id, state_type ), si) )

            # Reset state buffer
            
            state_buffer = []

        else: # Simply append
            sv.append( ((state_id, state_type ), si) )


    return sv


In [288]:
test_smooth_insertions( t_aln3_states_before_smoothing )

7 []
8 [((8, 'm'), 5)]
9 [((8, 'm'), 5), ((9, 'm'), 6)]
10 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None)]
11 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'), 7)]
12 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'), 7), ((12, 'm'), 8)]
13 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'), 7), ((12, 'm'), 8), ((13, 'm'), 9)]
14 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'), 7), ((12, 'm'), 8), ((13, 'm'), 9), ((14, 'm'), 10)]
15 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'), 7), ((12, 'm'), 8), ((13, 'm'), 9), ((14, 'm'), 10), ((15, 'm'), 11)]
16 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'), 7), ((12, 'm'), 8), ((13, 'm'), 9), ((14, 'm'), 10), ((15, 'm'), 11), ((16, 'm'), 12)]
17 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'), 7), ((12, 'm'), 8), ((13, 'm'), 9), ((14, 'm'), 10), ((15, 'm'), 11), ((16, 'm'), 12), ((17, 'm'), 13)]
18 [((8, 'm'), 5), ((9, 'm'), 6), ((10, 'd'), None), ((11, 'm'

In [290]:
t_aln4_created=[
    ((8, 'm'), 5),
 ((9, 'm'), 6),
 ((10, 'd'), None),
 ((11, 'm'), 7),
 ((12, 'm'), 8),
 ((13, 'm'), 9),
 ((14, 'm'), 10),
 ((15, 'm'), 11),
 ((16, 'm'), 12),
 ((17, 'm'), 13),
 ((18, 'm'), 14),
 ((19, 'i'), 15),
 ((19, 'i'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'i'), 23),
 ((26, 'i'), 24),
 ((27, 'i'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 ((57, 'm'), 51),
 ((58, 'm'), 52),
 ((59, 'm'), 53),
 ((60, 'd'), None),
 ((61, 'd'), None),
 ((62, 'm'), 54),
 ((63, 'm'), 55),
 ((64, 'm'), 56),
 ((65, 'm'), 57),
 ((66, 'm'), 58),
 ((67, 'm'), 59),
 ((68, 'm'), 60),
 ((69, 'm'), 61),
 ((70, 'm'), 62),
 ((71, 'm'), 63),
 ((72, 'm'), 64),
 ((73, 'd'), None),
 ((74, 'm'), 65),
 ((75, 'm'), 66),
 ((76, 'm'), 67),
 ((77, 'm'), 68),
 ((78, 'm'), 69),
 ((79, 'm'), 70),
 ((80, 'm'), 71),
 ((81, 'm'), 72),
 ((82, 'm'), 73),
 ((83, 'm'), 74),
 ((84, 'm'), 75),
 ((85, 'm'), 76),
 ((86, 'm'), 77),
 ((87, 'm'), 78),
 ((88, 'm'), 79),
 ((89, 'd'), None),
 ((90, 'm'), 80),
 ((91, 'm'), 81),
 ((92, 'm'), 82),
 ((93, 'm'), 83),
 ((94, 'm'), 84),
 ((95, 'm'), 85),
 ((96, 'm'), 86),
 ((97, 'm'), 87),
 ((98, 'm'), 88),
 ((99, 'm'), 89),
 ((100, 'm'), 90),
 ((101, 'm'), 91),
 ((102, 'm'), 92),
 ((103, 'm'), 93),
 ((104, 'm'), 94),
 ((105, 'm'), 95),
 ((106, 'm'), 96),
 ((107, 'm'), 97),
 ((108, 'm'), 98),
 ((109, 'm'), 99),
 ((110, 'm'), 100),
 ((111, 'm'), 101),
 ((112, 'm'), 102),
 ((113, 'm'), 103),
 ((114, 'm'), 104),
 ((115, 'm'), 105),
 ((116, 'm'), 106),
 ((117, 'm'), 107),
 ((118, 'm'), 108),
 ((119, 'm'), 109),
 ((120, 'm'), 110),
 ((121, 'm'), 111),
 ((122, 'm'), 112),
 ((123, 'm'), 113),
 ((124, 'm'), 114),
 ((125, 'm'), 115),
 ((126, 'm'), 116),
 ((127, 'm'), 117),
 ((128, 'm'), 118)
]

In [297]:
test_smooth_insertions( t_aln4_created )

28 [((25, 'i'), 23), ((26, 'i'), 24), ((27, 'i'), 25)] [(25, 'm'), (26, 'm'), (27, 'm'), (28, 'i')]


[((6, 'm'), 5),
 ((7, 'm'), 6),
 ((8, 'm'), 7),
 ((9, 'm'), 8),
 ((11, 'm'), 9),
 ((12, 'm'), 10),
 ((13, 'm'), 11),
 ((14, 'm'), 12),
 ((15, 'm'), 13),
 ((16, 'm'), 14),
 ((17, 'm'), 15),
 ((18, 'm'), 16),
 ((19, 'm'), 17),
 ((20, 'm'), 18),
 ((21, 'm'), 19),
 ((22, 'm'), 20),
 ((23, 'm'), 21),
 ((24, 'm'), 22),
 ((25, 'm'), 23),
 ((26, 'm'), 24),
 ((27, 'm'), 25),
 ((28, 'm'), 26),
 ((29, 'm'), 27),
 ((30, 'm'), 28),
 ((31, 'd'), None),
 ((32, 'd'), None),
 ((33, 'd'), None),
 ((34, 'd'), None),
 ((35, 'm'), 29),
 ((36, 'm'), 30),
 ((37, 'm'), 31),
 ((38, 'm'), 32),
 ((39, 'm'), 33),
 ((40, 'm'), 34),
 ((41, 'm'), 35),
 ((42, 'm'), 36),
 ((43, 'm'), 37),
 ((44, 'm'), 38),
 ((45, 'm'), 39),
 ((46, 'm'), 40),
 ((47, 'm'), 41),
 ((48, 'm'), 42),
 ((49, 'm'), 43),
 ((50, 'm'), 44),
 ((51, 'm'), 45),
 ((52, 'm'), 46),
 ((53, 'm'), 47),
 ((54, 'm'), 48),
 ((55, 'm'), 49),
 ((56, 'm'), 50),
 ((57, 'm'), 51),
 ((58, 'm'), 52),
 ((59, 'm'), 53),
 ((60, 'd'), None),
 ((61, 'd'), None),
 ((62, 

In [298]:
def _number_regions_n(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions):
    """
    General function to number a sequence and divide it into different regions

    @param sequence: The sequence string
    @param state_vector: The list of states from the aligned hmm
    @param state_string: A string of states for the scheme relative to IMGT (this is X for a direct equivalence, I if needs to be treated as insertion)
    @param region_string: A string of characters that indicate which hmm states are in each regions for this scheme (i.e. how should the sequence be divided up)
    @param region_index_dict: A dictionary converting the characters in region string to an index of the regions.
    @param rels: The difference of the numbering integer at the *start* of each region
    @param n_regions: The number of regions
    @param exclude_deletions: A list of region indices for which deletion states should not be included. Typically the CDRs.
                              These will be reannotated in the scheme function. Also allows the reset of insertions.

    @return: A list of lists where each region has been numbered according to the scheme. Some regions will need renumbering. This should be taken care of after the function called.

    """

    #state_vector = smooth_insertions( state_vector )

    _regions = [ [] for _ in range(n_regions) ]

    # Initialise the insertion index (-1 is a blank space) and the previous state.
    insertion = -1
    previous_state_id = 1
    previous_state_type = 'd'
    start_index, end_index  = None, None

    region = None

    # Iterate over the aligned state vector
    for (state_id, state_type ), si in state_vector:

        # Retrieve the region index
        if state_type != "i" or region is None: # BUG_FIX - JD 9/4/15 - do not allow a new region to start as an insertion.
            region = region_index_dict[region_string[state_id-1]]


        # Check the state_types
        if state_type == "m": # It is a match

            # Check whether this position is in the scheme as an independent state
            if state_string[state_id-1]=="I": # No, it should be treated as an insertion
                if previous_state_type != 'd': # Unless there was a deletion beforehand in which case this should be a real pos.
                    insertion +=1 # Increment the insertion annotation index
                rels[region] -= 1 # Update the relative numbering from the imgt states
            else: # Yes
                insertion = -1 # Reset the insertions

            # Add the numbering annotation to the appropriate region list
            _regions[region].append( ( (state_id + rels[region], alphabet[insertion] ), sequence[si]  ) )
            previous_state_id = state_id # Record the previous state ID
            if start_index is None:
                start_index = si
            end_index = si

            previous_state_type = state_type

        elif state_type == "i": # It is an insertion
            insertion +=1 # Increment the insertion annotation index

            # Add the numbering annotation to the appropriate region list
            _regions[region].append( ( (previous_state_id + rels[region], alphabet[insertion]), sequence[si]  ) )
            if start_index is None:
                start_index = si
            end_index = si

            previous_state_type = state_type

        else: # It is a deletion
            previous_state_type = state_type

            # Check whether this position is in the scheme as an independent state
            if state_string[state_id-1]=="I": # No, therefore irrelevant to the scheme.
                rels[region] -= 1 # Update the relative numbering from the imgt states
                continue

            insertion = -1 # Reset the insertions
            previous_state_id = state_id # Record the previous state ID, should not be needed (no delete to insert state transition)


        # Reset the inssertion index if necessary and allowed. (Means the insertion code is meaningless and will be reannotated)
        if insertion >= 25 and region in exclude_deletions:
            insertion = 0

        assert insertion < 25, "Too many insertions for numbering scheme to handle" # We ran out of letters.

    return _regions, start_index, end_index

def new_number_imgt(state_vector, sequence):
    """
    Apply the IMGT numbering scheme for heavy or light chains

    Rules should be implemented using two strings - the state string and the region string.

    There are 128 states in the HMMs. Treat X as a direct match in IMGT scheme, I is an insertion. (All X's for IMGT)
    XXXXXXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXX XXXXXXXXXXXXXXXXX XXXXXXXXXX XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXX
    11111111111111111111111111 222222222222 33333333333333333 4444444444 555555555555555555555555555555555555555 6666666666666 77777777777

    Regions - (N.B These do not match up with any particular definition of CDR)
    1. All positions before CDR1
    2. CDR1 positions
    3. Positions between CDR1/2
    4. CDR2 positions
    5. Positions between CDR2/3
    6. CDR positions 105 (inc) to 118 (exc)
    7. Positions after CDR3

    """

    # Set up the numbering

    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

    # Region string - regions that should be treated separately in putting the numbering together
    region_string = '11111111111111111111111111222222222222333333333333333334444444444555555555555555555555555555555555555555666666666666677777777777'

    region_index_dict = {
                         "1":0,
                         "2":1,
                         "3":2,
                         "4":3,
                         "5":4,
                         "6":5,
                         "7":6
                         }

    # Define how the scheme's numbering differs from IMGT at the start of each region.
    # This is updated in the loop below
    rels              =  {0:0,
                          1:0,
                          2:0,
                          3:0,
                          4:0,
                          5:0,
                          6:0,
                          7:0
                          }

    n_regions = 7

    exclude_deletions = [1,3,5]

    _regions, startindex, endindex = _number_regions_n(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)

    ###############
    # Renumbering #
    ###############

    _numbering = [ _regions[0], # Fw1
                   [],          # CDR1
                   _regions[2], # Fw2
                   [],          # CDR2
                   _regions[4], # Fw3
                   [],          # CDR3
                   _regions[6], # Fw4

                 ]

    # The alignment from HMMER should be correct for CDRs 1 and 2. Testing has shown not always the case and 'manual' renumbering
    # is required as with the other schemes.

    # CDR1
    # CDR1 has a range from 27 (inc.) to 39 (exc.) and has a theoretical maximum length of 12.
    cdr1seq    = "".join([ x[1] for x in _regions[1] if x[1] != "-" ])
    cdr1length = len(cdr1seq)
    si = 0
    prev_state = 26
    for ann in get_imgt_cdr(cdr1length, 12, 27, 39):
        if not ann:
            _numbering[1].append( ((prev_state+1, ' '), '-') )
            prev_state += 1
        else:
            _numbering[1].append( (ann, cdr1seq[si]) )
            prev_state = ann[0]
            si += 1

    # CDR2
    # CDR2 has a range from 56 (inc.) to 66 (exc.) and has a theoretical length of 10.
    cdr2seq    = "".join([ x[1] for x in _regions[3] if x[1] != "-" ])
    cdr2length = len(cdr2seq)
    si = 0
    prev_state = 55
    for ann in get_imgt_cdr(cdr2length, 10, 56, 66):
        if not ann:
            _numbering[3].append( ((prev_state+1, ' '), '-') )
            prev_state += 1
        else:
            _numbering[3].append( (ann, cdr2seq[si]) )
            prev_state = ann[0]
            si += 1

    # FW3. We allow the HMM to place insertions. Technically all insertion points are taken care of but in reality insertions can
    # and do occur. No specification of where the insertions should be placed.


    # CDR3
    # CDR3 has a range from 105 (inc.) to 118 (exc.). Insertions are placed on 112 and 111 symetrically. IMGT has a technical
    # maximum length of 65 (13 positions, 26*2 insertions) . In practice ANARCI will not recognise CDR3s of this length.
    cdr3seq    = "".join([ x[1] for x in _regions[5] if x[1] != "-" ])
    cdr3length = len(cdr3seq)
    if cdr3length > 117: return [], startindex, endindex # Too many insertions. Do not apply numbering.
    si = 0
    previous_state_id = 104
    for ann in get_imgt_cdr(cdr3length, 13, 105, 118):
        if ann is None:
            _numbering[5].append( ((previous_state_id+1, " "), "-"   ) )
            previous_state_id+=1
        else:
            _numbering[5].append( (ann, cdr3seq[si] ) )
            previous_state_id = ann[0]
            si+=1

    # Return the full vector and the start and end indices of the numbered region of the sequence
    return gap_missing( _numbering ), startindex, endindex

In [300]:
seq_3gk8_H

[('3gk8:H',
  'AVHLQGTELVKPGASAGVKLSCKASGYTFTNYDMNWVRQRPEQGLEWIGWIFPGDGSTRYNEKFKGKATLTTDKSSSTAYQLNRLTSEDSAVYFCARRGFHGSYSFAYWGQGTLVTVSGAKTTAPSVYPLAPAAGAAGAGSSVTLGCLVKGYFPEPVTLTWNSGSLSSGVHTFPAVLADLYTLSSSVTVTSSTWPAESITCNVAHPASSTKVDKKIEPRG')]

In [301]:
new_number_imgt(t_aln3_states_before_smoothing, seq_3gk8_H[0][1])

([((1, ' '), '-'),
  ((2, ' '), '-'),
  ((3, ' '), '-'),
  ((4, ' '), '-'),
  ((5, ' '), '-'),
  ((6, ' '), '-'),
  ((7, ' '), '-'),
  ((8, ' '), 'G'),
  ((9, ' '), 'T'),
  ((10, ' '), '-'),
  ((11, ' '), 'E'),
  ((12, ' '), 'L'),
  ((13, ' '), 'V'),
  ((14, ' '), 'K'),
  ((15, ' '), 'P'),
  ((16, ' '), 'G'),
  ((17, ' '), 'A'),
  ((18, ' '), 'S'),
  ((18, 'A'), 'A'),
  ((18, 'B'), 'G'),
  ((19, ' '), 'V'),
  ((20, ' '), 'K'),
  ((21, ' '), 'L'),
  ((22, ' '), 'S'),
  ((23, ' '), 'C'),
  ((24, ' '), 'K'),
  ((25, ' '), 'A'),
  ((26, ' '), 'S'),
  ((27, ' '), 'G'),
  ((28, ' '), 'Y'),
  ((29, ' '), 'T'),
  ((30, ' '), 'F'),
  ((31, ' '), '-'),
  ((32, ' '), '-'),
  ((33, ' '), '-'),
  ((34, ' '), '-'),
  ((35, ' '), 'T'),
  ((36, ' '), 'N'),
  ((37, ' '), 'Y'),
  ((38, ' '), 'D'),
  ((39, ' '), 'M'),
  ((40, ' '), 'N'),
  ((41, ' '), 'W'),
  ((42, ' '), 'V'),
  ((43, ' '), 'R'),
  ((44, ' '), 'Q'),
  ((45, ' '), 'R'),
  ((46, ' '), 'P'),
  ((47, ' '), 'E'),
  ((48, ' '), 'Q'),
  ((49, '

In [302]:
number_imgt(t_aln3_states_before_smoothing, seq_3gk8_H[0][1])

([((1, ' '), '-'),
  ((2, ' '), '-'),
  ((3, ' '), '-'),
  ((4, ' '), '-'),
  ((5, ' '), '-'),
  ((6, ' '), 'G'),
  ((7, ' '), 'T'),
  ((8, ' '), 'E'),
  ((9, ' '), 'L'),
  ((10, ' '), '-'),
  ((11, ' '), 'V'),
  ((12, ' '), 'K'),
  ((13, ' '), 'P'),
  ((14, ' '), 'G'),
  ((15, ' '), 'A'),
  ((16, ' '), 'S'),
  ((17, ' '), 'A'),
  ((18, ' '), 'G'),
  ((19, ' '), 'V'),
  ((20, ' '), 'K'),
  ((21, ' '), 'L'),
  ((22, ' '), 'S'),
  ((23, ' '), 'C'),
  ((24, ' '), 'K'),
  ((25, ' '), 'A'),
  ((26, ' '), 'S'),
  ((27, ' '), 'G'),
  ((28, ' '), 'Y'),
  ((29, ' '), 'T'),
  ((30, ' '), 'F'),
  ((31, ' '), '-'),
  ((32, ' '), '-'),
  ((33, ' '), '-'),
  ((34, ' '), '-'),
  ((35, ' '), 'T'),
  ((36, ' '), 'N'),
  ((37, ' '), 'Y'),
  ((38, ' '), 'D'),
  ((39, ' '), 'M'),
  ((40, ' '), 'N'),
  ((41, ' '), 'W'),
  ((42, ' '), 'V'),
  ((43, ' '), 'R'),
  ((44, ' '), 'Q'),
  ((45, ' '), 'R'),
  ((46, ' '), 'P'),
  ((47, ' '), 'E'),
  ((48, ' '), 'Q'),
  ((49, ' '), 'G'),
  ((50, ' '), 'L'),
  ((51, '